In [1]:
# %%capture cap --no-stderr
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib import ticker
from matplotlib.ticker import ScalarFormatter
from IPython.display import display, HTML
import regex as re # finding specific patterns in str
import textwrap # split text into equal parts
import collections # return repeated items in list
from collections import OrderedDict
import time
from datetime import datetime # attach current date to export filename
import sys
import ast # convert str back to tuple/list/int, etc
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from matplotlib.colors import LinearSegmentedColormap
import string

## INPUT EXCEL FILES
byonicfile = '20241218_expi-jn1-rbd_Byonic_inSol_TC-byonic' # Please paste filename here. if None, please leave it as empty string '' (NOTE: DO NOT INCLUDE .XLSX HERE)
byosfile = '20241218_expi-JN1_Byologic_inSol_TC' # Please paste filename here. if None, please leave it as empty string '' (NOTE: DO NOT INCLUDE .XLSX HERE)
pglycofile = '' # Please paste filename here. if None, please leave it as empty string '' (NOTE: DO NOT INCLUDE .XLSX HERE)

## INPUT TARGET PROTEIN SEQUENCE FOR MAJOR GLYCOFORM TABLE (PGLYCO SITES MAY BE WRONG, THE SEQ HERE CAN ALSO HELP COMPUTER DETECT THE WRONG SITES)
seq = 'MFVFLVLLPLVSSQRVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNFHHHHHH'

# ntu156_seq = 'ANSDCVQVNVTQLPGNENIIKDFLFSNFKEEGTVVVGGYYPTEVWYNCSRTATTTAYEYFSNIHAFYFDMEAMENSTGNARGKPLLFHVHGEPVSVIIYISAYRDDVQQRPLLKHGLVCITESRNIDYNQFTSNQWNSICTGNDRKIPFSVIPTDNGTKIYGLEWNDESVTAYISGHSYSLNINTNWFNNVTLLYSRSSTATWQKSAAYVYQGVSNFTYYKLNNTNGLKTYELCENYEYCTGHATNVFAPIAGGYIPDGFSFNNWFLLTNSSTFVSGRFVTNQPLLVNCLWPVPSFGVAAQEFCFEGAQFSQCNGVSLNNTVDVIRFNLNLTADVQSGMGATVFSLNTTGGVILEISCYNDTVSESSFYSYGEIPFGVTDGPRYCYVLYNGTALKYLGTLPPSVKEIAISKWGHFYINGYNFFSTFPIDCISFNLTTGASGAFWTIAYTSYTEALVQVENTAIKKVTYCNSHINNIKCSQLTANLQNGFYPVASSDVGLVNKSVVLLPIFFAHTAVNITIDLGMKRSGYGQPIASTLSNITLPMQDNNTDVYCVRSNQFSVYTHSTCKSSLWDNNFNQDCTDVLYATAVIKTGTCPFSFDKLNNYLTFNKLCLSLNPTGANCKFDVAARTRTNEQVVRSLYVIYEEGDNIVGVPSDNSGPHDLSVLHLDSCTDYNIYGRTGVGIIRQTNSTILSGLYYTSLSGDLLGFKNVSDGVVYSVTPCDVSAQAAVIDGAIVGAMTSINSELLGLTHWTTTPNFYYYSIYNYTNERTRGTAIDSNAVDCEPIITYSNIGVCKNGALVFINVTHSDGDVQPISTGNVTIPTNFTISVQVEYIQVYTTPVSIDCSRYVCNGNPRCNKLLTQYVSACQTIEQALAMSARLENMEVDSMLFVSENALKLASVEAFNSTEHLDPIYKEWPNIGGSWLGGLKDILPSHNSKRKYRSAIEDLLFDKVVTSGLGTVDEDYKRCTGGYDIADLVCAQYYNGIMVLPGVANDDKMTMYTASLAGGITLGALGGGAVAIPFAVAVQARLNYVALQTDVLNKNQQILANAFNQAIGNITQAFGKVNDAIHQTSQGLATVAKALVKVQDVVNTQGQALSHLTVQLQNNFQAISSSISDIYNRLDPPSADAQVDRLITGRLTALNAFVSQTLTRQAEVRASRQLAKDKVNECVRSQSQRFGFCGNGTHLFSLANAAPNGMIFFHTVLLPTAYETVTAWSGICASDGDRTFGLVVKDVQLTLFRNLDDKFYLTPRTMYQPRAATSSDFVQIEGCDVLFVNATVIDLPSIIPDYIDINQTVQDILENYRPNWTVPELTLDIFNATYLNLTGEIDDLEFRSEKLHNTTVELAILIDNINNTLVNLEWLNRIETYVKWPGYIPEAPRDGQAYVRKDGEWVLLSTFLGGRSSLEGPRFEGKPIPNPLLGLDSTRTGHHHHHH'

# uu4_seq = 'DAPHGVTLPHFNTSHNNSKFELNFYNFLQTWDIPPNTETILGGYLPYCDHEDNCGWYNFVYNNKVGPNAKYSYINTQNLNIPNVHGVYFDVREHNSDGVWDQIDRVGLLIAIHGTSHYSLLMVLQDGVEASQPHVAVKICHWNPGNISTYHQFDVNLGDGGQCVFNQRFSLDTVLTANDFYGFQWTDTYVDIYLGGTITKVWVVNDWSVVEASISSHWNALNYGYYIQFVNRTTYYAYNSTGGSNYTHLQLTECHTDYCAGYAKNVFVPIDGKIPEGFSFSNWFLLTDKSTLVQGRVLSSQPVFVQCLRPVPTWSNNTAVVHFKNDVFCPNVTADVLRFNLNFSDTDVYTDSTTDDQLHFTFEDNTTASITCYSSANVTDNQPASGSISHTPFVSNSYLCFANFSHSSVSRQFLGILPPTVREFAFGRDGSIFVNGYKYFSLQPIKSVNFSISSVENYGFWTIAYTNYTDVMVDVNGTVITRLFYCDSPLNRIKCQQLKHELPDGFYSASMLVKKDLPKTFVTMPQFYNWMNVTLHVVLNDIEKKADIILAGAPELASLADIHFEIAQANGSVVNVTSVCVQARQLALFYKYTSLQGLYTYSNLVQLQNYDCPFSPQQFNNYLQFETLCFDVSPAVAGCKWSLVHDVKWRTQFATITVSYKDGAMITTMPKAQLGFQDISNIVKDECTDYNIYGFQGTGIIRSTTSRLVAGLYYTSASGDLLGFKISTTGEIFTVVPCDLTAQAAVINDEIVGAITATNQTDLFEFVNHTWSRSARGSSPSTVNTYTMPQFYYITKWNNGTSSNCTSVITYSSFAICNTGEIKYVNVTHVEIVDDSVGVIKPVSTGNITIPKNFTVAVQAEYVQIQVKPVAVDCAKYVCNGNRHCLNLLTQYTSACQTIENSLNLGARLESLMLNDMITVSDRSLEFATVDKFNTTALGGEKLGGLYFDGLSSLLPPRVGMRSAVEDLLFNKVVTSGLGTVDDDYKKCSAGTDVADLVCAQYYNGIMVLPGVVDYNKMAMYTASLIGGMALGSITSAVAVPFSMQVQARLNYVALQTDVLQENQKILANAFNNAIGNITLALGKVSNAITTVSDGFNSMASALTKIQSVVNQQGEALSHLISQLQKNFQAISSSIAEIYNRLEKVEADAQVDRLITGRLAALNAYVAQTLTQYAEVKASRQLAMEKVNECVKSQSDRYGFCGNGTHLFSLVNSAPDGLLFFHTVLLPTEWEEVTAWSGICVNDTYAYLLKDFDHSIFSYNGTYMVTPRNMFQPRKPQMSDFVQITSCEVTFLNTTHTTFQEIVIDYIDINKTIADMLEQYHSNYTTPELDLQLEIFNQTKLNLTAEIDQLEQRADNLTNIAHELQQYIDNLNKTIVDLEWLNRIETYVKWPGSGGYIPEAPRDGQAYVRKDGEWVLLSTFLKGQDNSADIQHSGGRSSLEGPRFEGKPIPNPLLGLDSTRTGHHHHHH'

# fm2p seq
# seq = 'MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHAIHVSGTNGTKRFDNPVLPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIKVCEFQFCNDPFLGVYYHKNNKSWMESEFRVYSSANNCTFEYVSQPFLMDLEGKQGNFKNLREFVFKNIDGYFKIYSKHTPINLVRDLPQGFSALEPLVDLPIGINITRFQTLLALHRSYLTPGDSSSGWTAGAAAYYVGYLQPRTFLLKYNENGTITDAVDCALDPLSETKCTLKSFTVEKGIYQTSNFRVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNFNFNGLTGTGVLTESNKKFLPFQQFGRDIADTTDAVRDPQTLEILDITPCSFGGVSVITPGTNTSNQVAVLYQDVNCTEVPVAIHADQLTPTWRVYSTGSNVFQTRAGCLIGAEHVNNSYECDIPIGAGICASYQTQTNSPGSASSVASQSIIAYTMSLGAENSVAYSNNSIAIPTNFTISVTTEILPVSMTKTSVDCTMYICGDSTECSNLLLQYGSFCTQLNRALTGIAVEQDKNTQEVFAQVKQIYKTPPIKDFGGFNFSQILPDPSKPSKRSFIEDLLFNKVTLADAGFIKQYGDCLGDIAARDLICAQKFNGLTVLPPLLTDEMIAQYTSALLAGTITSGWTFGAGAALQIPFAMQMAYRFNGIGVTQNVLYENQKLIANQFNSAIGKIQDSLSSTASALGKLQDVVNQNAQALNTLVKQLSSNFGAISSVLNDILSRLDPPEAEVQIDRLITGRLQSLQTYVTQQLIRAAEIRASANLAATKMSECVLGQSKRVDFCGKGYHLMSFPQSAPHGVVFLHVTYVPAQEKNFTTAPAICHDGKAHFPREGVFVSNGTHWFVTQRNFYEPQIITTDNTFVSGNCDVVIGIVNNTVYDPLQPELDSFKEELDKYFKNHTSPDVDLGDISGINASVVNIQKEIDRLNEVAKNLNESLIDLQELGKYEQ'

# omicron seq
# seq = 'MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHVISGTNGTKRFDNPVLPFNDGVYFASIEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIKVCEFQFCNDPFLDHKNNKSWMESEFRVYSSANNCTFEYVSQPFLMDLEGKQGNFKNLREFVFKNIDGYFKIYSKHTPIIVREPEDLPQGFSALEPLVDLPIGINITRFQTLLALHRSYLTPGDSSSGWTAGAAAYYVGYLQPRTFLLKYNENGTITDAVDCALDPLSETKCTLKSFTVEKGIYQTSNFRVQPTESIVRFPNITNLCPFDEVFNATRFASVYAWNRKRISNCVADYSVLYNLAPFFTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGNIADYNYKLPDDFTGCVIAWNSNKLDSKVSGNYNYLYRLFRKSNLKPFERDISTEIYQAGNKPCNGVAGFNCYFPLRSYSFRPTYGVGHQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNFNFNGLKGTGVLTESNKKFLPFQQFGRDIADTTDAVRDPQTLEILDITPCSFGGVSVITPGTNTSNQVAVLYQGVNCTEVPVAIHADQLTPTWRVYSTGSNVFQTRAGCLIGAEYVNNSYECDIPIGAGICASYQTQTKSHGSASSVASQSIIAYTMSLGAENSVAYSNNSIAIPTNFTISVTTEILPVSMTKTSVDCTMYICGDSTECSNLLLQYGSFCTQLKRALTGIAVEQDKNTQEVFAQVKQIYKTPPIKYFGGFNFSQILPDPSKPSKRSFIEDLLFNKVTLADAGFIKQYGDCLGDIAARDLICAQKFKGLTVLPPLLTDEMIAQYTSALLAGTITSGWTFGAGAALQIPFAMQMAYRFNGIGVTQNVLYENQKLIANQFNSAIGKIQDSLSSTASALGKLQDVVNHNAQALNTLVKQLSSKFGAISSVLNDIFSRLDPPEAEVQIDRLITGRLQSLQTYVTQQLIRAAEIRASANLAATKMSECVLGQSKRVDFCGKGYHLMSFPQSAPHGVVFLHVTYVPAQEKNFTTAPAICHDGKAHFPREGVFVSNGTHWFVTQRNFYEPQIITTDNTFVSGNCDVVIGIVNNTVYDPLQPELDSFKEELDKYFKNHTSPDVDLGDISGINASVVNIQKEIDRLNEVAKNLNESLIDLQELGKYEQEFGSGGYIPEAPRDGQAYVRKDGEWVLLSTFLKGQDNSADIQHSGRPLESRGPFEQKLISEEDLNMHTGHHHHHH'

# uu4 seq
# seq = 'MDAMKRGLCCVLLLCGAVFVSPSASDAPHGVTLPHFNTSHNNSKFELNFYNFLQTWDIPPNTETILGGYLPYCDHEDNCGWYNFVYNNKVGPNAKYSYINTQNLNIPNVHGVYFDVREHNSDGVWDQIDRVGLLIAIHGTSHYSLLMVLQDGVEASQPHVAVKICHWNPGNISTYHQFDVNLGDGGQCVFNQRFSLDTVLTANDFYGFQWTDTYVDIYLGGTITKVWVVNDWSVVEASISSHWNALNYGYYIQFVNRTTYYAYNSTGGSNYTHLQLTECHTDYCAGYAKNVFVPIDGKIPEGFSFSNWFLLTDKSTLVQGRVLSSQPVFVQCLRPVPTWSNNTAVVHFKNDVFCPNVTADVLRFNLNFSDTDVYTDSTTDDQLHFTFEDNTTASITCYSSANVTDNQPASGSISHTPFVSNSYLCFANFSHSSVSRQFLGILPPTVREFAFGRDGSIFVNGYKYFSLQPIKSVNFSISSVENYGFWTIAYTNYTDVMVDVNGTVITRLFYCDSPLNRIKCQQLKHELPDGFYSASMLVKKDLPKTFVTMPQFYNWMNVTLHVVLNDIEKKADIILAGAPELASLADIHFEIAQANGSVVNVTSVCVQARQLALFYKYTSLQGLYTYSNLVQLQNYDCPFSPQQFNNYLQFETLCFDVSPAVAGCKWSLVHDVKWRTQFATITVSYKDGAMITTMPKAQLGFQDISNIVKDECTDYNIYGFQGTGIIRSTTSRLVAGLYYTSASGDLLGFKISTTGEIFTVVPCDLTAQAAVINDEIVGAITATNQTDLFEFVNHTWSRSARGSSPSTVNTYTMPQFYYITKWNNGTSSNCTSVITYSSFAICNTGEIKYVNVTHVEIVDDSVGVIKPVSTGNITIPKNFTVAVQAEYVQIQVKPVAVDCAKYVCNGNRHCLNLLTQYTSACQTIENSLNLGARLESLMLNDMITVSDRSLEFATVDKFNTTALGGEKLGGLYFDGLSSLLPPRVGMRSAVEDLLFNKVVTSGLGTVDDDYKKCSAGTDVADLVCAQYYNGIMVLPGVVDYNKMAMYTASLIGGMALGSITSAVAVPFSMQVQARLNYVALQTDVLQENQKILANAFNNAIGNITLALGKVSNAITTVSDGFNSMASALTKIQSVVNQQGEALSHLISQLQKNFQAISSSIAEIYNRLEKVEADAQVDRLITGRLAALNAYVAQTLTQYAEVKASRQLAMEKVNECVKSQSDRYGFCGNGTHLFSLVNSAPDGLLFFHTVLLPTEWEEVTAWSGICVNDTYAYLLKDFDHSIFSYNGTYMVTPRNMFQPRKPQMSDFVQITSCEVTFLNTTHTTFQEIVIDYIDINKTIADMLEQYHSNYTTPELDLQLEIFNQTKLNLTAEIDQLEQRADNLTNIAHELQQYIDNLNKTIVDLEWLNRIETYVKWPGSGGYIPEAPRDGQAYVRKDGEWVLLSTFLKGQDNSADIQHSGGRSSLEGPRFEGKPIPNPLLGLDSTRTGHHHHHH' # Please paste the pure aa. seq into '' (case insensitive)

# P1 (Gamma) seq
# seq = 'MFVFLVLLPLVSSQCVNFTNRTQLPSAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHAIHVSGTNGTKRFDNPVLPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIKVCEFQFCNYPFLGVYYHKNNKSWMESEFRVYSSANNCTFEYVSQPFLMDLEGKQGNFKNLSEFVFKNIDGYFKIYSKHTPINLVRDLPQGFSALEPLVDLPIGINITRFQTLLALHRSYLTPGDSSSGWTAGAAAYYVGYLQPRTFLLKYNENGTITDAVDCALDPLSETKCTLKSFTVEKGIYQTSNFRVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGTIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVKGFNCYFPLQSYGFQPTYGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNFNFNGLTGTGVLTESNKKFLPFQQFGRDIADTTDAVRDPQTLEILDITPCSFGGVSVITPGTNTSNQVAVLYQGVNCTEVPVAIHADQLTPTWRVYSTGSNVFQTRAGCLIGAEYVNNSYECDIPIGAGICASYQTQTNSPGSASSVASQSIIAYTMSLGAENSVAYSNNSIAIPTNFTISVTTEILPVSMTKTSVDCTMYICGDSTECSNLLLQYGSFCTQLNRALTGIAVEQDKNTQEVFAQVKQIYKTPPIKDFGGFNFSQILPDPSKPSKRSFIEDLLFNKVTLADAGFIKQYGDCLGDIAARDLICAQKFNGLTVLPPLLTDEMIAQYTSALLAGTITSGWTFGAGAALQIPFAMQMAYRFNGIGVTQNVLYENQKLIANQFNSAIGKIQDSLSSTASALGKLQDVVNQNAQALNTLVKQLSSNFGAISSVLNDILSRLDPPEAEVQIDRLITGRLQSLQTYVTQQLIRAAEIRASANLAAIKMSECVLGQSKRVDFCGKGYHLMSFPQSAPHGVVFLHVTYVPAQEKNFTTAPAICHDGKAHFPREGVFVSNGTHWFVTQRNFYEPQIITTDNTFVSGNCDVVIGIVNNTVYDPLQPELDSFKEELDKYFKNHTSPDVDLGDISGINASVVNIQKEIDRLNEVAKNLNESLIDLQELGKYEQEFGSGGYIPEAPRDGQAYVRKDGEWVLLSTFLKGQDNSADIQHSGRPLESRGPFEQKLISEEDLNMHTGHHHHHH'

# SA (Beta) seq
# seq = 'MFVFLVLLPLVSSQCVNFTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHAIHVSGTNGTKRFANPVLPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIKVCEFQFCNDPFLGVYYHKNNKSWMESEFRVYSSANNCTFEYVSQPFLMDLEGKQGNFKNLREFVFKNIDGYFKIYSKHTPINLVRGLPQGFSALEPLVDLPIGINITRFQTLHISYLTPGDSSSGWTAGAAAYYVGYLQPRTFLLKYNENGTITDAVDCALDPLSETKCTLKSFTVEKGIYQTSNFRVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGNIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVKGFNCYFPLQSYGFQPTYGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNFNFNGLTGTGVLTESNKKFLPFQQFGRDIADTTDAVRDPQTLEILDITPCSFGGVSVITPGTNTSNQVAVLYQGVNCTEVPVAIHADQLTPTWRVYSTGSNVFQTRAGCLIGAEHVNNSYECDIPIGAGICASYQTQTNSPGSASSVASQSIIAYTMSLGVENSVAYSNNSIAIPTNFTISVTTEILPVSMTKTSVDCTMYICGDSTECSNLLLQYGSFCTQLNRALTGIAVEQDKNTQEVFAQVKQIYKTPPIKDFGGFNFSQILPDPSKPSKRSFIEDLLFNKVTLADAGFIKQYGDCLGDIAARDLICAQKFNGLTVLPPLLTDEMIAQYTSALLAGTITSGWTFGAGAALQIPFAMQMAYRFNGIGVTQNVLYENQKLIANQFNSAIGKIQDSLSSTASALGKLQDVVNQNAQALNTLVKQLSSNFGAISSVLNDILSRLDPPEAEVQIDRLITGRLQSLQTYVTQQLIRAAEIRASANLAATKMSECVLGQSKRVDFCGKGYHLMSFPQSAPHGVVFLHVTYVPAQEKNFTTAPAICHDGKAHFPREGVFVSNGTHWFVTQRNFYEPQIITTDNTFVSGNCDVVIGIVNNTVYDPLQPELDSFKEELDKYFKNHTSPDVDLGDISGINASVVNIQKEIDRLNEVAKNLNESLIDLQELGKYEQEFGSGGYIPEAPRDGQAYVRKDGEWVLLSTFLKGQDNSADIQHSGRPLESRGPFEQKLISEEDLNMHTGHHHHHH'

# KAPPA
# seq = 'MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHAIHVSGTNGTKRFDNPVLPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIKVCEFQFCNDPFLDVYYHKNNKSWMKSEFRVYSSANNCTFEYVSQPFLMDLEGKQGNFKNLREFVFKNIDGYFKIYSKHTPINLVRDLPQGFSALEPLVDLPIGINITRFQTLLALHRSYLTPGDSSSGWTAGAAAYYVGYLQPRTFLLKYNENGTITDAVDCALDPLSETKCTLKSFTVEKGIYQTSNFRVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYRYRLFRKSNLKPFERDISTEIYQAGSTPCNGVQGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNFNFNGLTGTGVLTESNKKFLPFQQFGRDIADTTDAVRDPQTLEILDITPCSFGGVSVITPGTNTSNQVAVLYQGVNCTEVPVAIHADQLTPTWRVYSTGSNVFQTRAGCLIGAEHVNNSYECDIPIGAGICASYQTQTNSPGSASSVASQSIIAYTMSLGAENSVAYSNNSIAIPTNFTISVTTEILPVSMTKTSVDCTMYICGDSTECSNLLLQYGSFCTQLNRALTGIAVEQDKNTQEVFAQVKQIYKTPPIKDFGGFNFSQILPDPSKPSKRSFIEDLLFNKVTLADAGFIKQYGDCLGDIAARDLICAQKFNGLTVLPPLLTDEMIAQYTSALLAGTITSGWTFGAGAALQIPFAMQMAYRFNGIGVTQNVLYENQKLIANQFNSAIGKIQDSLSSTASALGKLQDVVNQNAQALNTLVKQLSSNFGAISSVLNDILSRLDPPEAEVQIDRLITGRLQSLQTYVTQQLIRAAEIRASANLAATKMSECVLGQSKRVDFCGKGYHLMSFPQSAPHGVVFLHVTYVPAHEKNFTTAPAICHDGKAHFPREGVFVSNGTDWFVTQRNFYEPQIITTDNTFVSGNCDVVIGIVNNTVYDPLQPELDSFKEELDKYFKNHTSPDVDLGDISGINASVVNIQKEIDRLNEVAKNLNESLIDLQELGKYEQEFGSGGYIPEAPRDGQAYVRKDGEWVLLSTFLKGQDNSADIQHSGRPLESRGPFEQKLISEEDLNMHTGHHHHHH'

# DELTA 
# seq = 'MFVFLVLLPLVSSQCVNLRTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHAIHVSGTNGTKRFDNPVLPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIKVCEFQFCNDPFLDVYYHKNNKSWMESGVYSSANNCTFEYVSQPFLMDLEGKQGNFKNLREFVFKNIDGYFKIYSKHTPINLVRDLPQGFSALEPLVDLPIGINITRFQTLLALHRSYLTPGDSSSGWTAGAAAYYVGYLQPRTFLLKYNENGTITDAVDCALDPLSETKCTLKSFTVEKGIYQTSNFRVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYRYRLFRKSNLKPFERDISTEIYQAGSKPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNFNFNGLTGTGVLTESNKKFLPFQQFGRDIADTTDAVRDPQTLEILDITPCSFGGVSVITPGTNTSNQVAVLYQGVNCTEVPVAIHADQLTPTWRVYSTGSNVFQTRAGCLIGAEHVNNSYECDIPIGAGICASYQTQTNSRGSASSVASQSIIAYTMSLGAENSVAYSNNSIAIPTNFTISVTTEILPVSMTKTSVDCTMYICGDSTECSNLLLQYGSFCTQLNRALTGIAVEQDKNTQEVFAQVKQIYKTPPIKDFGGFNFSQILPDPSKPSKRSFIEDLLFNKVTLADAGFIKQYGDCLGDIAARDLICAQKFNGLTVLPPLLTDEMIAQYTSALLAGTITSGWTFGAGAALQIPFAMQMAYRFNGIGVTQNVLYENQKLIANQFNSAIGKIQDSLSSTASALGKLQNVVNQNAQALNTLVKQLSSNFGAISSVLNDILSRLDPPEAEVQIDRLITGRLQSLQTYVTQQLIRAAEIRASANLAATKMSECVLGQSKRVDFCGKGYHLMSFPQSAPHGVVFLHVTYVPAQEKNFTTAPAICHDGKAHFPREGVFVSNGTHWFVTQRNFYEPQIITTDNTFVSGNCDVVIGIVNNTVYDPLQPELDSFKEELDKYFKNHTSPDVDLGDISGINASVVNIQKEIDRLNEVAKNLNESLIDLQELGKYEQEFGSGGYIPEAPRDGQAYVRKDGEWVLLSTFLKGQDNSADIQHSGRPLESRGPFEQKLISEEDLNMHTGHHHHHH'

# pedv ns seq
# seq = 'MDAMKRGLCCVLLLCGAVFVSPSASQDVTRCSAKTNFRRFFSKFNVQAPAVVVLGGYLPIGENQGVNSTWYCAGQHPTASGVHGIFVSHIRGGHGFEIGISQEPFDPSGYQLYLHKATNGNTNATARLRICQFPSIKTLGPTANNDVTTGRNCLFNKAIPAHMSEHSVVGITWDNDRVTVFSDKIYYFYFKNDWSRVATKCYNSGGCAMQYVYEPTYYMLNVTSAGEDGISYQPCTANCIGYAANVFATEPNGHIPEGFSFNNWFLLSNDSTLVHGKVVSNQPLLVNCLLAIPKIYGLGQFFSFNQTIDGVCNGAAVQRAPEALRFNINDTSVILAEGSIVLHTALGTNFSFVCSNSSNPHLATFAIPLGATQVPYYCFLKVDTYNSTVYKFLAVLPPTVREIVITKYGDVYVNGFGYLHLGLLDAVTINFTGHGTDDDVSGFWTIASTNFVDALIEVQGTAIQRILYCDDPVSQLKCSQVAFDLDDGFYPISSRNLLSHEQPISFVTLPSFNDHSFVNITVSASFGGHSGANLIASDTTINGFSSFCVDTRQFTISLFYNVTNSYGYVSNSQDSNCPFTLQSVNDYLSFSKFCVSTSLLASACTIDLFGYPEFGSGVKFTSLYFQFTKGELITGTPKPLEGVTDVSFMTLDVCTKYTIYGFKGEGIITLTNSSFLAGVYYTSDSGQLLPFKNVTSGAVYSVTPCSFSEQAAYVDDDIVGVISSLSSSTFNSTRELPGFFYHSNDGSNCTEPVLVYSNIGVCKSGSIGYVPSQSGQVKIAPTVTENISIPTNFSMSIKTEYLQLYNTPVSVDCATYVCNGNSRCKQLLTQYTAACKTIESALQLSARLESVEVNSMLTISEEALQLATISSFNGDGYNFTNVLGVSVYDPASGRVVQKRSFIEDLLFNKVVTNGLGTVDEDYKRCSNGRSVADLVCAQYYSGVMVLPGVVDAEKLHMYSASLIGGMVLGGFTSAAALPFSYAVQARLNYLALQTDVLQRNQQLLAESFNSAIGNITSAFESVKEAISQTSKGLNTVAHALTKVQEVVNSQGAALTQLTVQLQHNFQAISSSIDDIYSRLDILSADVQVDRLITGRLSALNAFVAQTLTKYTEVQASRKLAQQKVNECVKSQSQRYGFCGGDGEHIFSLVQAAPQGLLFLHTVLVPSDFVDVIAIAGLCVNDEIALTLREPGLVLFTHELQNHTATEYFVSSRRMFEPRKPTVSDFVQIESCVVTYVNLTRDQLPDVIPDYIDVNKTLDEILASLPNRTGPSLPLDVFNATYLNLTGEIADLEQRSESLRNTTEELQSLIYNINNTLVDLEWLNRVETYIKWP'

# pedv ns 2
# seq = 'MKSLTYFWLFLPVLSTLSLPQDVTRCSAKTNFRRFFSKFNVQAPAVVVLGGYLPIGENQGVNSTWYCAGQHPTASGVHGIFVSHIRGGHGFEIGISQEPFDPSGYQLYLHKATNGNTNATARLRICQFPSIKTLGPTANNDVTIGRNCLFNKAIPAHMSEHSVVGITWDNDRVTVFSDKIYYFYFKNDWSRVATKCYNSGGCAMQYVYEPTYYMLNVTSAGEDGISYQPCTANCIGYAANVFATEPNGHIPEGFSFNNWFLLSNDSTLVHGKVVSNQPLLVNCLLAIPKIYGLGQFFSFNQTIDGVCNGAAVQRAPEALRFNINDTSVILAEGSIVLHTALGTNFSFVCSNSSNPHLATFAIPLGATQVPYYCFFKVDTYNSTVYKFLAVLPPTVREIVITKYGDVYVNGFGYLHLGLLDAVTINFTGHGTDDDVSGFWTIASTNFVDALIEVQGTAIQRILYCDDPVSQLKCSQVAFDLDDGFYPFSSRNLLSHEQPISFVTLPSFNAHSFVNITVSASFGGHSGANLIASDTTINGFSSFCVDTRQFTISLSYNVTNSYGYVSNSQDSNCPFTLQSVNDYLSFSKFCVSTSLLASACTIDLFGYPEFGSGVKFTSLYFQFTKGELITGTPKPLEGVTDVSFMTLDVCTKYTIYGFKGEGIITLTNSSFLAGVYYTSDSGQLLAFKNVTSGAVYSVTPCSFSEQAAYVDDDIVGVISSLSSSTFNSTRELPGFFYHSNDGSNCTEPVLVYSNIGVCKSGSIGYVPSQSGQVKIAPTVTGNISIPTNFSMSIRTEYLQLYNTPVSVDCATYVCNGNSRCKQLLTQYTAACKTIESALQLSARLESVEVNSMLTISEEALQLATISSFNGDGYNFTNVLGVSVYDPARGRVVQKRSFIEDLLFNKVVTNGLGTVDEDYKRCSNGRSVADLVCAQYYSGVMVLPGVVDAEKLHMYSASLIGGMVLGGFTAAAALPFSYAVQARLNYLALQTDVLQRNQQLLAESFNSAIGNITSAFESVKEASSQTSRGLNTVAHALTKVQEVVNSQGAALTQLTVQLQHNFQAISSSIDDIYSRLDILSADVQVDRLITGRLSALNAFVAQTLTKYTEVQASRKLAQQKVNECVKSQSQRYGFCGGDGEHIFSLVQAAPQGLLFLHTVLVPSDFVDVIAIAGLCVNDEIALTLREPGLVLFTHELQNHTATEYFVSSRRMFEPRKPTVSDFVQIESCVVTYVNLTRDQLPDVIPDYIDVNKTRDEILASLPNRTGPSLPLDVFNATYLNLTGEIADLEQRSESLRNTTEELQSLIYNINNTLVDLEWLNRVETYIKWPWWVWLIIFIVLIFVVSLLVFCCISTGFCGCFGCCCACFSGCCRGPRLQPYEVFEKVHVQ'

# pedv os seq
# seq = 'MDAMKRGLCCVLLLCGAVFVSPSASDVTRCQSTINFRRFFSKFNVQAPAVVVLGGYLPSMNSSSWYCGTGLETASGVHGIFLSYIDSGQGFEIGISQEPFDPSGYQLYLHKATNGNQDAIARLRICQFPNNKTLGPSVNDVTTGRNCLFNKAIPAYMQDGKNIIVGITWDNDRVTVFADKIYHFYLKNEWSRVATRCYNKRSCAMQYVYTPTYYMLNVTSAGEDGIYYSLCTANCIGYAVNVFATDSNGHIPEGFSFNNWFLLSNDSTLLHGKVVSNQPLLVNCLLAIPKIYGLGQFFSFNQTMDGACNGVAAQRAPEALRFNINDTSVILAEGSIVLHTALGTNLSFVCSNSSDPHLSTFAIPLGATQVPYYCFLKVDTYNSTVYKFLAVLPPTVREIVITKYGDVYVNGFGYLHLGLLDAVTINFTGHGTDDDVSGFWTIASTNFVDALIEVQGTAIERILYCDDPVSQLKCSQVAFDLDDGFYPISSRNLLSHEQPISFVTLPSFNDHSFVNITVSASFGGHSGANVIASDTTINGFSSFCVDTRQFTISLFYNVTNIYGYVSTSQGSNCPFTLQSVNDYLSFSKFCVSTSLLASACTIDLFGYPDFGSGVKLTSLYFQFTKGELITGTPKPLQGVTDVSFMTLDVCTKYTIYGFKGEGVITLTNSSFLAGVYYTSDSGQLLAFKNVTSGAVYSVTPCSFSEQAAYVDDDIVGVISSLSNSTFNSTRELPGFFYHSNDGSNCTEPVLVYSNIGVCKSGSIGYVPSQSGQVKIAPTVTGNISIPTNFSMSIRTEYLQLYNTPVSVDCATYVCNGNSRCKQLLTQYTAACKTIESALQLSARLESVEVNSMLTISEEALQLATVSSFNGDGYNFTNVLGVSVYDPASGRVVQKRSFIEDLLFNKVVTNGLGTVDEDYKRCSKGRSVADLVCAQYYSGVMVLPGVVDAEKLHMYSASLIGGMVLGGFTAAAALPFSYAVQARLNYLALQTDVLQRNQQLLAESFNSAIGNITSAFESVKEAISQTSKGLNTVAHALTKVQEVVNSQGAALTQLTVQLQHNFQAISSSIDDIYSRLDSLSADVQVDRLITGRLSALNAFVAQTLTKYTEVQASRKLAQQKVNECVKSQSQRYGFCGGDGEHIFSLVQAAPQGLLFLHTVLVPGDFVNVIAIAGLCVNDEIALTLREPGLVLFTHELQNYTATEYFVSSRRMFEPRKPTFSDFVQIESCVVTYVNLTRDQLPDVIPDYIDVNKTLDEILASLPNRTGPSLPLDVFNATYLNLTGEIADLEQRSESLRNTTEELQSLIYNINNTLVDLEWLNRVETYIKWPGSGGYIPEAPRDGQAYVRKDGEWVLLSTFLKGQDNSADIQHSGGRSSLEGPRFEGKPIPNPLLGLDSTRTGHHHHHH'

# pedv os mut seq
# seq = 'DVTRCQSTINFRRFFSKFNVQAPAVVVLGGYLPSMNSSSWYCGTGLETASGVHGIFLSYIDSGQGFEIGISQEPFDPSGYQLYLHKATNGNQDAIARLRICQFPNNKTLGPSVNDVTTGRNCLFNKAIPAYMQDGKNIIVGITWDNDRVTVFADKIYHFYLKNEWSRVATRCYNKRSCAMQYVYTPTYYMLNVTSAGEDGIYYSLCTANCIGYAVNVFATDSNGHIPEGFSFNNWFLLSNDSTLLHGKVVSNQPLLVNCLLAIPKIYGLGQFFSFNQTMDGACNGVAAQRAPEALRFNINDTSVILAEGSIVLHTALGTNLSFVCSNSSDPHLSTFAIPLGATQVPYYCFLKVDTYNSTVYKFLAVLPPTVREIVITKYGDVYVNGFGYLHLGLLDAVTINFTGHGTDDDVSGFWTIASTNFVDALIEVQGTAIERILYCDDPVSQLKCSQVAFDLDDGFYPISSRNLLSHEQPISFVTLPSFNDHSFVNITVSASFGGHSGANVIASDTTINGFSSFCVDTRQFTISLFYNVTNIYGYVSTSQGSNCPFTLQSVNDYLSFSKFCVSTSLLASACTIDLFGYPDFGSGVKLTSLYFQFTKGELITGTPKPLQGVTDVSFMTLDVCTKYTIYGFKGEGVITLTNSSFLAGVYYTSDSGQLLAFKNVTSGAVYSVTPCSFSEQAAYVDDDIVGVISSLSSSTFNSTRELPGFFYHSNDGSNCTEPVLVYSNIGVCKSGSIGYVPSQSGQVKIAPTVTGNISIPTNFSMSIRTEYLQLYNTPVSVDCATYVCNGNSRCKQLLTQYTAACKTIESALQLSARLESVEVNSMLTISEEALQLATVSSFNGDGYNFTNVLGVSVYDPASGRVVQKRSFIEDLLFNKVVTNGLGTVDEDYKRCSKGRSVADLVCAQYYSGVMVLPGVVDAEKLHMYSASLIGGMVLGGFTAAAALPFSYAVQARLNYLALQTDVLQRNQQLLAESFNSAIGNITSAFESVKEAISQTSKGLNTVAHALTKVQEVVNSQGAALTQLTVQLQHNFQAISSSIDDIYSRLDSLSADVQVDRLITGRLSALNAFVAQTLTKYTEVQASRKLAQQKVNECVKSQSQRYGFCGGDGEHIFSLVQAAPQGLLFLHTVLVPGDFVNVIAIAGLCVNDEIALTLREPGLVLFTHELQNYTATEYFVSSRRMFEPRKPTFSDFVQIESCVVTYVNLTRDQLPDVIPDYIDVNKTLDEILASLPNRTGPSLPLDVFNATYLNLTGEIADLEQRSESLRNTTEELQSLIYNINNTLVDLEWLNRVETYIKWPGSGGYIPEAPRDGQAYVRKDGEWVLLSTFLKGQDNSADIQHSGRPLESRGPFEQKLISEEDLNMHTGHHHHHH'

# pedv pt52
# seq = 'LPQDVTRCSAKTNFRRFFSKFNVQAPAVVVLGGYLPIGENQGVNSTWYCAGQHPTASGVHGIFVSHIRGGHGFEIGISQEPFDPSGYQLYLHKATNGNTNATARLRICQFPSIKTLGPTANNDVTIGRNCLFNKAIPAHMSEHSVVGITWDNDRVTVFSDKIYYFYFKNDWSRVATKCYNSGGCAMQYVYEPTYYMLNVTSAGEDGISYQPCTANCIGYAANVFATEPNGHIPEGFSFNNWFLLSNDSTLVHGKVVSNQPLLVNCLLAIPKIYGLGQFFSFNQTIDGVCNGAAVQRAPEALRFNINDTSVILAEGSIVLHTALGTNFSFVCSNSSNPHLATFAIPLGATQVPYYCFFKVDTYNSTVYKFLAVLPPTVREIVITKYGDVYVNGFGYLHLGLLDAVTINFTGHGTDDDVSGFWTIASTNFVDALIEVQGTAIQRILYCDDPVSQLKCSQVAFDLDDGFYPFSSRNLLSHEQPISFVTLPSFNAHSFVNITVSASFGGHSGANLIASDTTINGFSSFCVDTRQFTISLSYNVTNSYGYVSNSQDSNCPFTLQSVNDYLSFSKFCVSTSLLASACTIDLFGYPEFGSGVKFTSLYFQFTKGELITGTPKPLEGVTDVSFMTLDVCTKYTIYGFKGEGIITLTNSSFLAGVYYTSDSGQLLAFKNVTSGAVYSVTPCSFSEQAAYVDDDIVGVISSLSSSTFNSTRELPGFFYHSNDGSNCTEPVLVYSNIGVCKSGSIGYVPSQSGQVKIAPTVTGNISIPTNFSMSIRTEYLQLYNTPVSVDCATYVCNGNSRCKQLLTQYTAACKTIESALQLSARLESVEVNSMLTISEEALQLATISSFNGDGYNFTNVLGVSVYDPARGRVVQKRSFIEDLLFNKVVTNGLGTVDEDYKRCSNGRSVADLVCAQYYSGVMVLPGVVDAEKLHMYSASLIGGMVLGGFTAAAALPFSYAVQARLNYLALQTDVLQRNQQLLAESFNSAIGNITSAFESVKEASSQTSRGLNTVAHALTKVQEVVNSQGAALTQLTVQLQHNFQAISSSIDDIYSRLDPPSADVQVDRLITGRLSALNAFVAQTLTKYTEVQASRKLAQQKVNECVKSQSQRYGFCGGDGEHIFSLVQAAPQGLLFLHTVLVPSDFVDVIAIAGLCVNDEIALTLREPGLVLFTHELQNHTATEYFVSSRRMFEPRKPTVSDFVQIESCVVTYVNLTRDQLPDVIPDYIDVNKTRDEILASLPNRTGPSLPLDVFNATYLNLTGEIADLEQRSESLRNTTEELQSLIYNINNTLVDLEWLNRVETYIKWPEFGSGGYIPEAPRDGQAYVRKDGEWVLLSTFLKGQDNSADIQHSGRPLESRGPFEQKLISEEDLNMHTGHHHHHH'

# MERS_S_fm2P_inHouse
# seq = 'MDSWFILVLLGSGLICVSASYVDVGPDSVKSACIEVDIQQTFFDKTWPRPIDVSKADGIIYPQGRTYSNITITYQGLFPYQGDHGDMYVYSAGHATGTTPQKLFVANYSQDVKQFANGFVVRIGAAANSTGTVIISPSTSATIRKIYPAFMLGSSVGNFSDGKMGRFFNHTLVLLPDGCGTLLRAFYCILEPRSGNHCPAGNSYTSFATYHTPATDCSDGNYNRNASLNSFKEYFNLRNCTFMYTYNITEDEILEWFGITQTAQGVHLFSSRYVDLYGGNMFQFATLPVYDTIKYYSIIPHSIRSIQSDRKAWAAFYVYKLQPLTFLLDFSVDGYIRRAIDCGFNDLSQLHCSYESFDVESGVYSVSSFEAKPSGSVVEQAEGVECDFSPLLSGTPPQVYNFKRLVFTNCNYNLTKLLSLFSVNDFTCSQISPAAIASNCYSSLILDYFSYPLSMKSDLSVSSAGPISQFNYKQSFSNPTCLILATVPHNLTTITKPLKYSYINKCSRLLSDDRTEVPQLVNANQYSPCVSIVPSTVWEDGDYYRKQLSPLEGGGWLVASGSTVAMTEQLQMGFGITVQYGTDTNSVCPKLEFANDTKIASQLGNCVEYSLYGVSGRGVFQNCTAVGVRQQRFVYDAYQNLVGYYSDDGNYYCLRACVSVPVSVIYDKETKTHATLFGSVACEHISSTMSQYSRSTRSMLKRRDSTYGPLQTPVGCVLGLVNSSLFVEDCKLPLGQSLCALPDTPSTLTPASVGSVPGEMRLASIAFNHPIQVDQLNSSYFKLSIPTNFSFGVTQEYIQTTIQKVTVDCKQYVCNGFQKCEQLLREYGQFCSKINQALHGANLRQDDSVRNLFASVKSSQSSPIIPGFGGDFNLTLLEPVSISTGSRSARSAIEDLLFDKVTIADPGYMQGYDDCMQQGPASARDLICAQYVAGYKVLPPLMDVNMEAAYTSSLLGSIAGVGWTAGLSSFAAIPFAQSIFYRLNGVGITQQVLSENQKLIANKFNQALGAMQTGFTTTNEAFQKVQDAVNNNAQALSKLASELSNTFGAISASIGDIIQRLDPPEQDAQIDRLINGRLTTLNAFVAQQLVRSESAALSAQLAKDKVNECVKAQSKRSGFCGQGTHIVSFVVNAPNGLYFMHVGYYPSNHIEVVSAYGLCDAANPTNCIAPVNGYFIKTNNTRIVDEWSYTGSSFYAPEPITSLNTKYVAPQVTYQNISTNLPPPLLGNSTGIDFQDELDEFFKNVSTSIPNFGSLTQINTTLLDLTYEMLSLQQVVKALNESYIDLKELGNYTYEFGSGGYIPEAPRDGQAYVRKDGEWVLLSTFLKGQDNSADIQHSGRPLESRGPFEQKLISEEDLNMHTGHHHHHH'

# SARS_S_2P_inHouse 
# seq = 'MFIFLLFLTLTSGSDLDRCTTFDDVQAPNYTQHTSSMRGVYYPDEIFRSDTLYLTQDLFLPFYSNVTGFHTINHTFDNPVIPFKDGIYFAATEKSNVVRGWVFGSTMNNKSQSVIIINNSTNVVIRACNFELCDNPFFAVSKPMGTQTHTMIFDNAFNCTFEYISDAFSLDVSEKSGNFKHLREFVFKNKDGFLYVYKGYQPIDVVRDLPSGFNTLKPIFKLPLGINITNFRAILTAFSPAQDTWGTSAAAYFVGYLKPTTFMLKYDENGTITDAVDCSQNPLAELKCSVKSFEIDKGIYQTSNFRVVPSGDVVRFPNITNLCPFGEVFNATKFPSVYAWERKKISNCVADYSVLYNSTFFSTFKCYGVSATKLNDLCFSNVYADSFVVKGDDVRQIAPGQTGVIADYNYKLPDDFMGCVLAWNTRNIDATSTGNYNYKYRYLRHGKLRPFERDISNVPFSPDGKPCTPPALNCYWPLNDYGFYTTTGIGYQPYRVVVLSFELLNAPATVCGPKLSTDLIKNQCVNFNFNGLTGTGVLTPSSKRFQPFQQFGRDVSDFTDSVRDPKTSEILDISPCSFGGVSVITPGTNASSEVAVLYQDVNCTDVSTAIHADQLTPAWRIYSTGNNVFQTQAGCLIGAEHVDTSYECDIPIGAGICASYHTVSLLRSTSQKSIVAYTMSLGADSSIAYSNNTIAIPTNFSISITTEVMPVSMAKTSVDCNMYICGDSTECANLLLQYGSFCTQLNRALSGIAAEQDRNTREVFAQVKQMYKTPTLKYFGGFNFSQILPDPLKPTKRSFIEDLLFNKVTLADAGFMKQYGECLGDINARDLICAQKFNGLTVLPPLLTDDMIAAYTAALVSGTATAGWTFGAGAALQIPFAMQMAYRFNGIGVTQNVLYENQKQIANQFNKAISQIQESLTTTSTALGKLQDVVNQNAQALNTLVKQLSSNFGAISSVLNDILSRLDPPEAEVQIDRLITGRLQSLQTYVTQQLIRAAEIRASANLAATKMSECVLGQSKRVDFCGKGYHLMSFPQAAPHGVVFLHVTYVPSQERNFTTAPAICHEGKAYFPREGVFVFNGTSWFITQRNFFSPQIITTDNTFVSGNCDVVIGIINNTVYDPLQPELDSFKEELDKYFKNHTSPDVDLGDISGINASVVNIQKEIDRLNEVAKNLNESLIDLQELGKYEQEFGSGGYIPEAPRDGQAYVRKDGEWVLLSTFLKGQDNSADIQHSGRPLESRGPFEQKLISEEDLNMHTGHHHHHH'

# HKU1
# seq = 'MFLIIFILPTTLAVIGDFNCTNSFINDYNKTIPRISEDVVDVSLGLGTYYVLNRVYLNTTLLFTGYFPKSGANFRDLALKGSIYLSTLWYKPPFLSDFNNGIFSKVKNTKLYVNNTLYSEFSTIVIGSVFVNTSYTIVVQPHNGILEITACQYTMCEYPHTVCKSKGSIRNESWHIDSSEPLCLFKKNFTYNVSADWLYFHFYQERGVFYAYYADVGMPTTFLFSLYLGTILSHYYVMPLTCNAISSNTDNETLEYWVTPLSRRQYLLNFDEHGVITNAVDCSSSFLSEIQCKTQSFAPNTGVYDLSGFTVKPVATVYRRIPNLPDCDIDNWLNNVSVPSPLNWERRIFSNCNFNLSTLLRLVHVDSFSCNNLDKSKIFGSCFNSITVDKFAIPNRRRDDLQLGSSGFLQSSNYKIDISSSSCQLYYSLPLVNVTINNFNPSSWNRRYGFGSFNLSSYDVVYSDHCFSVNSDFCPCADPSVVNSCAKSKPPSAICPAGTKYRHCDLDTTLYVKNWCRCSCLPDPISTYSPNTCPQKKVVVGIGEHCPGLGINEEKCGTQLNHSSCFCSPDAFLGWSFDSCISNNRCNIFSNFIFNGINSGTTCSNDLLYSNTEISTGVCVNYDLYGITGQGIFKEVSAAYYNNWQNLLYDSNGNIIGFKDFLTNKTYTILPCYSGRVSAAFYQNSSSPALLYRNLKCSYVLNNISFISQPFYFDSYLGCVLNAVNLTSYSVSSCDLRMGSGFCIDYALPSSGGSGSGISSPYRFVTFEPFNVSFVNDSVETVGGLFEIQIPTNFTIAGHEEFIQTSSPKVTIDCSAFVCSNYAACHDLLSEYGTFCDNINSILNEVNDLLDITQLQVANALMQGVTLSSNLNTNLHSDVDNIDFKSLLGCLGSQCGSSSRSLLEDLLFNKVKLSDVGFVEAYNNCTGGSEIRDLLCVQSFNGIKVLPPILSETQISGYTTAATVAAMFPPWSAAAGVPFSLNVQYRINGLGVTMDVLNKNQKLIANAFNKALLSIQNGFTATNSALAKIQSVVNANAQALNSLLQQLFNKFGAISSSLQEILSRLDPPEAQVQIDRLINGRLTALNAYVSQQLSDITLIKAGASRAIEKVNECVKSQSPRINFCGNGNHILSLVQNAPYGLLFIHFSYKPTSFKTVLVSPGLCLSGDRGIAPKQGYFIKQNDSWMFTGSSYYYPEPISDKNVVFMNSCSVNFTKAPFIYLNNSIPNLSDFEAELSLWFKNHTSIAPNLTFNSHINATFLDLYYEMNVIQESIKSLNEFGSGGYIPEAPRDGQAYVRKDGEWVLLSTFLKGQNSADIQHSGRPLESRGPFEQKLISEEDLNMHTGHHHHHH'

# OC43
# seq = 'MGILPSPGMPALLSLVSLLSVLLMGCVAETGTVIGDLKCTSDNINDKDTGPPPISTDTVDVTNGLGTYYVLDRVYLNTTLFLNGYYPTSGSTYRNMALKGSVLLSRLWFKPPFLSDFINGIFAKVKNTKVIKDRVMYSEFPAITIGSTFVNTSYSVVVQPRTINSTQDGDNKLQGLLEVSVCQYNMCEYPQTICHPNLGNHRKELWHLDTGVVSCLYKRNFTYDVNADYLYFHFYQEGGTFYAYFTDTGVVTKFLFNVYLGMALSHYYVMPLTCNSKLTLEYWVTPLTSRQYLLAFNQDGIIFNAVDCMSDFMSEIKCKTQSIAPPTGVYELNGYTVQPIADVYRRKPNLPNCNIEAWLNDKSVPSPLNWERKTFSNCNFNMSSLMSFIQADSFTCNNIDAAKIYGMCFSSITIDKFAIPNGRKVDLQLGNLGYLQSFNYRIDTTATSCQLYYNLPAANVSVSRFNPSTWNKRFGFIEDSVFKPRPAGVLTNHDVVYAQHCFKAPKNFCPCKLNGSCVGSGPGKNNGIGTCPAGTNYLTCDNLCTPDPITFTGTYKCPQTKSLVGIGEHCSGLAVKSDYCGGNSCTCRPQAFLGWSADSCLQGDKCNIFANFILHDVNSGLTCSTDLQKANTDIILGVCVNYDLYGILGQGIFVEVNATYYNSWQNLLYDSNGNLYGFRDYITNRTFMIRSCYSGRVSAAFHANSSEPALLFRNIKCNYVFNNSLTRQLQPINYFDSYLGCVVNAYNSTAISVQTCDLTVGSGYCVDYSKNGGSGGAITTGYRFTNFEPFTVNSVNDSLEPVGGLYEIQIPSEFTIGNMVEFIQTSSPKVTIDCAAFVCGDYAACKSQLVEYGSFCDNINAILTEVNELLDTTQLQVANSLMNGVTLSTKLKDGVNFNVDDINFSPVLGCLGSECSKASSRSAIEDLLFDKVKLSDVGFVEAYNNCTGGAEIRDLICVQSYKGIKVLPPLLSENQFSGYTLAATSASLFPPWTAAAGVPFYLNVQYRINGLGVTMDVLSQNQKLIANAFNNALYAIQEGFDATNSALVKIQAVVNANAEALNNLLQQLSNRFGAISASLQEILSRLDPPEAEAQIDRLINGRLTALNAYVSQQLSDSTLVKFSAAQAMEKVNECVKSQSSRINFCGNGNHIISLVQNAPYGLYFIHFSYVPTKYVTARVSPGLCIAGDRGIAPKSGYFVNVNNTWMYTGSGYYYPEPITENNVVVMSTCAVNYTKAPYVMLNTSIPNLPDFKEELDQWFKNQTSVAPDLSLDYINVTFLDLLGSGRENLYFQGGGGSGYIPEAPRDGQAYVRKDGEWVLLSTFLGHHHHHHHHGLNDIFEAQKIEWHE'

# inh-NL63-S-2P
# seq = 'MKLFLILLVLPLASCFFTCNSNANLSMLQLGVPDNSSTIVTGLLPTHWICANQSTSVYSANGFFYIDVGNHRSAFALHTGYYDVNQYYIYVTNEIGLNASVTLKICKFGINTTFDFLSNSSSSFDCIVNLLFTEQLGAPLGITISGETVRLHLYNVTRTFYVPAAYKLTKLSVKCYFNYSCVFSVVNATVTVNVTTHNGRVVNYTVCDDCNGYTDNIFSVQQDGRIPNGFPFNNWFLLTNGSTLVDGVSRLYQPLRLTCLWPVPGLKSSTGFVYFNATGSDVNCNGYQHNSVADVMRYNLNFSANSVDNLKSGVIVFKTLQYDVLFYCSNSSSGVLDTTIPFGPSSQPYYCFINSTINTTHVSTFVGVLPPTVREIVVARTGQFYINGFKYFDLGFIEAVNFNVTTASATDFWTVAFATFVDVLVNVSATKIQNLLYCDSPFEKLQCEHLQFGLQDGFYSANFLDDNVLPETYVALPIYYQHTDINFTATASFGGSCYVCKPHQVNISLNGNTSVCVRTSHFSIRYIYNRVKSGSPGDSSWHIYLKSGTCPFSFSKLNNFQKFKTICFSTVAVPGSCNFPLEATWHYTSYTIVGALYVTWSEGNSITGVPYPVSGIREFSNLVLNNCTKYNIYDYVGTGIIRSSNQSLAGGITYVSNSGNLLGFKNVSTGNIFIVTPCNQPDQVAVYQQSIIGAMTAVNESRYGLQNLLQLPNFYYVSNGGNNCTTAVMTYSNFGICADGSLIPVRLRNSSDNGISAIITANLSIPSNWTTSVQVEYLQITSIPIVVDCATYVCNGNPRCKNLLKQYTSACKTIEDALRLSAHLETNDVSSMLTFDSNAFSLANVTSFGDYNLSSVLPQRNIHSSRIAGRSALEDLLFSKVVTSGLGTVDVDYKSCTKGLSIADLACAQYYNGIMVLPGVADAERMAMYTGSLIGGMVLGGLTSAAAIPFSLALQARLNYVALQTDVLQENQKILAASFNKAINNIVASFSSVNDAITQTAEAIHTVTIALNKIQDVVNQQGSALNHLTSQLRHNFQAISNSIQAIYDRLDPPQADQQVDRLITGRLAALNAFVSQVLNKYTEVRSSRRLAQQKINECVKSQSNRYGFCGNGTHIFSIVNSAPDGLLFLHTVLLPTDYKNVKAWSGICVDGIYGYVLRQPNLVLYSDNGVFRVTSRVMFQPRLPVLSDFVQIYNCNVTFVNISRVELHTVIPDYVDVNKTLQEFAQNLPKYVKPNFDLTPFNLTYLNLSSELKQLEAKTASLFQTTVELQGLIDQINSTYVDLKLLNRFENEFGSGGYIPEAPRDGQAYVRKDGEWVLLSTFLKGQDNSADIQHSGRPLESRGPFEQKLISEEDLNMHTGHHHHHH'

# inh-229E-S-2P
# seq = 'MFVLLVAYALLHIAGCQTTNGTNTSHSVCNGCVGHSENVFAVESGGYIPSNFSFNNWFLLTNTSSVVDGVVRSFQPLLLNCLWSVSGSQFTTGFVYFNGTGRGACKGFYSNASSDVIRYNINFEENLRRGTILFKTSYGAVVFYCTNNTLVSGDAHIPSGTVLGNFYCFVNTTIGNETTSAFVGALPKTVREFVISRTGHFYINGYRYFSLGDVEAVNFNVTNAATIVCTVALASYADVLVNVSQTAIANIIYCNSVINRLRCDQLSFDVPDGFYSTSPIQPVELPMSIVSLPVYHKHTFIVLHVKFEHQRGPGKCYNCRPSVINITLANFNETKGPLCVDTSHFTTQFVDNVKLARWSASINTGNCPFSFGKVNNFVKFGSVCFSLKDIPGGCAMPIMANLVNHKSHNIGSLYVSWSDGDVITGVPKPVEGVSSFMNVTLNKCTKYNIYDVSGVGVIRISNDTFLNGITYTSTSGNLLGFKDVTNGTIYSITPCNPPDQLVVYQQAVVGAMLSENFTSYGFSNVVEMPKFFYASNGTYNCTDAVLTYSSFGVCADGSIIAVQPRNVSYDSVSAIVTANLSIPSNWTTSVQVEYLQITSKPIVVDCSTYVCNGNVRCVELLKQYTSACKTIEDALRNSAMLESADVSEMLTFDKKAFTLANVSSFGDYNLSSVIPSLPRSGSRVAGRSAIEDILFSKLVTSGLGTVDADYKKCTKGLSIADLACAQYYNGIMVLPGVADAERMAMYTGSLIGGIALGGLTSAASIPFSLAIQSRLNYVALQTDVLQENQKILAASFNKAMTNIVDAFTGVNDAITQTSQALQTVATALNKIQDVVNQQGNSLNHLTSQLRQNFQAISSSIQAIYDRLDPPQADQQVDRLITGRLAALNVFVSHTLTKYTEVRASRQLAQQKVNECVKSQSKRYGFCGNGTHIFSLVNAAPEGLVFLHTVLLPTQYKDVEAWSGLCVDGINGYVLRQPNLALYKEGNYYRITSRIMFEPRIPTIADFVQIENCNVTFVNISRSELQTIVPEYIDVNKTLQELSYKLPNYTVPDLVVEQYNQTILNLTSEISTLENKSAELNYTVQKLQTLIDNINSTLVDLKWLNRVETYIKWPWEFGSGGYIPEAPRDGQAYVRKDGEWVLLSTFLKGQDNSADIQHSGRPLESRGPFEQKLISEEDLNMHTGHHHHHH'

# caltech-229E-S-2P (change to native signal peptide)
# seq = 'GTQTTNGLNTSYSVCNGCVGYSENVFAVESGGYIPSDFAFNNWFLLTNTSSVVDGVVRSFQPLLLNCLWSVSGLRFTTGFVYFNGTGRGDCKGFSSDVLSDVIRYNLNFEENLRRGTILFKTSYGVVVFYCTNNTLVSGDAHIPFGTVLGNFYCFVNTTIGTETTSAFVGALPKTVREFVISRTGHFYINGYRYFTLGNVEAVNFNVTTAETTDFFTVALASYADVLVNVSQTSIANIIYCNSVINRLRCDQLSFYVPDGFYSTSPIQSVELPVSIVSLPVYHKHMFIVLYVDFKPQSGGGKCFNCYPAGVNITLANFNETKGPLCVDTSHFTTKYVAVYANVGRWSASINTGNCPFSFGKVNNFVKFGSVCFSLKDIPGGCAMPIVANWAYSKYYTIGTLYVSWSDGDGITGVPQPVEGVSSFMNVTLDKCTKYNIYDVSGVGVIRVSNDTFLNGITYTSTSGNLLGFKDVTKGTIYSITPCNPPDQLVVYQQAVVGAMLSENFTSYGFSNVVELPKFFYASNGTYNCTDAVLTYSSFGVCADGSIIAVQPRNVSYDSVSAIVTANLSIPSNWTISVQVEYLQITSTPIVVDCSTYVCNGNVRCVELLKQYTSACKTIEDALRNSARLESADVSEMLTFDKKAFTLANVSSFGDYNLSSVIPSLPTSGSRVAGRSAIEDILFSKIVTSGLGTVDADYKNCTKGLSIADLACAQYYNGIMVLPGVADAERMAMYTGSLIGGIALGGLTSAVSIPFSLAIQARLNYVALQTDVLQENQKILAASFNKAMTNIVDAFTGVNDAITQTSQALQTVATALNKIQDVVNQQGNSLNHLTSQLRQNFQAISSSIQAIYDRLDPPQADQQVDRLITGRLAALNVFVSHTLTKYTEVRASRQLAQQKVNECVKSQSKRYGFCGNGTHIFSIVNAAPEGLVFLHTVLLPTQYKDVEAWSGLCVDGTNGYVLRQPNLALYKEGNYYRITSRIMFEPRIPTMADFVQIENCNVTFVNISRSELQTIVPEYIDVNKTLQELSYKLPNYTVPDLVVEQYNQTILNLTSEISTLENKSAELNYTVQKLQTLIDNINSTLVDLKWLNRVETYIKGSGRENLYFQGGGGSGYIPEAPRDGQAYVRKDGEWVLLSTFLGHHHHHHHHGLNDIFEAQKIEWHE'
# ace2 v1
# seq = 'MSSSSWLLLSLVAVTAAQSTIEEQAKTFLDKFNAEAEDLFYQSSLASWNYNTNITEENVQNMNNAGDKWSAFLKEQSTLAQMYPLQEIQNLQVKLQLQALQQNGSSVLSEDKSKRLNTILNTMSTIYSTGKVCNPDNPQECLLLEPGLNEIMANSLDYNERLWAWESWRSEVGKQLRPLYEEYVVLKNEMARANHYEDYGDYWRGDYEVNGVDGYDYSRGQLIEDVEHTFEEIKPLYEHLHAYVRAKLMNAYPSYISPIGCLPAHLLGDMWGRFWTNLYSLTVPFGQKPNIDVTDAMVDQAWDAQRIFKEAEKFFVSVGLPNMTPGFWENSMLTDPGNVQKAVCHPTAWDLGKGDFRILMCTKVTMDDFLTAHHEMGHIQYDMAYLAQPFLLRNGANEGFHEAVGEIMSLSAATPKHLKSIGLLSPDFQEDNETEINFLLKQALTIVGTLPFTYMLEKWRWMVFKGEIPKDQWMKKWWEMKREIVGVVEPVPHDETYCDPASLFHVSNDYSFIRYYTRTLYQFQFQEALCQAAKHEGPLHKCDISNSTEAGQKLFNMLRLGKSEPWTLALENVVGAKNMNVRPLLNYFEPLFTWLKDQNKNSFVGWSTDWSPYADGSGGSGSGGSKGEELFTGVVPILVELDGDVNGHKFSVRGEGEGDATNGKLTLKFICTTGKLPVPWPTLVTTLTYGVQCFSRYPDHMKRHDFFKSAMPEGYVQERTISFKDDGTYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNFNSHNVYITADKQKNGIKANFKIRHNVEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSVLSKDPNEKRDHMVLLEFVTAAGITHGMDELYK'

# ace2 v2
# seq = 'MSSSSWLLLSLVAVTAAQSTIEEQAKYFLDKFNHEAEDLFYQSSLASWNYNTNITEENVQNMNNAGDKWSAFLKEQSTTAQMYPLQEIQNLTVKLQLQALQQNGSSVLSEDKSKRLNTILNTMSTIYSTGKVCNPDNPQECLLLEPGLNEIMANSLDYNERLWAWESWRSEVGKQLRPLYEEYVVLKNEMARANHYEDYGDYWRGDYEVNGVDGYDYSRGQLIEDVEHTFEEIKPLYEHLHAYVRAKLMNAYPSYISPIGCLPAHLLGDMWGRFWTNLYSLTVPFGQKPNIDVTDAMVDQAWDAQRIFKEAEKFFVSVGLPNMTQGFWEYSMLTDPGNVQKAVCHPTAWDLGKGDFRILMCTKVTMDDFLTAHHEMGHIQYDMAYLAQPFLLRNGANEGFHEAVGEIMSLSAATPKHLKSIGLLSPDFQEDNETEINFLLKQALTIVGTLPFTYMLEKWRWMVFKGEIPKDQWMKKWWEMKREIVGVVEPVPHDETYCDPASLFHVSNDYSFIRYYTRTLYQFQFQEALCQAAKHEGPLHKCDISNSTEAGQKLFNMLRLGKSEPWTLALENVVGAKNMNVRPLLNYFEPLFTWLKDQNKNSFVGWSTDWSPYADGSGGSGSGGSKGEELFTGVVPILVELDGDVNGHKFSVRGEGEGDATNGKLTLKFICTTGKLPVPWPTLVTTLTYGVQCFSRYPDHMKRHDFFKSAMPEGYVQERTISFKDDGTYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNFNSHNVYITADKQKNGIKANFKIRHNVEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSVLSKDPNEKRDHMVLLEFVTAAGITHGMDELYK'

# ace2 v3
# seq = 'MSSSSWLLLSLVAVTAAQSTIEEQVKYFLDKFNHEAEDLFYQSSLASWNYNTNITEENVQNMNNAGDKWSAFLKEQSTLAQMYPLQEIQNLQVKLQLQALQQNGSSVLSEDKSKRLNTILNTMSTIYSTGKVCNPDNPQECLLLEPGLNEIMANSLDYNERLWAWESWRSEVGKQLRPLYEEYVVLKNEMARANHYEDYGDYWRGDYEVNGVDGYDYSRGQLIEDVEHTFEEIKPLYEHLHAYVRAKLMNAYPSYISPIGCLPAHLLGDMWGRFWTNLYSLTVPFGQKPNIDVTDAMVDQAWDAQRIFKEAEKFFVSVGLPNMTPGFWENSMLTDPGNVQKAVCHPTAWDLGKGDFRILMCTKVTMDDFLTAHHEMGHIQYDMAYLAQPFLLRNGANEGFHEAVGEIMSLSAATPKHLKSIGLLSPDFQEDNETEINFLLKQALTIVGTLPFTYMLEKWRWMVFKGEIPKDQWMKKWWEMKREIVGVVEPVPHDETYCDPASLFHVSNDYSFIRYYTRTLYQFQFQEALCQAAKHEGPLHKCDISNSTEAGQKLFNMLRLGKSEPWTLALENVVGAKNMNVRPLLNYFEPLFTWLKDQNKNSFVGWSTDWSPYADGSGGSGSGGSKGEELFTGVVPILVELDGDVNGHKFSVRGEGEGDATNGKLTLKFICTTGKLPVPWPTLVTTLTYGVQCFSRYPDHMKRHDFFKSAMPEGYVQERTISFKDDGTYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNFNSHNVYITADKQKNGIKANFKIRHNVEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSVLSKDPNEKRDHMVLLEFVTAAGITHGMDELYK'

# ace2 v4
# seq = 'MSSSSWLLLSLVAVTAAQSTIEEQAKTFLDKFNAEAEDLFYQSSLASWNYNTNITEENVQNMNNAGDKWSAFLKEQSTTAQMYPLQEIQNLTVKLQLQALQQNGSSVLSEDKSKRLNTILNTMSTIYSTGKVCNPDNPQECLLLEPGLNEIMANSLDYNERLWAWESWRSEVGKQLRPLYEEYVVLKNEMARANHYEDYGDYWRGDYEVNGVDGYDYSRGQLIEDVEHTFEEIKPLYEHLHAYVRAKLMNAYPSYISPIGCLPAHLLGDMWGRFWTNLYSLTVPFGQKPNIDVTDAMVDQAWDAQRIFKEAEKFFVSVGLPNMTQGFWEYSMLTDPGNVQKAVCHPTAWDLGKGDFRILMCTKVTMDDFLTAHHEMGHIQYDMAYLAQPFLLRNGANEGFHEAVGEIMSLSAATPKHLKSIGLLSPDFQEDNETEINFLLKQALTIVGTLPFTYMLEKWRWMVFKGEIPKDQWMKKWWEMKREIVGVVEPVPHDETYCDPASLFHVSNDYSFIRYYTRTLYQFQFQEALCQAAKHEGPLHKCDISNSTEAGQKLFNMLRLGKSEPWTLALENVVGAKNMNVRPLLNYFEPLFTWLKDQNKNSFVGWSTDWSPYADGSGGSGSGGSKGEELFTGVVPILVELDGDVNGHKFSVRGEGEGDATNGKLTLKFICTTGKLPVPWPTLVTTLTYGVQCFSRYPDHMKRHDFFKSAMPEGYVQERTISFKDDGTYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNFNSHNVYITADKQKNGIKANFKIRHNVEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSVLSKDPNEKRDHMVLLEFVTAAGITHGMDELYK'

# ace2 v5
# seq = 'MSSSSWLLLSLVAVTAAQSTIEEQVKTFLDKFNHEAEDLFYQSSLASWNYNTNITEENVQNMNNAGDKWSAFLKEQSTLAQMYPLQEIQNLQVKLQLQALQQNGSSVLSEDKSKRLNTILNTMSTIYSTGKVCNPDNPQECLLLEPGLNEIMANSLDYNERLWAWESWRSEVGKQLRPLYEEYVVLKNEMARANHYEDYGDYWRGDYEVNGVDGYDYSRGQLIEDVEHTFEEIKPLYEHLHAYVRAKLMNAYPSYISPIGCLPAHLLGDMWGRFWTNLYSLTVPFGQKPNIDVTDAMVDQAWDAQRIFKEAEKFFVSVGLPNMTQGFWENSMLTDPGNVQKAVCHPTAWDLGKGDFRILMCTKVTMDDFLTAHHEMGHIQYDMAYLAQPFLLRNGANEGFHEAVGEIMSLSAATPKHLKSIGLLSPDFQEDNETEINFLLKQALTIVGTLPFTYMLEKWRWMVFKGEIPKDQWMKKWWEMKREIVGVVEPVPHDETYCDPASLFHVSNDYSFIRYYTRTLYQFQFQEALCQAAKHEGPLHKCDISNSTEAGQKLFNMLRLGKSEPWTLALENVVGAKNMNVRPLLNYFEPLFTWLKDQNKNSFVGWSTDWSPYADGSGGSGSGGSKGEELFTGVVPILVELDGDVNGHKFSVRGEGEGDATNGKLTLKFICTTGKLPVPWPTLVTTLTYGVQCFSRYPDHMKRHDFFKSAMPEGYVQERTISFKDDGTYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNFNSHNVYITADKQKNGIKANFKIRHNVEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSVLSKDPNEKRDHMVLLEFVTAAGITHGMDELYK'

# ace2 v6
# seq = 'MSSSSWLLLSLVAVTAAQSTIEEQAKYFLDKFNHEAEDLFYLSSLASWNYNTNITEENVQNMNNAGDKWSAFLKEQSTTAQMYPLQEIQNLQVKLQLQALQQNGSSVLSEDKSKRLNTILNTMSTIYSTGKVCNPDNPQECLLLEPGLNEIMANSLDYNERLWAWESWRSEVGKQLRPLYEEYVVLKNEMARANHYEDYGDYWRGDYEVNGVDGYDYSRGQLIEDVEHTFEEIKPLYEHLHAYVRAKLMNAYPSYISPIGCLPAHLLGDMWGRFWTNLYSLTVPFGQKPNIDVTDAMVDQAWDAQRIFKEAEKFFVSVGLPNMTPGFWEYSMLTDPGNVQKAVCHPTAWDLGKGDFRILMCTKVTMDDFLTAHHEMGHIQYDMAYLAQPFLLRNGANEGFHEAVGEIMSLSAATPKHLKSIGLLSPDFQEDNETEINFLLKQALTIVGTLPFTYMLEKWRWMVFKGEIPKDQWMKKWWEMKREIVGVVEPVPHDETYCDPASLFHVSNDYSFIRYYTRTLYQFQFQEALCQAAKHEGPLHKCDISNSTEAGQKLFNMLRLGKSEPWTLALENVVGAKNMNVRPLLNYFEPLFTWLKDQNKNSFVGWSTDWSPYADGSGGSGSGGSKGEELFTGVVPILVELDGDVNGHKFSVRGEGEGDATNGKLTLKFICTTGKLPVPWPTLVTTLTYGVQCFSRYPDHMKRHDFFKSAMPEGYVQERTISFKDDGTYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNFNSHNVYITADKQKNGIKANFKIRHNVEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSVLSKDPNEKRDHMVLLEFVTAAGITHGMDELYK'

# ace2 wt
# seq = 'MSSSSWLLLSLVAVTAAQSTIEEQAKTFLDKFNHEAEDLFYQSSLASWNYNTNITEENVQNMNNAGDKWSAFLKEQSTLAQMYPLQEIQNLTVKLQLQALQQNGSSVLSEDKSKRLNTILNTMSTIYSTGKVCNPDNPQECLLLEPGLNEIMANSLDYNERLWAWESWRSEVGKQLRPLYEEYVVLKNEMARANHYEDYGDYWRGDYEVNGVDGYDYSRGQLIEDVEHTFEEIKPLYEHLHAYVRAKLMNAYPSYISPIGCLPAHLLGDMWGRFWTNLYSLTVPFGQKPNIDVTDAMVDQAWDAQRIFKEAEKFFVSVGLPNMTQGFWENSMLTDPGNVQKAVCHPTAWDLGKGDFRILMCTKVTMDDFLTAHHEMGHIQYDMAYAAQPFLLRNGANEGFHEAVGEIMSLSAATPKHLKSIGLLSPDFQEDNETEINFLLKQALTIVGTLPFTYMLEKWRWMVFKGEIPKDQWMKKWWEMKREIVGVVEPVPHDETYCDPASLFHVSNDYSFIRYYTRTLYQFQFQEALCQAAKHEGPLHKCDISNSTEAGQKLFNMLRLGKSEPWTLALENVVGAKNMNVRPLLNYFEPLFTWLKDQNKNSFVGWSTDWSPYADGSGGSGSGGSKGEELFTGVVPILVELDGDVNGHKFSVRGEGEGDATNGKLTLKFICTTGKLPVPWPTLVTTLTYGVQCFSRYPDHMKRHDFFKSAMPEGYVQERTISFKDDGTYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNFNSHNVYITADKQKNGIKANFKIRHNVEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSVLSKDPNEKRDHMVLLEFVTAAGITHGMDELYK' 

# SARS-CoV-2_Spike_Omicron_fm_HexaPro
# seq = 'MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHVISGTNGTKRFDNPVLPFNDGVYFASIEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIKVCEFQFCNDPFLDHKNNKSWMESEFRVYSSANNCTFEYVSQPFLMDLEGKQGNFKNLREFVFKNIDGYFKIYSKHTPIIVREPEDLPQGFSALEPLVDLPIGINITRFQTLLALHRSYLTPGDSSSGWTAGAAAYYVGYLQPRTFLLKYNENGTITDAVDCALDPLSETKCTLKSFTVEKGIYQTSNFRVQPTESIVRFPNITNLCPFDEVFNATRFASVYAWNRKRISNCVADYSVLYNLAPFFTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGNIADYNYKLPDDFTGCVIAWNSNKLDSKVSGNYNYLYRLFRKSNLKPFERDISTEIYQAGNKPCNGVAGFNCYFPLRSYSFRPTYGVGHQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNFNFNGLKGTGVLTESNKKFLPFQQFGRDIADTTDAVRDPQTLEILDITPCSFGGVSVITPGTNTSNQVAVLYQGVNCTEVPVAIHADQLTPTWRVYSTGSNVFQTRAGCLIGAEYVNNSYECDIPIGAGICASYQTQTKSHGSASSVASQSIIAYTMSLGAENSVAYSNNSIAIPTNFTISVTTEILPVSMTKTSVDCTMYICGDSTECSNLLLQYGSFCTQLKRALTGIAVEQDKNTQEVFAQVKQIYKTPPIKYFGGFNFSQILPDPSKPSKRSPIEDLLFNKVTLADAGFIKQYGDCLGDIAARDLICAQKFKGLTVLPPLLTDEMIAQYTSALLAGTITSGWTFGAGPALQIPFPMQMAYRFNGIGVTQNVLYENQKLIANQFNSAIGKIQDSLSSTPSALGKLQDVVNHNAQALNTLVKQLSSKFGAISSVLNDIFSRLDPPEAEVQIDRLITGRLQSLQTYVTQQLIRAAEIRASANLAATKMSECVLGQSKRVDFCGKGYHLMSFPQSAPHGVVFLHVTYVPAQEKNFTTAPAICHDGKAHFPREGVFVSNGTHWFVTQRNFYEPQIITTDNTFVSGNCDVVIGIVNNTVYDPLQPELDSFKEELDKYFKNHTSPDVDLGDISGINASVVNIQKEIDRLNEVAKNLNESLIDLQELGKYEQEFGSGGYIPEAPRDGQAYVRKDGEWVLLSTFLKGQDNSADIQHSGRPLESRGPFEQKLISEEDLNMHTGHHHHHH'

# SARS_CoV2_S_Omicron
# seq = 'MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHVISGTNGTKRFDNPVLPFNDGVYFASIEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIKVCEFQFCNDPFLDHKNNKSWMESEFRVYSSANNCTFEYVSQPFLMDLEGKQGNFKNLREFVFKNIDGYFKIYSKHTPIIVREPEDLPQGFSALEPLVDLPIGINITRFQTLLALHRSYLTPGDSSSGWTAGAAAYYVGYLQPRTFLLKYNENGTITDAVDCALDPLSETKCTLKSFTVEKGIYQTSNFRVQPTESIVRFPNITNLCPFDEVFNATRFASVYAWNRKRISNCVADYSVLYNLAPFFTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGNIADYNYKLPDDFTGCVIAWNSNKLDSKVSGNYNYLYRLFRKSNLKPFERDISTEIYQAGNKPCNGVAGFNCYFPLRSYSFRPTYGVGHQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNFNFNGLKGTGVLTESNKKFLPFQQFGRDIADTTDAVRDPQTLEILDITPCSFGGVSVITPGTNTSNQVAVLYQGVNCTEVPVAIHADQLTPTWRVYSTGSNVFQTRAGCLIGAEYVNNSYECDIPIGAGICASYQTQTKSHGSASSVASQSIIAYTMSLGAENSVAYSNNSIAIPTNFTISVTTEILPVSMTKTSVDCTMYICGDSTECSNLLLQYGSFCTQLKRALTGIAVEQDKNTQEVFAQVKQIYKTPPIKYFGGFNFSQILPDPSKPSKRSFIEDLLFNKVTLADAGFIKQYGDCLGDIAARDLICAQKFKGLTVLPPLLTDEMIAQYTSALLAGTITSGWTFGAGAALQIPFAMQMAYRFNGIGVTQNVLYENQKLIANQFNSAIGKIQDSLSSTASALGKLQDVVNHNAQALNTLVKQLSSKFGAISSVLNDIFSRLDPPEAEVQIDRLITGRLQSLQTYVTQQLIRAAEIRASANLAATKMSECVLGQSKRVDFCGKGYHLMSFPQSAPHGVVFLHVTYVPAQEKNFTTAPAICHDGKAHFPREGVFVSNGTHWFVTQRNFYEPQIITTDNTFVSGNCDVVIGIVNNTVYDPLQPELDSFKEELDKYFKNHTSPDVDLGDISGINASVVNIQKEIDRLNEVAKNLNESLIDLQELGKYEQEFGSGGYIPEAPRDGQAYVRKDGEWVLLSTFLKGQDNSADIQHSGRPLESRGPFEQKLISEEDLNMHTGHHHHHH'

# Wuhan nCoV S-fm2P
# seq = 'MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHAIHVSGTNGTKRFDNPVLPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIKVCEFQFCNDPFLGVYYHKNNKSWMESEFRVYSSANNCTFEYVSQPFLMDLEGKQGNFKNLREFVFKNIDGYFKIYSKHTPINLVRDLPQGFSALEPLVDLPIGINITRFQTLLALHRSYLTPGDSSSGWTAGAAAYYVGYLQPRTFLLKYNENGTITDAVDCALDPLSETKCTLKSFTVEKGIYQTSNFRVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNFNFNGLTGTGVLTESNKKFLPFQQFGRDIADTTDAVRDPQTLEILDITPCSFGGVSVITPGTNTSNQVAVLYQDVNCTEVPVAIHADQLTPTWRVYSTGSNVFQTRAGCLIGAEHVNNSYECDIPIGAGICASYQTQTNSPGSASSVASQSIIAYTMSLGAENSVAYSNNSIAIPTNFTISVTTEILPVSMTKTSVDCTMYICGDSTECSNLLLQYGSFCTQLNRALTGIAVEQDKNTQEVFAQVKQIYKTPPIKDFGGFNFSQILPDPSKPSKRSFIEDLLFNKVTLADAGFIKQYGDCLGDIAARDLICAQKFNGLTVLPPLLTDEMIAQYTSALLAGTITSGWTFGAGAALQIPFAMQMAYRFNGIGVTQNVLYENQKLIANQFNSAIGKIQDSLSSTASALGKLQDVVNQNAQALNTLVKQLSSNFGAISSVLNDILSRLDPPEAEVQIDRLITGRLQSLQTYVTQQLIRAAEIRASANLAATKMSECVLGQSKRVDFCGKGYHLMSFPQSAPHGVVFLHVTYVPAQEKNFTTAPAICHDGKAHFPREGVFVSNGTHWFVTQRNFYEPQIITTDNTFVSGNCDVVIGIVNNTVYDPLQPELDSFKEELDKYFKNHTSPDVDLGDISGINASVVNIQKEIDRLNEVAKNLNESLIDLQELGKYEQ'
## ADJUSTABLE PARAMETERS
# HCD: light colors
lightgreen_hex = '#ccffcc' 
lightblue_hex = '#cce6ff'
lightorange_hex = '#ffebcc'
# ETD: normal colors
normalgreen_hex = '#99ff99'
normalblue_hex = '#99ccff'
normalorange_hex = '#ffd699' 
# GlycanSource B+P, B/P: deep colors
darkgreen_hex = '#66ff66' 
darkblue_hex = '#66b3ff'
# byos colors
lightpink_hex = '#ffb6c1'
deeppink_hex = '#ff1493'
yellow_hex = '#ffff66' 
# BYONIC RT TOLERANCE FOR UNIQUEPEP FILE <min.>, SET TO np.Inf TO CANCEL THIS CRITERION (COUNT PSM)
unique_byonic_rt = 3
# PGLYCO RT TOLERANCE FOR UNIQUEPEP FILE <sec.>, SET TO np.Inf TO CANCEL THIS CRITERION (COUNT PSM)
unique_pglyco_rt = 180

# BYONIC RT TOLERANCE FOR NGLYCOPEP FILE <min.>, SET TO np.Inf TO CANCEL THIS CRITERION (COUNT PSM)
nglyco_byonic_rt = np.Inf
# PGLYCO RT TOLERANCE FOR NGLYCOPEP FILE <sec.>, SET TO np.Inf TO CANCEL THIS CRITERION (COUNT PSM)
nglyco_pglyco_rt = np.Inf

# SCAN DIFFERENCE <scan num.> (ASSIGN PAIR)
byonic_scandif = 5
# PLOTTING CHOICES (whether to plot xicauc(byos)/int(byos)/mono(pglyco)/isotope(pglyco))
# if you do not want to plot anything, please set all the parameters to 'no'.
export_xicauc = 'yes' # set this to 'no' if byos file is not present
export_int = 'no' # set this to 'no' if byos file is not present 
export_mono = 'no' # set this to 'no' if pglyco file is not present
export_isotope = 'yes' # set this to 'no' if pglyco file is not present
# glycan category version (v1: nx, v2: unocuppied/oligoman/complex/complex+sia, v3: nx w/ man4&below-man9, v4: nx w/ man4&below-man9 & fucosylation)
plot_v1 = 'yes'
plot_v2 = 'yes'
plot_v3 = 'no'
plot_v4 = 'no'
# color hybrid yellow or not (default: n3 as hybrid)
colorHybrid = True # True/False # only for v3/v4
# TopN
topN = 5
# BYONIC SCORE CUTOFF (default=200)
BYONIC_SCORE_CUTOFF = 200
# PGLYCO PRECURSOR MZ ROUNDING DECIMAL
RoundTo = 1
# ROUNDING FOR MODS NUMBERS
ModsRoundTo = 4
# pass in the modification numbers in list (all the reuslts from byonic will be divided by the nums in this lst to find the most probable modification)
# the reuslts in byonic are calculated from the passed-in nums, so should be completely divided if it's the correct mods
mods_name = [] # IF NO MODS, LEAVE IT AS []. PLEASE PASS IN W/ THE SAME ORDER AS MODS_NUM (CAN ONLY PASS IN THE FOLLOWING CHOICES: 'SO3', 'Phospho', 'KDN', CASE-SENSITIVE)
mods_num = [] # IF NO MODS, LEAVE IT AS []. PLEASE PASS IN W/ THE SAME ORDER AS MODS_NAME
# ALLOW ATYPICAL N-GLYCOSYLATION SEQUON (e.g. N-X-C, the atypical sequons can only be detected by pglyco for the time being)
allow_atypical_sequon = 'no' # yes / no

############################### PLEASE DO NOT ALTER ANYTHING BELOW ###############################
# AUXILIARY FUNCTIONS
def get_key(val):
    # construct relation btw mods_name & mods_num
    mods_NameToNum = dict(zip(mods_name, mods_num))
    for key, value in mods_NameToNum.items():
        if val == value:
            return key
def move_df(df, move_col, insert_left_col):
    move_df = df[move_col]
    df.drop(labels=[move_col], axis=1, inplace = True)
    df.insert(df.columns.get_loc(insert_left_col) + 1, move_col, move_df)
def findNPosBySequon(seq):
#     pattern = '(?i)(N[ARNDBCEQZGHILKMFSTWYV]T)|(N[ARNDBCEQZGHILKMFSTWYV]S)' # case insensitive
    if allow_atypical_sequon == 'yes':
        pattern = '(?=((?i)(N[ARNDBCEQZGHILKMFSTWYV]T)|(N[ARNDBCEQZGHILKMFSTWYV]S)|(N[ARNDBCEQZGHILKMFSTWYV]C)))' # case insensive & able to find overlapping sequons
    else:
        pattern = '(?=((?i)(N[ARNDBCEQZGHILKMFSTWYV]T)|(N[ARNDBCEQZGHILKMFSTWYV]S)))'
    result = [(m.start(0)+1, seq[m.start(0):m.start(0)+3]) for m in re.finditer(pattern, seq, overlapped = True)]
    return result # [(site, 'sequon'), ...(...)]
def findPotentialOPos(seq):
    # find o pos by st
    opos = [m.start(0)+1 for m in re.finditer(r'(?i)(s)|(t)', seq)]
    return opos
def findUnionOnly(val): # for final major glycoform table
    color = 'red'
    weight = 'bold'
    bg_color = 'WhiteSmoke'
    return f'color: {color}; font-weight: {weight}; background-color: {bg_color}'
def sort_intstr_col(df, col_to_sort):
    # accepts int & str mixed type col, no other datatype is allowed
    lst = df[col_to_sort].tolist()
    lst.sort(key=lambda v: (isinstance(v, str), v))
    sorter = list(set(lst))
    # reindex the df with .loc by the new sorted lst
    df = df.set_index(col_to_sort).loc[lst]
#     df.drop_duplicates(inplace=True)
#     df.reset_index(inplace=True)
    df = df.drop_duplicates().reset_index()
#     df[col_to_sort] = df[col_to_sort].astype("category")
#     df[col_to_sort].cat.set_categories(sorter, inplace=True)
#     df = df.sort_values([col_to_sort]).reset_index(drop=True)
    return df
# get export filename based on input combo
def export_filename(byonicfile = '', byosfile = '', pglycofile = ''):
    names = [byonicfile, byosfile, pglycofile]
    names = [n for n in names if n != '']
    names = '_'.join(names)
    return names
# define glycansource function
def glycansource(df, colname):
    # glycan comprison: only present in byonic -> b, only present in pglyco -> p, both the same -> b+p, not the same -> b/p
    conditions = [
        (df['rounded_FinalGlycans[Byonic]'] != -1) & (df['GlycanComposition_ByonicStyle[pGlyco]'] == -1),
        (df['rounded_FinalGlycans[Byonic]'] == -1) & (df['GlycanComposition_ByonicStyle[pGlyco]'] != -1),
        (df['rounded_FinalGlycans[Byonic]'] != -1) & (df['GlycanComposition_ByonicStyle[pGlyco]'] != -1) & (df['rounded_FinalGlycans[Byonic]'] == df['GlycanComposition_ByonicStyle[pGlyco]']),
        (df['rounded_FinalGlycans[Byonic]'] != -1) & (df['GlycanComposition_ByonicStyle[pGlyco]'] != -1) & (df['rounded_FinalGlycans[Byonic]'] != df['GlycanComposition_ByonicStyle[pGlyco]'])]
    choices = ['B', 'P', 'B=P', 'B≠P'] 
    glycan_source = np.select(conditions, choices, -1) 
    df.insert(df.columns.get_loc(colname) + 1 , 'GlycanSource', glycan_source , True)
# define mask function
def threshold_masks_colorind(df):
    global b_hcd_mask, p_hcd_mask, both_hcd_mask, b_etd_mask, p_etd_mask, both_etd_mask
    global b_glycansource_mask, p_glycansource_mask, byos_exclusiveOr_mask, byos_and_mask, byos_bothsame_mask
    global lightgreen_ind, lightblue_ind, lightorange_ind, normalgreen_ind, normalblue_ind, normalorange_ind, deepgreen_ind, deepblue_ind, lightpink_ind, deeppink_ind, yellow_ind  

    if byonicfile != '' and byosfile != '' and pglycofile != '': # bbp all present
        # HCD 
        b_hcd_mask = (df['Fragment\r\nType[Byonic]'] == 'hcd') & (df['Score[Byonic]'] > BYONIC_SCORE_CUTOFF) & (df['PEP\r\n2D[Byonic]'].abs() < 0.001) & ((df['PepScore[pGlyco]'] <= 5) | (df['GlyScore[pGlyco]'] <= 4))
        p_hcd_mask = (df['FragmentType[pGlyco]'] == 'hcd') & ((df['Score[Byonic]'] <= BYONIC_SCORE_CUTOFF) | (df['PEP\r\n2D[Byonic]'].abs() >= 0.001)) & (df['PepScore[pGlyco]'] > 5) & (df['GlyScore[pGlyco]'] > 4)    
        both_hcd_mask = (df['Fragment\r\nType[Byonic]'] == 'hcd') & (df['FragmentType[pGlyco]'] == 'hcd') & (df['Score[Byonic]'] > BYONIC_SCORE_CUTOFF) & (df['PEP\r\n2D[Byonic]'].abs() < 0.001) & (df['PepScore[pGlyco]'] > 5) & (df['GlyScore[pGlyco]'] > 4)
        # ETD: remember byonic etd does not need threshold, so we only need to make sure that the row only contain byonic data, which means pglyco data will be -1
        b_etd_mask = (df['Fragment\r\nType[Byonic]'] == 'ethcd') & (df['Score[Byonic]'] != -1) & (df['PEP\r\n2D[Byonic]'] != -1) & ((df['PepScore[pGlyco]'] <= 5) | (df['GlyScore[pGlyco]'] <= 4)) 
        p_etd_mask = (df['FragmentType[pGlyco]'] == 'ethcd') & (df['Score[Byonic]'] == -1) & (df['PEP\r\n2D[Byonic]'] == -1) & (df['PepScore[pGlyco]'] > 5) & (df['GlyScore[pGlyco]'] > 4)    
        both_etd_mask = (df['Fragment\r\nType[Byonic]'] == 'ethcd') & (df['FragmentType[pGlyco]'] == 'ethcd') & (df['Score[Byonic]'] != -1) & (df['PEP\r\n2D[Byonic]'] != -1) & (df['PepScore[pGlyco]'] > 5) & (df['GlyScore[pGlyco]'] > 4)
        # (HCD OR ETD) & (B+P OR B/P) & passes threshold: parts of lightblue/blue & lightgreen/green will become deep colors
        b_glycansource_mask = ((df['GlycanSource'] == 'B=P') ^ (df['GlycanSource'] == 'B≠P')) & ((b_hcd_mask) ^ (b_etd_mask)) & (~((p_hcd_mask) ^ (p_etd_mask))) # hcd exclusive or etd & ~(p)
        p_glycansource_mask = ((df['GlycanSource'] == 'B=P') ^ (df['GlycanSource'] == 'B≠P')) & ((p_hcd_mask) ^ (p_etd_mask)) & (~((b_hcd_mask) ^ (b_etd_mask))) # hcd exclusive or etd & ~(b)   
        # comparison between byonic & byos
        byos_exclusiveOr_mask = (df['Calc.MH[Byos]'] != -1) & (df['Calc.\r\nMH[Byonic]'] != -1) & ((df['Calc.MH[Byos]'].round(decimals=2) != df['Calc.\r\nMH[Byonic]'].round(decimals=2))^(df['PureSequence[Byos]'] != df['PureSequence[Byonic]']))
        byos_and_mask = (df['Calc.MH[Byos]'] != -1) & (df['Calc.\r\nMH[Byonic]'] != -1) & (df['Calc.MH[Byos]'].round(decimals=2) != df['Calc.\r\nMH[Byonic]'].round(decimals=2)) & (df['PureSequence[Byos]'] != df['PureSequence[Byonic]'])
        byos_bothsame_mask = (df['Calc.MH[Byos]'] != -1) & (df['Calc.\r\nMH[Byonic]'] != -1) & (df['Calc.MH[Byos]'].round(decimals=2) == df['Calc.\r\nMH[Byonic]'].round(decimals=2)) & (df['PureSequence[Byos]'] == df['PureSequence[Byonic]'])
        # record df color indices
        lightgreen_ind = df.loc[b_hcd_mask].index.tolist()
        df.loc[b_hcd_mask, 'ColorCode'] = 'lightgreen(%s)'%lightgreen_hex
        
        lightblue_ind = df.loc[p_hcd_mask].index.tolist()
        df.loc[p_hcd_mask, 'ColorCode'] = 'lightblue(%s)'%lightblue_hex
        
        lightorange_ind = df.loc[both_hcd_mask].index.tolist()
        df.loc[both_hcd_mask, 'ColorCode'] = 'lightorange(%s)'%lightorange_hex
        
        normalgreen_ind = df.loc[b_etd_mask].index.tolist()
        df.loc[b_etd_mask, 'ColorCode'] = 'normalgreen(%s)'%normalgreen_hex
        
        normalblue_ind = df.loc[p_etd_mask].index.tolist()
        df.loc[p_etd_mask, 'ColorCode'] = 'normalblue(%s)'%normalblue_hex
            
        normalorange_ind = df.loc[both_etd_mask].index.tolist()
        df.loc[both_etd_mask, 'ColorCode'] = 'normalorange(%s)'%normalorange_hex
        
        deepgreen_ind = df.loc[b_glycansource_mask].index.tolist()
        df.loc[b_glycansource_mask, 'ColorCode'] = 'deepgreen(%s)'%darkgreen_hex
        
        deepblue_ind = df.loc[p_glycansource_mask].index.tolist()
        df.loc[p_glycansource_mask, 'ColorCode'] = 'deepblue(%s)'%darkblue_hex
        
        lightpink_ind = df.loc[byos_exclusiveOr_mask].index.tolist()
        
        deeppink_ind = df.loc[byos_and_mask].index.tolist()
        
        yellow_ind = df.loc[byos_bothsame_mask].index.tolist()
    elif byonicfile != '' and byosfile == '' and pglycofile != '': # byonic + pglyco
        # HCD 
        b_hcd_mask = (df['Fragment\r\nType[Byonic]'] == 'hcd') & (df['Score[Byonic]'] > BYONIC_SCORE_CUTOFF) & (df['PEP\r\n2D[Byonic]'].abs() < 0.001) & ((df['PepScore[pGlyco]'] <= 5) | (df['GlyScore[pGlyco]'] <= 4))
        p_hcd_mask = (df['FragmentType[pGlyco]'] == 'hcd') & ((df['Score[Byonic]'] <= BYONIC_SCORE_CUTOFF) | (df['PEP\r\n2D[Byonic]'].abs() >= 0.001)) & (df['PepScore[pGlyco]'] > 5) & (df['GlyScore[pGlyco]'] > 4)    
        both_hcd_mask = (df['Fragment\r\nType[Byonic]'] == 'hcd') & (df['FragmentType[pGlyco]'] == 'hcd') & (df['Score[Byonic]'] > BYONIC_SCORE_CUTOFF) & (df['PEP\r\n2D[Byonic]'].abs() < 0.001) & (df['PepScore[pGlyco]'] > 5) & (df['GlyScore[pGlyco]'] > 4)
        # ETD: remember byonic etd does not need threshold, so we only need ot make sure that the row only contain byonic data, which means pglyco data will be -1
        b_etd_mask = (df['Fragment\r\nType[Byonic]'] == 'ethcd') & (df['Score[Byonic]'] != -1) & (df['PEP\r\n2D[Byonic]'] != -1) & ((df['PepScore[pGlyco]'] <= 5) | (df['GlyScore[pGlyco]'] <= 4)) 
        p_etd_mask = (df['FragmentType[pGlyco]'] == 'ethcd') & (df['Score[Byonic]'] == -1) & (df['PEP\r\n2D[Byonic]'] == -1) & (df['PepScore[pGlyco]'] > 5) & (df['GlyScore[pGlyco]'] > 4)    
        both_etd_mask = (df['Fragment\r\nType[Byonic]'] == 'ethcd') & (df['FragmentType[pGlyco]'] == 'ethcd') & (df['Score[Byonic]'] != -1) & (df['PEP\r\n2D[Byonic]'] != -1) & (df['PepScore[pGlyco]'] > 5) & (df['GlyScore[pGlyco]'] > 4)
        # (HCD OR ETD) & (B+P OR B/P) & passes threshold: parts of lightblue/blue & lightgreen/green will become deep colors
        b_glycansource_mask = ((df['GlycanSource'] == 'B=P') ^ (df['GlycanSource'] == 'B≠P')) & ((b_hcd_mask) ^ (b_etd_mask)) & (~((p_hcd_mask) ^ (p_etd_mask))) # hcd exclusive or etd & ~(p)
        p_glycansource_mask = ((df['GlycanSource'] == 'B=P') ^ (df['GlycanSource'] == 'B≠P')) & ((p_hcd_mask) ^ (p_etd_mask)) & (~((b_hcd_mask) ^ (b_etd_mask))) # hcd exclusive or etd & ~(b)   
        # record df color indices
        lightgreen_ind = df.loc[b_hcd_mask].index.tolist()
        df.loc[b_hcd_mask, 'ColorCode'] = 'lightgreen(%s)'%lightgreen_hex
        
        lightblue_ind = df.loc[p_hcd_mask].index.tolist()
        df.loc[p_hcd_mask, 'ColorCode'] = 'lightblue(%s)'%lightblue_hex
        
        lightorange_ind = df.loc[both_hcd_mask].index.tolist()
        df.loc[both_hcd_mask, 'ColorCode'] = 'lightorange(%s)'%lightorange_hex
        
        normalgreen_ind = df.loc[b_etd_mask].index.tolist()
        df.loc[b_etd_mask, 'ColorCode'] = 'normalgreen(%s)'%normalgreen_hex
        
        normalblue_ind = df.loc[p_etd_mask].index.tolist()
        df.loc[p_etd_mask, 'ColorCode'] = 'normalblue(%s)'%normalblue_hex
            
        normalorange_ind = df.loc[both_etd_mask].index.tolist()
        df.loc[both_etd_mask, 'ColorCode'] = 'normalorange(%s)'%normalorange_hex
        
        deepgreen_ind = df.loc[b_glycansource_mask].index.tolist()
        df.loc[b_glycansource_mask, 'ColorCode'] = 'deepgreen(%s)'%darkgreen_hex
        
        deepblue_ind = df.loc[p_glycansource_mask].index.tolist()
        df.loc[p_glycansource_mask, 'ColorCode'] = 'deepblue(%s)'%darkblue_hex
    elif byonicfile != '' and byosfile == '' and pglycofile == '': # only byonic
        # HCD 
        b_hcd_mask = (df['Fragment\r\nType'] == 'hcd') & (df['Score'] > BYONIC_SCORE_CUTOFF) & (df['PEP\r\n2D'].abs() < 0.001)
        # ETD: remember byonic etd does not need threshold, so we only need ot make sure that the row only contain byonic data, which means pglyco data will be -1
        b_etd_mask = (df['Fragment\r\nType'] == 'ethcd') & (df['Score'] != -1) & (df['PEP\r\n2D'] != -1) 
        # record df color indices
        lightgreen_ind = df.loc[b_hcd_mask].index.tolist()
        df.loc[b_hcd_mask, 'ColorCode'] = 'lightgreen(%s)'%lightgreen_hex
        
        normalgreen_ind = df.loc[b_etd_mask].index.tolist()
        df.loc[b_etd_mask, 'ColorCode'] = 'normalgreen(%s)'%normalgreen_hex
    elif byonicfile == '' and byosfile == '' and pglycofile != '': # only pglyco
        # HCD 
        p_hcd_mask = (df['FragmentType'] == 'hcd') & (df['PepScore'] > 5) & (df['GlyScore'] > 4)    
        # ETD: remember byonic etd does not need threshold, so we only need ot make sure that the row only contain byonic data, which means pglyco data will be -1
        p_etd_mask = (df['FragmentType'] == 'ethcd') & (df['PepScore'] > 5) & (df['GlyScore'] > 4)    
        # record df color indices
        lightblue_ind = df.loc[p_hcd_mask].index.tolist()
        df.loc[p_hcd_mask, 'ColorCode'] = 'lightblue(%s)'%lightblue_hex
        
        normalblue_ind = df.loc[p_etd_mask].index.tolist()
        df.loc[p_etd_mask, 'ColorCode'] = 'normalblue(%s)'%normalblue_hex
    elif byonicfile != '' and byosfile != '' and pglycofile == '': # byonic + byos
        # HCD 
        b_hcd_mask = (df['Fragment\r\nType[Byonic]'] == 'hcd') & (df['Score[Byonic]'] > BYONIC_SCORE_CUTOFF) & (df['PEP\r\n2D[Byonic]'].abs() < 0.001)
        # ETD: remember byonic etd does not need threshold, so we only need ot make sure that the row only contain byonic data, which means pglyco data will be -1
        b_etd_mask = (df['Fragment\r\nType[Byonic]'] == 'ethcd') & (df['Score[Byonic]'] != -1) & (df['PEP\r\n2D[Byonic]'] != -1)
        # comparison between byonic & byos
        byos_exclusiveOr_mask = (df['Calc.MH[Byos]'] != -1) & (df['Calc.\r\nMH[Byonic]'] != -1) & ((df['Calc.MH[Byos]'].round(decimals=2) != df['Calc.\r\nMH[Byonic]'].round(decimals=2))^(df['PureSequence[Byos]'] != df['PureSequence[Byonic]']))
        byos_and_mask = (df['Calc.MH[Byos]'] != -1) & (df['Calc.\r\nMH[Byonic]'] != -1) & (df['Calc.MH[Byos]'].round(decimals=2) != df['Calc.\r\nMH[Byonic]'].round(decimals=2)) & (df['PureSequence[Byos]'] != df['PureSequence[Byonic]'])
        byos_bothsame_mask = (df['Calc.MH[Byos]'] != -1) & (df['Calc.\r\nMH[Byonic]'] != -1) & (df['Calc.MH[Byos]'].round(decimals=2) == df['Calc.\r\nMH[Byonic]'].round(decimals=2)) & (df['PureSequence[Byos]'] == df['PureSequence[Byonic]'])
        # record df color indices
        lightgreen_ind = df.loc[b_hcd_mask].index.tolist()
        df.loc[b_hcd_mask, 'ColorCode'] = 'lightgreen(%s)'%lightgreen_hex
        
        normalgreen_ind = df.loc[b_etd_mask].index.tolist()
        df.loc[b_etd_mask, 'ColorCode'] = 'normalgreen(%s)'%normalgreen_hex
        
        lightpink_ind = df.loc[byos_exclusiveOr_mask].index.tolist()
        
        deeppink_ind = df.loc[byos_and_mask].index.tolist()
        
        yellow_ind = df.loc[byos_bothsame_mask].index.tolist()
def bg_color(x):
    if byonicfile != '' and byosfile != '' and pglycofile != '': # bbp all present
        # byonic & pglyco colors
        # HCD: light colors
        c1 = 'background-color: %s'%lightgreen_hex 
        c2 = 'background-color: %s'%lightblue_hex 
        c3 = 'background-color: %s'%lightorange_hex 
        # ETD: normal colors
        c4 = 'background-color: %s'%normalgreen_hex 
        c5 = 'background-color: %s'%normalblue_hex 
        c6 = 'background-color: %s'%normalorange_hex 
        # GlycanSource B+P, B/P: deep colors
        c7 = 'background-color: %s'%darkgreen_hex 
        c8 = 'background-color: %s'%darkblue_hex 
        # byos colors
        c9 = 'background-color: %s'%lightpink_hex 
        c10 = 'background-color: %s'%deeppink_hex
        c11 = 'background-color: %s'%yellow_hex 
        c = '' 
        #DataFrame with same index and columns names as original filled empty strings
        df1 =  pd.DataFrame(c, index=x.index, columns=x.columns)    
        # byonic & byos coloring range
        bb_range = [col for col in df1.columns.tolist() if '[Byos]' in col]
        # byonic & pglyco coloring range
        bp_range = [col for col in df1.columns.tolist() if '[Byonic]' in col or '[pGlyco]' in col]
        # modify values of df1 column by boolean mask
        # HCD: light colors (c1-c3)
        df1.loc[b_hcd_mask, bp_range] = c1
        print('<Color Summary>\n%s rows will be colored light green (%s).'%(len(df1.loc[b_hcd_mask, bp_range]), lightgreen_hex))
        df1.loc[p_hcd_mask, bp_range] = c2 
        print('%s rows will be colored light blue (%s).'%(len(df1.loc[p_hcd_mask, bp_range]), lightblue_hex))
        df1.loc[both_hcd_mask, bp_range] = c3
        print('%s rows will be colored light orange (%s).'%(len(df1.loc[both_hcd_mask, bp_range]), lightorange_hex))
        # ETD: normal colors (c4-c6)
        df1.loc[b_etd_mask, bp_range] = c4
        print('%s rows will be colored green (%s).'%(len(df1.loc[b_etd_mask, bp_range]), normalgreen_hex))
        df1.loc[p_etd_mask, bp_range] = c5
        print('%s rows will be colored blue (%s).'%(len(df1.loc[p_etd_mask, bp_range]), normalblue_hex))
        df1.loc[both_etd_mask, bp_range] = c6
        print('%s rows will be colored orange (%s).'%(len(df1.loc[both_etd_mask, bp_range]), normalorange_hex))
        # GlycanSource B+P, B/P: deep colors (c7-c8)
        df1.loc[b_glycansource_mask, bp_range] = c7
        print('%s rows will be colored dark green (%s).'%(len(df1.loc[b_glycansource_mask, bp_range]), darkgreen_hex))
        df1.loc[p_glycansource_mask, bp_range] = c8
        print('%s rows will be colored dark blue (%s).'%(len(df1.loc[p_glycansource_mask, bp_range]), darkblue_hex))
        # byos colors (c9-c11)
        df1.loc[byos_exclusiveOr_mask, bb_range] = c9
        print('%s rows will be colored light pink (%s).'%(len(df1.loc[byos_exclusiveOr_mask, bb_range]), lightpink_hex))
        df1.loc[byos_and_mask, bb_range] = c10
        print('%s rows will be colored deep pink (%s).'%(len(df1.loc[byos_and_mask, bb_range]), deeppink_hex))
        df1.loc[byos_bothsame_mask, bb_range] = c11
        print('%s rows will be colored yellow (%s).'%(len(df1.loc[byos_bothsame_mask, bb_range]), yellow_hex))
        
        bp_white = len(df1) - (len(df1.loc[b_hcd_mask, bp_range]) + len(df1.loc[p_hcd_mask, bp_range]) \
                          + len(df1.loc[both_hcd_mask, bp_range]) + len(df1.loc[b_etd_mask, bp_range]) + len(df1.loc[p_etd_mask, bp_range]) \
                          + len(df1.loc[both_etd_mask, bp_range]))
        bb_white = len(df1) - (len(df1.loc[byos_exclusiveOr_mask, bb_range]) + len(df1.loc[byos_and_mask, bb_range]) + len(df1.loc[byos_bothsame_mask, bb_range]))
        print('%s rows will be colorless in byonic & pglyco data.'%bp_white)
        print('%s rows will be colorless in byos data (absent data in certain scans).'%bb_white)
        return df1
    elif byonicfile != '' and byosfile == '' and pglycofile != '': # byonic + pglyco
        # byonic & pglyco colors
        # HCD: light colors
        c1 = 'background-color: %s'%lightgreen_hex 
        c2 = 'background-color: %s'%lightblue_hex 
        c3 = 'background-color: %s'%lightorange_hex 
        # ETD: normal colors
        c4 = 'background-color: %s'%normalgreen_hex 
        c5 = 'background-color: %s'%normalblue_hex 
        c6 = 'background-color: %s'%normalorange_hex 
        # GlycanSource B+P, B/P: deep colors
        c7 = 'background-color: %s'%darkgreen_hex 
        c8 = 'background-color: %s'%darkblue_hex
        c = '' 
        #DataFrame with same index and columns names as original filled empty strings
        df1 =  pd.DataFrame(c, index=x.index, columns=x.columns) 
        # byonic & pglyco coloring range
        bp_range = [col for col in df1.columns.tolist() if '[Byonic]' in col or '[pGlyco]' in col]
        # modify values of df1 column by boolean mask
        # HCD: light colors (c1-c3)
        df1.loc[b_hcd_mask, bp_range] = c1
        print('<Color Summary>\n%s rows will be colored light green (%s).'%(len(df1.loc[b_hcd_mask, bp_range]), lightgreen_hex))
        df1.loc[p_hcd_mask, bp_range] = c2 
        print('%s rows will be colored light blue (%s).'%(len(df1.loc[p_hcd_mask, bp_range]), lightblue_hex))
        df1.loc[both_hcd_mask, bp_range] = c3
        print('%s rows will be colored light orange (%s).'%(len(df1.loc[both_hcd_mask, bp_range]), lightorange_hex))
        # ETD: normal colors (c4-c6)
        df1.loc[b_etd_mask, bp_range] = c4
        print('%s rows will be colored green (%s).'%(len(df1.loc[b_etd_mask, bp_range]), normalgreen_hex))
        df1.loc[p_etd_mask, bp_range] = c5
        print('%s rows will be colored blue (%s).'%(len(df1.loc[p_etd_mask, bp_range]), normalblue_hex))
        df1.loc[both_etd_mask, bp_range] = c6
        print('%s rows will be colored orange (%s).'%(len(df1.loc[both_etd_mask, bp_range]), normalorange_hex))
        # GlycanSource B+P, B/P: deep colors (c7-c8)
        df1.loc[b_glycansource_mask, bp_range] = c7
        print('%s rows will be colored dark green (%s).'%(len(df1.loc[b_glycansource_mask, bp_range]), darkgreen_hex))
        df1.loc[p_glycansource_mask, bp_range] = c8
        print('%s rows will be colored dark blue (%s).'%(len(df1.loc[p_glycansource_mask, bp_range]), darkblue_hex))
        
        bp_white = len(df1) - (len(df1.loc[b_hcd_mask, bp_range]) + len(df1.loc[p_hcd_mask, bp_range]) \
                          + len(df1.loc[both_hcd_mask, bp_range]) + len(df1.loc[b_etd_mask, bp_range]) + len(df1.loc[p_etd_mask, bp_range]) \
                          + len(df1.loc[both_etd_mask, bp_range]))
        print('%s rows will be colorless in byonic & pglyco data.'%bp_white)
        return df1
    elif byonicfile != '' and byosfile == '' and pglycofile == '': # only byonic
        # byonic colors
        # HCD: light colors
        c1 = 'background-color: %s'%lightgreen_hex
        # ETD: normal colors
        c4 = 'background-color: %s'%normalgreen_hex
        c = '' 
        #DataFrame with same index and columns names as original filled empty strings
        df1 =  pd.DataFrame(c, index=x.index, columns=x.columns) 
        # modify values of df1 column by boolean mask
        # HCD: light colors (c1-c3)
        df1.loc[b_hcd_mask, :] = c1
        print('<Color Summary>\n%s rows will be colored light green (%s).'%(len(df1.loc[b_hcd_mask, :]), lightgreen_hex))
        # ETD: normal colors (c4-c6)
        df1.loc[b_etd_mask, :] = c4
        print('%s rows will be colored green (%s).'%(len(df1.loc[b_etd_mask, :]), normalgreen_hex))
        
        bp_white = len(df1) - (len(df1.loc[b_hcd_mask, :]) + len(df1.loc[b_etd_mask, :]))
        print('%s rows will be colorless in byonic data.'%bp_white)
        return df1
    elif byonicfile == '' and byosfile == '' and pglycofile != '': # only pglyco
        # pglyco colors
        # HCD: light colors
        c2 = 'background-color: %s'%lightblue_hex
        # ETD: normal colors
        c5 = 'background-color: %s'%normalblue_hex
        c = '' 
        #DataFrame with same index and columns names as original filled empty strings
        df1 =  pd.DataFrame(c, index=x.index, columns=x.columns) 
        # modify values of df1 column by boolean mask
        # HCD: light colors (c1-c3)
        df1.loc[p_hcd_mask, :] = c2 
        print('%s rows will be colored light blue (%s).'%(len(df1.loc[p_hcd_mask, :]), lightblue_hex))
        # ETD: normal colors (c4-c6)
        df1.loc[p_etd_mask, :] = c5
        print('%s rows will be colored blue (%s).'%(len(df1.loc[p_etd_mask, :]), normalblue_hex))
        
        bp_white = len(df1) - (len(df1.loc[p_hcd_mask, :]) + len(df1.loc[p_etd_mask, :]))
        print('%s rows will be colorless in byonic & pglyco data.'%bp_white)
        return df1
    elif byonicfile != '' and byosfile != '' and pglycofile == '': # byonic + byos
        # byonic colors
        # HCD: light colors
        c1 = 'background-color: %s'%lightgreen_hex
        # ETD: normal colors
        c4 = 'background-color: %s'%normalgreen_hex
        # byos colors
        c9 = 'background-color: %s'%lightpink_hex 
        c10 = 'background-color: %s'%deeppink_hex
        c11 = 'background-color: %s'%yellow_hex
        c = '' 
        #DataFrame with same index and columns names as original filled empty strings
        df1 =  pd.DataFrame(c, index=x.index, columns=x.columns)    
        # byonic & byos coloring range
        bb_range = [col for col in df1.columns.tolist() if '[Byos]' in col]
        # byonic & pglyco coloring range
        bp_range = [col for col in df1.columns.tolist() if '[Byonic]' in col]
        # modify values of df1 column by boolean mask
        # HCD: light colors (c1-c3)
        df1.loc[b_hcd_mask, bp_range] = c1
        print('<Color Summary>\n%s rows will be colored light green (%s).'%(len(df1.loc[b_hcd_mask, bp_range]), lightgreen_hex))
        # ETD: normal colors (c4-c6)
        df1.loc[b_etd_mask, bp_range] = c4
        print('%s rows will be colored green (%s).'%(len(df1.loc[b_etd_mask, bp_range]), normalgreen_hex))
        # byos colors (c9-c11)
        df1.loc[byos_exclusiveOr_mask, bb_range] = c9
        print('%s rows will be colored light pink (%s).'%(len(df1.loc[byos_exclusiveOr_mask, bb_range]), lightpink_hex))
        df1.loc[byos_and_mask, bb_range] = c10
        print('%s rows will be colored deep pink (%s).'%(len(df1.loc[byos_and_mask, bb_range]), deeppink_hex))
        df1.loc[byos_bothsame_mask, bb_range] = c11
        print('%s rows will be colored yellow (%s).'%(len(df1.loc[byos_bothsame_mask, bb_range]), yellow_hex))
        bp_white = len(df1) - (len(df1.loc[b_hcd_mask, bp_range]) + len(df1.loc[b_etd_mask, bp_range]))
        bb_white = len(df1) - (len(df1.loc[byos_exclusiveOr_mask, bb_range]) + len(df1.loc[byos_and_mask, bb_range]) + len(df1.loc[byos_bothsame_mask, bb_range]))
        print('%s rows will be colorless in byonic & pglyco data.'%bp_white)
        print('%s rows will be colorless in byos data (absent data in certain scans).'%bb_white)
        return df1

def v3(df): # for v3
    n = [int(lst[0]) if lst!=[] else 0 for lst in df['Glycans ↓'].str.findall(r'HexNAc\((\d+)\)').tolist()]
    h = [int(lst[0]) if lst!=[] else 0 for lst in df['Glycans ↓'].str.findall(r'Hex\((\d+)\)').tolist()]
    nxhy = [(n[i], h[i]) for i in range(len(n))]
    # nxhy_fix = ['N(%s)'%(t[0]) if t[0]!=2 else 'N(2)H(5) ↓' if t[0]==2 and t[1]<=5 else 'N(2)H(9) ↑' if t[0]==2 and t[1]>=9 else 'N(%s)H(%s)'%(t[0], t[1]) for t in nxhy]
    nxhy_fix = ['N(2)H(5) ↓' if t[0]==2 and t[1]<=5 else 'N(2)H(9) ↑' if t[0]==2 and t[1]>=9 else 'N(%s)H(%s)'%(t[0], t[1]) if t[0]==2 and 5<t[1]<9 else 'N(%s)'%(t[0]) for t in nxhy]
    df['N(x)H(y)'] = nxhy_fix
    nxhy_fix_order = sorted(list(set(nxhy_fix)))
    nxhy_fix_order = sorted(nxhy_fix_order, key=lambda x: int(re.findall(r'[0-9]+', x)[0]))
    return df, nxhy_fix_order

def v4(df): # for v4 (add ' ' to align hexnac number)
    f = [int(lst[0]) if lst!=[] else 0 for lst in df['Glycans ↓'].str.findall(r'Fuc\((\d+)\)').tolist()]
    n = [int(lst[0]) if lst!=[] else 0 for lst in df['Glycans ↓'].str.findall(r'HexNAc\((\d+)\)').tolist()]
    h = [int(lst[0]) if lst!=[] else 0 for lst in df['Glycans ↓'].str.findall(r'Hex\((\d+)\)').tolist()]
    fnxhy = [(f[i], n[i], h[i]) for i in range(len(n))]
    # fnxhy_fix = [' N(%s)'%(t[1]) if t[1]!=2 and t[0]==0 else ' N(2)H(5) ↓' if t[1]==2 and t[2]<=5 and t[0]==0 else ' N(2)H(9) ↑' if t[1]==2 and t[2]>=9 and t[0]==0 else ' N(%s)H(%s)'%(t[1], t[2]) if t[1]==2 and t[0]==0 else 'FN(%s)'%(t[1]) for t in fnxhy]
    fnxhy_fix = [' N(2)H(5) ↓' if t[1]==2 and t[2]<=5 and t[0]==0 else ' N(2)H(9) ↑' if t[1]==2 and t[2]>=9 and t[0]==0 else ' N(%s)H(%s)'%(t[1], t[2]) if t[1]==2 and 5<t[2]<9 and t[0]==0 else ' N(%s)'%(t[1]) if t[1]!=2 and t[0]==0 else 'FN(%s)'%(t[1]) for t in fnxhy]
    df['(F)N(x)H(y)'] = fnxhy_fix
    fnxhy_fix_order = sorted(list(set(fnxhy_fix)))
    fnxhy_fix_order = sorted(fnxhy_fix_order, key=lambda x: int(re.findall(r'[0-9]+', x)[0]))
    return df, fnxhy_fix_order

# PLOTTING FUNC
def plot_clustered_stacked(dfall, nsum, labels=None, title='', neugc_exist=None, detailedHighMan=False, fucosylation=False, hybrid=colorHybrid, tickW=2, tickL=15, spineW=2, xlabelsize=15, ticklabelsize=15, xlabel_rotation=90, xlabelpad=5, legend_fontsize=15, legend_handleL=3, hatch_lineW=1, bar_labelpad=0, bar_labelfontsize=7):
    # NOTE THE DEFAULT STYLE OF THIS FUNC IS V1/V2 (V1: original Yu-Chun version, V2: original Yu-Chun version, V3: detailedHighMan=True,fucosylation=False, hybrid=TorF(color or not) / V4: detailedHighMan=True,fucosylation=True, hybrid=TorF(color or not))
    # Given a list of dataframes, with identical columns and index, create a clustered stacked bar plot
    # labels is a list of the names of the dataframe, used for the legend
    # title is a string for the title of the plot
    # H is the hatch used for identification of the different dataframe
    n_df = len(dfall)
    n_col = len(dfall[0].columns) 
    n_ind = len(dfall[0].index)
    k = dfall[0].columns.tolist()
    
    # colormaps
    gw = plt.cm.gray(np.linspace(0, 1, 5))[-2:]
    single_p = plt.cm.Purples(np.linspace(0, 1, 10))[5]
    single_b = plt.cm.Blues(np.linspace(0, 1, 10))[5]
    single_r = plt.cm.Reds(np.linspace(0, 1, 5))[3]
    single_g = plt.cm.Greens(np.linspace(0, 1, 4))[2]

    if neugc_exist is None: # this section caters to v1/v3/v4 plotting
        # extract the largest n number in col to determine the color dict range
        # so that the color-nnumber relation is locked
        # n_max = ast.literal_eval(re.findall(r'N\((\d+)\)', k[-1])[0])
        n_max = 12 # default to 12 to make sure the gradients are the same across dif plots
        b = plt.cm.Blues(np.linspace(0, 1, n_max))[2:] # can be more than N(10) # for v1 & v3
        if fucosylation == True: # Reds for fucosylation 
            fuc = plt.cm.Reds(np.linspace(0, 1, n_max+2))[2:] # just like complex having unlimited range
        else:
            pass

        if detailedHighMan == True:
            highman_g = plt.cm.Greens(np.linspace(0, 1, 8))[2:7] # man5&below-man9&above
        else:
            pass

        if hybrid == True: 
            hybrid_y = plt.cm.spring(np.linspace(0, 1, 6))[-1] # only n3(orange). fn3 will be colored red also for clarity
            hybrid_b = plt.cm.Blues(np.linspace(0, 1, n_max))[3:] # n3 will be colored orange, so blue starts from n4
        else:
            pass 

    # combine them and build a new colormap
    if neugc_exist == True: # v2, complex+sia in red
        colors = np.vstack((gw, single_g, single_b, single_r)) 
        # construct 1 to 1 color dict
        n = ['N(0)', 'N(1)', 'OligoMannose', 'Complex', 'Complex+Sia'] # limited categories
        c = [colors[i] for i in range(len(n))]
        nc_dict = dict(zip(n, c)) # given k lst, can get v = c. v = nc_dict[k[k_cnt]]
    elif neugc_exist == False: # v2, complex+sia in purple
        colors = np.vstack((gw, single_g, single_b, single_p)) 
        # construct 1 to 1 color dict
        n = ['N(0)', 'N(1)', 'OligoMannose', 'Complex', 'Complex+Sia'] # limited categories
        c = [colors[i] for i in range(len(n))]
        nc_dict = dict(zip(n, c)) # given k lst, can get v = c. v = nc_dict[k[k_cnt]]
    else: # v1/v3/v4
        if detailedHighMan == False and fucosylation == False: # v1
            colors = np.vstack((gw, single_g, b)) # b is unlimited cmap determined by n_max
            # construct 1 to 1 color dict
            n = ['N(%s)'%i for i in range(n_max+1)] # e.g. N(0)-N(10), can be more than 11 cols # unlimited categories
            c = [colors[i] for i in range(len(n))]
            nc_dict = dict(zip(n, c)) # given k lst, can get v = c. v = nc_dict[k[k_cnt]]
        elif detailedHighMan == True and fucosylation == False: # v3
            if hybrid == True:
                colors = np.vstack((gw, highman_g, hybrid_y, hybrid_b)) # b is unlimited cmap determined by n_max
                # construct 1 to 1 color dict
                n = ['N(%s)'%i for i in range(n_max+1) if i!=2]
                highman = ['N(2)H(%s) ↓'%(i) if i==5 else 'N(2)H(%s) ↑'%(i) if i==9 else 'N(2)H(%s)'%(i) for i in range(5, 10, 1)]
                n[2:2] = highman
                c = [colors[i] for i in range(len(n))]
                nc_dict = dict(zip(n, c)) # given k lst, can get v = c. v = nc_dict[k[k_cnt]]
            else:
                colors = np.vstack((gw, highman_g, b)) # b is unlimited cmap determined by n_max
                # construct 1 to 1 color dict
                n = ['N(%s)'%i for i in range(n_max+1) if i!=2]
                highman = ['N(2)H(%s) ↓'%(i) if i==5 else 'N(2)H(%s) ↑'%(i) if i==9 else 'N(2)H(%s)'%(i) for i in range(5, 10, 1)]
                n[2:2] = highman
                c = [colors[i] for i in range(len(n))]
                nc_dict = dict(zip(n, c)) # given k lst, can get v = c. v = nc_dict[k[k_cnt]]
        elif detailedHighMan == True and fucosylation == True: # v4
            if hybrid == True:
                colors = np.vstack((gw, highman_g, hybrid_y, hybrid_b, fuc)) # b & cyans are unlimited cmap determined by n_max
                # construct 1 to 1 color dict
                n = [' N(%s)'%i for i in range(n_max+1) if i!=2]
                highman = [' N(2)H(%s) ↓'%(i) if i==5 else ' N(2)H(%s) ↑'%(i) if i==9 else ' N(2)H(%s)'%(i) for i in range(5, 10, 1)]
                n[2:2] = highman
                # append fucosylation for cyans colormap
                f = ['FN(%s)'%i for i in range(1, n_max+1)]
                n.extend(f)
                c = [colors[i] for i in range(len(n))]
                nc_dict = dict(zip(n, c)) # given k lst, can get v = c. v = nc_dict[k[k_cnt]]
            else:
                colors = np.vstack((gw, highman_g, b, fuc)) # b & cyans are unlimited cmap determined by n_max
                # construct 1 to 1 color dict
                n = [' N(%s)'%i for i in range(n_max+1) if i!=2]
                highman = [' N(2)H(%s) ↓'%(i) if i==5 else ' N(2)H(%s) ↑'%(i) if i==9 else ' N(2)H(%s)'%(i) for i in range(5, 10, 1)]
                n[2:2] = highman
                # append fucosylation for cyans colormap
                f = ['FN(%s)'%i for i in range(1, n_max+1)]
                n.extend(f)
                c = [colors[i] for i in range(len(n))]
                nc_dict = dict(zip(n, c)) # given k lst, can get v = c. v = nc_dict[k[k_cnt]]

    # make tick & spine thick.
    fig, axe = plt.subplots()
    axes = plt.gca()
    axes.xaxis.set_tick_params(width=tickW, length=tickL)
    axes.yaxis.set_tick_params(width=tickW, length=tickL)
    for axis in ['top','bottom','left','right']:
        axes.spines[axis].set_linewidth(spineW)
        
    # start plotting
    bar_label = [string.ascii_uppercase[i] for i in range(n_df)]
    bar_label_cnt = 0
    for df in dfall : # for each data frame
        if 'norm' in labels[0]: # add edge to the bar
            axe = df.plot(kind='bar', stacked=True, ax=axe, legend=False, grid=False, figsize =(20, 10), edgecolor = 'k')  # make bar plots
        else: # sum: no edge for clarity
            axe = df.plot(kind='bar', stacked=True, ax=axe, legend=False, grid=False, figsize =(20, 10))  # make bar plots
    # get the y aixs range
    ymin, ymax = axes.get_ylim()
    # get the handles we want to modify in each site, h: 44 barcontainers, l: 4 full N(x) cols
    h,l = axe.get_legend_handles_labels() 
    # print('h:%s\n'%h) 44 <BarContainer object of 29 artists> 
    for i in range(0, n_df * n_col, n_col): # len(h) = n_col * n_df -> 0, 11, 22, 33
        k_cnt = 0 # iterate thru k for rect color fix
        for j, pa in enumerate(h[i:i+n_col]): # each pa.patches contains 29 <matplotlib.patches.Rectangle object>
            # print('pa.patches:%s\n'%pa.patches)
            rect_cnt = 0
            for rect in pa.patches: # each rect here is one single N(X) small rect of one single xicauc(int...etc) big rect. Note this loops thru the small rects spanning dif n-sites (iterate thru 29 small rects)
                rect.set_x(rect.get_x() + 1 / float(n_df + 1) * i / float(n_col))    
                rect.set_width(1 / float(n_df + 1))
                axe.text(rect.get_x(), nsum[bar_label_cnt][rect_cnt] + bar_labelpad, bar_label[bar_label_cnt], ha='left', va='bottom')
                rect.set_facecolor(nc_dict[k[k_cnt]]) # use this to fix rect color & legend
                rect_cnt += 1
            k_cnt += 1
        bar_label_cnt += 1
    
    axe.tick_params(axis='both', labelsize=ticklabelsize)
    axe.set_xticks((np.arange(0, 2 * n_ind, 2) + 1 / float(n_df + 1)) / 2.)
    axe.set_xticklabels(df.index, rotation = xlabel_rotation, fontweight="bold") 
    axe.set_xlabel(df.index.name, fontweight="bold", fontsize = xlabelsize, labelpad = xlabelpad)
    axe.ticklabel_format(axis='y')
    axe.yaxis.offsetText.set_fontsize(30)
    axe.yaxis.set_major_formatter(ScalarFormatter(useMathText=True))
    for label in axe.get_yticklabels():
        label.set_fontweight('bold')
    axe.set_title(title)
    
    # legend properties
    params = {'legend.fontsize': legend_fontsize, 'legend.handlelength': legend_handleL, 'hatch.linewidth': hatch_lineW}
    legend_properties = {'weight':'bold'}
    plt.rcParams.update(params)
    
    # Add invisible data to add another legend 
    l1 = axe.legend(h[:n_col], l[:n_col], prop=legend_properties, loc=[1.01, 0.5])
    
    legend_elements = [] # for df legend (xicauc, int.. are from dif. df) 
    if labels is not None:      
        for i in range(n_df):
            label = '%s: %s'%(bar_label[i], labels[i])
            each_bullet = Patch(label = label)
            legend_elements.append(each_bullet)
        plt.legend(handles=legend_elements, handlelength = 0, prop=legend_properties, loc=[1.01, 0.1]) # set handleL=0 to hide Patch
    axe.add_artist(l1)
    
    return fig

def ModifiedGlycanTypeAnalysis(df, glycans_col): # this modified version is only for single sites (from quant)
    N = df[glycans_col].str.findall(r'HexNAc\((\d+)\)')
    N = [[sum(list(map(int, i)))] if type(i) == list else np.nan for i in N]
    df['hexnac'] = N

    H = df[glycans_col].str.findall(r'Hex\((\d+)\)')
    H = [[sum(list(map(int, i)))] if type(i) == list else np.nan for i in H]
    df['hex'] = H

    F = df[glycans_col].str.findall(r'Fuc\((\d+)\)')
    F = [[sum(list(map(int, i)))] if type(i) == list else np.nan for i in F]
    df['fuc'] = F

    NA = df[glycans_col].str.findall(r'NeuAc\((\d+)\)')
    NA = [[sum(list(map(int, i)))] if type(i) == list else np.nan for i in NA]
    df['neuac'] = NA

    NG = df[glycans_col].str.findall(r'NeuGc\((\d+)\)')
    NG = [[sum(list(map(int, i)))] if type(i) == list else np.nan for i in NG]
    df['neugc'] = NG

    df['glycancode'] = df['hexnac'] + df['hex'] + df['fuc'] + df['neuac'] + df['neugc']
    df.loc[(df[glycans_col] == 'Unoccupied'), 'glycancode'] = 'Unoccupied'

    # extract numbers in each glyco composition gp
    df['hexnac'] = [i[0] if type(i) == list else i for i in df['hexnac']]
    df['hex'] = [i[0] if type(i) == list else i for i in df['hex']]
    df['hex-hexnac'] = df['hex']-df['hexnac']
    df['fuc'] = [i[0] if type(i) == list else i for i in df['fuc']]
    df['neuac'] = [i[0] if type(i) == list else i for i in df['neuac']]
    df['neugc'] = [i[0] if type(i) == list else i for i in df['neugc']]

    # HIGHMAN (24000~29000)
    df.loc[(df['hexnac']==2)&(df['hex']<=9)&(df['hex']>=4), 'GlycanTypeAnalysis'] = 'Man'
    df['GlycanTypeAnalysis'] = df['GlycanTypeAnalysis'] + df['hex'].astype('Int64').astype(str)
    # ONLY CORE
    df.loc[(df['hexnac']==2)&(df['hex']==3), 'GlycanTypeAnalysis'] = 'Only core!'
    # THE CORE IS NOT COMPLETE
    df.loc[(df['hexnac']<2)|(df['hex']<3), 'GlycanTypeAnalysis'] = 'The core is not complete!' # any one of them is below 23000 composition could happen
    # PRECURSOR (>29000)
    df.loc[(df['hexnac']==2)&(df['hex']>9), 'GlycanTypeAnalysis'] = 'Potential N-glycan precursor'
    # SINGLE COMPLEX & HYBRID (REMEMBER TO CONSIDER FUCOSE)
    # single complex no f
    df.loc[((df['hexnac']==3)&(df['hex-hexnac']<2)&(df['fuc']==0)), 'GlycanTypeAnalysis'] = 'A1' 
    df.loc[((df['hexnac']==4)&(df['hex-hexnac']<2)&(df['fuc']==0)), 'GlycanTypeAnalysis'] = 'A2/A1B' 
    df.loc[((df['hexnac']==5)&(df['hex-hexnac']<2)&(df['fuc']==0)), 'GlycanTypeAnalysis'] = 'A3/A2B' 
    df.loc[((df['hexnac']>=6)&(df['hex-hexnac']<2)&(df['fuc']==0)), 'GlycanTypeAnalysis'] = 'A4/A3B' 
    # single complex w/ f 
    df.loc[((df['hexnac']==3)&(df['hex-hexnac']<2)&(df['fuc']!=0)), 'GlycanTypeAnalysis'] = 'FA1' 
    df.loc[((df['hexnac']==4)&(df['hex-hexnac']<2)&(df['fuc']!=0)), 'GlycanTypeAnalysis'] = 'FA2/FA1B' 
    df.loc[((df['hexnac']==5)&(df['hex-hexnac']<2)&(df['fuc']!=0)), 'GlycanTypeAnalysis'] = 'FA3/FA2B' 
    df.loc[((df['hexnac']>=6)&(df['hex-hexnac']<2)&(df['fuc']!=0)), 'GlycanTypeAnalysis'] = 'FA4/FA3B' 
    # single hybrid no f
    df.loc[((df['hexnac']>=3)&(df['hex-hexnac']>=2)&(df['fuc']==0)), 'GlycanTypeAnalysis'] = 'Hybrid' 
    # single hybrid w/ f
    df.loc[((df['hexnac']>=3)&(df['hex-hexnac']>=2)&(df['fuc']!=0)), 'GlycanTypeAnalysis'] = 'Fhybrid' 
    # UNOCCUPIED
    df.loc[(df[glycans_col]=='Unoccupied'), 'GlycanTypeAnalysis'] = 'Unoccupied'
    # DROP REDUNDANT COLS
    df = df.drop(['glycancode', 'hexnac', 'hex', 'fuc', 'neuac', 'neugc', 'hex-hexnac'], axis = 1) 
    # MOVE RESULT
    move_df(df, 'GlycanTypeAnalysis', glycans_col)
    # APPLY 4 MAJOR CLASSES (HIGHMAN/HYBRID/COMPLEX/UNOCCUPIED) BASED ON THE DETAILED GLYCAN TYPE ANALYSIS ABOVE
    # delete man4 & below-core structure (just like danny's version of bar/pie charts)
    df = df.loc[(df['GlycanTypeAnalysis']!='Only core!')&(df['GlycanTypeAnalysis']!='The core is not complete!')&(df['GlycanTypeAnalysis']!='Potential N-glycan precursor')&(df['GlycanTypeAnalysis']!='Man4')]
    df.loc[(df['GlycanTypeAnalysis'].str.contains('Man')),'Classes'] = 'HighMannose'
    df.loc[(df['GlycanTypeAnalysis'].str.contains('hybrid', case=False)),'Classes'] = 'Hybrid'
    df.loc[(df['GlycanTypeAnalysis'].str.contains('A|B')),'Classes'] = 'Complex'
    df.loc[(df['GlycanTypeAnalysis'].str.contains('Unoccupied')),'Classes'] = 'Unoccupied'
    return df

def normalizer(df, site_col): 
    # calculate the sum within each n-site, then normalize xicauc/ int/ mono/ iso by these sum
    if "sumpersite_xicauc" in df.columns.tolist() and "sumpersite_int" in df.columns.tolist():
        df['sumpersite_xicauc'] = df.groupby(site_col)['e_sum_XIC\r\nAUC[Byos]'].transform(lambda x: x[x!=-1].sum())
        df['sumpersite_int'] = df.groupby(site_col)['f_sum_Apex Int.\r\n(Posit)[Byos]'].transform(lambda x: x[x!=-1].sum())
        # change the real absent data back to -1 (also make sure divide by 0 won't appen)
        df.loc[df['a_norm_XIC\r\nAUC[Byos]'] == -1, ['sumpersite_xicauc', 'sumpersite_int']] = -1 
        # using groupby transform sum function to add 8 cols (byos xicauc/byos area int/pglycomono/pglycoisotope) recording summed values & normalized values (same site & same glycan & seq can be dif)
        df['a_norm_XIC\r\nAUC[Byos]'] = df['e_sum_XIC\r\nAUC[Byos]']/df['sumpersite_xicauc']
        df['b_norm_Apex Int.\r\n(Posit)[Byos]'] = df['f_sum_Apex Int.\r\n(Posit)[Byos]']/df['sumpersite_int']
        # change the real absent data back to -1
        df.loc[df['sumpersite_xicauc'] == -1, ['a_norm_XIC\r\nAUC[Byos]']] = -1
        df.loc[df['sumpersite_int'] == -1, ['b_norm_Apex Int.\r\n(Posit)[Byos]']] = -1
        # clean off cols
        df = df.drop(['sumpersite_xicauc', 'sumpersite_int'], axis=1)
    if "sumpersite_mono" in df.columns.tolist() and "sumpersite_iso" in df.columns.tolist():
        df['sumpersite_mono'] = df.groupby(site_col)['g_sum_MonoArea[pGlyco]'].transform(lambda x: x[x!=-1].sum())
        df['sumpersite_iso'] = df.groupby(site_col)['h_sum_IsotopeArea[pGlyco]'].transform(lambda x: x[x!=-1].sum())
        # change the real absent data back to -1 (also make sure divide by 0 won't appen)
        df.loc[df['c_norm_MonoArea[pGlyco]'] == -1, ['sumpersite_mono', 'sumpersite_iso']] = -1
        # using groupby transform sum function to add 8 cols (byos xicauc/byos area int/pglycomono/pglycoisotope) recording summed values & normalized values (same site & same glycan & seq can be dif)
        df['c_norm_MonoArea[pGlyco]'] = df['g_sum_MonoArea[pGlyco]']/df['sumpersite_mono']
        df['d_norm_IsotopeArea[pGlyco]'] = df['h_sum_IsotopeArea[pGlyco]']/df['sumpersite_iso']
        # change the real absent data back to -1
        df.loc[df['sumpersite_mono'] == -1, ['c_norm_MonoArea[pGlyco]']] = -1
        df.loc[df['sumpersite_iso'] == -1, ['d_norm_IsotopeArea[pGlyco]']] = -1
        # clean off cols
        df = df.drop(['sumpersite_mono', 'sumpersite_iso'], axis=1)
    # # change the real absent data back to -1 (also make sure divide by 0 won't appen)
    # df.loc[df['c_norm_MonoArea[pGlyco]'] == -1, ['sumpersite_mono', 'sumpersite_iso']] = -1
    # df.loc[df['a_norm_XIC\r\nAUC[Byos]'] == -1, ['sumpersite_xicauc', 'sumpersite_int']] = -1 
    # # using groupby transform sum function to add 8 cols (byos xicauc/byos area int/pglycomono/pglycoisotope) recording summed values & normalized values (same site & same glycan & seq can be dif)
    # df['a_norm_XIC\r\nAUC[Byos]'] = df['e_sum_XIC\r\nAUC[Byos]']/df['sumpersite_xicauc']
    # df['b_norm_Apex Int.\r\n(Posit)[Byos]'] = df['f_sum_Apex Int.\r\n(Posit)[Byos]']/df['sumpersite_int']
    # df['c_norm_MonoArea[pGlyco]'] = df['g_sum_MonoArea[pGlyco]']/df['sumpersite_mono']
    # df['d_norm_IsotopeArea[pGlyco]'] = df['h_sum_IsotopeArea[pGlyco]']/df['sumpersite_iso']
    # # change the real absent data back to -1
    # df.loc[df['sumpersite_xicauc'] == -1, ['a_norm_XIC\r\nAUC[Byos]']] = -1
    # df.loc[df['sumpersite_int'] == -1, ['b_norm_Apex Int.\r\n(Posit)[Byos]']] = -1 
    # df.loc[df['sumpersite_mono'] == -1, ['c_norm_MonoArea[pGlyco]']] = -1
    # df.loc[df['sumpersite_iso'] == -1, ['d_norm_IsotopeArea[pGlyco]']] = -1    
    # clean off cols
    # df = df.drop(['sumpersite_xicauc', 'sumpersite_int', 'sumpersite_mono', 'sumpersite_iso'], axis=1)
    return df

def findMajorClass(df, site_col, glycan_col): # select the major class from highman/hybrid/complex/unoccupied 
    # REMEMBER TO SWITCH TO ISOTOPE AREA IF XICAUC ARE ALL -1 (MISSING DATA)
    # return a df only w/ major class data within each nsite
    if "a_norm_XIC\r\nAUC[Byos]" in df.columns.tolist(): 
        df['xicauc_class_sum'] = df.groupby([site_col, 'Classes'])['a_norm_XIC\r\nAUC[Byos]'].transform(lambda x: x[x!=-1].sum())
        df.loc[(df['xicauc_class_sum']!=0), 'mix_quant_sum'] = df['xicauc_class_sum']
        if "d_norm_IsotopeArea[pGlyco]" in df.columns.tolist(): # both are present. But pglyco can find sites that are not found by byonic
            df['isotope_class_sum'] = df.groupby([site_col, 'Classes'])['d_norm_IsotopeArea[pGlyco]'].transform(lambda x: x[x!=-1].sum())
            df.loc[(df['xicauc_class_sum']==0), 'mix_quant_sum'] = df['isotope_class_sum']
    if "d_norm_IsotopeArea[pGlyco]" in df.columns.tolist() and "a_norm_XIC\r\nAUC[Byos]" not in df.columns.tolist(): # only pglyco 
        df['isotope_class_sum'] = df.groupby([site_col, 'Classes'])['d_norm_IsotopeArea[pGlyco]'].transform(lambda x: x[x!=-1].sum())
        df.loc[(df['isotope_class_sum']!=0), 'mix_quant_sum'] = df['isotope_class_sum']
    # df.loc[(df['xicauc_class_sum']!=0), 'mix_quant_sum'] = df['xicauc_class_sum']
    # df.loc[(df['xicauc_class_sum']==0), 'mix_quant_sum'] = df['isotope_class_sum']
    if "xicauc_class_sum" in df.columns.tolist():
        df = df.drop(['xicauc_class_sum'], axis=1)
    if "isotope_class_sum" in df.columns.tolist():
        df = df.drop(['isotope_class_sum'], axis=1)
    major_class = df.loc[df.groupby([site_col])['mix_quant_sum'].idxmax()]['Classes'].tolist()
    site = sorted(list(set(df[site_col].tolist())))
    
    major_class_cnt = 0
    for each_site in site:
        df.loc[(df[site_col]==each_site)&(df['Classes']==major_class[major_class_cnt]), 'mkr'] = 'pass'
        major_class_cnt += 1

    df = df.loc[(df['mkr']=='pass')].drop(['mkr', 'mix_quant_sum', 'Classes'], axis=1)
    return df

# scantime grouping function
def ScantimeGp(df, gp_by = [], score_col = '', rt_col = '', software = '', purpose = ''):
    # copy df to avoid altering df info
    mock_df = df.copy()
    
    rnd = 1
    # record original cols
    col = mock_df.columns.tolist()
    # start grouping
    # 1st run
    mock_df['to_subtract1'] = mock_df.loc[(mock_df.groupby(gp_by)[score_col].idxmax()), rt_col]
    mock_df['to_subtract_fillna1'] = mock_df.groupby(gp_by)['to_subtract1'].transform(lambda x: x.fillna(x.mean()))
    mock_df['rt_dif1'] = (mock_df[rt_col] - mock_df['to_subtract_fillna1']).abs()
    # if re_dif < 3, then reset the dif to 0
    if software == '[Byonic]' and purpose == 'forUnique':
        mock_df['rt_dif_reset1'] = ['same1' if dif < unique_byonic_rt else dif for dif in mock_df['rt_dif1']]
    elif software == '[pGlyco]' and purpose == 'forUnique':
        mock_df['rt_dif_reset1'] = ['same1' if dif < unique_pglyco_rt else dif for dif in mock_df['rt_dif1']]
    elif software == '[Byonic]' and purpose == 'forNglyco':
        mock_df['rt_dif_reset1'] = ['same1' if dif < nglyco_byonic_rt else dif for dif in mock_df['rt_dif1']]
    elif software == '[pGlyco]' and purpose == 'forNglyco':
        mock_df['rt_dif_reset1'] = ['same1' if dif < nglyco_pglyco_rt else dif for dif in mock_df['rt_dif1']]
    rnd += 1

    while mock_df[mock_df.columns.tolist()[-1]].apply(lambda x: isinstance(x, float)).sum() != 0:
        mock_df['to_subtract%s'%rnd] = mock_df.loc[(mock_df.loc[(mock_df['rt_dif_reset%s'%(rnd-1)]!='same%s'%(rnd-1))&(mock_df['rt_dif_reset%s'%(rnd-1)]!='')].groupby(gp_by)[score_col].idxmax()), rt_col]
        mock_df['to_subtract_fillna%s'%rnd] = mock_df.loc[(mock_df['rt_dif_reset%s'%(rnd-1)]!='same%s'%(rnd-1))&(mock_df['rt_dif_reset%s'%(rnd-1)]!='')].groupby(gp_by)['to_subtract%s'%rnd].transform(lambda x: x.fillna(x.mean()))
        mock_df['rt_dif%s'%rnd] = (mock_df[rt_col] - mock_df['to_subtract_fillna%s'%rnd]).abs()

        if software == '[Byonic]' and purpose == 'forUnique':
            mock_df['rt_dif_reset%s'%rnd] = ['same%s'%rnd if dif < unique_byonic_rt else dif for dif in mock_df['rt_dif%s'%rnd]]
        elif software == '[pGlyco]' and purpose == 'forUnique':
            mock_df['rt_dif_reset%s'%rnd] = ['same%s'%rnd if dif < unique_pglyco_rt else dif for dif in mock_df['rt_dif%s'%rnd]]
        elif software == '[Byonic]' and purpose == 'forNglyco':
            mock_df['rt_dif_reset%s'%rnd] = ['same%s'%rnd if dif < nglyco_byonic_rt else dif for dif in mock_df['rt_dif%s'%rnd]]
        elif software == '[pGlyco]' and purpose == 'forNglyco':
            mock_df['rt_dif_reset%s'%rnd] = ['same%s'%rnd if dif < nglyco_pglyco_rt else dif for dif in mock_df['rt_dif%s'%rnd]]
            
        mock_df['rt_dif_reset%s'%rnd] = mock_df['rt_dif_reset%s'%rnd].fillna('')
        rnd += 1
        
    for i in range(1, rnd):
        mock_df.loc[(mock_df['rt_dif_reset%s'%i].str.contains('same')==True), f'rt_dif_reset_{purpose}{software}'] = mock_df['rt_dif_reset%s'%i]

    mock_df = mock_df.loc[:, [c for c in mock_df.columns if c in col]+[f'rt_dif_reset_{purpose}{software}']]

    return mock_df

# PREPROCESSING FUNCTIONS
def pglyco_preprocessor(pglycofile):
    if pglycofile != '':
        # read in pglyco as df
        pglyco_df = pd.read_excel('%s.xlsx'%pglycofile, header = 0)
        pglyco_df = pglyco_df.fillna('N/A')
        pglyco_df = pglyco_df.sort_values(by=['Scan'])
        pglyco_df = pglyco_df.reset_index(drop = True)

        # output column name
        print('Original pglyco columns:\n%s\n'%pglyco_df.columns)

        # replace _x000D_ w/ \r if exists
        fixed_colname = [i.replace('_x000D_', '\r') if '_x000D_' in i else i for i in pglyco_df.columns]
        pglyco_df.columns = fixed_colname
        print('Fixed pglyco columns:\n%s\n'%pglyco_df.columns)

        # record original data size
        print('Original pglyco data size:\nrow: %s\ncol: %s\n'%(pglyco_df.shape[0], pglyco_df.shape[1]))
        
        # due to some unknown error in pglyco, we need to reject the rows w/ ETDScan before HCD scan number (because etd should happen after hcd).
        if 'ETDScan' in pglyco_df.columns.tolist():
            pglyco_df = pglyco_df.loc[(pglyco_df['ETDScan']>pglyco_df['Scan'])^(pglyco_df['ETDScan']==-1)]
            
        # check if prosites are correct (pglyco might assign wrong nsite due to intrinsic bug)
        if seq != '':
            # potential nsites
            result = findNPosBySequon(seq)
            nsite = [t[0] for t in result]
            # potential osites
            osite = findPotentialOPos(seq)
            # find union of n&osites
            nosites = list(set.union(set(nsite), set(osite)))
            prosite = pglyco_df['ProSites'].tolist()
            site_error = np.setdiff1d(prosite,nosites)
            if len(site_error)!= 0: # print out wrong site & terminate processing for file checking
                sys.exit(f'Wrong N/O site (site: {list(site_error)}) detected! Preprocessing terminated for file checking.\nPredicted N-sites: {nsite};\nPredicted O-sites: {osite}')
            else:
                pass
        else:
            pass

        # change J back to N as column named peptide(J-->N)
        pglyco_df['Peptide'] = pglyco_df['Peptide'].str.replace('J','N')

        # analyze sequon in pglyco file
        pglyco_sequon = pglyco_df['Peptide'].str.findall('(?i)(N[ARNDBCEQZGHILKMFSTWYV]T)|(N[ARNDBCEQZGHILKMFSTWYV]S)').tolist()
        pglyco_sequon_lst = []
        for t in pglyco_sequon:
            t = str(t)
            res = re.findall('[ARNDBCEQZGHILKMFSTWYVP]', t)
            res = ''.join(res)
            res = textwrap.wrap(res, 3)
            if res == []:
                pglyco_sequon_lst.append('N/A')
            elif len(res) == 1:
                pglyco_sequon_lst.append(res[0])
            else:
                pglyco_sequon_lst.append(res)

        pglyco_sequon_lst = [tuple(i) if type(i) == list else i for i in pglyco_sequon_lst]
        pglyco_df.insert(pglyco_df.columns.get_loc('Peptide') + 1 , 'Sequon', pglyco_sequon_lst , True)

        # extract compenents numbers
        pglyco_df['N'] = [0 if i==[] else int(i[0]) for i in pglyco_df['GlycanComposition'].str.findall(r'N\((\d+)\)')]
        pglyco_df['H'] = [0 if i==[] else int(i[0]) for i in pglyco_df['GlycanComposition'].str.findall(r'H\((\d+)\)')]
        pglyco_df['F'] = [0 if i==[] else int(i[0]) for i in pglyco_df['GlycanComposition'].str.findall(r'F\((\d+)\)')]
        pglyco_df['X'] = [0 if i==[] else int(i[0]) for i in pglyco_df['GlycanComposition'].str.findall(r'X\((\d+)\)')]
        pglyco_df['A'] = [0 if i==[] else int(i[0]) for i in pglyco_df['GlycanComposition'].str.findall(r'A\((\d+)\)')]
        pglyco_df['G'] = [0 if i==[] else int(i[0]) for i in pglyco_df['GlycanComposition'].str.findall(r'G\((\d+)\)')]
        pglyco_df['HA'] = [0 if i==[] else int(i[0]) for i in pglyco_df['GlycanComposition'].str.findall(r'HA\((\d+)\)')]
        pglyco_df['HS'] = [0 if i==[] else int(i[0]) for i in pglyco_df['GlycanComposition'].str.findall(r'HS\((\d+)\)')]
        pglyco_df['MN'] = [0 if i==[] else int(i[0]) for i in pglyco_df['GlycanComposition'].str.findall(r'MN\((\d+)\)')]
        pglyco_df['KDN'] = [0 if i==[] else int(i[0]) for i in pglyco_df['GlycanComposition'].str.findall(r'KDN\((\d+)\)')]
        pglyco_df['pH'] = [0 if i==[] else int(i[0]) for i in pglyco_df['GlycanComposition'].str.findall(r'pH\((\d+)\)')]
        pglyco_df['aH'] = [0 if i==[] else int(i[0]) for i in pglyco_df['GlycanComposition'].str.findall(r'aH\((\d+)\)')]
        pglyco_df['PG'] = [0 if i==[] else int(i[0]) for i in pglyco_df['GlycanComposition'].str.findall(r'PG\((\d+)\)')]
        pglyco_df['sH'] = [0 if i==[] else int(i[0]) for i in pglyco_df['GlycanComposition'].str.findall(r'sH\((\d+)\)')]
        pglyco_df['sN'] = [0 if i==[] else int(i[0]) for i in pglyco_df['GlycanComposition'].str.findall(r'sN\((\d+)\)')]

        # add back the modified glycans
        pglyco_df['H'] += pglyco_df['aH']
        pglyco_df['H'] += pglyco_df['sH']
        pglyco_df['H'] += pglyco_df['pH']
        pglyco_df['N'] += pglyco_df['sN']
        pglyco_df['s(H+N)'] = pglyco_df['sH'] + pglyco_df['sN']

        # convert to final symbols (str)
        pglyco_df['N'] = ['' if i==0 else 'HexNAc(%s)'%(i) for i in pglyco_df['N']]
        pglyco_df['H'] = ['' if i==0 else 'Hex(%s)'%(i) for i in pglyco_df['H']]
        pglyco_df['s(H+N)'] = ['' if i==0 else 'SO3(%s)'%(i) for i in pglyco_df['s(H+N)']]
        pglyco_df['aH'] = ['' if i==0 else ' %s'%(np.round(17.02654007*i, ModsRoundTo)) for i in pglyco_df['aH']]
        pglyco_df['pH'] = ['' if i==0 else 'Phospho(%s)'%(i) for i in pglyco_df['pH']]
        pglyco_df['F'] = ['' if i==0 else 'Fuc(%s)'%(i) for i in pglyco_df['F']]
        pglyco_df['A'] = ['' if i==0 else 'NeuAc(%s)'%(i) for i in pglyco_df['A']]
        pglyco_df['G'] = ['' if i==0 else 'NeuGc(%s)'%(i) for i in pglyco_df['G']]
        pglyco_df['X'] = ['' if i==0 else 'Pent(%s)'%(i) for i in pglyco_df['X']]
        pglyco_df['HA'] = ['' if i==0 else ' %s'%(np.round(176.032088*i, ModsRoundTo)) for i in pglyco_df['HA']]
        pglyco_df['HS'] = ['' if i==0 else ' %s'%(np.round(161.0688002*i, ModsRoundTo)) for i in pglyco_df['HS']]
        pglyco_df['MN'] = ['' if i==0 else ' %s'%(np.round(275.1004901*i, ModsRoundTo)) for i in pglyco_df['MN']]
        pglyco_df['KDN'] = ['' if i==0 else 'KDN(%s)'%(i) for i in pglyco_df['KDN']]
        pglyco_df['PG'] = ['' if i==0 else ' %s'%((299.12296*i, ModsRoundTo)) for i in pglyco_df['PG']]

        # combine mods to mimic byonic format
        pglyco_df['GlycanComposition_ByonicStyle'] = pglyco_df['N']+pglyco_df['H']+pglyco_df['F']+pglyco_df['X']+pglyco_df['A']+pglyco_df['G']\
                                                +pglyco_df['s(H+N)']+pglyco_df['pH']+pglyco_df['KDN']+pglyco_df['aH']+pglyco_df['HA']+pglyco_df['HS']+pglyco_df['MN']+pglyco_df['PG']

        pglyco_df = pglyco_df.drop(['N','H','F','X','A','G','HA','HS','MN','KDN','pH','aH','PG','sH','sN','s(H+N)'], axis=1)
        move_df(pglyco_df, 'GlycanComposition_ByonicStyle', 'GlycanComposition')

        # if the etdscan is not -1, duplicate the row and change the duplicated scan to etdscan (micmic byonic format)
        pglyco_df.insert(pglyco_df.columns.get_loc('Scan') + 1 , 'FragmentType', 'hcd' , True) # insert 'fragment type' col
        if 'ETDScan' in pglyco_df.columns.tolist():
            row_to_duplicate = pglyco_df[pglyco_df['ETDScan'] != -1].copy() # missing ETDScan in pglyco is represented as -1
            row_to_duplicate['FragmentType'] = 'ethcd'
            row_to_duplicate['Scan'] = row_to_duplicate['ETDScan']
            pglyco_df = pd.concat([pglyco_df, row_to_duplicate]) # duplicate w/ index
        else:
            pass

        # rounding precursor mz col to the desired decimal
        pglyco_df['rounded_PrecursorMZ'] = pglyco_df.round({'PrecursorMZ': RoundTo})['PrecursorMZ']
        move_df(pglyco_df, 'rounded_PrecursorMZ', 'PrecursorMZ')

        pglyco_df = pglyco_df.sort_values(by = ['Scan']) # sort by scan directly, the duplicated ethcd rows can be separated from hcd rows
        print('----- pGlyco data preprocessing completed. -----\n')
        return pglyco_df

def byonic_preprocessor(byonicfile):
    if byonicfile != '':
        byonic_df = pd.read_excel('%s.xlsx'%byonicfile, header = 0)
        byonic_df = byonic_df.fillna('N/A')
        print('Original byonic columns:\n%s\n'%byonic_df.columns)
        # replace _x000D_ w/ \r if exists
        fixed_colname = [i.replace('_x000D_', '\r') if '_x000D_' in i else i for i in byonic_df.columns]
        byonic_df.columns = fixed_colname
        print('Fixed byonic columns:\n%s\n'%byonic_df.columns)
        # record original data size
        print('Original byonic data size:\nrow: %s\ncol: %s\n'%(byonic_df.shape[0], byonic_df.shape[1]))
        # extract 'scan' from 'scan #' in byonic file & add a 'Scan' column
        byonic_scan = byonic_df['Scan #'].tolist()
        byonic_scan_lst = []
        for scan in byonic_scan:
            scan = scan.split(' ')[-1].split('=')[-1]
            scan = int(scan)
            byonic_scan_lst.append(scan)
        byonic_df.insert(byonic_df.columns.get_loc('Scan\r\nTime') + 1 , 'Scan', byonic_scan_lst , True)
        byonic_df = byonic_df.sort_values(by = ['Scan'])
        byonic_df = byonic_df.reset_index(drop = True)
        # add 'PureSequence' column to byonic file
        if 'Sequence\r\n(unformatted)' in byonic_df.columns: # deal w/ dif byonic version
            byonic_seq = byonic_df['Sequence\r\n(unformatted)'].str[2:-2].str.findall('(?i)[-ARNDBCEQZGHILKMFSTWYVP]').tolist()
            
            new_byonic_seq = []
            for lst in byonic_seq:
                minus_pos = [m.start(0) for m in re.finditer('[-]', ''.join(lst))]
                delete_minus_pos = [i for i in minus_pos if i!=0]
                lst = [i for j, i in enumerate(lst) if j not in delete_minus_pos]
                new_byonic_seq.append(lst)
            
            byonic_seq = [''.join(each_pure) for each_pure in new_byonic_seq]
            byonic_df.insert(byonic_df.columns.get_loc('Sequence\r\n(unformatted)') + 1 , 'PureSequence', byonic_seq , True)
            
            byonic_seq_forSequonNsite = byonic_df['Sequence\r\n(unformatted)'].str.findall('(?i)[-ARNDBCEQZGHILKMFSTWYVP]').tolist()
            
            new_byonic_seq_forSequonNsite = []
            for lst in byonic_seq_forSequonNsite:
                minus_pos = [m.start(0) for m in re.finditer('[-]', ''.join(lst))]
                delete_minus_pos = [i for i in minus_pos if i!=0]
                lst = [i for j, i in enumerate(lst) if j not in delete_minus_pos]
                new_byonic_seq_forSequonNsite.append(lst)
                
            byonic_seq_forSequonNsite = [''.join(each_pure) for each_pure in new_byonic_seq_forSequonNsite]
            byonic_seq_forSequonNsite = pd.Series(byonic_seq_forSequonNsite) # preserve aa outside dots
        elif 'Sequence' in byonic_df.columns:
            byonic_seq = byonic_df['Sequence'].str[2:-2].str.findall('(?i)[-ARNDBCEQZGHILKMFSTWYVP]').tolist()
            
            new_byonic_seq = []
            for lst in byonic_seq:
                minus_pos = [m.start(0) for m in re.finditer('[-]', ''.join(lst))]
                delete_minus_pos = [i for i in minus_pos if i!=0]
                lst = [i for j, i in enumerate(lst) if j not in delete_minus_pos]
                new_byonic_seq.append(lst)
            
            byonic_seq = [''.join(each_pure) for each_pure in new_byonic_seq]
            byonic_df.insert(byonic_df.columns.get_loc('Sequence') + 1 , 'PureSequence', byonic_seq , True)
            
            byonic_seq_forSequonNsite = byonic_df['Sequence'].str.findall('(?i)[-ARNDBCEQZGHILKMFSTWYVP]').tolist()
            
            new_byonic_seq_forSequonNsite = []
            for lst in byonic_seq_forSequonNsite:
                minus_pos = [m.start(0) for m in re.finditer('[-]', ''.join(lst))]
                delete_minus_pos = [i for i in minus_pos if i!=0]
                lst = [i for j, i in enumerate(lst) if j not in delete_minus_pos]
                new_byonic_seq_forSequonNsite.append(lst)
                
            byonic_seq_forSequonNsite = [''.join(each_pure) for each_pure in new_byonic_seq_forSequonNsite]
            byonic_seq_forSequonNsite = pd.Series(byonic_seq_forSequonNsite)
        else:
            sys.exit('Please check if sequence column name has changed. This app will stop.') 
        # # CHECK IF N GLYCAN IS SEARCHED   
        # if sum(byonic_df['Mods\r\n(variable)'].str.contains('NGlycan')==True) > 0:   
        # add 'Sequon' & 'N-site' column to byonic file
        byonic_sequon_lst = []
        n_site_lst = []
        pureseq_cnt = 0 # this is byonic_seq_forSequonNsite
        pos = byonic_df['Pos.'].tolist()
        pos_cnt = 0
        byonic_sequon = byonic_seq_forSequonNsite.str.findall('(?=((?i)(N[ARNDBCEQZGHILKMFSTWYV]T)|(N[ARNDBCEQZGHILKMFSTWYV]S)))').tolist()
        for t in byonic_sequon:
            t = [tuple(set(i)) for i in t]
            t = str(t)
            res = re.findall('(?i)[ARNDBCEQZGHILKMFSTWYVP]', t)
            res = ''.join(res)
            res = textwrap.wrap(res, 3)
            if res == []:
                byonic_sequon_lst.append('N/A')
                n_site_lst.append('N/A')
                pureseq_cnt += 1
                pos_cnt += 1
            elif len(res) == 1: # single sequon -> single n-site
                byonic_sequon_lst.append(res[0])
                regex = re.compile(r'(' + '|'.join(res) + r')')
                each_n_site_pos = [m.start() for m in regex.finditer(byonic_seq_forSequonNsite[pureseq_cnt])][0]
                each_n_site = pos[pos_cnt] -1 + each_n_site_pos # -1 to fix the 'dots problem'
                n_site_lst.append(each_n_site)
                pureseq_cnt += 1
                pos_cnt += 1
            else: # non-single sequon
                byonic_sequon_lst.append(res)
                regex = re.compile(r'(' + '|'.join(res) + r')')
                each_n_site_pos = [m.start() for m in regex.finditer(byonic_seq_forSequonNsite[pureseq_cnt], overlapped=True)]
                each_n_site = list(pos[pos_cnt] -1 + np.array(each_n_site_pos))
                n_site_lst.append(each_n_site)
                pureseq_cnt += 1
                pos_cnt += 1
        # convert lists within list to tuple for later usage ('cos lists are unhashable, which may cause some problems later)
        byonic_sequon_lst = [tuple(i) if type(i) == list else i for i in byonic_sequon_lst]
        n_site_lst = [tuple(i) if type(i) == list else i for i in n_site_lst]
        byonic_df.insert(byonic_df.columns.get_loc('PureSequence') + 1 , 'Sequon', byonic_sequon_lst , True)
        byonic_df.insert(0, 'N-site(SequonBased)', n_site_lst , True)
        # ONLY FOR HCD//ETD FILES: add N/A col 'Pair' for later modification (pair: cal.m/z same, cal.m same, pureseq same, scan difference <= 5, hcd before ethcd)
        if 'ethcd' in byonic_df['Fragment\r\nType'].tolist():
            byonic_df.insert(byonic_df.columns.get_loc('Scan\r\nTime') + 1 , 'Pair', 'N/A' , True)
            potential_pair = byonic_df[byonic_df.duplicated(subset=['Calc.\r\nm/z', 'Calc.\r\nMH', 'PureSequence'], keep=False)].sort_values(['Calc.\r\nm/z'])
            # print(potential_pair)
            mz_gp = sorted(list(set(potential_pair['Calc.\r\nm/z'].tolist())))
            pair_cnt = 0
            all_hcd_ind = []
            all_etd_ind = []
            for i in mz_gp:
                gp = potential_pair[potential_pair['Calc.\r\nm/z'] == i][potential_pair.duplicated(subset=['Calc.\r\nMH', 'PureSequence'], keep=False)]
                gp = gp.sort_values(['Scan'])
                # skip the gp w/o ethcd & find pairs: last criterion -> scan dif <= 5
                if 'hcd' in gp['Fragment\r\nType'].tolist() and 'ethcd' in gp['Fragment\r\nType'].tolist():
                    # from hcd find the nearest ethcd below
                    pair_candidate = gp[(gp['Fragment\r\nType'] == 'hcd') & (gp['Fragment\r\nType'].shift(-1) == 'ethcd') & (gp['Scan'].shift(-1) - gp['Scan'] <= byonic_scandif)]
                    hcd_iloc = [gp.index.get_loc(ind) for ind in pair_candidate.index] # get the positions of the hcd in pairs in gp df
                    etd_iloc = list(np.array(hcd_iloc) + 1)
                    hcd_ind = pair_candidate.index.tolist()
                    etd_ind = [gp.index[i] for i in etd_iloc] 
                    all_hcd_ind.extend(hcd_ind)
                    all_etd_ind.extend(etd_ind)
                else:
                    pass
            all_hcd_ind.sort()
            all_etd_ind.sort()
            byonic_df['Pair'].iloc[all_hcd_ind] = ['pair%s'%(i+1) for i in range(len(all_hcd_ind))]
            byonic_df['Pair'].iloc[all_etd_ind] = ['pair%s'%(i+1) for i in range(len(all_etd_ind))]
        else:
            pass 

        if byonic_df['Glycans'].str.contains(r'\d+\.\d+').sum() > 0:

            # record the starting cols
            ori_col = byonic_df.columns.tolist()

            byonic_df['GlycansModNum'] = byonic_df['Glycans'].str.replace('; ', ' ')
            byonic_df['GlycansModNum'] = ['N/A' if i==[] else np.array([float(j) for j in i]) if len(i)>1 else float(i[0]) for i in byonic_df['GlycansModNum'].str.findall(r'\d+\.\d+')]
            
            if mods_name != [] and mods_num != []:
                # start finding the possible mods
                for num in mods_num:
                    byonic_df['/%s'%(get_key(num))] = byonic_df['GlycansModNum'].apply(lambda x: list(x/num) if type(x)==np.ndarray else x/num if type(x)==float else None)
                    byonic_df['/%s'%(get_key(num))] = byonic_df['/%s'%(get_key(num))].apply(lambda x: '' if x is not None and type(x)==float and x.is_integer()==False else '%s(%s)'%(get_key(num), int(x)) if x is not None and type(x)==float and x.is_integer()==True else np.char.array(['' if j.is_integer()==False else '%s(%s)'%(get_key(num), int(j)) for j in x]) if x is not None and type(x)==list else '')

                byonic_df['final'] = byonic_df[['/%s'%(name) for name in mods_name]].sum(axis=1)
            else:
                byonic_df['final'] = ''

            byonic_df['PureGlycans'] = byonic_df['Glycans'].str.replace(r' \d+\.\d+', '', regex=True).to_frame()
            byonic_df['PureGlycans'] = [i[0] if len(i)==1 else np.char.array(i) for i in byonic_df['PureGlycans'].str.split(';')]

            byonic_df['FinalGlycans'] = byonic_df['PureGlycans'] + byonic_df['final']
            byonic_df['FinalGlycans'] = [';'.join(list(i)) if type(i)==np.chararray else i for i in byonic_df['FinalGlycans']]

            # retrieve the mods numbers in glycans if needed
            byonic_df['GlycansModNum'] = byonic_df.round({'GlycansModNum': ModsRoundTo})['GlycansModNum'] # round mods numbers for later comparison
            byonic_df.loc[(byonic_df['Glycans'].str.contains(r'\d+\.\d+')==True)&(byonic_df['final'].apply(lambda x: len(x)==0 or all(v=='' for v in x))), 'FinalGlycans'] = byonic_df['Glycans']

            # round all the remaining mods numbers to 4 decimals in the final glycans col
            byonic_df['rounded_GlycansModNum(1)'] = ['' if i==[] else ' '+ np.char.array((np.round(np.array([float(v) for v in i]), ModsRoundTo)).astype(str)) for i in byonic_df['FinalGlycans'].str.findall(r' \d+\.\d+')]
            byonic_df.loc[(byonic_df['rounded_GlycansModNum(1)'].apply(lambda x: len(x)!=0)), 'rounded_GlycansModNum(2)'] = byonic_df['PureGlycans'] + byonic_df['rounded_GlycansModNum(1)']
            byonic_df.loc[(byonic_df['rounded_GlycansModNum(2)'].isnull()), 'rounded_FinalGlycans'] = byonic_df['FinalGlycans']
            byonic_df.loc[~(byonic_df['rounded_GlycansModNum(2)'].isnull()), 'rounded_FinalGlycans'] = byonic_df['rounded_GlycansModNum(2)']
            byonic_df['rounded_FinalGlycans'] = ['; '.join(i) if type(i)==np.chararray else i for i in byonic_df['rounded_FinalGlycans']]

            # preserve only FinalGlycans & rounded_FinalGlycans
            byonic_df = byonic_df[ori_col + ['FinalGlycans', 'rounded_FinalGlycans']] 

            move_df(byonic_df, 'FinalGlycans', 'Glycans')
            move_df(byonic_df, 'rounded_FinalGlycans', 'FinalGlycans')
        else:
            byonic_df['rounded_FinalGlycans'] = byonic_df['Glycans']
            move_df(byonic_df, 'rounded_FinalGlycans', 'Glycans')
            
        print('----- Byonic data preprocessing completed. -----\n')
        return byonic_df

def byos_preprocessor(byosfile):
    if byosfile != '':
        byos_df = pd.read_excel('%s.xlsx'%byosfile, header = 0)
        # output column name
        print('Original byos columns:\n%s\n'%byos_df.columns)
        # replace _x000D_ w/ \r if exists
        fixed_colname = [i.replace('_x000D_', '\r') if '_x000D_' in i else i for i in byos_df.columns]
        byos_df.columns = fixed_colname
        print('Fixed byos columns:\n%s\n'%byos_df.columns)
        # add calc.MH col to the right side of byos calc.M col
        byos_df['Calc.MH'] = byos_df['Calc.M'] + 1.0073
        # extract needed columns: Scan Number(s)(Posit), MS Alias name, XIC area summed, XIC AUC, Apex Int.(Posit), Calc.M, Sequence
        byos_df = byos_df[['Scan Number(s)\r\n(Posit)', 'Glycans', 'MS2 Search\r\nAlias name', 'XIC area\r\nsummed', 'XIC\r\nAUC', 'Apex Int.\r\n(Posit)', 'Calc.M', 'Calc.MH', 'Sequence']]
        byos_df = byos_df.fillna('N/A')
        # output extracted column name
        print('Extracted byos columns:\n%s\n'%byos_df.columns)
        # drop the row w/ multiple scan numbers (data type would be str)
        byos_df = byos_df[byos_df['Scan Number(s)\r\n(Posit)'].apply(lambda x: isinstance(x, int))]
        # sort byos by scan
        byos_df = byos_df.sort_values(by=['Scan Number(s)\r\n(Posit)'])
        byos_df = byos_df.reset_index(drop = True)
        # add 'PureSequence' col
        byos_df['Sequence'] = byos_df['Sequence'].str.upper()
        byos_df['PureSequence'] = byos_df['Sequence'].str[2:-2]
        # display(HTML(byos_df.to_html()))
        print('----- Byos data preprocessing completed. -----\n')
        return byos_df

# MAIN PROCESSING FUNC
def main():
    # TIMESTAMP
    start_time = time.time()
    # CURRENT DATE
    date = datetime.now().strftime('%Y%m%d')
    # PREPARE EXPORT FILENAME
    filename = export_filename(byonicfile, byosfile, pglycofile)
    # FILE PREPROCESSING
    byonic_df = byonic_preprocessor(byonicfile)
    byos_df =  byos_preprocessor(byosfile)
    pglyco_df = pglyco_preprocessor(pglycofile)
    # INPUT COMBO CONTROL FLOW
    if byonicfile != '' and byosfile != '' and pglycofile != '': # bbp all present
        print('----- Start combining Byonic & Byos & pGlyco. -----\n')
        # combined data based on 'Scan'
        byonic_scanasid = byonic_df.copy()
        new_byonic_col = [n + '[Byonic]' if n != 'Scan' else n for n in byonic_scanasid.columns]
        byonic_scanasid.columns = new_byonic_col
        pglyco_scanasid = pglyco_df.copy()
        new_pglyco_col = [n + '[pGlyco]' if n != 'Scan' else n for n in pglyco_scanasid.columns]
        pglyco_scanasid.columns = new_pglyco_col
        byos_scanasid = byos_df.copy()
        new_byos_col = [n + '[Byos]' if n != 'Scan Number(s)\r\n(Posit)' else n for n in byos_scanasid.columns]
        byos_scanasid.columns = new_byos_col
        byonic_scanasid = byonic_scanasid.set_index('Scan')
        pglyco_scanasid = pglyco_scanasid.set_index('Scan')
        byos_scanasid = byos_scanasid.set_index('Scan Number(s)\r\n(Posit)')
        # align scan & concat (all align on row to make row number all the same)
        a1, a2 = byonic_scanasid.align(pglyco_scanasid, join = 'outer', axis = 0) # row: a1 = a2
        a1, a3 = a1.align(byos_scanasid, join = 'outer', axis = 0) # row: a1 = a2 = a3
        a2, a3 = a2.align(byos_scanasid, join = 'outer', axis = 0) # row: a2 = a1 = a3
        all_combined_df = pd.concat([a1,a3,a2], axis = 1)
        all_combined_df.index.name = 'Scan'
        all_combined_df.reset_index(level=0, inplace=True)
        # change all nan to blank -1
        all_combined_df = all_combined_df.fillna(-1)
        move_df(all_combined_df, 'ProSites[pGlyco]', 'N-site(SequonBased)[Byonic]')
        ## result post-processing 
        glycansource(all_combined_df, 'Scan')
        print('\nCombined data shape:\nrow --> %s, column --> %s'%(all_combined_df.shape[0], all_combined_df.shape[1]))
        ## style apply for excel export
        # color the rows below the threshold (threshold [byonic: score > BYONIC_SCORE_CUTOFF & pep2d < 0.001; pglyco: PepScore>5 & GlyScore>4])
        # color code => #ffedcc -> light orange for byonic; #add8e6 -> light blue for pglyco; #FFB6C1 -> light pink for byos w/ dif calm ^ seq from byonic; #FF1493 -> deep pink for byos w/ dif calm & seq from byonic; #FFFF00 -> yellow for byos & byonic all the same
        threshold_masks_colorind(all_combined_df)
        byonic_colored_id = list(set(lightgreen_ind + normalgreen_ind + lightorange_ind + normalorange_ind + deepgreen_ind)) # byonic pass threshold (include below-threshold ethcd w/o pair) 
        byonic_colored_id_for_pairfile = byonic_colored_id.copy()
        pglyco_colored_id = list(set(lightblue_ind + normalblue_ind + lightorange_ind + normalorange_ind + deepblue_ind)) # pglyco pass threshold
        pglyco_belowthreshold_id = [i for i in all_combined_df.index.tolist() if i not in pglyco_colored_id]
        ## analyze PSM (need to pass threshold)
        # need to record the ind of below-threshold ethcd w/o pair in advance (will be excluded later, so we should not count PSM as well)
        # so here we are forced to find pair in advance (since we are forced to list psm in all file)
        if 'Pair[Byonic]' in all_combined_df.columns.tolist(): # pair is to identify etd with pass-threshold hcd pair for later rescue
            # ONLY FOR HCD/ETD FILES: prepare '_Pair' file (as long as byonic passes threshold & have pair)
            all_combined_df = all_combined_df.astype({'Pair[Byonic]': 'str'})
            pair_df = all_combined_df.loc[byonic_colored_id]
            pair_df = pair_df[pair_df['Pair[Byonic]'].str.contains('pair')]
#             pair_df = pair_df[pair_df.duplicated(subset=['Pair[Byonic]'], keep=False)] # only paired data included in _Pair
            if 'ethcd' in all_combined_df['Fragment\r\nType[Byonic]'].tolist(): 
                hcd_pair_num = set(pair_df.loc[(pair_df['Fragment\r\nType[Byonic]']=='hcd'), 'Pair[Byonic]'].tolist())
                pair_df.drop(pair_df[(pair_df['Fragment\r\nType[Byonic]']=='ethcd')&(~(pair_df['Pair[Byonic]'].isin(hcd_pair_num)))].index, inplace=True)
        else:
            pass

        if 'ethcd' in all_combined_df['Fragment\r\nType[Byonic]'].tolist(): # if etd data is present (i.e. have pair)
            byonic_excluded_ethcd_ind =  all_combined_df.loc[((all_combined_df['Fragment\r\nType[Byonic]'] == 'ethcd')&((all_combined_df['Score[Byonic]'] <= BYONIC_SCORE_CUTOFF)|(all_combined_df['PEP\r\n2D[Byonic]'].abs() >= 0.001))&(~(all_combined_df['Pair[Byonic]'].isin(pair_df['Pair[Byonic]']))))].index.tolist()
            byonic_pass_id = [i for i in byonic_colored_id if i not in byonic_excluded_ethcd_ind]
            byonic_belowthreshold_id = [i for i in all_combined_df.index.tolist() if i not in byonic_pass_id] 
        else: # all hcd, all in (cause here all the hcd are above threshold, i.e. all colored)
            byonic_pass_id = byonic_colored_id 
            byonic_belowthreshold_id = [i for i in all_combined_df.index.tolist() if i not in byonic_pass_id] 

        # using groupby size function to count psm & add psm columns (new criterion: add fragment type, RT/scantime dif < 3 min (180s). is regarded as the same, >= 3 min. should be seen as dif.)
        # byonic & byos
        all_combined_df = all_combined_df.astype({'N-site(SequonBased)[Byonic]': 'str'}) # convert list & tuple to str for later groupby function
        b_df_pass = all_combined_df.loc[byonic_pass_id]
        b_df_pass_forUnique = ScantimeGp(b_df_pass, gp_by = ['Fragment\r\nType[Byonic]', 'N-site(SequonBased)[Byonic]',  'Calc.\r\nm/z[Byonic]','rounded_FinalGlycans[Byonic]', 'PureSequence[Byonic]'], score_col = 'Score[Byonic]', rt_col = 'Scan\r\nTime[Byonic]', software = '[Byonic]', purpose = 'forUnique')
        b_df_pass_forNglyco = ScantimeGp(b_df_pass, gp_by = ['N-site(SequonBased)[Byonic]',  'Calc.\r\nm/z[Byonic]','rounded_FinalGlycans[Byonic]', 'PureSequence[Byonic]'], score_col = 'Score[Byonic]', rt_col = 'Scan\r\nTime[Byonic]', software = '[Byonic]', purpose = 'forNglyco')
        byonic_psm = b_df_pass_forNglyco.groupby(['N-site(SequonBased)[Byonic]', 'Calc.\r\nm/z[Byonic]','rounded_FinalGlycans[Byonic]', 'PureSequence[Byonic]', 'rt_dif_reset_forNglyco[Byonic]'])['N-site(SequonBased)[Byonic]'].transform('size')
        
        all_combined_df.loc[(byonic_pass_id), 'PSM[Byonic]'] = byonic_psm
        move_df(all_combined_df, 'PSM[Byonic]', 'N-site(SequonBased)[Byonic]')
        all_combined_df.loc[(byonic_pass_id), 'rt_dif_reset_forUnique[Byonic]'] = b_df_pass_forUnique['rt_dif_reset_forUnique[Byonic]']
        all_combined_df.loc[(byonic_pass_id), 'rt_dif_reset_forNglyco[Byonic]'] = b_df_pass_forNglyco['rt_dif_reset_forNglyco[Byonic]']
        all_combined_df.loc[(all_combined_df['N-site(SequonBased)[Byonic]'] == 'N/A'), 'PSM[Byonic]'] = 'N/A' # if n-site is 'N/A', do not count psm
        all_combined_df.loc[byonic_belowthreshold_id, 'PSM[Byonic]'] = 'N/A' # do not count psm for rows below threshold
        all_combined_df.loc[(all_combined_df['N-site(SequonBased)[Byonic]'] == '-1'), 'PSM[Byonic]'] = -1 # if site is '-1', then psm set to -1
        # pglyco
        all_combined_df = all_combined_df.astype({'ProSites[pGlyco]': 'int'})
        all_combined_df = all_combined_df.astype({'ProSites[pGlyco]': 'str'}) # convert list & tuple to str for later groupby function
        p_df_pass = all_combined_df.loc[pglyco_colored_id]
        p_df_pass_forUnique = ScantimeGp(p_df_pass, gp_by = ['FragmentType[pGlyco]', 'ProSites[pGlyco]', 'rounded_PrecursorMZ[pGlyco]','GlycanComposition_ByonicStyle[pGlyco]', 'Peptide[pGlyco]'], score_col = 'PepScore[pGlyco]', rt_col = 'RT[pGlyco]', software = '[pGlyco]', purpose = 'forUnique')
        p_df_pass_forNglyco = ScantimeGp(p_df_pass, gp_by = ['ProSites[pGlyco]', 'rounded_PrecursorMZ[pGlyco]','GlycanComposition_ByonicStyle[pGlyco]', 'Peptide[pGlyco]'], score_col = 'PepScore[pGlyco]', rt_col = 'RT[pGlyco]', software = '[pGlyco]', purpose = 'forNglyco')
        
        pglyco_psm = p_df_pass_forNglyco.groupby(['ProSites[pGlyco]', 'rounded_PrecursorMZ[pGlyco]','GlycanComposition_ByonicStyle[pGlyco]', 'Peptide[pGlyco]', 'rt_dif_reset_forNglyco[pGlyco]'])['ProSites[pGlyco]'].transform('size')
        all_combined_df.loc[(pglyco_colored_id), 'PSM[pGlyco]'] = pglyco_psm
        move_df(all_combined_df, 'PSM[pGlyco]', 'ProSites[pGlyco]')
        all_combined_df.loc[(pglyco_colored_id), 'rt_dif_reset_forUnique[pGlyco]'] = p_df_pass_forUnique['rt_dif_reset_forUnique[pGlyco]']
        all_combined_df.loc[(pglyco_colored_id), 'rt_dif_reset_forNglyco[pGlyco]'] = p_df_pass_forNglyco['rt_dif_reset_forNglyco[pGlyco]']
        all_combined_df.loc[(all_combined_df['ProSites[pGlyco]'] == 'N/A'), 'PSM[pGlyco]'] = 'N/A' # if n-site is 'N/A', do not count psm
        all_combined_df.loc[pglyco_belowthreshold_id, 'PSM[pGlyco]'] = 'N/A' # do not count psm for rows below threshold
        all_combined_df.loc[(all_combined_df['ProSites[pGlyco]'] == '-1'), 'PSM[pGlyco]'] = -1 # if site is '-1', then psm set to -1
        all_forsimple_df = all_combined_df.copy() # to retain rt_dif_reset for simple df groupy and get unique row
        all_combined_df = all_combined_df.drop(['rt_dif_reset_forUnique[Byonic]', 'rt_dif_reset_forNglyco[Byonic]', 'rt_dif_reset_forUnique[pGlyco]', 'rt_dif_reset_forNglyco[pGlyco]'], axis = 1)
        print('\n----- Exporting "_All" file... This may take some time, please wait. -----\n')
        all_combined_df.style.apply(bg_color, axis=None).to_excel(f'{date}_All_{filename}.xlsx', index = False)  
        print('\n----- "_All" file exported. -----\n')
        
        if 'Pair[Byonic]' in all_combined_df.columns.tolist(): # pair is to identify etd with pass-threshold hcd pair for later rescue
            # ONLY FOR HCD/ETD FILES: prepare '_Pair' file (as long as byonic passes threshold & have pair)
            all_combined_df = all_combined_df.astype({'Pair[Byonic]': 'str'})
            pair_df = all_combined_df.loc[byonic_colored_id_for_pairfile]
            pair_df = pair_df[pair_df['Pair[Byonic]'].str.contains('pair')]
#             pair_df = pair_df[pair_df.duplicated(subset=['Pair[Byonic]'], keep=False)] # only paired data included in _Pair
            hcd_pair_num = set(pair_df.loc[(pair_df['Fragment\r\nType[Byonic]']=='hcd'), 'Pair[Byonic]'].tolist())
            pair_df.drop(pair_df[(pair_df['Fragment\r\nType[Byonic]']=='ethcd')&(~(pair_df['Pair[Byonic]'].isin(hcd_pair_num)))].index, inplace=True)
            threshold_masks_colorind(pair_df)
            print('\n----- Exporting "_Pair" file... This may take some time, please wait. -----\n')
            pair_df.style.apply(bg_color, axis=None).to_excel(f'{date}_Pair_{filename}.xlsx', index = False)  
            print('\n----- "_Pair" file exported. -----\n')
        else:
            pass

        print('----- Start preparing simplified version. -----\n')
        # extract rows included in colored indices list separately
        byonicbyos_col_range = ['Scan'] + [col for col in all_forsimple_df.columns.tolist() if '[Byonic]' in col or '[Byos]' in col]
        pglyco_col_range = ['Scan'] + [col for col in all_forsimple_df.columns.tolist() if '[pGlyco]' in col]
        simple_byonicbyos = all_forsimple_df.loc[byonic_pass_id, byonicbyos_col_range] # extract colored (pass threshold) rows - below-threshold ethcd w/o pair
        simple_pglyco = all_forsimple_df.loc[pglyco_colored_id, pglyco_col_range] # extract colored (pass threshold) rows
        
        simple_byonicbyos_forUnique = simple_byonicbyos[(simple_byonicbyos['N-site(SequonBased)[Byonic]'] != '-1')] # preserve n/a sites for potential o glycan data in unique
        simple_byonicbyos_forNglyco = simple_byonicbyos[(simple_byonicbyos['N-site(SequonBased)[Byonic]'] != 'N/A')&(simple_byonicbyos['N-site(SequonBased)[Byonic]'] != '-1')] # for simplified, n-site must exist
        
        simple_pglyco_forUnique = simple_pglyco[(simple_pglyco['ProSites[pGlyco]'] != '-1')] # preserve n/a sites for potential o glycan data in unique
        simple_pglyco_forNglyco = simple_pglyco[(simple_pglyco['ProSites[pGlyco]'] != 'N/A')&(simple_pglyco['ProSites[pGlyco]'] != '-1')] # for simplified, n-site must exist

        # for nglycopep
        simple_pglyco_forNglyco = simple_pglyco_forNglyco.loc[simple_pglyco_forNglyco['FragmentType[pGlyco]'] == 'hcd'] # pglyco etd has the same values as hcd, so only let hcd enter unique file
        
        # clean data by dropping duplicates        
        simple_byonicbyos_forUnique = simple_byonicbyos_forUnique.drop_duplicates().reset_index(drop=True)
        simple_byonicbyos_forNglyco = simple_byonicbyos_forNglyco.drop_duplicates().reset_index(drop=True)

        simple_pglyco_forUnique = simple_pglyco_forUnique.drop_duplicates().reset_index(drop=True)
        simple_pglyco_forNglyco = simple_pglyco_forNglyco.drop_duplicates().reset_index(drop=True)
        
        # HIGHSCORE SELECTION: get unique calc.m/z by groupby & idxmax function to find highest score within each gp and preserve (remember to split the simple_df into byonicbyos & pglyco parts to avoid dropping non-target rows)
        simple_byonicbyos_forUnique = simple_byonicbyos_forUnique.groupby(['Fragment\r\nType[Byonic]', 'N-site(SequonBased)[Byonic]', 'Calc.\r\nm/z[Byonic]','rounded_FinalGlycans[Byonic]', 'PureSequence[Byonic]', 'rt_dif_reset_forUnique[Byonic]'], as_index=False).apply(lambda x: x.loc[x['Score[Byonic]'].idxmax()]).reset_index(drop=True)
        simple_pglyco_forUnique = simple_pglyco_forUnique.groupby(['FragmentType[pGlyco]', 'ProSites[pGlyco]', 'rounded_PrecursorMZ[pGlyco]','GlycanComposition_ByonicStyle[pGlyco]', 'Peptide[pGlyco]', 'rt_dif_reset_forUnique[pGlyco]'], as_index=False).apply(lambda x: x.loc[x['PepScore[pGlyco]'].idxmax()]).reset_index(drop=True)

        simple_byonicbyos_forNglyco = simple_byonicbyos_forNglyco.groupby(['N-site(SequonBased)[Byonic]', 'Calc.\r\nm/z[Byonic]','rounded_FinalGlycans[Byonic]', 'PureSequence[Byonic]', 'rt_dif_reset_forNglyco[Byonic]'], as_index=False).apply(lambda x: x.loc[x['Score[Byonic]'].idxmax()]).reset_index(drop=True)
        simple_pglyco_forNglyco = simple_pglyco_forNglyco.groupby(['ProSites[pGlyco]', 'rounded_PrecursorMZ[pGlyco]','GlycanComposition_ByonicStyle[pGlyco]', 'Peptide[pGlyco]', 'rt_dif_reset_forNglyco[pGlyco]'], as_index=False).apply(lambda x: x.loc[x['PepScore[pGlyco]'].idxmax()]).reset_index(drop=True)

        # set scan to index & concat simple_byonicbyos & simple_pglyco by scan (for unique)
        simple_byonicbyos_forUnique = simple_byonicbyos_forUnique.set_index('Scan')
        simple_pglyco_forUnique = simple_pglyco_forUnique.set_index('Scan')
        a1, a2 = simple_byonicbyos_forUnique.align(simple_pglyco_forUnique, join = 'outer', axis = 0) # row: a1 = a2
        id_df_forUnique = pd.concat([a1,a2], axis = 1)
        id_df_forUnique.index.name = 'Scan'
        id_df_forUnique.reset_index(level=0, inplace=True)
        # change all nan to blank -1
        id_df_forUnique = id_df_forUnique.fillna(-1)
        # extract nglycan numbers in Mods & use it to calculate byonic pep mass 
        mods_nglycan_sum = [sum([ast.literal_eval(v) for v in lst]) if type(lst) == list else -1 for lst in id_df_forUnique['Mods\r\n(variable)[Byonic]'].str.findall(r'Glycan / (\d+.\d+)').tolist()]
        id_df_forUnique['glycan_mods_sum[Byonic]'] = mods_nglycan_sum
        id_df_forUnique['PepMS[Byonic]'] = id_df_forUnique['Calc.\r\nMH[Byonic]'] - id_df_forUnique['glycan_mods_sum[Byonic]']
        id_df_forUnique.loc[(id_df_forUnique['glycan_mods_sum[Byonic]'] == -1), 'PepMS[Byonic]'] = -1
        move_df(id_df_forUnique, 'ProSites[pGlyco]', 'N-site(SequonBased)[Byonic]')
        move_df(id_df_forUnique, 'PepMS[Byonic]', 'Fragment\r\nType[Byonic]')
        # ast.literaleval byonicbyos n-site & sorting: byonic site -> pglyco site -> byonic peptide mass -> pglyco peptide mass -> byonic glycans -> pglyco glycans in byonic style -> byonic clam/z -> pglcyo calm/z
        id_df_forUnique['N-site(SequonBased)[Byonic]'] = [ast.literal_eval(i) if type(i) == str and '(' not in i and i!='N/A' else i for i in id_df_forUnique['N-site(SequonBased)[Byonic]'].tolist()] # convert str back to int & tuple
        id_df_forUnique['ProSites[pGlyco]'] = [ast.literal_eval(i) if type(i) == str and '(' not in i and i!='N/A' else i for i in id_df_forUnique['ProSites[pGlyco]'].tolist()] # convert str back to int & tuple
        
        id_df_forUnique.loc[(id_df_forUnique['N-site(SequonBased)[Byonic]'] == -1), 'mock_site[Byonic]'] = id_df_forUnique['ProSites[pGlyco]']
        id_df_forUnique.loc[(id_df_forUnique['N-site(SequonBased)[Byonic]'] != -1), 'mock_site[Byonic]'] = id_df_forUnique['N-site(SequonBased)[Byonic]']
        # drop intermediate cols
        id_df_forUnique = id_df_forUnique.drop(['rt_dif_reset_forUnique[Byonic]', 'rt_dif_reset_forNglyco[Byonic]', 'rt_dif_reset_forUnique[pGlyco]', 'rt_dif_reset_forNglyco[pGlyco]', 'glycan_mods_sum[Byonic]'], axis = 1) # drop the intermediate cols
        glycansource(id_df_forUnique, 'Scan')
        move_df(id_df_forUnique, 'mock_site[Byonic]', 'GlycanSource')
        col_order = id_df_forUnique.columns.tolist()
        # sorting order: byonic site -> pglyco site -> byonic peptide mass -> pglyco peptide mass -> byonic glycans -> pglyco glycans in byonic style -> byonic calm/z -> pglcyo calm/z => all of this can be acheive with sort_values to multi cols.

        id_df_forUnique = sort_intstr_col(id_df_forUnique, 'Charge[pGlyco]')
        print('Done sorting by Charge[pGlyco] --> ', end = '')
        id_df_forUnique = sort_intstr_col(id_df_forUnique, 'PrecursorMH[pGlyco]')
        print('Done sorting by PrecursorMH[pGlyco] --> ', end = '')
        id_df_forUnique = sort_intstr_col(id_df_forUnique, 'PeptideMH[pGlyco]')
        print('Done sorting by PeptideMH[pGlyco] --> ', end = '')
        id_df_forUnique = sort_intstr_col(id_df_forUnique, 'GlycanComposition_ByonicStyle[pGlyco]')
        print('Done sorting by GlycanComposition_ByonicStyle[pGlyco] --> ', end = '')
        id_df_forUnique = sort_intstr_col(id_df_forUnique, 'Peptide[pGlyco]')
        print('Done sorting by Peptide[pGlyco] --> ', end = '')
        id_df_forUnique = sort_intstr_col(id_df_forUnique, 'z[Byonic]')
        print('Done sorting by z[Byonic] --> ', end = '')
        id_df_forUnique = sort_intstr_col(id_df_forUnique, 'Calc.\r\nMH[Byonic]')
        print('Done sorting by Calc.MH[Byonic] --> ', end = '')
        id_df_forUnique = sort_intstr_col(id_df_forUnique, 'PepMS[Byonic]')
        print('Done sorting by PepMS[Byonic] --> ', end = '')
        id_df_forUnique = sort_intstr_col(id_df_forUnique, 'rounded_FinalGlycans[Byonic]')
        print('Done sorting by rounded_FinalGlycans[Byonic] --> ', end = '')
        id_df_forUnique = sort_intstr_col(id_df_forUnique, 'PureSequence[Byonic]')
        print('Done sorting by PureSequence[Byonic] --> ', end = '')
        id_df_forUnique = sort_intstr_col(id_df_forUnique, 'mock_site[Byonic]')
        print('Done sorting by mock_site[Byonic] (added during processing)')

        # convert '-1' back to int -1 for correct glycansource analysis
        id_df_forUnique['rounded_FinalGlycans[Byonic]'] = [int(i) if i == '-1' else i for i in id_df_forUnique['rounded_FinalGlycans[Byonic]']]
        id_df_forUnique['GlycanComposition_ByonicStyle[pGlyco]'] = [int(i) if i == '-1' else i for i in id_df_forUnique['GlycanComposition_ByonicStyle[pGlyco]']]
        id_df_forUnique = id_df_forUnique[col_order]

        print('\n----- Exporting "_UniquePep" file... This may take some time, please wait. -----\n')
        threshold_masks_colorind(id_df_forUnique)
        id_df_forUnique.style.apply(bg_color, axis=None).to_excel(f'{date}_UniquePep_{filename}.xlsx', index = False)  
        print('\n----- "_UniquePep" file exported. -----\n')

        # set scan to index & concat simple_byonicbyos & simple_pglyco by scan (for nglyco)
        simple_byonicbyos_forNglyco = simple_byonicbyos_forNglyco.set_index('Scan')
        simple_pglyco_forNglyco = simple_pglyco_forNglyco.set_index('Scan')
        a1, a2 = simple_byonicbyos_forNglyco.align(simple_pglyco_forNglyco, join = 'outer', axis = 0) # row: a1 = a2
        id_df_forNglyco = pd.concat([a1,a2], axis = 1)
        id_df_forNglyco.index.name = 'Scan'
        id_df_forNglyco.reset_index(level=0, inplace=True)
        # change all nan to blank -1
        id_df_forNglyco = id_df_forNglyco.fillna(-1)
        # extract nglycan numbers in Mods & use it to calculate byonic pep mass 
        mods_nglycan_sum = [sum([ast.literal_eval(v) for v in lst]) if type(lst) == list else -1 for lst in id_df_forNglyco['Mods\r\n(variable)[Byonic]'].str.findall(r'Glycan / (\d+.\d+)').tolist()]
        id_df_forNglyco['glycan_mods_sum[Byonic]'] = mods_nglycan_sum
        id_df_forNglyco['PepMS[Byonic]'] = id_df_forNglyco['Calc.\r\nMH[Byonic]'] - id_df_forNglyco['glycan_mods_sum[Byonic]']
        id_df_forNglyco.loc[(id_df_forNglyco['glycan_mods_sum[Byonic]'] == -1), 'PepMS[Byonic]'] = -1
        move_df(id_df_forNglyco, 'ProSites[pGlyco]', 'N-site(SequonBased)[Byonic]')
        move_df(id_df_forNglyco, 'PepMS[Byonic]', 'Fragment\r\nType[Byonic]')
        # ast.literaleval byonicbyos n-site & sorting: byonic site -> pglyco site -> byonic peptide mass -> pglyco peptide mass -> byonic glycans -> pglyco glycans in byonic style -> byonic clam/z -> pglcyo calm/z
        id_df_forNglyco['N-site(SequonBased)[Byonic]'] = [ast.literal_eval(i) if type(i) == str and '(' not in i else i for i in id_df_forNglyco['N-site(SequonBased)[Byonic]'].tolist()] # convert str back to int & tuple
        id_df_forNglyco['ProSites[pGlyco]'] = [ast.literal_eval(i) if type(i) == str and '(' not in i else i for i in id_df_forNglyco['ProSites[pGlyco]'].tolist()] # convert str back to int & tuple

        id_df_forNglyco.loc[(id_df_forNglyco['N-site(SequonBased)[Byonic]'] == -1), 'mock_site[Byonic]'] = id_df_forNglyco['ProSites[pGlyco]']
        id_df_forNglyco.loc[(id_df_forNglyco['N-site(SequonBased)[Byonic]'] != -1), 'mock_site[Byonic]'] = id_df_forNglyco['N-site(SequonBased)[Byonic]']
        # drop intermediate cols
        id_df_forNglyco = id_df_forNglyco.drop(['rt_dif_reset_forUnique[Byonic]', 'rt_dif_reset_forNglyco[Byonic]', 'rt_dif_reset_forUnique[pGlyco]', 'rt_dif_reset_forNglyco[pGlyco]', 'glycan_mods_sum[Byonic]'], axis = 1) # drop the intermediate cols
        glycansource(id_df_forNglyco, 'Scan')
        move_df(id_df_forNglyco, 'mock_site[Byonic]', 'GlycanSource')
        col_order = id_df_forNglyco.columns.tolist()
        # sorting order: byonic site -> pglyco site -> byonic peptide mass -> pglyco peptide mass -> byonic glycans -> pglyco glycans in byonic style -> byonic calm/z -> pglcyo calm/z => all of this can be acheive with sort_values to multi cols.

        id_df_forNglyco = sort_intstr_col(id_df_forNglyco, 'Charge[pGlyco]')
        print('Done sorting by Charge[pGlyco] --> ', end = '')
        id_df_forNglyco = sort_intstr_col(id_df_forNglyco, 'PrecursorMH[pGlyco]')
        print('Done sorting by PrecursorMH[pGlyco] --> ', end = '')
        id_df_forNglyco = sort_intstr_col(id_df_forNglyco, 'PeptideMH[pGlyco]')
        print('Done sorting by PeptideMH[pGlyco] --> ', end = '')
        id_df_forNglyco = sort_intstr_col(id_df_forNglyco, 'GlycanComposition_ByonicStyle[pGlyco]')
        print('Done sorting by GlycanComposition_ByonicStyle[pGlyco] --> ', end = '')
        id_df_forNglyco = sort_intstr_col(id_df_forNglyco, 'Peptide[pGlyco]')
        print('Done sorting by Peptide[pGlyco] --> ', end = '')
        id_df_forNglyco = sort_intstr_col(id_df_forNglyco, 'z[Byonic]')
        print('Done sorting by z[Byonic] --> ', end = '')
        id_df_forNglyco = sort_intstr_col(id_df_forNglyco, 'Calc.\r\nMH[Byonic]')
        print('Done sorting by Calc.MH[Byonic] --> ', end = '')
        id_df_forNglyco = sort_intstr_col(id_df_forNglyco, 'PepMS[Byonic]')
        print('Done sorting by PepMS[Byonic] --> ', end = '')
        id_df_forNglyco = sort_intstr_col(id_df_forNglyco, 'rounded_FinalGlycans[Byonic]')
        print('Done sorting by rounded_FinalGlycans[Byonic] --> ', end = '')
        id_df_forNglyco = sort_intstr_col(id_df_forNglyco, 'PureSequence[Byonic]')
        print('Done sorting by PureSequence[Byonic] --> ', end = '')
        id_df_forNglyco = sort_intstr_col(id_df_forNglyco, 'mock_site[Byonic]')
        print('Done sorting by mock_site[Byonic] (added during processing)')

        # convert '-1' back to int -1 for correct glycansource analysis
        id_df_forNglyco['rounded_FinalGlycans[Byonic]'] = [int(i) if i == '-1' else i for i in id_df_forNglyco['rounded_FinalGlycans[Byonic]']]
        id_df_forNglyco['GlycanComposition_ByonicStyle[pGlyco]'] = [int(i) if i == '-1' else i for i in id_df_forNglyco['GlycanComposition_ByonicStyle[pGlyco]']]
        id_df_forNglyco = id_df_forNglyco[col_order]

        print('\n----- Exporting "_NglycoPep" file... This may take some time, please wait. -----\n')
        threshold_masks_colorind(id_df_forNglyco)
        id_df_forNglyco.style.apply(bg_color, axis=None).to_excel(f'{date}_NglycoPep_{filename}.xlsx', index = False)  
        print('\n----- "_NglycoPep" file exported. -----\n')

        print('----- Start preparing quantification file of N-glycosylation . -----\n')
        # only single sites for quant (quant_df offers important info. for later multiIndex df construction)
        quant = id_df_forNglyco[id_df_forNglyco['N-site(SequonBased)[Byonic]'].apply(lambda x: isinstance(x, int))]
        # avoid adding values from below-threshold rows: if deepgreen -> change value to -1, if deepblue -> change value to -1.(-1 to make lambda function easier to write)
        threshold_masks_colorind(quant)
        quant.loc[deepgreen_ind, ['MonoArea[pGlyco]', 'IsotopeArea[pGlyco]']] = -1
        quant.loc[deepblue_ind, ['XIC\r\nAUC[Byos]', 'Apex Int.\r\n(Posit)[Byos]']] = -1
        # calculate the sum within each n-site, then normalize xicauc/ int/ mono/ iso by these sum
        quant['sumpersite_xicauc'] = quant.groupby(['N-site(SequonBased)[Byonic]'])['XIC\r\nAUC[Byos]'].transform(lambda x: x[x!=-1].sum())
        quant['sumpersite_int'] = quant.groupby(['N-site(SequonBased)[Byonic]'])['Apex Int.\r\n(Posit)[Byos]'].transform(lambda x: x[x!=-1].sum())
        quant['sumpersite_mono'] = quant.groupby(['ProSites[pGlyco]'])['MonoArea[pGlyco]'].transform(lambda x: x[x!=-1].sum())
        quant['sumpersite_iso'] = quant.groupby(['ProSites[pGlyco]'])['IsotopeArea[pGlyco]'].transform(lambda x: x[x!=-1].sum())
        # convert all the 'N/A' in byonic glycans to 'Unoccupied'
        quant.loc[(quant['rounded_FinalGlycans[Byonic]'] == 'N/A'), ['rounded_FinalGlycans[Byonic]']] = 'Unoccupied'
        # using groupby transform sum function to add 8 cols (byos xicauc/byos area int/pglycomono/pglycoisotope) recording summed values & normalized values (same site & same glycan & seq can be dif)
        quant['e_sum_XIC\r\nAUC[Byos]'] = quant.groupby(['N-site(SequonBased)[Byonic]', 'rounded_FinalGlycans[Byonic]'])['XIC\r\nAUC[Byos]'].transform(lambda x: x[x != -1].sum())
        quant['f_sum_Apex Int.\r\n(Posit)[Byos]'] = quant.groupby(['N-site(SequonBased)[Byonic]', 'rounded_FinalGlycans[Byonic]'])['Apex Int.\r\n(Posit)[Byos]'].transform(lambda x: x[x != -1].sum())
        quant['g_sum_MonoArea[pGlyco]'] = quant.groupby(['ProSites[pGlyco]', 'GlycanComposition_ByonicStyle[pGlyco]'])['MonoArea[pGlyco]'].transform(lambda x: x[x != -1].sum())
        quant['h_sum_IsotopeArea[pGlyco]'] = quant.groupby(['ProSites[pGlyco]', 'GlycanComposition_ByonicStyle[pGlyco]'])['IsotopeArea[pGlyco]'].transform(lambda x: x[x != -1].sum())
        quant['a_norm_XIC\r\nAUC[Byos]'] = quant['e_sum_XIC\r\nAUC[Byos]']/quant['sumpersite_xicauc']
        quant['b_norm_Apex Int.\r\n(Posit)[Byos]'] = quant['f_sum_Apex Int.\r\n(Posit)[Byos]']/quant['sumpersite_int']
        quant['c_norm_MonoArea[pGlyco]'] = quant['g_sum_MonoArea[pGlyco]']/quant['sumpersite_mono']
        quant['d_norm_IsotopeArea[pGlyco]'] = quant['h_sum_IsotopeArea[pGlyco]']/quant['sumpersite_iso']
        # since it's possible to have real 0 from calculation, change the real absent data back to -1 in sum_ & norm_
        quant.loc[quant['MonoArea[pGlyco]'] == -1, ['g_sum_MonoArea[pGlyco]', 'h_sum_IsotopeArea[pGlyco]', 'c_norm_MonoArea[pGlyco]', 'd_norm_IsotopeArea[pGlyco]']] = -1
        quant.loc[quant['XIC\r\nAUC[Byos]'] == -1, ['e_sum_XIC\r\nAUC[Byos]', 'f_sum_Apex Int.\r\n(Posit)[Byos]', 'a_norm_XIC\r\nAUC[Byos]', 'b_norm_Apex Int.\r\n(Posit)[Byos]']] = -1 
        # quant.to_excel('20210714_quant_debug.xlsx', index = False)
        # extract the needed cols only & split byonicbyos/pglyco, drop all -1 rows, glycans as index to outer union concat data again
        quant_bb = quant[['GlycanSource', 'N-site(SequonBased)[Byonic]', 'rounded_FinalGlycans[Byonic]', 'MS2 Search\r\nAlias name[Byos]', 'e_sum_XIC\r\nAUC[Byos]', 'f_sum_Apex Int.\r\n(Posit)[Byos]', 'a_norm_XIC\r\nAUC[Byos]', 'b_norm_Apex Int.\r\n(Posit)[Byos]']].rename(columns={'GlycanSource':'GlycanSource[Byonic]'})
        quant_p = quant[['GlycanSource', 'ProSites[pGlyco]', 'GlycanComposition_ByonicStyle[pGlyco]', 'g_sum_MonoArea[pGlyco]', 'h_sum_IsotopeArea[pGlyco]', 'c_norm_MonoArea[pGlyco]', 'd_norm_IsotopeArea[pGlyco]']].rename(columns={'GlycanSource':'GlycanSource[pGlyco]'})
        quant_bb = quant_bb.loc[quant_bb['N-site(SequonBased)[Byonic]'] != -1]
        quant_p = quant_p.loc[quant_p['ProSites[pGlyco]'] != -1]
        quant_bb = quant_bb.set_index('rounded_FinalGlycans[Byonic]')
        quant_p = quant_p.set_index('GlycanComposition_ByonicStyle[pGlyco]')
        a1, a2 = quant_bb.align(quant_p, join = 'outer', axis = 0) 
        quant_glycanid = pd.concat([a1,a2], axis = 1)
        quant_glycanid.index.name = 'Glycans ↓'
        quant_glycanid.reset_index(level=0, inplace=True)
        # add N(x) col
        nx = ['N(0)' if lst == [] else 'N(%s)'%(lst[0]) for lst in quant_glycanid['Glycans ↓'].str.findall(r'HexNAc\((\d+)\)').tolist()]
        quant_glycanid.insert(0 , 'N(x) ↓', nx , True)
        move_df(quant_glycanid, 'ProSites[pGlyco]', 'N-site(SequonBased)[Byonic]')
        quant_glycanid = quant_glycanid.drop('MS2 Search\r\nAlias name[Byos]', axis = 1)
        # change all nan to blank -1
        quant_glycanid = quant_glycanid.fillna(-1)
        quant_glycanid = quant_glycanid.drop_duplicates() # remember to drop duplicates after splitting & reconcat df
        # generate 5 versions: bp_intersection (both sites are present at the same time <intersection>), bp_union (n site union), only b (xicauc/int), only p (mono/isotope)
        ## onlyb
        onlyb = quant_glycanid.loc[(quant_glycanid['N-site(SequonBased)[Byonic]'] != -1), [col for col in quant_glycanid.columns.tolist() if 'pGlyco' not in col]]
        onlyb_forunion = onlyb.copy().drop_duplicates()
        onlyb = onlyb.drop('GlycanSource[Byonic]', axis = 1).drop_duplicates()
        # clean off all no data rows
        onlyb = onlyb.loc[~((onlyb['e_sum_XIC\r\nAUC[Byos]']==-1)&(onlyb['f_sum_Apex Int.\r\n(Posit)[Byos]']==-1)&(onlyb['a_norm_XIC\r\nAUC[Byos]']==-1)&(onlyb['b_norm_Apex Int.\r\n(Posit)[Byos]']==-1))]
        nx = list(set(onlyb['N(x) ↓'].tolist()))
        nx = [int(re.findall(r'[0-9]+', i)[0]) if type(i) == str else i for i in nx]
        nx.sort(key=lambda v: (isinstance(v, str), v))
        b_new_nx = ['N(%s)'%(str(i)) if type(i) == int and i != -1 else i for i in nx]

        ## onlyp
        onlyp = quant_glycanid.loc[(quant_glycanid['ProSites[pGlyco]'] != -1), [col for col in quant_glycanid.columns.tolist() if 'Byonic' not in col and 'Byos' not in col]]
        onlyp_forunion = onlyp.copy().drop_duplicates()
        onlyp = onlyp.drop('GlycanSource[pGlyco]', axis = 1).drop_duplicates()
        # clean off all no data rows
        onlyp = onlyp.loc[~((onlyp['g_sum_MonoArea[pGlyco]']==-1)&(onlyp['h_sum_IsotopeArea[pGlyco]']==-1)&(onlyp['c_norm_MonoArea[pGlyco]']==-1)&(onlyp['d_norm_IsotopeArea[pGlyco]']==-1))]
        nx = list(set(onlyp['N(x) ↓'].tolist()))
        nx = [int(re.findall(r'[0-9]+', i)[0]) if type(i) == str else i for i in nx]
        nx.sort(key=lambda v: (isinstance(v, str), v))
        p_new_nx = ['N(%s)'%(str(i)) if type(i) == int and i != -1 else i for i in nx]

        ## bp_union
        onlyb_union = onlyb_forunion.rename(columns={'N-site(SequonBased)[Byonic]':'N-site(Byonic ∪ pGlyco) →', 'GlycanSource[Byonic]':'GlycanSource'}) 
        onlyp_union = onlyp_forunion.rename(columns={'ProSites[pGlyco]':'N-site(Byonic ∪ pGlyco) →', 'GlycanSource[pGlyco]':'GlycanSource'}) 
        a1, a2 = onlyb_union.drop('N(x) ↓', axis = 1).align(onlyp_union.drop('N(x) ↓', axis = 1), join='outer', axis = 1) # align col only
        bp_union = pd.concat([a1,a2])
        bp_union = bp_union.drop('GlycanSource', axis = 1).drop_duplicates().sort_values('Glycans ↓').reset_index(drop = True).fillna(-1)
        # add N(x) col
        nx = ['N(0)' if lst == [] else 'N(%s)'%(lst[0]) for lst in bp_union['Glycans ↓'].str.findall(r'HexNAc\((\d+)\)').tolist()]
        bp_union.insert(0 , 'N(x) ↓', nx , True)
        nx = list(set(bp_union['N(x) ↓'].tolist()))
        nx = [int(re.findall(r'[0-9]+', i)[0]) if type(i) == str else i for i in nx]
        nx.sort(key=lambda v: (isinstance(v, str), v))
        bp_union_new_nx = ['N(%s)'%(str(i)) if type(i) == int and i != -1 else i for i in nx]

        ## bp_intersection (ignore glycansource, some glycans will be deleted due to this, so we need to normalize again)
        # compare glycans from b/p using quant_glycanid df
        bp_inter = quant_glycanid.loc[(quant_glycanid['N-site(SequonBased)[Byonic]']==quant_glycanid['ProSites[pGlyco]'])].drop(['GlycanSource[Byonic]', 'GlycanSource[pGlyco]', 'ProSites[pGlyco]'], axis=1).rename(columns={'N-site(SequonBased)[Byonic]':'N-site(Byonic ∪ pGlyco) →'}).drop_duplicates().reset_index(drop=True)
        # bp_inter.to_excel('bp_inter.xlsx', index=False)
        # normalize again
        # calculate the sum within each n-site, then normalize xicauc/ int/ mono/ iso by these sum
        bp_inter['sumpersite_xicauc'] = bp_inter.groupby(['N-site(Byonic ∪ pGlyco) →'])['e_sum_XIC\r\nAUC[Byos]'].transform(lambda x: x[x!=-1].sum())
        bp_inter['sumpersite_int'] = bp_inter.groupby(['N-site(Byonic ∪ pGlyco) →'])['f_sum_Apex Int.\r\n(Posit)[Byos]'].transform(lambda x: x[x!=-1].sum())
        bp_inter['sumpersite_mono'] = bp_inter.groupby(['N-site(Byonic ∪ pGlyco) →'])['g_sum_MonoArea[pGlyco]'].transform(lambda x: x[x!=-1].sum())
        bp_inter['sumpersite_iso'] = bp_inter.groupby(['N-site(Byonic ∪ pGlyco) →'])['h_sum_IsotopeArea[pGlyco]'].transform(lambda x: x[x!=-1].sum())
        # change the real absent data back to -1
        bp_inter.loc[bp_inter['c_norm_MonoArea[pGlyco]'] == -1, ['sumpersite_mono', 'sumpersite_iso']] = -1
        bp_inter.loc[bp_inter['a_norm_XIC\r\nAUC[Byos]'] == -1, ['sumpersite_xicauc', 'sumpersite_int']] = -1 
        # using groupby transform sum function to add 8 cols (byos xicauc/byos area int/pglycomono/pglycoisotope) recording summed values & normalized values (same site & same glycan & seq can be dif)
        bp_inter['a_norm_XIC\r\nAUC[Byos]'] = bp_inter['e_sum_XIC\r\nAUC[Byos]']/bp_inter['sumpersite_xicauc']
        bp_inter['b_norm_Apex Int.\r\n(Posit)[Byos]'] = bp_inter['f_sum_Apex Int.\r\n(Posit)[Byos]']/bp_inter['sumpersite_int']
        bp_inter['c_norm_MonoArea[pGlyco]'] = bp_inter['g_sum_MonoArea[pGlyco]']/bp_inter['sumpersite_mono']
        bp_inter['d_norm_IsotopeArea[pGlyco]'] = bp_inter['h_sum_IsotopeArea[pGlyco]']/bp_inter['sumpersite_iso']
        # change the real absent data back to -1
        bp_inter.loc[bp_inter['sumpersite_xicauc'] == -1, ['a_norm_XIC\r\nAUC[Byos]']] = -1
        bp_inter.loc[bp_inter['sumpersite_int'] == -1, ['b_norm_Apex Int.\r\n(Posit)[Byos]']] = -1 
        bp_inter.loc[bp_inter['sumpersite_mono'] == -1, ['c_norm_MonoArea[pGlyco]']] = -1
        bp_inter.loc[bp_inter['sumpersite_iso'] == -1, ['d_norm_IsotopeArea[pGlyco]']] = -1 
        # bp_inter.to_excel('bp_inter_norm_test.xlsx', index = False)
        
        # drop cols & make sure the order of the cols (a-h)
        bp_inter = bp_inter.drop(['sumpersite_xicauc', 'sumpersite_int', 'sumpersite_mono', 'sumpersite_iso'], axis=1)
        bp_inter = bp_inter[['Glycans ↓', 'N-site(Byonic ∪ pGlyco) →', 'a_norm_XIC\r\nAUC[Byos]', 'b_norm_Apex Int.\r\n(Posit)[Byos]', 'c_norm_MonoArea[pGlyco]', 'd_norm_IsotopeArea[pGlyco]', 'e_sum_XIC\r\nAUC[Byos]', 'f_sum_Apex Int.\r\n(Posit)[Byos]', 'g_sum_MonoArea[pGlyco]', 'h_sum_IsotopeArea[pGlyco]']]
        # add N(x) col
        nx = ['N(0)' if lst == [] else 'N(%s)'%(lst[0]) for lst in bp_inter['Glycans ↓'].str.findall(r'HexNAc\((\d+)\)').tolist()]
        bp_inter.insert(0 , 'N(x) ↓', nx , True)
        nx = list(set(bp_inter['N(x) ↓'].tolist()))
        nx = [int(re.findall(r'[0-9]+', i)[0]) if type(i) == str else i for i in nx]
        nx.sort(key=lambda v: (isinstance(v, str), v))
        bp_intersection_new_nx = ['N(%s)'%(str(i)) if type(i) == int and i != -1 else i for i in nx]

        ## onlyb_top10
        onlyb_top10 = onlyb.copy()
        # onlyb_top10.to_excel('20210722_onlyb_top10_debug.xlsx', index=False)
        # xicauc_top10 = onlyb_top10.groupby('N-site(SequonBased)[Byonic]')['a_norm_XIC\r\nAUC[Byos]'].nlargest(topN).droplevel(level = 0).index.tolist()
        xicauc_top10 = onlyb_top10.groupby('N-site(SequonBased)[Byonic]').apply(pd.DataFrame.nlargest, n=5, columns='a_norm_XIC\r\nAUC[Byos]').droplevel(level=0).index.tolist()
        int_top10 = onlyb_top10.groupby('N-site(SequonBased)[Byonic]').apply(pd.DataFrame.nlargest, n=5, columns='b_norm_Apex Int.\r\n(Posit)[Byos]').droplevel(level=0).index.tolist()
        onlyb_top10.loc[[i for i in onlyb_top10.index.tolist() if i not in xicauc_top10], 'a_norm_XIC\r\nAUC[Byos]'] = -1
        onlyb_top10.loc[[i for i in onlyb_top10.index.tolist() if i not in int_top10], 'b_norm_Apex Int.\r\n(Posit)[Byos]'] = -1
        onlyb_top10 = onlyb_top10.loc[(onlyb_top10['a_norm_XIC\r\nAUC[Byos]'] != -1)|(onlyb_top10['b_norm_Apex Int.\r\n(Posit)[Byos]'] != -1)].reset_index(drop=True).drop(['N(x) ↓', 'e_sum_XIC\r\nAUC[Byos]', 'f_sum_Apex Int.\r\n(Posit)[Byos]', 'N(x) ↓'], axis = 1)

        ## onlyp_top10
        onlyp_top10 = onlyp.copy()
        # onlyp_top10.to_excel('20210722_onlyp_top10_debug.xlsx', index=False)
        mono_top10 = onlyp_top10.groupby('ProSites[pGlyco]').apply(pd.DataFrame.nlargest, n=5, columns='c_norm_MonoArea[pGlyco]').droplevel(level=0).index.tolist()
        iso_top10 = onlyp_top10.groupby('ProSites[pGlyco]').apply(pd.DataFrame.nlargest, n=5, columns='d_norm_IsotopeArea[pGlyco]').droplevel(level=0).index.tolist()
        onlyp_top10.loc[[i for i in onlyp_top10.index.tolist() if i not in mono_top10], 'c_norm_MonoArea[pGlyco]'] = -1
        onlyp_top10.loc[[i for i in onlyp_top10.index.tolist() if i not in iso_top10], 'd_norm_IsotopeArea[pGlyco]'] = -1
        onlyp_top10 = onlyp_top10.loc[(onlyp_top10['c_norm_MonoArea[pGlyco]'] != -1)|(onlyp_top10['d_norm_IsotopeArea[pGlyco]'] != -1)].reset_index(drop=True).drop(['N(x) ↓', 'g_sum_MonoArea[pGlyco]', 'h_sum_IsotopeArea[pGlyco]', 'N(x) ↓'], axis = 1)

        ## bp_top10
        onlyb_top10 = onlyb_top10.rename(columns={'N-site(SequonBased)[Byonic]': f'N-site(Byonic ∪ pGlyco [Top{topN}]) →'}) 
        onlyp_top10 = onlyp_top10.rename(columns={'ProSites[pGlyco]':f'N-site(Byonic ∪ pGlyco [Top{topN}]) →'})
        a1, a2 = onlyb_top10.align(onlyp_top10, join='outer', axis = 1) # align cols only
        bptop10 = pd.concat([a1,a2]) # concat vertically 
        bptop10 = bptop10.fillna(-1).sort_values('Glycans ↓').reset_index(drop = True)
        # add N(x) col
        nx = ['N(0)' if lst == [] else 'N(%s)'%(lst[0]) for lst in bptop10['Glycans ↓'].str.findall(r'HexNAc\((\d+)\)').tolist()]
        bptop10.insert(0 , 'N(x) ↓', nx , True)
        nx = list(set(bptop10['N(x) ↓'].tolist()))
        nx = [int(re.findall(r'[0-9]+', i)[0]) if type(i) == str else i for i in nx]
        nx.sort(key=lambda v: (isinstance(v, str), v))
        bptop10_new_nx = ['N(%s)'%(str(i)) if type(i) == int and i != -1 else i for i in nx]

        print('----- Start summary table construction. -----\n')
        # start grouping: N(X) -> byonic glycans -> byonic sites -> .agg({all 8 cols, apply .mean() & .mean()/total XXX})
        # bp_intersection
        bp_intersection_gp = bp_inter.groupby(['N(x) ↓', 'Glycans ↓', 'N-site(Byonic ∪ pGlyco) →']).agg({'a_norm_XIC\r\nAUC[Byos]': lambda x: x[x!= -1].mean(), 'b_norm_Apex Int.\r\n(Posit)[Byos]': lambda x: x[x!= -1].mean() \
                                                                                              , 'c_norm_MonoArea[pGlyco]': lambda x: x[x!= -1].mean(), 'd_norm_IsotopeArea[pGlyco]': lambda x: x[x!= -1].mean()})
        # fix the str '10' sorted before '2' problem
        bp_intersection_gp = bp_intersection_gp.reindex(bp_intersection_new_nx, level = 0)
        # unstack byonic n-sites, pglyco n-sites
        bp_intersection_gp = bp_intersection_gp.unstack(level=-1)
        # multiIndex col swaplevel
        bp_intersection_gp.columns = bp_intersection_gp.columns.swaplevel(0, 1)
        # sort is necessary for the right table structure (but since it will sort every level, so we need to manually adjust level3 back to the original order)
        bp_intersection_gp.sort_index(axis=1, level=[0, 1], inplace=True)
        bp_intersection_gp.columns = bp_intersection_gp.columns.set_names(['N-site(Byonic ∩ pGlyco [Glycan B=P]) →', 'Quant. →'])
        # delete a-h in Quant. (the alphabet is just for sorting)
        bp_intersection_gp.columns.set_levels(['norm_XIC\r\nAUC[Byos]', 'norm_Apex Int.\r\n(Posit)[Byos]', 'norm_MonoArea[pGlyco]', 'norm_IsotopeArea[pGlyco]'], level = 1, inplace=True)

        # bp_union
        bp_union_gp = bp_union.groupby(['N(x) ↓', 'Glycans ↓', 'N-site(Byonic ∪ pGlyco) →']).agg({'a_norm_XIC\r\nAUC[Byos]': lambda x: x[x!= -1].mean(), 'b_norm_Apex Int.\r\n(Posit)[Byos]': lambda x: x[x!= -1].mean() \
                                                                                              , 'c_norm_MonoArea[pGlyco]': lambda x: x[x!= -1].mean(), 'd_norm_IsotopeArea[pGlyco]': lambda x: x[x!= -1].mean()})

        # fix the str '10' sorted before '2' problem
        bp_union_gp = bp_union_gp.reindex(bp_union_new_nx, level = 0)
        # unstack byonic n-sites, pglyco n-sites
        bp_union_gp = bp_union_gp.unstack(level=-1)
        # multiIndex col swaplevel
        bp_union_gp.columns = bp_union_gp.columns.swaplevel(0, 1)
        # sort is necessary for the right table structure (but since it will sort every level, so we need to manually adjust level3 back to the original order)
        bp_union_gp.sort_index(axis=1, level=[0, 1], inplace=True)
        bp_union_gp.columns = bp_union_gp.columns.set_names(['N-site(Byonic ∪ pGlyco) →', 'Quant. →'])
        # delete a-h in Quant. (the alphabet is just for sorting)
        bp_union_gp.columns.set_levels(['norm_XIC\r\nAUC[Byos]', 'norm_Apex Int.\r\n(Posit)[Byos]', 'norm_MonoArea[pGlyco]', 'norm_IsotopeArea[pGlyco]'], level = 1, inplace=True)


        # onlyb
        onlyb_gp = onlyb.groupby(['N(x) ↓', 'Glycans ↓', 'N-site(SequonBased)[Byonic]']).agg({'e_sum_XIC\r\nAUC[Byos]': lambda x: x.mean(), 'f_sum_Apex Int.\r\n(Posit)[Byos]': lambda x: x.mean() \
                                                                                              , 'a_norm_XIC\r\nAUC[Byos]': lambda x: x.mean(), 'b_norm_Apex Int.\r\n(Posit)[Byos]': lambda x: x.mean()})

        # fix the str '10' sorted before '2' problem
        onlyb_gp = onlyb_gp.reindex(b_new_nx, level = 0)
        # unstack byonic n-sites
        onlyb_gp = onlyb_gp.unstack(level=-1)
        # multiIndex col swaplevel
        onlyb_gp.columns = onlyb_gp.columns.swaplevel(0, 1)
        # sort is necessary for the right table structure (but since it will sort every level, so we need to manually adjust level3 back to the original order)
        onlyb_gp.sort_index(axis=1, level=[0, 1], inplace=True)
        onlyb_gp.columns = onlyb_gp.columns.set_names(['N-site(SequonBased)[Byonic] →', 'Quant. →'])
        # delete a-h in Quant. (the alphabet is just for sorting)
        onlyb_gp.columns.set_levels(['norm_XIC\r\nAUC[Byos]', 'norm_Apex Int.\r\n(Posit)[Byos]', 'sum_XIC\r\nAUC[Byos]', 'sum_Apex Int.\r\n(Posit)[Byos]'], level = 1, inplace=True)

        # onlyp
        onlyp_gp = onlyp.groupby(['N(x) ↓', 'Glycans ↓', 'ProSites[pGlyco]']).agg({'g_sum_MonoArea[pGlyco]': lambda x: x.mean(), 'h_sum_IsotopeArea[pGlyco]': lambda x: x.mean() \
                                                                                              , 'c_norm_MonoArea[pGlyco]': lambda x: x.mean(), 'd_norm_IsotopeArea[pGlyco]': lambda x: x.mean()})

        # fix the str '10' sorted before '2' problem
        onlyp_gp = onlyp_gp.reindex(p_new_nx, level = 0)
        # unstack byonic n-sites, pglyco n-sites
        onlyp_gp = onlyp_gp.unstack(level=-1)
        # multiIndex col swaplevel
        onlyp_gp.columns = onlyp_gp.columns.swaplevel(0, 1)
        # sort is necessary for the right table structure (but since it will sort every level, so we need to manually adjust level3 back to the original order)
        onlyp_gp.sort_index(axis=1, level=[0, 1], inplace=True)
        onlyp_gp.columns = onlyp_gp.columns.set_names(['ProSites[pGlyco] →', 'Quant. →'])
        # delete a-h in Quant. (the alphabet is just for sorting)
        onlyp_gp.columns.set_levels(['norm_MonoArea[pGlyco]', 'norm_IsotopeArea[pGlyco]', 'sum_MonoArea[pGlyco]', 'sum_IsotopeArea[pGlyco]'], level = 1, inplace=True)

        # bp_top10
        bptop10_gp = bptop10.groupby(['N(x) ↓', 'Glycans ↓', f'N-site(Byonic ∪ pGlyco [Top{topN}]) →']).agg({'a_norm_XIC\r\nAUC[Byos]': lambda x: x[x!= -1].mean(), 'b_norm_Apex Int.\r\n(Posit)[Byos]': lambda x: x[x!= -1].mean() \
                                                                                              , 'c_norm_MonoArea[pGlyco]': lambda x: x[x!= -1].mean(), 'd_norm_IsotopeArea[pGlyco]': lambda x: x[x!= -1].mean()})

        # fix the str '10' sorted before '2' problem
        bptop10_gp = bptop10_gp.reindex(bptop10_new_nx, level = 0)
        # unstack byonic n-sites, pglyco n-sites
        bptop10_gp = bptop10_gp.unstack(level=-1)
        # multiIndex col swaplevel
        bptop10_gp.columns = bptop10_gp.columns.swaplevel(0, 1)
        # sort is necessary for the right table structure (but since it will sort every level, so we need to manually adjust level3 back to the original order)
        bptop10_gp.sort_index(axis=1, level=[0, 1], inplace=True)
        bptop10_gp.columns = bptop10_gp.columns.set_names([f'N-site(Byonic ∪ pGlyco [Top{topN}]) →', 'Quant. →'])
        # delete a-h in Quant. (the alphabet is just for sorting)
        bptop10_gp.columns.set_levels(['norm_XIC\r\nAUC[Byos]', 'norm_Apex Int.\r\n(Posit)[Byos]', 'norm_MonoArea[pGlyco]', 'norm_IsotopeArea[pGlyco]'], level = 1, inplace=True)

        print('\n----- Exporting "_Quant" files... This may take some time, please wait. -----\n')
        
        with pd.ExcelWriter(f'{date}_Quant_{filename}.xlsx') as writer:           
            # onlyb
            onlyb_gp.replace(to_replace = -1, value = np.nan , inplace = True)
            onlyb_gp.style.background_gradient(cmap ='Greens').highlight_null(null_color='white').to_excel(writer, sheet_name = 'Sheet1(QuantB)')
            # onlyp
            onlyp_gp.replace(to_replace = -1, value = np.nan , inplace = True) 
            onlyp_gp.style.background_gradient(cmap ='Blues').highlight_null(null_color='white').to_excel(writer, sheet_name = 'Sheet2(QuantP)')
            # bp_intersection
            bp_intersection_gp.replace(to_replace = -1, value = np.nan , inplace = True)
            bp_intersection_gp.style.background_gradient(cmap ='Reds').highlight_null(null_color='white').to_excel(writer, sheet_name = 'Sheet3(QuantBPinter)') 
            # bp_union
            bp_union_gp.replace(to_replace = -1, value = np.nan , inplace = True)
            bp_union_gp.style.background_gradient(cmap ='Reds').highlight_null(null_color='white').to_excel(writer, sheet_name = 'Sheet4(QuantBPunion)') 
            # bp_top10
            bptop10_gp.replace(to_replace = -1, value = np.nan , inplace = True)
            bptop10_gp.style.background_gradient(cmap ='Reds').highlight_null(null_color='white').to_excel(writer, sheet_name = f'Sheet5(Top{topN})') 
            
        print('\n----- "_Quant" file exported. -----\n')
        
        print('----- Start plotting. -----\n')
        bpunionNORM_dfall_lstv1, bpunionNORM_label_lstv1, bpunionNORM_dfall_lstv2, bpunionNORM_label_lstv2 \
        , bpunionNORM_dfall_lstv3, bpunionNORM_label_lstv3, bpunionNORM_dfall_lstv4, bpunionNORM_label_lstv4 \
        , bptop10NORM_dfall_lstv1, bptop10NORM_label_lstv1, bptop10NORM_dfall_lstv2, bptop10NORM_label_lstv2 \
        , bptop10NORM_dfall_lstv3, bptop10NORM_label_lstv3, bptop10NORM_dfall_lstv4, bptop10NORM_label_lstv4 \
        , bpinterNORM_dfall_lstv1, bpinterNORM_label_lstv1, bpinterNORM_dfall_lstv2, bpinterNORM_label_lstv2 \
        , bpinterNORM_dfall_lstv3, bpinterNORM_label_lstv3, bpinterNORM_dfall_lstv4, bpinterNORM_label_lstv4 \
        , bNORM_dfall_lstv1, bNORM_label_lstv1, bNORM_dfall_lstv2, bNORM_label_lstv2 \
        , bNORM_dfall_lstv3, bNORM_label_lstv3, bNORM_dfall_lstv4, bNORM_label_lstv4 \
        , bSUM_dfall_lstv1, bSUM_label_lstv1, bSUM_nsum_lstv1, bSUM_dfall_lstv2, bSUM_label_lstv2, bSUM_nsum_lstv2 \
        , bSUM_dfall_lstv3, bSUM_label_lstv3, bSUM_nsum_lstv3, bSUM_dfall_lstv4, bSUM_label_lstv4, bSUM_nsum_lstv4 \
        , pNORM_dfall_lstv1, pNORM_label_lstv1, pNORM_dfall_lstv2, pNORM_label_lstv2 \
        , pNORM_dfall_lstv3, pNORM_label_lstv3, pNORM_dfall_lstv4, pNORM_label_lstv4 \
        , pSUM_dfall_lstv1, pSUM_label_lstv1, pSUM_nsum_lstv1, pSUM_dfall_lstv2, pSUM_label_lstv2, pSUM_nsum_lstv2 \
        , pSUM_dfall_lstv3, pSUM_label_lstv3, pSUM_nsum_lstv3, pSUM_dfall_lstv4, pSUM_label_lstv4, pSUM_nsum_lstv4 \
        = [], [], [], [], [], [], [], [], [], [] \
        , [], [], [], [], [], [], [], [], [], [] \
        , [], [], [], [], [], [], [], [], [], [] \
        , [], [], [], [], [], [], [], [], [], [] \
        , [], [], [], [], [], [], [], [], [], [] \
        , [], [], [], [], [], [], [], [], [], [] \
        , [], [], [], []
        
        if plot_v2 == 'yes':  
            col_order = ['N(0)', 'N(1)', 'OligoMannose', 'Complex', 'Complex+Sia'] # col order for v2 plotting
            # necessary preprocessings for v2 plotting (modified from v1 nx)
            # bp union v2
            bp_union.loc[(bp_union['N(x) ↓'] == 'N(0)')|(bp_union['N(x) ↓'] == 'N(1)'), 'nx_version2'] = bp_union['N(x) ↓']
            bp_union.loc[(bp_union['N(x) ↓'] == 'N(2)'), 'nx_version2'] = 'OligoMannose'
            bp_union.loc[(bp_union['N(x) ↓'] != 'N(0)')&(bp_union['N(x) ↓'] != 'N(1)')&(bp_union['N(x) ↓'] != 'N(2)'), 'nx_version2'] = 'Complex'
            bp_union.loc[(bp_union['N(x) ↓'] != 'N(0)')&(bp_union['N(x) ↓'] != 'N(1)')&(bp_union['N(x) ↓'] != 'N(2)')&(bp_union['Glycans ↓'].str.contains('NeuAc|NeuGc')), 'nx_version2'] = 'Complex+Sia'
            bpunion_neugc_exist = sum(bp_union['Glycans ↓'].str.contains('NeuGc')) > 0 # Bool, if T, red. else purple
            # bp top10 (extract from bp union) v2
            bptop10.loc[(bptop10['N(x) ↓'] == 'N(0)')|(bptop10['N(x) ↓'] == 'N(1)'), 'nx_version2'] = bptop10['N(x) ↓']
            bptop10.loc[(bptop10['N(x) ↓'] == 'N(2)'), 'nx_version2'] = 'OligoMannose'
            bptop10.loc[(bptop10['N(x) ↓'] != 'N(0)')&(bptop10['N(x) ↓'] != 'N(1)')&(bptop10['N(x) ↓'] != 'N(2)'), 'nx_version2'] = 'Complex'
            bptop10.loc[(bptop10['N(x) ↓'] != 'N(0)')&(bptop10['N(x) ↓'] != 'N(1)')&(bptop10['N(x) ↓'] != 'N(2)')&(bptop10['Glycans ↓'].str.contains('NeuAc|NeuGc')), 'nx_version2'] = 'Complex+Sia'
            bptop10_neugc_exist = sum(bptop10['Glycans ↓'].str.contains('NeuGc')) > 0 # Bool, if T, red. else purple
            # bp inter v2
            bp_inter.loc[(bp_inter['N(x) ↓'] == 'N(0)')|(bp_inter['N(x) ↓'] == 'N(1)'), 'nx_version2'] = bp_inter['N(x) ↓']
            bp_inter.loc[(bp_inter['N(x) ↓'] == 'N(2)'), 'nx_version2'] = 'OligoMannose'
            bp_inter.loc[(bp_inter['N(x) ↓'] != 'N(0)')&(bp_inter['N(x) ↓'] != 'N(1)')&(bp_inter['N(x) ↓'] != 'N(2)'), 'nx_version2'] = 'Complex'
            bp_inter.loc[(bp_inter['N(x) ↓'] != 'N(0)')&(bp_inter['N(x) ↓'] != 'N(1)')&(bp_inter['N(x) ↓'] != 'N(2)')&(bp_inter['Glycans ↓'].str.contains('NeuAc|NeuGc')), 'nx_version2'] = 'Complex+Sia'
            bpinter_neugc_exist = sum(bp_inter['Glycans ↓'].str.contains('NeuGc')) > 0 # Bool, if T, red. else purple
            # onlyb v2
            onlyb.loc[(onlyb['N(x) ↓'] == 'N(0)')|(onlyb['N(x) ↓'] == 'N(1)'), 'nx_version2'] = onlyb['N(x) ↓']
            onlyb.loc[(onlyb['N(x) ↓'] == 'N(2)'), 'nx_version2'] = 'OligoMannose'
            onlyb.loc[(onlyb['N(x) ↓'] != 'N(0)')&(onlyb['N(x) ↓'] != 'N(1)')&(onlyb['N(x) ↓'] != 'N(2)'), 'nx_version2'] = 'Complex'
            onlyb.loc[(onlyb['N(x) ↓'] != 'N(0)')&(onlyb['N(x) ↓'] != 'N(1)')&(onlyb['N(x) ↓'] != 'N(2)')&(onlyb['Glycans ↓'].str.contains('NeuAc|NeuGc')), 'nx_version2'] = 'Complex+Sia'
            b_neugc_exist = sum(onlyb['Glycans ↓'].str.contains('NeuGc')) > 0 # Bool, if T, red. else purple
            # onlyp v2
            onlyp.loc[(onlyp['N(x) ↓'] == 'N(0)')|(onlyp['N(x) ↓'] == 'N(1)'), 'nx_version2'] = onlyp['N(x) ↓']
            onlyp.loc[(onlyp['N(x) ↓'] == 'N(2)'), 'nx_version2'] = 'OligoMannose'
            onlyp.loc[(onlyp['N(x) ↓'] != 'N(0)')&(onlyp['N(x) ↓'] != 'N(1)')&(onlyp['N(x) ↓'] != 'N(2)'), 'nx_version2'] = 'Complex'
            onlyp.loc[(onlyp['N(x) ↓'] != 'N(0)')&(onlyp['N(x) ↓'] != 'N(1)')&(onlyp['N(x) ↓'] != 'N(2)')&(onlyp['Glycans ↓'].str.contains('NeuAc|NeuGc')), 'nx_version2'] = 'Complex+Sia'
            p_neugc_exist = sum(onlyp['Glycans ↓'].str.contains('NeuGc')) > 0 # Bool, if T, red. else purple
        
        if plot_v3 == 'yes':
            # necessary preprocessings for v3 plotting (modified from v1 nx)
            # bp union v3
            bp_union, union_nxhy_fix_order = v3(bp_union)[0], v3(bp_union)[1]
            # bp top10 (extract from bp union) v3
            bptop10, top10_nxhy_fix_order = v3(bptop10)[0], v3(bptop10)[1]            
            # bp inter v3
            bp_inter, inter_nxhy_fix_order = v3(bp_inter)[0], v3(bp_inter)[1]             
            # onlyb v3
            onlyb, onlyb_nxhy_fix_order = v3(onlyb)[0], v3(onlyb)[1]            
            # onlyp v3
            onlyp, onlyp_nxhy_fix_order = v3(onlyp)[0], v3(onlyp)[1]

        if plot_v4 == 'yes':
            # necessary preprocessings for v4 plotting (modified from v1 nx)
            # bp union v4
            bp_union, union_fnxhy_fix_order = v4(bp_union)[0], v4(bp_union)[1]
            # bp top10 (extract from bp union) v4
            bptop10, top10_fnxhy_fix_order = v4(bptop10)[0], v4(bptop10)[1]             
            # bp inter v4
            bp_inter, inter_fnxhy_fix_order = v4(bp_inter)[0], v4(bp_inter)[1]            
            # onlyb v4
            onlyb, onlyb_fnxhy_fix_order = v4(onlyb)[0], v4(onlyb)[1]            
            # onlyp v4
            onlyp, onlyp_fnxhy_fix_order = v4(onlyp)[0], v4(onlyp)[1]

        # rename bp inter colname
        bp_inter = bp_inter.rename(columns={'N-site(Byonic ∪ pGlyco) →':'N-site(Byonic ∩ pGlyco [Glycan B=P]) →'})
        # make sure that sites are presented as int not float
        bp_union = bp_union.astype({'N-site(Byonic ∪ pGlyco) →':'Int64'})
        bptop10 = bptop10.astype({f'N-site(Byonic ∪ pGlyco [Top{topN}]) →':'Int64'})
        bp_inter = bp_inter.astype({'N-site(Byonic ∩ pGlyco [Glycan B=P]) →':'Int64'})
        onlyb = onlyb.astype({'N-site(SequonBased)[Byonic]':'Int64'})
        onlyp = onlyp.astype({'ProSites[pGlyco]':'Int64'})

        # split bp_union into: xicauc_df / int_df / mono_df / iso_df for later clustered stacked plot (norm only)
        if export_xicauc == 'yes':
            if plot_v1 == 'yes':
                # bpunion norm v1
                bpunion_xicaucNORM_df = bp_union.loc[:, ['N(x) ↓', 'N-site(Byonic ∪ pGlyco) →', 'a_norm_XIC\r\nAUC[Byos]']]
                bpunion_xicaucNORM_df.replace(to_replace = -1, value = np.nan , inplace = True)
                bpunion_xicaucNORM_df = bpunion_xicaucNORM_df.pivot_table(index='N-site(Byonic ∪ pGlyco) →', values='a_norm_XIC\r\nAUC[Byos]', columns='N(x) ↓', aggfunc=np.sum).reindex(columns=bp_union_new_nx)
                bpunionNORM_dfall_lstv1.append(bpunion_xicaucNORM_df)
                bpunionNORM_label_lstv1.append('norm_XIC AUC[Byos]')
                # bptop10 norm v1
                bptop10_xicaucNORM_df = bptop10.loc[:, ['N(x) ↓', f'N-site(Byonic ∪ pGlyco [Top{topN}]) →', 'a_norm_XIC\r\nAUC[Byos]']]
                bptop10_xicaucNORM_df.replace(to_replace = -1, value = np.nan , inplace = True)
                bptop10_xicaucNORM_df = bptop10_xicaucNORM_df.pivot_table(index=f'N-site(Byonic ∪ pGlyco [Top{topN}]) →', values='a_norm_XIC\r\nAUC[Byos]', columns='N(x) ↓', aggfunc=np.sum).reindex(columns=bptop10_new_nx)
                bptop10NORM_dfall_lstv1.append(bptop10_xicaucNORM_df)
                bptop10NORM_label_lstv1.append('norm_XIC AUC[Byos]')
                # bpinter norm v1
                bpinter_xicaucNORM_df = bp_inter.loc[:, ['N(x) ↓', 'N-site(Byonic ∩ pGlyco [Glycan B=P]) →', 'a_norm_XIC\r\nAUC[Byos]']]
                bpinter_xicaucNORM_df.replace(to_replace = -1, value = np.nan , inplace = True)
                bpinter_xicaucNORM_df = bpinter_xicaucNORM_df.pivot_table(index='N-site(Byonic ∩ pGlyco [Glycan B=P]) →', values='a_norm_XIC\r\nAUC[Byos]', columns='N(x) ↓', aggfunc=np.sum).reindex(columns=bp_intersection_new_nx)
                bpinterNORM_dfall_lstv1.append(bpinter_xicaucNORM_df)
                bpinterNORM_label_lstv1.append('norm_XIC AUC[Byos]')
                # onlyb norm v1
                b_xicaucNORM_df = onlyb.loc[:, ['N(x) ↓', 'N-site(SequonBased)[Byonic]', 'a_norm_XIC\r\nAUC[Byos]']]
                b_xicaucNORM_df.replace(to_replace = -1, value = np.nan , inplace = True)
                b_xicaucNORM_df = b_xicaucNORM_df.pivot_table(index='N-site(SequonBased)[Byonic]', values='a_norm_XIC\r\nAUC[Byos]', columns='N(x) ↓', aggfunc=np.sum).reindex(columns=b_new_nx)
                bNORM_dfall_lstv1.append(b_xicaucNORM_df)
                bNORM_label_lstv1.append('norm_XIC AUC[Byos]')
                # onlyb sum v1
                b_xicaucSUM_df = onlyb.loc[:, ['N(x) ↓', 'N-site(SequonBased)[Byonic]', 'e_sum_XIC\r\nAUC[Byos]']]
                b_xicaucSUM_df.replace(to_replace = -1, value = np.nan , inplace = True)
                b_xicaucSUM_df = b_xicaucSUM_df.pivot_table(index='N-site(SequonBased)[Byonic]', values='e_sum_XIC\r\nAUC[Byos]', columns='N(x) ↓', aggfunc=np.sum).reindex(columns=b_new_nx)
                b_xicaucSUM_df_nsum = [b_xicaucSUM_df.loc[i].sum() for i in b_xicaucSUM_df.index.tolist()] # for A bar label
                bSUM_dfall_lstv1.append(b_xicaucSUM_df)
                bSUM_label_lstv1.append('sum_XIC AUC[Byos]')
                bSUM_nsum_lstv1.append(b_xicaucSUM_df_nsum)

            if plot_v2 == 'yes':
                # bpuion norm v2
                bpunion_xicaucNORMv2_df = bp_union.loc[:, ['nx_version2', 'N-site(Byonic ∪ pGlyco) →', 'a_norm_XIC\r\nAUC[Byos]']]
                bpunion_xicaucNORMv2_df.replace(to_replace = -1, value = np.nan , inplace = True)
                bpunion_xicaucNORMv2_df = bpunion_xicaucNORMv2_df.pivot_table(index='N-site(Byonic ∪ pGlyco) →', values='a_norm_XIC\r\nAUC[Byos]', columns='nx_version2', aggfunc=np.sum).reindex(columns=col_order)
                bpunionNORM_dfall_lstv2.append(bpunion_xicaucNORMv2_df)
                bpunionNORM_label_lstv2.append('norm_XIC AUC[Byos]')
                # bptop10 norm v2
                bptop10_xicaucNORMv2_df = bptop10.loc[:, ['nx_version2', f'N-site(Byonic ∪ pGlyco [Top{topN}]) →', 'a_norm_XIC\r\nAUC[Byos]']]
                bptop10_xicaucNORMv2_df.replace(to_replace = -1, value = np.nan , inplace = True)
                bptop10_xicaucNORMv2_df = bptop10_xicaucNORMv2_df.pivot_table(index=f'N-site(Byonic ∪ pGlyco [Top{topN}]) →', values='a_norm_XIC\r\nAUC[Byos]', columns='nx_version2', aggfunc=np.sum).reindex(columns=col_order)
                bptop10NORM_dfall_lstv2.append(bptop10_xicaucNORMv2_df)
                bptop10NORM_label_lstv2.append('norm_XIC AUC[Byos]')
                # bpinter norm v2
                bpinter_xicaucNORMv2_df = bp_inter.loc[:, ['nx_version2', 'N-site(Byonic ∩ pGlyco [Glycan B=P]) →', 'a_norm_XIC\r\nAUC[Byos]']]
                bpinter_xicaucNORMv2_df.replace(to_replace = -1, value = np.nan , inplace = True)
                bpinter_xicaucNORMv2_df = bpinter_xicaucNORMv2_df.pivot_table(index='N-site(Byonic ∩ pGlyco [Glycan B=P]) →', values='a_norm_XIC\r\nAUC[Byos]', columns='nx_version2', aggfunc=np.sum).reindex(columns=col_order)
                bpinterNORM_dfall_lstv2.append(bpinter_xicaucNORMv2_df)
                bpinterNORM_label_lstv2.append('norm_XIC AUC[Byos]')
                # onlyb norm v2
                b_xicaucNORMv2_df = onlyb.loc[:, ['nx_version2', 'N-site(SequonBased)[Byonic]', 'a_norm_XIC\r\nAUC[Byos]']]
                b_xicaucNORMv2_df.replace(to_replace = -1, value = np.nan , inplace = True)
                b_xicaucNORMv2_df = b_xicaucNORMv2_df.pivot_table(index='N-site(SequonBased)[Byonic]', values='a_norm_XIC\r\nAUC[Byos]', columns='nx_version2', aggfunc=np.sum).reindex(columns=col_order)
                bNORM_dfall_lstv2.append(b_xicaucNORMv2_df)
                bNORM_label_lstv2.append('norm_XIC AUC[Byos]')
                # onlyb sum v2
                b_xicaucSUMv2_df = onlyb.loc[:, ['nx_version2', 'N-site(SequonBased)[Byonic]', 'e_sum_XIC\r\nAUC[Byos]']]
                b_xicaucSUMv2_df.replace(to_replace = -1, value = np.nan , inplace = True)
                b_xicaucSUMv2_df = b_xicaucSUMv2_df.pivot_table(index='N-site(SequonBased)[Byonic]', values='e_sum_XIC\r\nAUC[Byos]', columns='nx_version2', aggfunc=np.sum).reindex(columns=col_order)
                b_xicaucSUMv2_df_nsum = [b_xicaucSUMv2_df.loc[i].sum() for i in b_xicaucSUMv2_df.index.tolist()] # for A bar label
                bSUM_dfall_lstv2.append(b_xicaucSUMv2_df)
                bSUM_label_lstv2.append('sum_XIC AUC[Byos]')
                bSUM_nsum_lstv2.append(b_xicaucSUMv2_df_nsum)

            if plot_v3 == 'yes':
                # bpuion norm v3
                bpunion_xicaucNORMv3_df = bp_union.loc[:, ['N(x)H(y)', 'N-site(Byonic ∪ pGlyco) →', 'a_norm_XIC\r\nAUC[Byos]']]
                bpunion_xicaucNORMv3_df.replace(to_replace = -1, value = np.nan , inplace = True)
                bpunion_xicaucNORMv3_df = bpunion_xicaucNORMv3_df.pivot_table(index='N-site(Byonic ∪ pGlyco) →', values='a_norm_XIC\r\nAUC[Byos]', columns='N(x)H(y)', aggfunc=np.sum).reindex(columns=union_nxhy_fix_order)
                bpunionNORM_dfall_lstv3.append(bpunion_xicaucNORMv3_df)
                bpunionNORM_label_lstv3.append('norm_XIC AUC[Byos]')
                # bptop10 norm v3
                bptop10_xicaucNORMv3_df = bptop10.loc[:, ['N(x)H(y)', f'N-site(Byonic ∪ pGlyco [Top{topN}]) →', 'a_norm_XIC\r\nAUC[Byos]']]
                bptop10_xicaucNORMv3_df.replace(to_replace = -1, value = np.nan , inplace = True)
                bptop10_xicaucNORMv3_df = bptop10_xicaucNORMv3_df.pivot_table(index=f'N-site(Byonic ∪ pGlyco [Top{topN}]) →', values='a_norm_XIC\r\nAUC[Byos]', columns='N(x)H(y)', aggfunc=np.sum).reindex(columns=top10_nxhy_fix_order)
                bptop10NORM_dfall_lstv3.append(bptop10_xicaucNORMv3_df)
                bptop10NORM_label_lstv3.append('norm_XIC AUC[Byos]')
                # bpinter norm v3
                bpinter_xicaucNORMv3_df = bp_inter.loc[:, ['N(x)H(y)', 'N-site(Byonic ∩ pGlyco [Glycan B=P]) →', 'a_norm_XIC\r\nAUC[Byos]']]
                bpinter_xicaucNORMv3_df.replace(to_replace = -1, value = np.nan , inplace = True)
                bpinter_xicaucNORMv3_df = bpinter_xicaucNORMv3_df.pivot_table(index='N-site(Byonic ∩ pGlyco [Glycan B=P]) →', values='a_norm_XIC\r\nAUC[Byos]', columns='N(x)H(y)', aggfunc=np.sum).reindex(columns=inter_nxhy_fix_order)
                bpinterNORM_dfall_lstv3.append(bpinter_xicaucNORMv3_df)
                bpinterNORM_label_lstv3.append('norm_XIC AUC[Byos]')
                # onlyb norm v3
                b_xicaucNORMv3_df = onlyb.loc[:, ['N(x)H(y)', 'N-site(SequonBased)[Byonic]', 'a_norm_XIC\r\nAUC[Byos]']]
                b_xicaucNORMv3_df.replace(to_replace = -1, value = np.nan , inplace = True)
                b_xicaucNORMv3_df = b_xicaucNORMv3_df.pivot_table(index='N-site(SequonBased)[Byonic]', values='a_norm_XIC\r\nAUC[Byos]', columns='N(x)H(y)', aggfunc=np.sum).reindex(columns=onlyb_nxhy_fix_order)
                bNORM_dfall_lstv3.append(b_xicaucNORMv3_df)
                bNORM_label_lstv3.append('norm_XIC AUC[Byos]')
                # onlyb sum v3
                b_xicaucSUMv3_df = onlyb.loc[:, ['N(x)H(y)', 'N-site(SequonBased)[Byonic]', 'e_sum_XIC\r\nAUC[Byos]']]
                b_xicaucSUMv3_df.replace(to_replace = -1, value = np.nan , inplace = True)
                b_xicaucSUMv3_df = b_xicaucSUMv3_df.pivot_table(index='N-site(SequonBased)[Byonic]', values='e_sum_XIC\r\nAUC[Byos]', columns='N(x)H(y)', aggfunc=np.sum).reindex(columns=onlyb_nxhy_fix_order)
                b_xicaucSUMv3_df_nsum = [b_xicaucSUMv3_df.loc[i].sum() for i in b_xicaucSUMv3_df.index.tolist()] # for A bar label
                bSUM_dfall_lstv3.append(b_xicaucSUMv3_df)
                bSUM_label_lstv3.append('sum_XIC AUC[Byos]')
                bSUM_nsum_lstv3.append(b_xicaucSUMv3_df_nsum)

            if plot_v4 == 'yes':
                # bpuion norm v4
                bpunion_xicaucNORMv4_df = bp_union.loc[:, ['(F)N(x)H(y)', 'N-site(Byonic ∪ pGlyco) →', 'a_norm_XIC\r\nAUC[Byos]']]
                bpunion_xicaucNORMv4_df.replace(to_replace = -1, value = np.nan , inplace = True)
                bpunion_xicaucNORMv4_df = bpunion_xicaucNORMv4_df.pivot_table(index='N-site(Byonic ∪ pGlyco) →', values='a_norm_XIC\r\nAUC[Byos]', columns='(F)N(x)H(y)', aggfunc=np.sum).reindex(columns=union_fnxhy_fix_order)
                bpunionNORM_dfall_lstv4.append(bpunion_xicaucNORMv4_df)
                bpunionNORM_label_lstv4.append('norm_XIC AUC[Byos]')
                # bptop10 norm v4
                bptop10_xicaucNORMv4_df = bptop10.loc[:, ['(F)N(x)H(y)', f'N-site(Byonic ∪ pGlyco [Top{topN}]) →', 'a_norm_XIC\r\nAUC[Byos]']]
                bptop10_xicaucNORMv4_df.replace(to_replace = -1, value = np.nan , inplace = True)
                bptop10_xicaucNORMv4_df = bptop10_xicaucNORMv4_df.pivot_table(index=f'N-site(Byonic ∪ pGlyco [Top{topN}]) →', values='a_norm_XIC\r\nAUC[Byos]', columns='(F)N(x)H(y)', aggfunc=np.sum).reindex(columns=top10_fnxhy_fix_order)
                bptop10NORM_dfall_lstv4.append(bptop10_xicaucNORMv4_df)
                bptop10NORM_label_lstv4.append('norm_XIC AUC[Byos]')
                # bpinter norm v4
                bpinter_xicaucNORMv4_df = bp_inter.loc[:, ['(F)N(x)H(y)', 'N-site(Byonic ∩ pGlyco [Glycan B=P]) →', 'a_norm_XIC\r\nAUC[Byos]']]
                bpinter_xicaucNORMv4_df.replace(to_replace = -1, value = np.nan , inplace = True)
                bpinter_xicaucNORMv4_df = bpinter_xicaucNORMv4_df.pivot_table(index='N-site(Byonic ∩ pGlyco [Glycan B=P]) →', values='a_norm_XIC\r\nAUC[Byos]', columns='(F)N(x)H(y)', aggfunc=np.sum).reindex(columns=inter_fnxhy_fix_order)
                bpinterNORM_dfall_lstv4.append(bpinter_xicaucNORMv4_df)
                bpinterNORM_label_lstv4.append('norm_XIC AUC[Byos]')
                # onlyb norm v4
                b_xicaucNORMv4_df = onlyb.loc[:, ['(F)N(x)H(y)', 'N-site(SequonBased)[Byonic]', 'a_norm_XIC\r\nAUC[Byos]']]
                b_xicaucNORMv4_df.replace(to_replace = -1, value = np.nan , inplace = True)
                b_xicaucNORMv4_df = b_xicaucNORMv4_df.pivot_table(index='N-site(SequonBased)[Byonic]', values='a_norm_XIC\r\nAUC[Byos]', columns='(F)N(x)H(y)', aggfunc=np.sum).reindex(columns=onlyb_fnxhy_fix_order)
                bNORM_dfall_lstv4.append(b_xicaucNORMv4_df)
                bNORM_label_lstv4.append('norm_XIC AUC[Byos]')
                # onlyb sum v4
                b_xicaucSUMv4_df = onlyb.loc[:, ['(F)N(x)H(y)', 'N-site(SequonBased)[Byonic]', 'e_sum_XIC\r\nAUC[Byos]']]
                b_xicaucSUMv4_df.replace(to_replace = -1, value = np.nan , inplace = True)
                b_xicaucSUMv4_df = b_xicaucSUMv4_df.pivot_table(index='N-site(SequonBased)[Byonic]', values='e_sum_XIC\r\nAUC[Byos]', columns='(F)N(x)H(y)', aggfunc=np.sum).reindex(columns=onlyb_fnxhy_fix_order)
                b_xicaucSUMv4_df_nsum = [b_xicaucSUMv4_df.loc[i].sum() for i in b_xicaucSUMv4_df.index.tolist()] # for A bar label
                bSUM_dfall_lstv4.append(b_xicaucSUMv4_df)
                bSUM_label_lstv4.append('sum_XIC AUC[Byos]')
                bSUM_nsum_lstv4.append(b_xicaucSUMv4_df_nsum)    
        else:
            pass
            
        if export_int == 'yes': 
            if plot_v1 == 'yes':
                # bp union norm v1
                bpunion_intNORM_df = bp_union.loc[:, ['N(x) ↓', 'N-site(Byonic ∪ pGlyco) →', 'b_norm_Apex Int.\r\n(Posit)[Byos]']] 
                bpunion_intNORM_df.replace(to_replace = -1, value = np.nan , inplace = True)
                bpunion_intNORM_df = bpunion_intNORM_df.pivot_table(index='N-site(Byonic ∪ pGlyco) →', values='b_norm_Apex Int.\r\n(Posit)[Byos]', columns='N(x) ↓', aggfunc=np.sum).reindex(columns=bp_union_new_nx)
                bpunionNORM_dfall_lstv1.append(bpunion_intNORM_df)
                bpunionNORM_label_lstv1.append('norm_Apex Int.(Posit)[Byos]')
                # bptop10 norm v1
                bptop10_intNORM_df = bptop10.loc[:, ['N(x) ↓', f'N-site(Byonic ∪ pGlyco [Top{topN}]) →', 'b_norm_Apex Int.\r\n(Posit)[Byos]']] 
                bptop10_intNORM_df.replace(to_replace = -1, value = np.nan , inplace = True)
                bptop10_intNORM_df = bptop10_intNORM_df.pivot_table(index=f'N-site(Byonic ∪ pGlyco [Top{topN}]) →', values='b_norm_Apex Int.\r\n(Posit)[Byos]', columns='N(x) ↓', aggfunc=np.sum).reindex(columns=bptop10_new_nx)
                bptop10NORM_dfall_lstv1.append(bptop10_intNORM_df)
                bptop10NORM_label_lstv1.append('norm_Apex Int.(Posit)[Byos]')
                # bp inter norm v1
                bpinter_intNORM_df = bp_inter.loc[:, ['N(x) ↓', 'N-site(Byonic ∩ pGlyco [Glycan B=P]) →', 'b_norm_Apex Int.\r\n(Posit)[Byos]']]
                bpinter_intNORM_df.replace(to_replace = -1, value = np.nan , inplace = True)
                bpinter_intNORM_df = bpinter_intNORM_df.pivot_table(index='N-site(Byonic ∩ pGlyco [Glycan B=P]) →', values='b_norm_Apex Int.\r\n(Posit)[Byos]', columns='N(x) ↓', aggfunc=np.sum).reindex(columns=bp_intersection_new_nx)
                bpinterNORM_dfall_lstv1.append(bpinter_intNORM_df)
                bpinterNORM_label_lstv1.append('norm_Apex Int.(Posit)[Byos]')
                # onlyb norm v1 
                b_intNORM_df = onlyb.loc[:, ['N(x) ↓', 'N-site(SequonBased)[Byonic]', 'b_norm_Apex Int.\r\n(Posit)[Byos]']] 
                b_intNORM_df.replace(to_replace = -1, value = np.nan , inplace = True)
                b_intNORM_df = b_intNORM_df.pivot_table(index='N-site(SequonBased)[Byonic]', values='b_norm_Apex Int.\r\n(Posit)[Byos]', columns='N(x) ↓', aggfunc=np.sum).reindex(columns=b_new_nx)
                bNORM_dfall_lstv1.append(b_intNORM_df)
                bNORM_label_lstv1.append('norm_Apex Int.(Posit)[Byos]')
                # onlyb sum v1
                b_intSUM_df = onlyb.loc[:, ['N(x) ↓', 'N-site(SequonBased)[Byonic]', 'f_sum_Apex Int.\r\n(Posit)[Byos]']] 
                b_intSUM_df.replace(to_replace = -1, value = np.nan , inplace = True)
                b_intSUM_df = b_intSUM_df.pivot_table(index='N-site(SequonBased)[Byonic]', values='f_sum_Apex Int.\r\n(Posit)[Byos]', columns='N(x) ↓', aggfunc=np.sum).reindex(columns=b_new_nx)
                b_intSUM_df_nsum = [b_intSUM_df.loc[i].sum() for i in b_intSUM_df.index.tolist()] # for B bar label
                bSUM_dfall_lstv1.append(b_intSUM_df)
                bSUM_label_lstv1.append('sum_Apex Int.(Posit)[Byos]')
                bSUM_nsum_lstv1.append(b_intSUM_df_nsum)
            if plot_v2 == 'yes':
                # bp union norm v2
                bpunion_intNORMv2_df = bp_union.loc[:, ['nx_version2', 'N-site(Byonic ∪ pGlyco) →', 'b_norm_Apex Int.\r\n(Posit)[Byos]']]
                bpunion_intNORMv2_df.replace(to_replace = -1, value = np.nan , inplace = True)
                bpunion_intNORMv2_df = bpunion_intNORMv2_df.pivot_table(index='N-site(Byonic ∪ pGlyco) →', values='b_norm_Apex Int.\r\n(Posit)[Byos]', columns='nx_version2', aggfunc=np.sum).reindex(columns=col_order)
                bpunionNORM_dfall_lstv2.append(bpunion_intNORMv2_df)
                bpunionNORM_label_lstv2.append('norm_Apex Int.(Posit)[Byos]')
                # bptop10 norm v2
                bptop10_intNORMv2_df = bptop10.loc[:, ['nx_version2', f'N-site(Byonic ∪ pGlyco [Top{topN}]) →', 'b_norm_Apex Int.\r\n(Posit)[Byos]']]
                bptop10_intNORMv2_df.replace(to_replace = -1, value = np.nan , inplace = True)
                bptop10_intNORMv2_df = bptop10_intNORMv2_df.pivot_table(index=f'N-site(Byonic ∪ pGlyco [Top{topN}]) →', values='b_norm_Apex Int.\r\n(Posit)[Byos]', columns='nx_version2', aggfunc=np.sum).reindex(columns=col_order)
                bptop10NORM_dfall_lstv2.append(bptop10_intNORMv2_df)
                bptop10NORM_label_lstv2.append('norm_Apex Int.(Posit)[Byos]')
                # bp inter norm v2
                bpinter_intNORMv2_df = bp_inter.loc[:, ['nx_version2', 'N-site(Byonic ∩ pGlyco [Glycan B=P]) →', 'b_norm_Apex Int.\r\n(Posit)[Byos]']]
                bpinter_intNORMv2_df.replace(to_replace = -1, value = np.nan , inplace = True)
                bpinter_intNORMv2_df = bpinter_intNORMv2_df.pivot_table(index='N-site(Byonic ∩ pGlyco [Glycan B=P]) →', values='b_norm_Apex Int.\r\n(Posit)[Byos]', columns='nx_version2', aggfunc=np.sum).reindex(columns=col_order)
                bpinterNORM_dfall_lstv2.append(bpinter_intNORMv2_df)
                bpinterNORM_label_lstv2.append('norm_Apex Int.(Posit)[Byos]')
                # onlyb norm v2
                b_intNORMv2_df = onlyb.loc[:, ['nx_version2', 'N-site(SequonBased)[Byonic]', 'b_norm_Apex Int.\r\n(Posit)[Byos]']] 
                b_intNORMv2_df.replace(to_replace = -1, value = np.nan , inplace = True)
                b_intNORMv2_df = b_intNORMv2_df.pivot_table(index='N-site(SequonBased)[Byonic]', values='b_norm_Apex Int.\r\n(Posit)[Byos]', columns='nx_version2', aggfunc=np.sum).reindex(columns=col_order)
                bNORM_dfall_lstv2.append(b_intNORMv2_df)
                bNORM_label_lstv2.append('norm_Apex Int.(Posit)[Byos]')
                # onlyb sum v2
                b_intSUMv2_df = onlyb.loc[:, ['nx_version2', 'N-site(SequonBased)[Byonic]', 'f_sum_Apex Int.\r\n(Posit)[Byos]']] 
                b_intSUMv2_df.replace(to_replace = -1, value = np.nan , inplace = True)
                b_intSUMv2_df = b_intSUMv2_df.pivot_table(index='N-site(SequonBased)[Byonic]', values='f_sum_Apex Int.\r\n(Posit)[Byos]', columns='nx_version2', aggfunc=np.sum).reindex(columns=col_order)
                b_intSUMv2_df_nsum = [b_intSUMv2_df.loc[i].sum() for i in b_intSUMv2_df.index.tolist()] # for B bar label
                bSUM_dfall_lstv2.append(b_intSUMv2_df)
                bSUM_label_lstv2.append('sum_Apex Int.(Posit)[Byos]')
                bSUM_nsum_lstv2.append(b_intSUMv2_df_nsum)
            if plot_v3 == 'yes':
                bpunion_intNORMv3_df = bp_union.loc[:, ['N(x)H(y)', 'N-site(Byonic ∪ pGlyco) →', 'b_norm_Apex Int.\r\n(Posit)[Byos]']]
                bpunion_intNORMv3_df.replace(to_replace = -1, value = np.nan , inplace = True)
                bpunion_intNORMv3_df = bpunion_intNORMv3_df.pivot_table(index='N-site(Byonic ∪ pGlyco) →', values='b_norm_Apex Int.\r\n(Posit)[Byos]', columns='N(x)H(y)', aggfunc=np.sum).reindex(columns=union_nxhy_fix_order)
                bpunionNORM_dfall_lstv3.append(bpunion_intNORMv3_df)
                bpunionNORM_label_lstv3.append('norm_Apex Int.(Posit)[Byos]')
                # bptop10 norm v3
                bptop10_intNORMv3_df = bptop10.loc[:, ['N(x)H(y)', f'N-site(Byonic ∪ pGlyco [Top{topN}]) →', 'b_norm_Apex Int.\r\n(Posit)[Byos]']]
                bptop10_intNORMv3_df.replace(to_replace = -1, value = np.nan , inplace = True)
                bptop10_intNORMv3_df = bptop10_intNORMv3_df.pivot_table(index=f'N-site(Byonic ∪ pGlyco [Top{topN}]) →', values='b_norm_Apex Int.\r\n(Posit)[Byos]', columns='N(x)H(y)', aggfunc=np.sum).reindex(columns=top10_nxhy_fix_order)
                bptop10NORM_dfall_lstv3.append(bptop10_intNORMv3_df)
                bptop10NORM_label_lstv3.append('norm_Apex Int.(Posit)[Byos]')
                # bpinter norm v3
                bpinter_intNORMv3_df = bp_inter.loc[:, ['N(x)H(y)', 'N-site(Byonic ∩ pGlyco [Glycan B=P]) →', 'b_norm_Apex Int.\r\n(Posit)[Byos]']]
                bpinter_intNORMv3_df.replace(to_replace = -1, value = np.nan , inplace = True)
                bpinter_intNORMv3_df = bpinter_intNORMv3_df.pivot_table(index='N-site(Byonic ∩ pGlyco [Glycan B=P]) →', values='b_norm_Apex Int.\r\n(Posit)[Byos]', columns='N(x)H(y)', aggfunc=np.sum).reindex(columns=inter_nxhy_fix_order)
                bpinterNORM_dfall_lstv3.append(bpinter_intNORMv3_df)
                bpinterNORM_label_lstv3.append('norm_Apex Int.(Posit)[Byos]')
                # onlyb norm v3
                b_intNORMv3_df = onlyb.loc[:, ['N(x)H(y)', 'N-site(SequonBased)[Byonic]', 'b_norm_Apex Int.\r\n(Posit)[Byos]']]
                b_intNORMv3_df.replace(to_replace = -1, value = np.nan , inplace = True)
                b_intNORMv3_df = b_intNORMv3_df.pivot_table(index='N-site(SequonBased)[Byonic]', values='b_norm_Apex Int.\r\n(Posit)[Byos]', columns='N(x)H(y)', aggfunc=np.sum).reindex(columns=onlyb_nxhy_fix_order)
                bNORM_dfall_lstv3.append(b_intNORMv3_df)
                bNORM_label_lstv3.append('norm_Apex Int.(Posit)[Byos]')
                # onlyb sum v3
                b_intSUMv3_df = onlyb.loc[:, ['N(x)H(y)', 'N-site(SequonBased)[Byonic]', 'f_sum_Apex Int.\r\n(Posit)[Byos]']]
                b_intSUMv3_df.replace(to_replace = -1, value = np.nan , inplace = True)
                b_intSUMv3_df = b_intSUMv3_df.pivot_table(index='N-site(SequonBased)[Byonic]', values='f_sum_Apex Int.\r\n(Posit)[Byos]', columns='N(x)H(y)', aggfunc=np.sum).reindex(columns=onlyb_nxhy_fix_order)
                b_intSUMv3_df_nsum = [b_intSUMv3_df.loc[i].sum() for i in b_intSUMv3_df.index.tolist()] # for A bar label
                bSUM_dfall_lstv3.append(b_intSUMv3_df)
                bSUM_label_lstv3.append('sum_Apex Int.(Posit)[Byos]')
                bSUM_nsum_lstv3.append(b_intSUMv3_df_nsum)
            if plot_v4 == 'yes':
                # bpuion norm v4
                bpunion_intNORMv4_df = bp_union.loc[:, ['(F)N(x)H(y)', 'N-site(Byonic ∪ pGlyco) →', 'b_norm_Apex Int.\r\n(Posit)[Byos]']]
                bpunion_intNORMv4_df.replace(to_replace = -1, value = np.nan , inplace = True)
                bpunion_intNORMv4_df = bpunion_intNORMv4_df.pivot_table(index='N-site(Byonic ∪ pGlyco) →', values='b_norm_Apex Int.\r\n(Posit)[Byos]', columns='(F)N(x)H(y)', aggfunc=np.sum).reindex(columns=union_fnxhy_fix_order)
                bpunionNORM_dfall_lstv4.append(bpunion_intNORMv4_df)
                bpunionNORM_label_lstv4.append('norm_Apex Int.(Posit)[Byos]')
                # bptop10 norm v4
                bptop10_intNORMv4_df = bptop10.loc[:, ['(F)N(x)H(y)', f'N-site(Byonic ∪ pGlyco [Top{topN}]) →', 'b_norm_Apex Int.\r\n(Posit)[Byos]']]
                bptop10_intNORMv4_df.replace(to_replace = -1, value = np.nan , inplace = True)
                bptop10_intNORMv4_df = bptop10_intNORMv4_df.pivot_table(index=f'N-site(Byonic ∪ pGlyco [Top{topN}]) →', values='b_norm_Apex Int.\r\n(Posit)[Byos]', columns='(F)N(x)H(y)', aggfunc=np.sum).reindex(columns=top10_fnxhy_fix_order)
                bptop10NORM_dfall_lstv4.append(bptop10_intNORMv4_df)
                bptop10NORM_label_lstv4.append('norm_Apex Int.(Posit)[Byos]')
                # bpinter norm v4
                bpinter_intNORMv4_df = bp_inter.loc[:, ['(F)N(x)H(y)', 'N-site(Byonic ∩ pGlyco [Glycan B=P]) →', 'b_norm_Apex Int.\r\n(Posit)[Byos]']]
                bpinter_intNORMv4_df.replace(to_replace = -1, value = np.nan , inplace = True)
                bpinter_intNORMv4_df = bpinter_intNORMv4_df.pivot_table(index='N-site(Byonic ∩ pGlyco [Glycan B=P]) →', values='b_norm_Apex Int.\r\n(Posit)[Byos]', columns='(F)N(x)H(y)', aggfunc=np.sum).reindex(columns=inter_fnxhy_fix_order)
                bpinterNORM_dfall_lstv4.append(bpinter_intNORMv4_df)
                bpinterNORM_label_lstv4.append('norm_Apex Int.(Posit)[Byos]')
                # onlyb norm v4
                b_intNORMv4_df = onlyb.loc[:, ['(F)N(x)H(y)', 'N-site(SequonBased)[Byonic]', 'b_norm_Apex Int.\r\n(Posit)[Byos]']]
                b_intNORMv4_df.replace(to_replace = -1, value = np.nan , inplace = True)
                b_intNORMv4_df = b_intNORMv4_df.pivot_table(index='N-site(SequonBased)[Byonic]', values='b_norm_Apex Int.\r\n(Posit)[Byos]', columns='(F)N(x)H(y)', aggfunc=np.sum).reindex(columns=onlyb_fnxhy_fix_order)
                bNORM_dfall_lstv4.append(b_intNORMv4_df)
                bNORM_label_lstv4.append('norm_Apex Int.(Posit)[Byos]')
                # onlyb sum v4
                b_intSUMv4_df = onlyb.loc[:, ['(F)N(x)H(y)', 'N-site(SequonBased)[Byonic]', 'f_sum_Apex Int.\r\n(Posit)[Byos]']]
                b_intSUMv4_df.replace(to_replace = -1, value = np.nan , inplace = True)
                b_intSUMv4_df = b_intSUMv4_df.pivot_table(index='N-site(SequonBased)[Byonic]', values='f_sum_Apex Int.\r\n(Posit)[Byos]', columns='(F)N(x)H(y)', aggfunc=np.sum).reindex(columns=onlyb_fnxhy_fix_order)
                b_intSUMv4_df_nsum = [b_intSUMv4_df.loc[i].sum() for i in b_intSUMv4_df.index.tolist()] # for A bar label
                bSUM_dfall_lstv4.append(b_intSUMv4_df)
                bSUM_label_lstv4.append('sum_Apex Int.(Posit)[Byos]')
                bSUM_nsum_lstv4.append(b_intSUMv4_df_nsum)       
        else:
            pass

        if export_mono == 'yes':
            if plot_v1 == 'yes':
                # bp union norm v1
                bpunion_monoNORM_df = bp_union.loc[:, ['N(x) ↓', 'N-site(Byonic ∪ pGlyco) →', 'c_norm_MonoArea[pGlyco]']]
                bpunion_monoNORM_df.replace(to_replace = -1, value = np.nan , inplace = True)
                bpunion_monoNORM_df = bpunion_monoNORM_df.pivot_table(index='N-site(Byonic ∪ pGlyco) →', values='c_norm_MonoArea[pGlyco]', columns='N(x) ↓', aggfunc=np.sum).reindex(columns=bp_union_new_nx)
                bpunionNORM_dfall_lstv1.append(bpunion_monoNORM_df)
                bpunionNORM_label_lstv1.append('norm_MonoArea[pGlyco]')
                # bptop10 norm v1
                bptop10_monoNORM_df = bptop10.loc[:, ['N(x) ↓', f'N-site(Byonic ∪ pGlyco [Top{topN}]) →', 'c_norm_MonoArea[pGlyco]']]
                bptop10_monoNORM_df.replace(to_replace = -1, value = np.nan , inplace = True)
                bptop10_monoNORM_df = bptop10_monoNORM_df.pivot_table(index=f'N-site(Byonic ∪ pGlyco [Top{topN}]) →', values='c_norm_MonoArea[pGlyco]', columns='N(x) ↓', aggfunc=np.sum).reindex(columns=bptop10_new_nx)
                bptop10NORM_dfall_lstv1.append(bptop10_monoNORM_df)
                bptop10NORM_label_lstv1.append('norm_MonoArea[pGlyco]')
                # bp inter norm v1
                bpinter_monoNORM_df = bp_inter.loc[:, ['N(x) ↓', 'N-site(Byonic ∩ pGlyco [Glycan B=P]) →', 'c_norm_MonoArea[pGlyco]']]
                bpinter_monoNORM_df.replace(to_replace = -1, value = np.nan , inplace = True)
                bpinter_monoNORM_df = bpinter_monoNORM_df.pivot_table(index='N-site(Byonic ∩ pGlyco [Glycan B=P]) →', values='c_norm_MonoArea[pGlyco]', columns='N(x) ↓', aggfunc=np.sum).reindex(columns=bp_intersection_new_nx)
                bpinterNORM_dfall_lstv1.append(bpinter_monoNORM_df)
                bpinterNORM_label_lstv1.append('norm_MonoArea[pGlyco]')
                # onlyp norm v1
                p_monoNORM_df = onlyp.loc[:, ['N(x) ↓', 'ProSites[pGlyco]', 'c_norm_MonoArea[pGlyco]']]
                p_monoNORM_df.replace(to_replace = -1, value = np.nan , inplace = True)
                p_monoNORM_df = p_monoNORM_df.pivot_table(index='ProSites[pGlyco]', values='c_norm_MonoArea[pGlyco]', columns='N(x) ↓', aggfunc=np.sum).reindex(columns=p_new_nx)
                pNORM_dfall_lstv1.append(p_monoNORM_df)
                pNORM_label_lstv1.append('norm_MonoArea[pGlyco]')
                # onlyp sum v1
                p_monoSUM_df = onlyp.loc[:, ['N(x) ↓', 'ProSites[pGlyco]', 'g_sum_MonoArea[pGlyco]']]
                p_monoSUM_df.replace(to_replace = -1, value = np.nan , inplace = True)
                p_monoSUM_df = p_monoSUM_df.pivot_table(index='ProSites[pGlyco]', values='g_sum_MonoArea[pGlyco]', columns='N(x) ↓', aggfunc=np.sum).reindex(columns=p_new_nx)
                p_monoSUM_df_nsum = [p_monoSUM_df.loc[i].sum() for i in p_monoSUM_df.index.tolist()] # for A bar label
                pSUM_dfall_lstv1.append(p_monoSUM_df)
                pSUM_label_lstv1.append('sum_MonoArea[pGlyco]')
                pSUM_nsum_lstv1.append(p_monoSUM_df_nsum)

            if plot_v2 == 'yes':
                # bp union norm v2
                bpunion_monoNORMv2_df = bp_union.loc[:, ['nx_version2', 'N-site(Byonic ∪ pGlyco) →', 'c_norm_MonoArea[pGlyco]']]
                bpunion_monoNORMv2_df.replace(to_replace = -1, value = np.nan , inplace = True)
                bpunion_monoNORMv2_df = bpunion_monoNORMv2_df.pivot_table(index='N-site(Byonic ∪ pGlyco) →', values='c_norm_MonoArea[pGlyco]', columns='nx_version2', aggfunc=np.sum).reindex(columns=col_order)
                bpunionNORM_dfall_lstv2.append(bpunion_monoNORMv2_df)
                bpunionNORM_label_lstv2.append('norm_MonoArea[pGlyco]')
                # bptop10 norm v2
                bptop10_monoNORMv2_df = bptop10.loc[:, ['nx_version2', f'N-site(Byonic ∪ pGlyco [Top{topN}]) →', 'c_norm_MonoArea[pGlyco]']]
                bptop10_monoNORMv2_df.replace(to_replace = -1, value = np.nan , inplace = True)
                bptop10_monoNORMv2_df = bptop10_monoNORMv2_df.pivot_table(index=f'N-site(Byonic ∪ pGlyco [Top{topN}]) →', values='c_norm_MonoArea[pGlyco]', columns='nx_version2', aggfunc=np.sum).reindex(columns=col_order)
                bptop10NORM_dfall_lstv2.append(bptop10_monoNORMv2_df)
                bptop10NORM_label_lstv2.append('norm_MonoArea[pGlyco]')
                # bp inter norm v2
                bpinter_monoNORMv2_df = bp_inter.loc[:, ['nx_version2', 'N-site(Byonic ∩ pGlyco [Glycan B=P]) →', 'c_norm_MonoArea[pGlyco]']]
                bpinter_monoNORMv2_df.replace(to_replace = -1, value = np.nan , inplace = True)
                bpinter_monoNORMv2_df = bpinter_monoNORMv2_df.pivot_table(index='N-site(Byonic ∩ pGlyco [Glycan B=P]) →', values='c_norm_MonoArea[pGlyco]', columns='nx_version2', aggfunc=np.sum).reindex(columns=col_order)
                bpinterNORM_dfall_lstv2.append(bpinter_monoNORMv2_df)
                bpinterNORM_label_lstv2.append('norm_MonoArea[pGlyco]')
                # onlyp norm v2
                p_monoNORMv2_df = onlyp.loc[:, ['nx_version2', 'ProSites[pGlyco]', 'c_norm_MonoArea[pGlyco]']]
                p_monoNORMv2_df.replace(to_replace = -1, value = np.nan , inplace = True)
                p_monoNORMv2_df = p_monoNORMv2_df.pivot_table(index='ProSites[pGlyco]', values='c_norm_MonoArea[pGlyco]', columns='nx_version2', aggfunc=np.sum).reindex(columns=col_order)
                pNORM_dfall_lstv2.append(p_monoNORMv2_df)
                pNORM_label_lstv2.append('norm_MonoArea[pGlyco]')
                # onlyp sum v2
                p_monoSUMv2_df = onlyp.loc[:, ['nx_version2', 'ProSites[pGlyco]', 'g_sum_MonoArea[pGlyco]']]
                p_monoSUMv2_df.replace(to_replace = -1, value = np.nan , inplace = True)
                p_monoSUMv2_df = p_monoSUMv2_df.pivot_table(index='ProSites[pGlyco]', values='g_sum_MonoArea[pGlyco]', columns='nx_version2', aggfunc=np.sum).reindex(columns=col_order)
                p_monoSUMv2_df_nsum = [p_monoSUMv2_df.loc[i].sum() for i in p_monoSUMv2_df.index.tolist()] # for A bar label
                pSUM_dfall_lstv2.append(p_monoSUMv2_df)
                pSUM_label_lstv2.append('sum_MonoArea[pGlyco]')
                pSUM_nsum_lstv2.append(p_monoSUMv2_df_nsum)

            if plot_v3 == 'yes':
                bpunion_monoNORMv3_df = bp_union.loc[:, ['N(x)H(y)', 'N-site(Byonic ∪ pGlyco) →', 'c_norm_MonoArea[pGlyco]']]
                bpunion_monoNORMv3_df.replace(to_replace = -1, value = np.nan , inplace = True)
                bpunion_monoNORMv3_df = bpunion_monoNORMv3_df.pivot_table(index='N-site(Byonic ∪ pGlyco) →', values='c_norm_MonoArea[pGlyco]', columns='N(x)H(y)', aggfunc=np.sum).reindex(columns=union_nxhy_fix_order)
                bpunionNORM_dfall_lstv3.append(bpunion_monoNORMv3_df)
                bpunionNORM_label_lstv3.append('norm_MonoArea[pGlyco]')
                # bptop10 norm v3
                bptop10_monoNORMv3_df = bptop10.loc[:, ['N(x)H(y)', f'N-site(Byonic ∪ pGlyco [Top{topN}]) →', 'c_norm_MonoArea[pGlyco]']]
                bptop10_monoNORMv3_df.replace(to_replace = -1, value = np.nan , inplace = True)
                bptop10_monoNORMv3_df = bptop10_monoNORMv3_df.pivot_table(index=f'N-site(Byonic ∪ pGlyco [Top{topN}]) →', values='c_norm_MonoArea[pGlyco]', columns='N(x)H(y)', aggfunc=np.sum).reindex(columns=top10_nxhy_fix_order)
                bptop10NORM_dfall_lstv3.append(bptop10_monoNORMv3_df)
                bptop10NORM_label_lstv3.append('norm_MonoArea[pGlyco]')
                # bpinter norm v3
                bpinter_monoNORMv3_df = bp_inter.loc[:, ['N(x)H(y)', 'N-site(Byonic ∩ pGlyco [Glycan B=P]) →', 'c_norm_MonoArea[pGlyco]']]
                bpinter_monoNORMv3_df.replace(to_replace = -1, value = np.nan , inplace = True)
                bpinter_monoNORMv3_df = bpinter_monoNORMv3_df.pivot_table(index='N-site(Byonic ∩ pGlyco [Glycan B=P]) →', values='c_norm_MonoArea[pGlyco]', columns='N(x)H(y)', aggfunc=np.sum).reindex(columns=inter_nxhy_fix_order)
                bpinterNORM_dfall_lstv3.append(bpinter_monoNORMv3_df)
                bpinterNORM_label_lstv3.append('norm_MonoArea[pGlyco]')
                # onlyp norm v3
                p_monoNORMv3_df = onlyb.loc[:, ['N(x)H(y)', 'ProSites[pGlyco]', 'c_norm_MonoArea[pGlyco]']]
                p_monoNORMv3_df.replace(to_replace = -1, value = np.nan , inplace = True)
                p_monoNORMv3_df = p_monoNORMv3_df.pivot_table(index='ProSites[pGlyco]', values='c_norm_MonoArea[pGlyco]', columns='N(x)H(y)', aggfunc=np.sum).reindex(columns=onlyp_nxhy_fix_order)
                pNORM_dfall_lstv3.append(p_monoNORMv3_df)
                pNORM_label_lstv3.append('norm_MonoArea[pGlyco]')
                # onlyp sum v3
                p_monoSUMv3_df = onlyp.loc[:, ['N(x)H(y)', 'ProSites[pGlyco]', 'g_sum_MonoArea[pGlyco]']]
                p_monoSUMv3_df.replace(to_replace = -1, value = np.nan , inplace = True)
                p_monoSUMv3_df = p_monoSUMv3_df.pivot_table(index='ProSites[pGlyco]', values='g_sum_MonoArea[pGlyco]', columns='N(x)H(y)', aggfunc=np.sum).reindex(columns=onlyp_nxhy_fix_order)
                p_monoSUMv3_df_nsum = [p_monoSUMv3_df.loc[i].sum() for i in p_monoSUMv3_df.index.tolist()] # for A bar label
                pSUM_dfall_lstv3.append(p_monoSUMv3_df)
                pSUM_label_lstv3.append('sum_MonoArea[pGlyco]')
                pSUM_nsum_lstv3.append(p_monoSUMv3_df_nsum)

            if plot_v4 == 'yes':
                # bpuion norm v4
                bpunion_monoNORMv4_df = bp_union.loc[:, ['(F)N(x)H(y)', 'N-site(Byonic ∪ pGlyco) →', 'c_norm_MonoArea[pGlyco]']]
                bpunion_monoNORMv4_df.replace(to_replace = -1, value = np.nan , inplace = True)
                bpunion_monoNORMv4_df = bpunion_monoNORMv4_df.pivot_table(index='N-site(Byonic ∪ pGlyco) →', values='c_norm_MonoArea[pGlyco]', columns='(F)N(x)H(y)', aggfunc=np.sum).reindex(columns=union_fnxhy_fix_order)
                bpunionNORM_dfall_lstv4.append(bpunion_monoNORMv4_df)
                bpunionNORM_label_lstv4.append('norm_MonoArea[pGlyco]')
                # bptop10 norm v4
                bptop10_monoNORMv4_df = bptop10.loc[:, ['(F)N(x)H(y)', f'N-site(Byonic ∪ pGlyco [Top{topN}]) →', 'c_norm_MonoArea[pGlyco]']]
                bptop10_monoNORMv4_df.replace(to_replace = -1, value = np.nan , inplace = True)
                bptop10_monoNORMv4_df = bptop10_monoNORMv4_df.pivot_table(index=f'N-site(Byonic ∪ pGlyco [Top{topN}]) →', values='c_norm_MonoArea[pGlyco]', columns='(F)N(x)H(y)', aggfunc=np.sum).reindex(columns=top10_fnxhy_fix_order)
                bptop10NORM_dfall_lstv4.append(bptop10_monoNORMv4_df)
                bptop10NORM_label_lstv4.append('norm_MonoArea[pGlyco]')
                # bpinter norm v4
                bpinter_monoNORMv4_df = bp_inter.loc[:, ['(F)N(x)H(y)', 'N-site(Byonic ∩ pGlyco [Glycan B=P]) →', 'c_norm_MonoArea[pGlyco]']]
                bpinter_monoNORMv4_df.replace(to_replace = -1, value = np.nan , inplace = True)
                bpinter_monoNORMv4_df = bpinter_monoNORMv4_df.pivot_table(index='N-site(Byonic ∩ pGlyco [Glycan B=P]) →', values='c_norm_MonoArea[pGlyco]', columns='(F)N(x)H(y)', aggfunc=np.sum).reindex(columns=inter_fnxhy_fix_order)
                bpinterNORM_dfall_lstv4.append(bpinter_monoNORMv4_df)
                bpinterNORM_label_lstv4.append('norm_MonoArea[pGlyco]')
                # onlyp norm v4
                p_monoNORMv4_df = onlyp.loc[:, ['(F)N(x)H(y)', 'ProSites[pGlyco]', 'c_norm_MonoArea[pGlyco]']]
                p_monoNORMv4_df.replace(to_replace = -1, value = np.nan , inplace = True)
                p_monoNORMv4_df = p_monoNORMv4_df.pivot_table(index='N-site(SequonBased)[Byonic]', values='c_norm_MonoArea[pGlyco]', columns='(F)N(x)H(y)', aggfunc=np.sum).reindex(columns=onlyp_fnxhy_fix_order)
                pNORM_dfall_lstv4.append(p_monoNORMv4_df)
                pNORM_label_lstv4.append('norm_MonoArea[pGlyco]')
                # onlyp sum v4
                p_monoSUMv4_df = onlyp.loc[:, ['(F)N(x)H(y)', 'ProSites[pGlyco]', 'g_sum_MonoArea[pGlyco]']]
                p_monoSUMv4_df.replace(to_replace = -1, value = np.nan , inplace = True)
                p_monoSUMv4_df = p_monoSUMv4_df.pivot_table(index='ProSites[pGlyco]', values='g_sum_MonoArea[pGlyco]', columns='(F)N(x)H(y)', aggfunc=np.sum).reindex(columns=onlyp_fnxhy_fix_order)
                p_monoSUMv4_df_nsum = [p_monoSUMv4_df.loc[i].sum() for i in p_monoSUMv4_df.index.tolist()] # for A bar label
                pSUM_dfall_lstv4.append(p_monoSUMv4_df)
                pSUM_label_lstv4.append('sum_MonoArea[pGlyco]')
                pSUM_nsum_lstv4.append(p_monoSUMv4_df_nsum)     
        else:
            pass

        if export_isotope == 'yes':
            if plot_v1 == 'yes':
                # bp union norm v1
                bpunion_isoNORM_df = bp_union.loc[:, ['N(x) ↓', 'N-site(Byonic ∪ pGlyco) →', 'd_norm_IsotopeArea[pGlyco]']]
                bpunion_isoNORM_df.replace(to_replace = -1, value = np.nan , inplace = True)
                bpunion_isoNORM_df = bpunion_isoNORM_df.pivot_table(index='N-site(Byonic ∪ pGlyco) →', values='d_norm_IsotopeArea[pGlyco]', columns='N(x) ↓', aggfunc=np.sum).reindex(columns=bp_union_new_nx)
                bpunionNORM_dfall_lstv1.append(bpunion_isoNORM_df)
                bpunionNORM_label_lstv1.append('norm_IsotopeArea[pGlyco]')
                # bp top10 norm v1
                bptop10_isoNORM_df = bptop10.loc[:, ['N(x) ↓', f'N-site(Byonic ∪ pGlyco [Top{topN}]) →', 'd_norm_IsotopeArea[pGlyco]']]
                bptop10_isoNORM_df.replace(to_replace = -1, value = np.nan , inplace = True)
                bptop10_isoNORM_df = bptop10_isoNORM_df.pivot_table(index=f'N-site(Byonic ∪ pGlyco [Top{topN}]) →', values='d_norm_IsotopeArea[pGlyco]', columns='N(x) ↓', aggfunc=np.sum).reindex(columns=bptop10_new_nx)
                bptop10NORM_dfall_lstv1.append(bptop10_isoNORM_df)
                bptop10NORM_label_lstv1.append('norm_IsotopeArea[pGlyco]')
                # bp inter norm v1
                bpinter_isoNORM_df = bp_inter.loc[:, ['N(x) ↓', 'N-site(Byonic ∩ pGlyco [Glycan B=P]) →', 'd_norm_IsotopeArea[pGlyco]']]
                bpinter_isoNORM_df.replace(to_replace = -1, value = np.nan , inplace = True)
                bpinter_isoNORM_df = bpinter_isoNORM_df.pivot_table(index='N-site(Byonic ∩ pGlyco [Glycan B=P]) →', values='d_norm_IsotopeArea[pGlyco]', columns='N(x) ↓', aggfunc=np.sum).reindex(columns=bp_intersection_new_nx)
                bpinterNORM_dfall_lstv1.append(bpinter_isoNORM_df)
                bpinterNORM_label_lstv1.append('norm_IsotopeArea[pGlyco]')
                # onlyp norm v1
                p_isoNORM_df = onlyp.loc[:, ['N(x) ↓', 'ProSites[pGlyco]', 'd_norm_IsotopeArea[pGlyco]']]
                p_isoNORM_df.replace(to_replace = -1, value = np.nan , inplace = True)
                p_isoNORM_df = p_isoNORM_df.pivot_table(index='ProSites[pGlyco]', values='d_norm_IsotopeArea[pGlyco]', columns='N(x) ↓', aggfunc=np.sum).reindex(columns=p_new_nx)
                pNORM_dfall_lstv1.append(p_isoNORM_df)
                pNORM_label_lstv1.append('norm_IsotopeArea[pGlyco]')
                # onlyp sum v1
                p_isoSUM_df = onlyp.loc[:, ['N(x) ↓', 'ProSites[pGlyco]', 'h_sum_IsotopeArea[pGlyco]']] 
                p_isoSUM_df.replace(to_replace = -1, value = np.nan , inplace = True)
                p_isoSUM_df = p_isoSUM_df.pivot_table(index='ProSites[pGlyco]', values='h_sum_IsotopeArea[pGlyco]', columns='N(x) ↓', aggfunc=np.sum).reindex(columns=p_new_nx)
                p_isoSUM_df_nsum = [p_isoSUM_df.loc[i].sum() for i in p_isoSUM_df.index.tolist()] # for B bar label
                pSUM_dfall_lstv1.append(p_isoSUM_df)
                pSUM_label_lstv1.append('sum_IsotopeArea[pGlyco]')
                pSUM_nsum_lstv1.append(p_isoSUM_df_nsum)

            if plot_v2 == 'yes':
                # bp union norm v2
                bpunion_isoNORMv2_df = bp_union.loc[:, ['nx_version2', 'N-site(Byonic ∪ pGlyco) →', 'd_norm_IsotopeArea[pGlyco]']]
                bpunion_isoNORMv2_df.replace(to_replace = -1, value = np.nan , inplace = True)
                bpunion_isoNORMv2_df = bpunion_isoNORMv2_df.pivot_table(index='N-site(Byonic ∪ pGlyco) →', values='d_norm_IsotopeArea[pGlyco]', columns='nx_version2', aggfunc=np.sum).reindex(columns=col_order)
                bpunionNORM_dfall_lstv2.append(bpunion_isoNORMv2_df)
                bpunionNORM_label_lstv2.append('norm_IsotopeArea[pGlyco]')
                # bp top10 norm v2
                bptop10_isoNORMv2_df = bptop10.loc[:, ['nx_version2', f'N-site(Byonic ∪ pGlyco [Top{topN}]) →', 'd_norm_IsotopeArea[pGlyco]']]
                bptop10_isoNORMv2_df.replace(to_replace = -1, value = np.nan , inplace = True)
                bptop10_isoNORMv2_df = bptop10_isoNORMv2_df.pivot_table(index=f'N-site(Byonic ∪ pGlyco [Top{topN}]) →', values='d_norm_IsotopeArea[pGlyco]', columns='nx_version2', aggfunc=np.sum).reindex(columns=col_order)
                bptop10NORM_dfall_lstv2.append(bptop10_isoNORMv2_df)
                bptop10NORM_label_lstv2.append('norm_IsotopeArea[pGlyco]')
                # bp inter norm v2
                bpinter_isoNORMv2_df = bp_inter.loc[:, ['nx_version2', 'N-site(Byonic ∩ pGlyco [Glycan B=P]) →', 'd_norm_IsotopeArea[pGlyco]']]
                bpinter_isoNORMv2_df.replace(to_replace = -1, value = np.nan , inplace = True)
                bpinter_isoNORMv2_df = bpinter_isoNORMv2_df.pivot_table(index='N-site(Byonic ∩ pGlyco [Glycan B=P]) →', values='d_norm_IsotopeArea[pGlyco]', columns='nx_version2', aggfunc=np.sum).reindex(columns=col_order)
                bpinterNORM_dfall_lstv2.append(bpinter_isoNORMv2_df)
                bpinterNORM_label_lstv2.append('norm_IsotopeArea[pGlyco]')
                # onlyp norm v2
                p_isoNORMv2_df = onlyp.loc[:, ['nx_version2', 'ProSites[pGlyco]', 'd_norm_IsotopeArea[pGlyco]']]
                p_isoNORMv2_df.replace(to_replace = -1, value = np.nan , inplace = True)
                p_isoNORMv2_df = p_isoNORMv2_df.pivot_table(index='ProSites[pGlyco]', values='d_norm_IsotopeArea[pGlyco]', columns='nx_version2', aggfunc=np.sum).reindex(columns=col_order)
                pNORM_dfall_lstv2.append(p_isoNORMv2_df)
                pNORM_label_lstv2.append('norm_IsotopeArea[pGlyco]')
                # onlyp sum v2
                p_isoSUMv2_df = onlyp.loc[:, ['nx_version2', 'ProSites[pGlyco]', 'h_sum_IsotopeArea[pGlyco]']] 
                p_isoSUMv2_df.replace(to_replace = -1, value = np.nan , inplace = True)
                p_isoSUMv2_df = p_isoSUMv2_df.pivot_table(index='ProSites[pGlyco]', values='h_sum_IsotopeArea[pGlyco]', columns='nx_version2', aggfunc=np.sum).reindex(columns=col_order)
                p_isoSUMv2_df_nsum = [p_isoSUMv2_df.loc[i].sum() for i in p_isoSUMv2_df.index.tolist()] # for B bar label
                pSUM_dfall_lstv2.append(p_isoSUMv2_df)
                pSUM_label_lstv2.append('sum_IsotopeArea[pGlyco]')
                pSUM_nsum_lstv2.append(p_isoSUMv2_df_nsum)

            if plot_v3 == 'yes':
                bpunion_isoNORMv3_df = bp_union.loc[:, ['N(x)H(y)', 'N-site(Byonic ∪ pGlyco) →', 'd_norm_IsotopeArea[pGlyco]']]
                bpunion_isoNORMv3_df.replace(to_replace = -1, value = np.nan , inplace = True)
                bpunion_isoNORMv3_df = bpunion_isoNORMv3_df.pivot_table(index='N-site(Byonic ∪ pGlyco) →', values='d_norm_IsotopeArea[pGlyco]', columns='N(x)H(y)', aggfunc=np.sum).reindex(columns=union_nxhy_fix_order)
                bpunionNORM_dfall_lstv3.append(bpunion_isoNORMv3_df)
                bpunionNORM_label_lstv3.append('norm_IsotopeArea[pGlyco]')
                # bptop10 norm v3
                bptop10_isoNORMv3_df = bptop10.loc[:, ['N(x)H(y)', f'N-site(Byonic ∪ pGlyco [Top{topN}]) →', 'd_norm_IsotopeArea[pGlyco]']]
                bptop10_isoNORMv3_df.replace(to_replace = -1, value = np.nan , inplace = True)
                bptop10_isoNORMv3_df = bptop10_isoNORMv3_df.pivot_table(index=f'N-site(Byonic ∪ pGlyco [Top{topN}]) →', values='d_norm_IsotopeArea[pGlyco]', columns='N(x)H(y)', aggfunc=np.sum).reindex(columns=top10_nxhy_fix_order)
                bptop10NORM_dfall_lstv3.append(bptop10_isoNORMv3_df)
                bptop10NORM_label_lstv3.append('norm_IsotopeArea[pGlyco]')
                # bpinter norm v3
                bpinter_isoNORMv3_df = bp_inter.loc[:, ['N(x)H(y)', 'N-site(Byonic ∩ pGlyco [Glycan B=P]) →', 'd_norm_IsotopeArea[pGlyco]']]
                bpinter_isoNORMv3_df.replace(to_replace = -1, value = np.nan , inplace = True)
                bpinter_isoNORMv3_df = bpinter_isoNORMv3_df.pivot_table(index='N-site(Byonic ∩ pGlyco [Glycan B=P]) →', values='d_norm_IsotopeArea[pGlyco]', columns='N(x)H(y)', aggfunc=np.sum).reindex(columns=inter_nxhy_fix_order)
                bpinterNORM_dfall_lstv3.append(bpinter_isoNORMv3_df)
                bpinterNORM_label_lstv3.append('norm_IsotopeArea[pGlyco]')
                # onlyp norm v3
                p_isoNORMv3_df = onlyp.loc[:, ['N(x)H(y)', 'ProSites[pGlyco]', 'd_norm_IsotopeArea[pGlyco]']]
                p_isoNORMv3_df.replace(to_replace = -1, value = np.nan , inplace = True)
                p_isoNORMv3_df = p_isoNORMv3_df.pivot_table(index='ProSites[pGlyco]', values='d_norm_IsotopeArea[pGlyco]', columns='N(x)H(y)', aggfunc=np.sum).reindex(columns=onlyp_nxhy_fix_order)
                pNORM_dfall_lstv3.append(p_isoNORMv3_df)
                pNORM_label_lstv3.append('norm_IsotopeArea[pGlyco]')
                # onlyp sum v3
                p_isoSUMv3_df = onlyp.loc[:, ['N(x)H(y)', 'ProSites[pGlyco]', 'h_sum_IsotopeArea[pGlyco]']]
                p_isoSUMv3_df.replace(to_replace = -1, value = np.nan , inplace = True)
                p_isoSUMv3_df = p_isoSUMv3_df.pivot_table(index='ProSites[pGlyco]', values='h_sum_IsotopeArea[pGlyco]', columns='N(x)H(y)', aggfunc=np.sum).reindex(columns=onlyp_nxhy_fix_order)
                p_isoSUMv3_df_nsum = [p_isoSUMv3_df.loc[i].sum() for i in p_isoSUMv3_df.index.tolist()] # for A bar label
                pSUM_dfall_lstv3.append(p_isoSUMv3_df)
                pSUM_label_lstv3.append('sum_IsotopeArea[pGlyco]')
                pSUM_nsum_lstv3.append(p_isoSUMv3_df_nsum)

            if plot_v4 == 'yes':
                # bpuion norm v4
                bpunion_isoNORMv4_df = bp_union.loc[:, ['(F)N(x)H(y)', 'N-site(Byonic ∪ pGlyco) →', 'd_norm_IsotopeArea[pGlyco]']]
                bpunion_isoNORMv4_df.replace(to_replace = -1, value = np.nan , inplace = True)
                bpunion_isoNORMv4_df = bpunion_isoNORMv4_df.pivot_table(index='N-site(Byonic ∪ pGlyco) →', values='d_norm_IsotopeArea[pGlyco]', columns='(F)N(x)H(y)', aggfunc=np.sum).reindex(columns=union_fnxhy_fix_order)
                bpunionNORM_dfall_lstv4.append(bpunion_isoNORMv4_df)
                bpunionNORM_label_lstv4.append('norm_IsotopeArea[pGlyco]')
                # bptop10 norm v4
                bptop10_isoNORMv4_df = bptop10.loc[:, ['(F)N(x)H(y)', f'N-site(Byonic ∪ pGlyco [Top{topN}]) →', 'd_norm_IsotopeArea[pGlyco]']]
                bptop10_isoNORMv4_df.replace(to_replace = -1, value = np.nan , inplace = True)
                bptop10_isoNORMv4_df = bptop10_isoNORMv4_df.pivot_table(index=f'N-site(Byonic ∪ pGlyco [Top{topN}]) →', values='d_norm_IsotopeArea[pGlyco]', columns='(F)N(x)H(y)', aggfunc=np.sum).reindex(columns=top10_fnxhy_fix_order)
                bptop10NORM_dfall_lstv4.append(bptop10_isoNORMv4_df)
                bptop10NORM_label_lstv4.append('norm_IsotopeArea[pGlyco]')
                # bpinter norm v4
                bpinter_isoNORMv4_df = bp_inter.loc[:, ['(F)N(x)H(y)', 'N-site(Byonic ∩ pGlyco [Glycan B=P]) →', 'd_norm_IsotopeArea[pGlyco]']]
                bpinter_isoNORMv4_df.replace(to_replace = -1, value = np.nan , inplace = True)
                bpinter_isoNORMv4_df = bpinter_isoNORMv4_df.pivot_table(index='N-site(Byonic ∩ pGlyco [Glycan B=P]) →', values='d_norm_IsotopeArea[pGlyco]', columns='(F)N(x)H(y)', aggfunc=np.sum).reindex(columns=inter_fnxhy_fix_order)
                bpinterNORM_dfall_lstv4.append(bpinter_isoNORMv4_df)
                bpinterNORM_label_lstv4.append('norm_IsotopeArea[pGlyco]')
                # onlyp norm v4
                p_isoNORMv4_df = onlyp.loc[:, ['(F)N(x)H(y)', 'ProSites[pGlyco]', 'd_norm_IsotopeArea[pGlyco]']]
                p_isoNORMv4_df.replace(to_replace = -1, value = np.nan , inplace = True)
                p_isoNORMv4_df = p_isoNORMv4_df.pivot_table(index='ProSites[pGlyco]', values='d_norm_IsotopeArea[pGlyco]', columns='(F)N(x)H(y)', aggfunc=np.sum).reindex(columns=onlyp_fnxhy_fix_order)
                pNORM_dfall_lstv4.append(p_isoNORMv4_df)
                pNORM_label_lstv4.append('norm_IsotopeArea[pGlyco]')
                # onlyp sum v4
                p_isoSUMv4_df = onlyp.loc[:, ['(F)N(x)H(y)', 'ProSites[pGlyco]', 'h_sum_IsotopeArea[pGlyco]']]
                p_isoSUMv4_df.replace(to_replace = -1, value = np.nan , inplace = True)
                p_isoSUMv4_df = p_isoSUMv4_df.pivot_table(index='ProSites[pGlyco]', values='h_sum_IsotopeArea[pGlyco]', columns='(F)N(x)H(y)', aggfunc=np.sum).reindex(columns=onlyp_fnxhy_fix_order)
                p_isoSUMv4_df_nsum = [p_isoSUMv4_df.loc[i].sum() for i in p_isoSUMv4_df.index.tolist()] # for A bar label
                pSUM_dfall_lstv4.append(p_isoSUMv4_df)
                pSUM_label_lstv4.append('sum_IsotopeArea[pGlyco]')
                pSUM_nsum_lstv4.append(p_isoSUMv4_df_nsum)    
        else:
            pass
        
        if export_xicauc == export_int == export_mono == export_isotope == 'no': # no plots will be exported
            pass
        else: # plotting criterium: cannot be empty list

            if bpunionNORM_dfall_lstv1 != []:
                # bp_union NORMv1
                fig = plot_clustered_stacked(bpunionNORM_dfall_lstv1, [[1]*len(bpunionNORM_dfall_lstv1[0].index) for i in range(len(bpunionNORM_dfall_lstv1))], bpunionNORM_label_lstv1)
                fig.tight_layout()
                fig.savefig(f'{date}_bpUnionNORMv1_{filename}.png', bbox_inches='tight')
            if bpunionNORM_dfall_lstv2 != []:
                # bp_union NORMv2
                fig = plot_clustered_stacked(bpunionNORM_dfall_lstv2, [[1]*len(bpunionNORM_dfall_lstv2[0].index) for i in range(len(bpunionNORM_dfall_lstv2))], bpunionNORM_label_lstv2, neugc_exist=bpunion_neugc_exist)
                fig.tight_layout()
                fig.savefig(f'{date}_bpUnionNORMv2_{filename}.png', bbox_inches='tight')
            if bpunionNORM_dfall_lstv3 != []: # detailedHighMan=True, fucosylation=False(default), hybrid=colorHybrid(default=True)
                # bp_union NORMv3
                fig = plot_clustered_stacked(bpunionNORM_dfall_lstv3, [[1]*len(bpunionNORM_dfall_lstv3[0].index) for i in range(len(bpunionNORM_dfall_lstv3))], bpunionNORM_label_lstv3, detailedHighMan=True)
                fig.tight_layout()
                fig.savefig(f'{date}_bpUnionNORMv3_{filename}.png', bbox_inches='tight')
            if bpunionNORM_dfall_lstv4 != []: # detailedHighMan=True, fucosylation=True, hybrid=colorHybrid(default=True)
                # bp_union NORMv4
                fig = plot_clustered_stacked(bpunionNORM_dfall_lstv4, [[1]*len(bpunionNORM_dfall_lstv4[0].index) for i in range(len(bpunionNORM_dfall_lstv4))], bpunionNORM_label_lstv4, detailedHighMan=True, fucosylation=True)
                fig.tight_layout()
                fig.savefig(f'{date}_bpUnionNORMv4_{filename}.png', bbox_inches='tight')


            if bptop10NORM_dfall_lstv1 != []:
                # bptop10 NORMv1
                fig = plot_clustered_stacked(bptop10NORM_dfall_lstv1, [[1]*len(bptop10NORM_dfall_lstv1[0].index) for i in range(len(bptop10NORM_dfall_lstv1))], bptop10NORM_label_lstv1)
                fig.tight_layout()
                fig.savefig(f'{date}_bptop{topN}NORMv1_{filename}.png', bbox_inches='tight')
            if bptop10NORM_dfall_lstv2 != []:
                # bptop10 NORMv2
                fig = plot_clustered_stacked(bptop10NORM_dfall_lstv2, [[1]*len(bptop10NORM_dfall_lstv2[0].index) for i in range(len(bptop10NORM_dfall_lstv2))], bptop10NORM_label_lstv2, neugc_exist=bptop10_neugc_exist)
                fig.tight_layout()
                fig.savefig(f'{date}_bptop{topN}NORMv2_{filename}.png', bbox_inches='tight')
            if bptop10NORM_dfall_lstv3 != []:
                # bptop10 NORMv3
                fig = plot_clustered_stacked(bptop10NORM_dfall_lstv3, [[1]*len(bptop10NORM_dfall_lstv3[0].index) for i in range(len(bptop10NORM_dfall_lstv3))], bptop10NORM_label_lstv3, detailedHighMan=True)
                fig.tight_layout()
                fig.savefig(f'{date}_bptop{topN}NORMv3_{filename}.png', bbox_inches='tight')
            if bptop10NORM_dfall_lstv4 != []:
                # bptop10 NORMv4
                fig = plot_clustered_stacked(bptop10NORM_dfall_lstv4, [[1]*len(bptop10NORM_dfall_lstv4[0].index) for i in range(len(bptop10NORM_dfall_lstv4))], bptop10NORM_label_lstv4, detailedHighMan=True, fucosylation=True)
                fig.tight_layout()
                fig.savefig(f'{date}_bptop{topN}NORMv4_{filename}.png', bbox_inches='tight')


            if bpinterNORM_dfall_lstv1 != []:
                # bp_intersection NORMv1
                fig = plot_clustered_stacked(bpinterNORM_dfall_lstv1, [[1]*len(bpinterNORM_dfall_lstv1[0].index) for i in range(len(bpinterNORM_dfall_lstv1))], bpinterNORM_label_lstv1)
                fig.tight_layout()
                fig.savefig(f'{date}_bpInterNORMv1_{filename}.png', bbox_inches='tight')
            if bpinterNORM_dfall_lstv2 != []:
                # bp_intersection NORMv2
                fig = plot_clustered_stacked(bpinterNORM_dfall_lstv2, [[1]*len(bpinterNORM_dfall_lstv2[0].index) for i in range(len(bpinterNORM_dfall_lstv2))], bpinterNORM_label_lstv2, neugc_exist=bpinter_neugc_exist)
                fig.tight_layout()
                fig.savefig(f'{date}_bpInterNORMv2_{filename}.png', bbox_inches='tight')
            if bpinterNORM_dfall_lstv3 != []:
                # bp_intersection NORMv3
                fig = plot_clustered_stacked(bpinterNORM_dfall_lstv3, [[1]*len(bpinterNORM_dfall_lstv3[0].index) for i in range(len(bpinterNORM_dfall_lstv3))], bpinterNORM_label_lstv3, detailedHighMan=True)
                fig.tight_layout()
                fig.savefig(f'{date}_bpInterNORMv3_{filename}.png', bbox_inches='tight')
            if bpinterNORM_dfall_lstv4 != []:
                # bp_intersection NORMv4
                fig = plot_clustered_stacked(bpinterNORM_dfall_lstv4, [[1]*len(bpinterNORM_dfall_lstv4[0].index) for i in range(len(bpinterNORM_dfall_lstv4))], bpinterNORM_label_lstv4, detailedHighMan=True, fucosylation=True)
                fig.tight_layout()
                fig.savefig(f'{date}_bpInterNORMv4_{filename}.png', bbox_inches='tight')


            if bNORM_dfall_lstv1 != []:
                # onlyb NORMv1
                fig = plot_clustered_stacked(bNORM_dfall_lstv1, [[1]*len(bNORM_dfall_lstv1[0].index) for i in range(len(bNORM_dfall_lstv1))],bNORM_label_lstv1)
                fig.tight_layout()
                fig.savefig(f'{date}_onlybNORMv1_{filename}.png', bbox_inches='tight')
            if bNORM_dfall_lstv2 != []:
                # onlyb NORMv2
                fig = plot_clustered_stacked(bNORM_dfall_lstv2, [[1]*len(bNORM_dfall_lstv2[0].index) for i in range(len(bNORM_dfall_lstv2))],bNORM_label_lstv2, neugc_exist=b_neugc_exist)
                fig.tight_layout()
                fig.savefig(f'{date}_onlybNORMv2_{filename}.png', bbox_inches='tight')
            if bNORM_dfall_lstv3 != []:
                # onlyb NORMv3
                fig = plot_clustered_stacked(bNORM_dfall_lstv3, [[1]*len(bNORM_dfall_lstv3[0].index) for i in range(len(bNORM_dfall_lstv3))],bNORM_label_lstv3, detailedHighMan=True)
                fig.tight_layout()
                fig.savefig(f'{date}_onlybNORMv3_{filename}.png', bbox_inches='tight')
            if bNORM_dfall_lstv4 != []:
                # onlyb NORMv4
                fig = plot_clustered_stacked(bNORM_dfall_lstv4, [[1]*len(bNORM_dfall_lstv4[0].index) for i in range(len(bNORM_dfall_lstv4))],bNORM_label_lstv4, detailedHighMan=True, fucosylation=True)
                fig.tight_layout()
                fig.savefig(f'{date}_onlybNORMv4_{filename}.png', bbox_inches='tight')


            if bSUM_dfall_lstv1 != []:
                # onlyb SUMv1
                fig = plot_clustered_stacked(bSUM_dfall_lstv1, bSUM_nsum_lstv1,bSUM_label_lstv1)
                fig.tight_layout()
                fig.savefig(f'{date}_onlybSUMv1_{filename}.png', bbox_inches='tight')
            if bSUM_dfall_lstv2 != []:
                # onlyb SUMv2
                fig = plot_clustered_stacked(bSUM_dfall_lstv2,bSUM_nsum_lstv2 ,bSUM_label_lstv2, neugc_exist=b_neugc_exist)
                fig.tight_layout()
                fig.savefig(f'{date}_onlybSUMv2_{filename}.png', bbox_inches='tight')
            if bSUM_dfall_lstv3 != []:
                # onlyb SUMv3
                fig = plot_clustered_stacked(bSUM_dfall_lstv3, bSUM_nsum_lstv3,bSUM_label_lstv3, detailedHighMan=True)
                fig.tight_layout()
                fig.savefig(f'{date}_onlybSUMv3_{filename}.png', bbox_inches='tight')
            if bSUM_dfall_lstv4 != []:
                # onlyb SUMv4
                fig = plot_clustered_stacked(bSUM_dfall_lstv4,bSUM_nsum_lstv4 ,bSUM_label_lstv4, detailedHighMan=True, fucosylation=True)
                fig.tight_layout()
                fig.savefig(f'{date}_onlybSUMv4_{filename}.png', bbox_inches='tight')


            if pNORM_dfall_lstv1 != []:
                # onlyp NORMv1
                fig = plot_clustered_stacked(pNORM_dfall_lstv1,[[1]*len(pNORM_dfall_lstv1[0].index) for i in range(len(pNORM_dfall_lstv1))],pNORM_label_lstv1)
                fig.tight_layout()
                fig.savefig(f'{date}_onlypNORMv1_{filename}.png', bbox_inches='tight')
            if pNORM_dfall_lstv2 != []:
                # onlyp NORMv2
                fig = plot_clustered_stacked(pNORM_dfall_lstv2,[[1]*len(pNORM_dfall_lstv2[0].index) for i in range(len(pNORM_dfall_lstv2))],pNORM_label_lstv2, neugc_exist=p_neugc_exist)
                fig.tight_layout()
                fig.savefig(f'{date}_onlypNORMv2_{filename}.png', bbox_inches='tight')
            if pNORM_dfall_lstv3 != []:
                # onlyp NORMv3
                fig = plot_clustered_stacked(pNORM_dfall_lstv3,[[1]*len(pNORM_dfall_lstv3[0].index) for i in range(len(pNORM_dfall_lstv3))],pNORM_label_lstv3, detailedHighMan=True)
                fig.tight_layout()
                fig.savefig(f'{date}_onlypNORMv3_{filename}.png', bbox_inches='tight')
            if pNORM_dfall_lstv4 != []:
                # onlyp NORMv4
                fig = plot_clustered_stacked(pNORM_dfall_lstv4,[[1]*len(pNORM_dfall_lstv4[0].index) for i in range(len(pNORM_dfall_lstv4))],pNORM_label_lstv4, detailedHighMan=True, fucosylation=True)
                fig.tight_layout()
                fig.savefig(f'{date}_onlypNORMv4_{filename}.png', bbox_inches='tight')


            if pSUM_dfall_lstv1 != []:
                # onlyp SUMv1
                fig = plot_clustered_stacked(pSUM_dfall_lstv1,pSUM_nsum_lstv1,pSUM_label_lstv1)
                fig.tight_layout()
                fig.savefig(f'{date}_onlypSUMv1_{filename}.png', bbox_inches='tight')
            if pSUM_dfall_lstv2 != []:
                # onlyp SUMv2
                fig = plot_clustered_stacked(pSUM_dfall_lstv2,pSUM_nsum_lstv2 ,pSUM_label_lstv2, neugc_exist=p_neugc_exist)
                fig.tight_layout()
                fig.savefig(f'{date}_onlypSUMv2_{filename}.png', bbox_inches='tight')
            if pSUM_dfall_lstv3 != []:
                # onlyp SUMv3
                fig = plot_clustered_stacked(pSUM_dfall_lstv3,pSUM_nsum_lstv3,pSUM_label_lstv3, detailedHighMan=True)
                fig.tight_layout()
                fig.savefig(f'{date}_onlypSUMv3_{filename}.png', bbox_inches='tight')
            if pSUM_dfall_lstv4 != []:
                # onlyp SUMv4
                fig = plot_clustered_stacked(pSUM_dfall_lstv4,pSUM_nsum_lstv4 ,pSUM_label_lstv4, detailedHighMan=True, fucosylation=True)
                fig.tight_layout()
                fig.savefig(f'{date}_onlypSUMv4_{filename}.png', bbox_inches='tight')


        if seq != '':
            print('----- Start preparing MajorGlycoformTable. -----\n')
            # prepare needed lists for the table cols (no./nsite/sequon are the indices, major glycoform is the col)
            result = findNPosBySequon(seq)
            nsite = [t[0] for t in result]
            nsite.append('')
            sequon = [t[1] for t in result]
            sequon.append('')
            no = [i+1 for i in range(len(nsite)-1)]
            no.append('Intersection/Union')
            table = [(no[i], nsite[i], sequon[i]) for i in range(len(no))]
            table = pd.DataFrame(table)
            table.columns = ['No.', 'N-site', 'Sequon']
            # rename col & extract xicauc max and its glycoform
            bp_union = bp_union.rename(columns={'Glycans ↓':'Major Glycoform(union)'}).drop_duplicates().reset_index(drop=True)
            bp_inter = bp_inter.rename(columns={'Glycans ↓':'Major Glycoform(inter)'}).drop_duplicates().reset_index(drop=True)
            # (1 OF 2 WAYS TO DETERMINE MAJOR GLYCOFORM) TABLE1: CHOOSE HIGHEST XICAUC (yu-chun's version: from quant, so man4 & other below-core are not cleaned)
            # bp_union = bp_union.groupby('N-site(Byonic ∪ pGlyco) →').apply(lambda x: x.loc[x['a_norm_XIC\r\nAUC[Byos]'].idxmax(), ['N-site(Byonic ∪ pGlyco) →', 'Major Glycoform(union)']]).reset_index(drop=True)
            bp_union_forTable1 = bp_union.groupby('N-site(Byonic ∪ pGlyco) →').apply(lambda x: x.loc[x['a_norm_XIC\r\nAUC[Byos]'].idxmax(), ['N-site(Byonic ∪ pGlyco) →', 'Major Glycoform(union)']] if x['a_norm_XIC\r\nAUC[Byos]'].mean()!=-1 else x.loc[x['d_norm_IsotopeArea[pGlyco]'].idxmax(), ['N-site(Byonic ∪ pGlyco) →', 'Major Glycoform(union)']]).reset_index(drop=True)
            bp_inter_forTable1 = bp_inter.groupby('N-site(Byonic ∩ pGlyco [Glycan B=P]) →').apply(lambda x: x.loc[x['a_norm_XIC\r\nAUC[Byos]'].idxmax(), ['N-site(Byonic ∩ pGlyco [Glycan B=P]) →', 'Major Glycoform(inter)']]).reset_index(drop=True)      
            bp_union_forTable1 = bp_union_forTable1.set_index('N-site(Byonic ∪ pGlyco) →')
            bp_inter_forTable1 = bp_inter_forTable1.set_index('N-site(Byonic ∩ pGlyco [Glycan B=P]) →')
            combined = pd.concat([bp_inter_forTable1, bp_union_forTable1], axis=1)
            combined.index.name = 'N-site'
            combined = combined.reset_index()
            combined.loc[(combined['Major Glycoform(inter)']!=combined['Major Glycoform(union)']), 'compare'] = 'color'
            dif_num = len(combined.loc[(combined['Major Glycoform(inter)'].isnull())&(combined['compare']=='color')])
            union_num = len(combined)
            inter_num = union_num - dif_num
            # map the result to the empty df
            table1 = table.merge(combined, on=['N-site'], how='outer')
            table1.loc[table1.index[-1], 'Major Glycoform(union)'] = f'{inter_num}/{union_num}'
            # style apply to union only data
            table1 = table1.fillna('').style.applymap(findUnionOnly, subset=pd.IndexSlice[(table1['compare']=='color'), 'Major Glycoform(union)'])
            table1.data = table1.data.drop('compare', axis=1)
            # (2 OF 2 WAYS TO DETERMINE MAJOR GLYCOFORM) TABLE2: CHOOSE MAJOR CLASS (HIGHMAN/HYBRID/COMPLEX/UNOCCUPIED) -> CHOOSE HIGHEST XICAUC WITHIN MAJOR CLASS (danny's version: man4 & other below-core are cleaned, just like data used for xicauc bar/pie charts)
            # clean man4 & below-core structure & add glycan classes col by ModifiedGlycanTypeAnalysis func (highman/hybrid/complex/unoccupied) 
            bp_union_forTable2 = ModifiedGlycanTypeAnalysis(bp_union, 'Major Glycoform(union)')
            bp_inter_forTable2 = ModifiedGlycanTypeAnalysis(bp_inter, 'Major Glycoform(inter)')
            bp_union_forTable2 = normalizer(bp_union_forTable2, 'N-site(Byonic ∪ pGlyco) →')
            bp_inter_forTable2 = normalizer(bp_inter_forTable2, 'N-site(Byonic ∩ pGlyco [Glycan B=P]) →')
            # bp_union_forTable2.to_excel('bp_union_forTable2_norm.xlsx')
            # bp_inter_forTable2.to_excel('bp_inter_forTable2_norm.xlsx')
            # find major class then find the xicauc / isotope area max within major class
            bp_union_forTable2 = findMajorClass(bp_union_forTable2, 'N-site(Byonic ∪ pGlyco) →', 'Major Glycoform(union)')
            bp_inter_forTable2 = findMajorClass(bp_inter_forTable2, 'N-site(Byonic ∩ pGlyco [Glycan B=P]) →', 'Major Glycoform(inter)')
            # bp_union_forTable2.to_excel('bp_union_forTable2_majorclass.xlsx')
            # bp_inter_forTable2.to_excel('bp_inter_forTable2_majorclass.xlsx')
            
            bp_union_forTable2 = bp_union_forTable2.groupby('N-site(Byonic ∪ pGlyco) →').apply(lambda x: x.loc[x['a_norm_XIC\r\nAUC[Byos]'].idxmax(), ['N-site(Byonic ∪ pGlyco) →', 'Major Glycoform(union)']] if x['a_norm_XIC\r\nAUC[Byos]'].mean()!=-1 else x.loc[x['d_norm_IsotopeArea[pGlyco]'].idxmax(), ['N-site(Byonic ∪ pGlyco) →', 'Major Glycoform(union)']]).reset_index(drop=True)
            bp_inter_forTable2 = bp_inter_forTable2.groupby('N-site(Byonic ∩ pGlyco [Glycan B=P]) →').apply(lambda x: x.loc[x['a_norm_XIC\r\nAUC[Byos]'].idxmax(), ['N-site(Byonic ∩ pGlyco [Glycan B=P]) →', 'Major Glycoform(inter)']]).reset_index(drop=True)      
    #         bp_union_forTable2.to_excel('bp_union_forTable2_gp.xlsx')
    #         bp_inter_forTable2.to_excel('bp_inter_forTable2_gp.xlsx')
            bp_union_forTable2 = bp_union_forTable2.set_index('N-site(Byonic ∪ pGlyco) →')
            bp_inter_forTable2 = bp_inter_forTable2.set_index('N-site(Byonic ∩ pGlyco [Glycan B=P]) →')
            combined = pd.concat([bp_inter_forTable2, bp_union_forTable2], axis=1)
            combined.index.name = 'N-site'
            combined = combined.reset_index()
            combined.loc[(combined['Major Glycoform(inter)']!=combined['Major Glycoform(union)']), 'compare'] = 'color'
            dif_num = len(combined.loc[(combined['Major Glycoform(inter)'].isnull())&(combined['compare']=='color')])
            union_num = len(combined)
            inter_num = union_num - dif_num
            # map the result to the empty df
            table2 = table.merge(combined, on=['N-site'], how='outer')
            table2.loc[table2.index[-1], 'Major Glycoform(union)'] = f'{inter_num}/{union_num}'
            # style apply to union only data
            table2 = table2.fillna('').style.applymap(findUnionOnly, subset=pd.IndexSlice[(table2['compare']=='color'), 'Major Glycoform(union)'])
            table2.data = table2.data.drop('compare', axis=1)

            with pd.ExcelWriter(f'{date}_MajorGlycoformTable_{filename}.xlsx') as writer:   
                table1.to_excel(writer, sheet_name = 'XICAUC max. within site', index=False)
                table2.to_excel(writer, sheet_name = 'XICAUC max. within major class', index=False)

            print('\n----- MajorGlycoformTable exported. -----\n')
    elif byonicfile != '' and byosfile == '' and pglycofile != '': # byonic + pglyco 
        print('----- Start combining Byonic & pGlyco. -----\n')
        # combined data based on 'Scan'
        byonic_scanasid = byonic_df.copy()
        new_byonic_col = [n + '[Byonic]' if n != 'Scan' else n for n in byonic_scanasid.columns]
        byonic_scanasid.columns = new_byonic_col
        pglyco_scanasid = pglyco_df.copy()
        new_pglyco_col = [n + '[pGlyco]' if n != 'Scan' else n for n in pglyco_scanasid.columns]
        pglyco_scanasid.columns = new_pglyco_col
        byonic_scanasid = byonic_scanasid.set_index('Scan')
        pglyco_scanasid = pglyco_scanasid.set_index('Scan')
        # align scan & concat (all align on row to make row number all the same)
        a1, a2 = byonic_scanasid.align(pglyco_scanasid, join = 'outer', axis = 0) # row: a1 = a2
        all_combined_df = pd.concat([a1,a2], axis = 1)
        all_combined_df.index.name = 'Scan'
        all_combined_df.reset_index(level=0, inplace=True)
        # change all nan to blank -1
        all_combined_df = all_combined_df.fillna(-1)
        all_combined_df.to_excel('20210819_all.xlsx', index=False)
        move_df(all_combined_df, 'ProSites[pGlyco]', 'N-site(SequonBased)[Byonic]')
        ## result post-processing 
        glycansource(all_combined_df, 'Scan')
        print('\nCombined data shape:\nrow --> %s, column --> %s'%(all_combined_df.shape[0], all_combined_df.shape[1]))
        ## style apply for excel export
        # color the rows below the threshold (threshold [byonic: score > BYONIC_SCORE_CUTOFF & pep2d < 0.001; pglyco: PepScore>5 & GlyScore>4])
        # color code => #ffedcc -> light orange for byonic; #add8e6 -> light blue for pglyco; #FFB6C1 -> light pink for byos w/ dif calm ^ seq from byonic; #FF1493 -> deep pink for byos w/ dif calm & seq from byonic; #FFFF00 -> yellow for byos & byonic all the same
        threshold_masks_colorind(all_combined_df)

        byonic_colored_id = list(set(lightgreen_ind + normalgreen_ind + lightorange_ind + normalorange_ind + deepgreen_ind)) # byonic pass threshold (include below-threshold ethcd w/o pair) 
        byonic_colored_id_for_pairfile = byonic_colored_id.copy()
        pglyco_colored_id = list(set(lightblue_ind + normalblue_ind + lightorange_ind + normalorange_ind + deepblue_ind)) # pglyco pass threshold
        pglyco_belowthreshold_id = [i for i in all_combined_df.index.tolist() if i not in pglyco_colored_id]
        ## analyze PSM (need to pass threshold)
        # need to record the ind of below-threshold ethcd w/o pair in advance (will be excluded later, so we should not count PSM as well)
        # so here we are forced to find pair in advance (since we are forced to list psm in all file)
        if 'Pair[Byonic]' in all_combined_df.columns.tolist(): # pair is to identify etd with pass-threshold hcd pair for later rescue
            # ONLY FOR HCD/ETD FILES: prepare '_Pair' file (as long as byonic passes threshold & have pair)
            all_combined_df = all_combined_df.astype({'Pair[Byonic]': 'str'})
            pair_df = all_combined_df.loc[byonic_colored_id]
            pair_df = pair_df[pair_df['Pair[Byonic]'].str.contains('pair')]
            pair_df = pair_df[pair_df.duplicated(subset=['Pair[Byonic]'], keep=False)] # only paired data included in _Pair
        else:
            pass

        if 'ethcd' in all_combined_df['Fragment\r\nType[Byonic]'].tolist(): # if etd data is present (i.e. have pair)
            byonic_excluded_ethcd_ind =  all_combined_df.loc[((all_combined_df['Fragment\r\nType[Byonic]'] == 'ethcd')&((all_combined_df['Score[Byonic]'] <= BYONIC_SCORE_CUTOFF)|(all_combined_df['PEP\r\n2D[Byonic]'].abs() >= 0.001))&(~(all_combined_df['Pair[Byonic]'].isin(pair_df['Pair[Byonic]']))))].index.tolist()
            byonic_pass_id = [i for i in byonic_colored_id if i not in byonic_excluded_ethcd_ind]
            byonic_belowthreshold_id = [i for i in all_combined_df.index.tolist() if i not in byonic_pass_id] 
        else: # all hcd, all in (cause here all the hcd are above threshold, i.e. all colored)
            byonic_pass_id = byonic_colored_id 
            byonic_belowthreshold_id = [i for i in all_combined_df.index.tolist() if i not in byonic_pass_id] 

        # using groupby size function to count psm & add psm columns (new criterion: add fragment type, RT/scantime dif < 3 min (180s). is regarded as the same, >= 3 min. should be seen as dif.)
        # byonic & byos
        all_combined_df = all_combined_df.astype({'N-site(SequonBased)[Byonic]': 'str'}) # convert list & tuple to str for later groupby function
        b_df_pass = all_combined_df.loc[byonic_pass_id]
        b_df_pass_forUnique = ScantimeGp(b_df_pass, gp_by = ['Fragment\r\nType[Byonic]', 'N-site(SequonBased)[Byonic]',  'Calc.\r\nm/z[Byonic]','rounded_FinalGlycans[Byonic]', 'PureSequence[Byonic]'], score_col = 'Score[Byonic]', rt_col = 'Scan\r\nTime[Byonic]', software = '[Byonic]', purpose = 'forUnique')
        b_df_pass_forNglyco = ScantimeGp(b_df_pass, gp_by = ['N-site(SequonBased)[Byonic]',  'Calc.\r\nm/z[Byonic]','rounded_FinalGlycans[Byonic]', 'PureSequence[Byonic]'], score_col = 'Score[Byonic]', rt_col = 'Scan\r\nTime[Byonic]', software = '[Byonic]', purpose = 'forNglyco')
        # b_df_pass_forNglyco.to_excel('b_df_pass_forNglyco.xlsx', index=False)
        byonic_psm = b_df_pass_forNglyco.groupby(['N-site(SequonBased)[Byonic]', 'Calc.\r\nm/z[Byonic]','rounded_FinalGlycans[Byonic]', 'PureSequence[Byonic]', 'rt_dif_reset_forNglyco[Byonic]'])['N-site(SequonBased)[Byonic]'].transform('size')
        
        all_combined_df.loc[(byonic_pass_id), 'PSM[Byonic]'] = byonic_psm
        move_df(all_combined_df, 'PSM[Byonic]', 'N-site(SequonBased)[Byonic]')
        all_combined_df.loc[(byonic_pass_id), 'rt_dif_reset_forUnique[Byonic]'] = b_df_pass_forUnique['rt_dif_reset_forUnique[Byonic]']
        all_combined_df.loc[(byonic_pass_id), 'rt_dif_reset_forNglyco[Byonic]'] = b_df_pass_forNglyco['rt_dif_reset_forNglyco[Byonic]']
        all_combined_df.loc[(all_combined_df['N-site(SequonBased)[Byonic]'] == 'N/A'), 'PSM[Byonic]'] = 'N/A' # if n-site is 'N/A', do not count psm
        all_combined_df.loc[byonic_belowthreshold_id, 'PSM[Byonic]'] = 'N/A' # do not count psm for rows below threshold
        all_combined_df.loc[(all_combined_df['N-site(SequonBased)[Byonic]'] == '-1'), 'PSM[Byonic]'] = -1 # if site is '-1', then psm set to -1
        # pglyco
        all_combined_df = all_combined_df.astype({'ProSites[pGlyco]': 'int'})
        all_combined_df = all_combined_df.astype({'ProSites[pGlyco]': 'str'}) # convert list & tuple to str for later groupby function
        p_df_pass = all_combined_df.loc[pglyco_colored_id]
        p_df_pass_forUnique = ScantimeGp(p_df_pass, gp_by = ['FragmentType[pGlyco]', 'ProSites[pGlyco]', 'rounded_PrecursorMZ[pGlyco]','GlycanComposition_ByonicStyle[pGlyco]', 'Peptide[pGlyco]'], score_col = 'PepScore[pGlyco]', rt_col = 'RT[pGlyco]', software = '[pGlyco]', purpose = 'forUnique')
        p_df_pass_forNglyco = ScantimeGp(p_df_pass, gp_by = ['ProSites[pGlyco]', 'rounded_PrecursorMZ[pGlyco]','GlycanComposition_ByonicStyle[pGlyco]', 'Peptide[pGlyco]'], score_col = 'PepScore[pGlyco]', rt_col = 'RT[pGlyco]', software = '[pGlyco]', purpose = 'forNglyco')
        
        pglyco_psm = p_df_pass_forNglyco.groupby(['ProSites[pGlyco]', 'rounded_PrecursorMZ[pGlyco]','GlycanComposition_ByonicStyle[pGlyco]', 'Peptide[pGlyco]', 'rt_dif_reset_forNglyco[pGlyco]'])['ProSites[pGlyco]'].transform('size')
        all_combined_df.loc[(pglyco_colored_id), 'PSM[pGlyco]'] = pglyco_psm
        move_df(all_combined_df, 'PSM[pGlyco]', 'ProSites[pGlyco]')
        all_combined_df.loc[(pglyco_colored_id), 'rt_dif_reset_forUnique[pGlyco]'] = p_df_pass_forUnique['rt_dif_reset_forUnique[pGlyco]']
        all_combined_df.loc[(pglyco_colored_id), 'rt_dif_reset_forNglyco[pGlyco]'] = p_df_pass_forNglyco['rt_dif_reset_forNglyco[pGlyco]']
        all_combined_df.loc[(all_combined_df['ProSites[pGlyco]'] == 'N/A'), 'PSM[pGlyco]'] = 'N/A' # if n-site is 'N/A', do not count psm
        all_combined_df.loc[pglyco_belowthreshold_id, 'PSM[pGlyco]'] = 'N/A' # do not count psm for rows below threshold
        all_combined_df.loc[(all_combined_df['ProSites[pGlyco]'] == '-1'), 'PSM[pGlyco]'] = -1 # if site is '-1', then psm set to -1
        all_forsimple_df = all_combined_df.copy() # to retain rt_dif_reset for simple df groupy and get unique row
        all_combined_df = all_combined_df.drop(['rt_dif_reset_forUnique[Byonic]', 'rt_dif_reset_forNglyco[Byonic]', 'rt_dif_reset_forUnique[pGlyco]', 'rt_dif_reset_forNglyco[pGlyco]'], axis = 1)
        print('\n----- Exporting "_All" file... This may take some time, please wait. -----\n')
        all_combined_df.style.apply(bg_color, axis=None).to_excel(f'{date}_All_{filename}.xlsx', index = False)  
        print('\n----- "_All" file exported. -----\n')
        
        if 'Pair[Byonic]' in all_combined_df.columns.tolist(): # pair is to identify etd with pass-threshold hcd pair for later rescue
            # ONLY FOR HCD/ETD FILES: prepare '_Pair' file (as long as byonic passes threshold & have pair)
            all_combined_df = all_combined_df.astype({'Pair[Byonic]': 'str'})
            pair_df = all_combined_df.loc[byonic_colored_id_for_pairfile]
            pair_df = pair_df[pair_df['Pair[Byonic]'].str.contains('pair')]
            pair_df = pair_df[pair_df.duplicated(subset=['Pair[Byonic]'], keep=False)] # only paired data included in _Pair
            threshold_masks_colorind(pair_df)
            print('\n----- Exporting "_Pair" file... This may take some time, please wait. -----\n')
            pair_df.style.apply(bg_color, axis=None).to_excel(f'{date}_Pair_{filename}.xlsx', index = False)  
            print('\n----- "_Pair" file exported. -----\n')
        else:
            pass

        print('----- Start preparing simplified version. -----\n')
        # extract rows included in colored indices list separately
        byonicbyos_col_range = ['Scan'] + [col for col in all_forsimple_df.columns.tolist() if '[Byonic]' in col or '[Byos]' in col]
        pglyco_col_range = ['Scan'] + [col for col in all_forsimple_df.columns.tolist() if '[pGlyco]' in col]
        simple_byonicbyos = all_forsimple_df.loc[byonic_pass_id, byonicbyos_col_range] # extract colored (pass threshold) rows - below-threshold ethcd w/o pair
        simple_pglyco = all_forsimple_df.loc[pglyco_colored_id, pglyco_col_range] # extract colored (pass threshold) rows
        
        simple_byonicbyos_forUnique = simple_byonicbyos[(simple_byonicbyos['N-site(SequonBased)[Byonic]'] != '-1')] # preserve n/a sites for potential o glycan data in unique
        simple_byonicbyos_forNglyco = simple_byonicbyos[(simple_byonicbyos['N-site(SequonBased)[Byonic]'] != 'N/A')&(simple_byonicbyos['N-site(SequonBased)[Byonic]'] != '-1')] # for simplified, n-site must exist
        
        simple_pglyco_forUnique = simple_pglyco[(simple_pglyco['ProSites[pGlyco]'] != '-1')] # preserve n/a sites for potential o glycan data in unique
        simple_pglyco_forNglyco = simple_pglyco[(simple_pglyco['ProSites[pGlyco]'] != 'N/A')&(simple_pglyco['ProSites[pGlyco]'] != '-1')] # for simplified, n-site must exist

        # for nglycopep
        simple_pglyco_forNglyco = simple_pglyco_forNglyco.loc[simple_pglyco_forNglyco['FragmentType[pGlyco]'] == 'hcd'] # pglyco etd has the same values as hcd, so only let hcd enter unique file
        
        # clean data by dropping duplicates        
        simple_byonicbyos_forUnique = simple_byonicbyos_forUnique.drop_duplicates().reset_index(drop=True)
        simple_byonicbyos_forNglyco = simple_byonicbyos_forNglyco.drop_duplicates().reset_index(drop=True)

        simple_pglyco_forUnique = simple_pglyco_forUnique.drop_duplicates().reset_index(drop=True)
        simple_pglyco_forNglyco = simple_pglyco_forNglyco.drop_duplicates().reset_index(drop=True)
        
        # HIGHSCORE SELECTION: get unique calc.m/z by groupby & idxmax function to find highest score within each gp and preserve (remember to split the simple_df into byonicbyos & pglyco parts to avoid dropping non-target rows)
        simple_byonicbyos_forUnique = simple_byonicbyos_forUnique.groupby(['Fragment\r\nType[Byonic]', 'N-site(SequonBased)[Byonic]', 'Calc.\r\nm/z[Byonic]','rounded_FinalGlycans[Byonic]', 'PureSequence[Byonic]', 'rt_dif_reset_forUnique[Byonic]'], as_index=False).apply(lambda x: x.loc[x['Score[Byonic]'].idxmax()]).reset_index(drop=True)
        simple_pglyco_forUnique = simple_pglyco_forUnique.groupby(['FragmentType[pGlyco]', 'ProSites[pGlyco]', 'rounded_PrecursorMZ[pGlyco]','GlycanComposition_ByonicStyle[pGlyco]', 'Peptide[pGlyco]', 'rt_dif_reset_forUnique[pGlyco]'], as_index=False).apply(lambda x: x.loc[x['PepScore[pGlyco]'].idxmax()]).reset_index(drop=True)

        simple_byonicbyos_forNglyco = simple_byonicbyos_forNglyco.groupby(['N-site(SequonBased)[Byonic]', 'Calc.\r\nm/z[Byonic]','rounded_FinalGlycans[Byonic]', 'PureSequence[Byonic]', 'rt_dif_reset_forNglyco[Byonic]'], as_index=False).apply(lambda x: x.loc[x['Score[Byonic]'].idxmax()]).reset_index(drop=True)
        simple_pglyco_forNglyco = simple_pglyco_forNglyco.groupby(['ProSites[pGlyco]', 'rounded_PrecursorMZ[pGlyco]','GlycanComposition_ByonicStyle[pGlyco]', 'Peptide[pGlyco]', 'rt_dif_reset_forNglyco[pGlyco]'], as_index=False).apply(lambda x: x.loc[x['PepScore[pGlyco]'].idxmax()]).reset_index(drop=True)

        # set scan to index & concat simple_byonicbyos & simple_pglyco by scan (for unique)
        simple_byonicbyos_forUnique = simple_byonicbyos_forUnique.set_index('Scan')
        simple_pglyco_forUnique = simple_pglyco_forUnique.set_index('Scan')
        a1, a2 = simple_byonicbyos_forUnique.align(simple_pglyco_forUnique, join = 'outer', axis = 0) # row: a1 = a2
        id_df_forUnique = pd.concat([a1,a2], axis = 1)
        id_df_forUnique.index.name = 'Scan'
        id_df_forUnique.reset_index(level=0, inplace=True)
        # change all nan to blank -1
        id_df_forUnique = id_df_forUnique.fillna(-1)
        # extract nglycan numbers in Mods & use it to calculate byonic pep mass 
        mods_nglycan_sum = [sum([ast.literal_eval(v) for v in lst]) if type(lst) == list else -1 for lst in id_df_forUnique['Mods\r\n(variable)[Byonic]'].str.findall(r'Glycan / (\d+.\d+)').tolist()]
        id_df_forUnique['glycan_mods_sum[Byonic]'] = mods_nglycan_sum
        id_df_forUnique['PepMS[Byonic]'] = id_df_forUnique['Calc.\r\nMH[Byonic]'] - id_df_forUnique['glycan_mods_sum[Byonic]']
        id_df_forUnique.loc[(id_df_forUnique['glycan_mods_sum[Byonic]'] == -1), 'PepMS[Byonic]'] = -1
        move_df(id_df_forUnique, 'ProSites[pGlyco]', 'N-site(SequonBased)[Byonic]')
        move_df(id_df_forUnique, 'PepMS[Byonic]', 'Fragment\r\nType[Byonic]')
        # ast.literaleval byonicbyos n-site & sorting: byonic site -> pglyco site -> byonic peptide mass -> pglyco peptide mass -> byonic glycans -> pglyco glycans in byonic style -> byonic clam/z -> pglcyo calm/z
        id_df_forUnique['N-site(SequonBased)[Byonic]'] = [ast.literal_eval(i) if type(i) == str and '(' not in i and i!='N/A' else i for i in id_df_forUnique['N-site(SequonBased)[Byonic]'].tolist()] # convert str back to int & tuple
        id_df_forUnique['ProSites[pGlyco]'] = [ast.literal_eval(i) if type(i) == str and '(' not in i and i!='N/A' else i for i in id_df_forUnique['ProSites[pGlyco]'].tolist()] # convert str back to int & tuple
        
        id_df_forUnique.loc[(id_df_forUnique['N-site(SequonBased)[Byonic]'] == -1), 'mock_site[Byonic]'] = id_df_forUnique['ProSites[pGlyco]']
        id_df_forUnique.loc[(id_df_forUnique['N-site(SequonBased)[Byonic]'] != -1), 'mock_site[Byonic]'] = id_df_forUnique['N-site(SequonBased)[Byonic]']
        # drop intermediate cols
        id_df_forUnique = id_df_forUnique.drop(['rt_dif_reset_forUnique[Byonic]', 'rt_dif_reset_forNglyco[Byonic]', 'rt_dif_reset_forUnique[pGlyco]', 'rt_dif_reset_forNglyco[pGlyco]', 'glycan_mods_sum[Byonic]'], axis = 1) # drop the intermediate cols
        glycansource(id_df_forUnique, 'Scan')
        move_df(id_df_forUnique, 'mock_site[Byonic]', 'GlycanSource')
        col_order = id_df_forUnique.columns.tolist()
        # sorting order: byonic site -> pglyco site -> byonic peptide mass -> pglyco peptide mass -> byonic glycans -> pglyco glycans in byonic style -> byonic calm/z -> pglcyo calm/z => all of this can be acheive with sort_values to multi cols.

        id_df_forUnique = sort_intstr_col(id_df_forUnique, 'Charge[pGlyco]')
        print('Done sorting by Charge[pGlyco] --> ', end = '')
        id_df_forUnique = sort_intstr_col(id_df_forUnique, 'PrecursorMH[pGlyco]')
        print('Done sorting by PrecursorMH[pGlyco] --> ', end = '')
        id_df_forUnique = sort_intstr_col(id_df_forUnique, 'PeptideMH[pGlyco]')
        print('Done sorting by PeptideMH[pGlyco] --> ', end = '')
        id_df_forUnique = sort_intstr_col(id_df_forUnique, 'GlycanComposition_ByonicStyle[pGlyco]')
        print('Done sorting by GlycanComposition_ByonicStyle[pGlyco] --> ', end = '')
        id_df_forUnique = sort_intstr_col(id_df_forUnique, 'Peptide[pGlyco]')
        print('Done sorting by Peptide[pGlyco] --> ', end = '')
        id_df_forUnique = sort_intstr_col(id_df_forUnique, 'z[Byonic]')
        print('Done sorting by z[Byonic] --> ', end = '')
        id_df_forUnique = sort_intstr_col(id_df_forUnique, 'Calc.\r\nMH[Byonic]')
        print('Done sorting by Calc.MH[Byonic] --> ', end = '')
        id_df_forUnique = sort_intstr_col(id_df_forUnique, 'PepMS[Byonic]')
        print('Done sorting by PepMS[Byonic] --> ', end = '')
        id_df_forUnique = sort_intstr_col(id_df_forUnique, 'rounded_FinalGlycans[Byonic]')
        print('Done sorting by rounded_FinalGlycans[Byonic] --> ', end = '')
        id_df_forUnique = sort_intstr_col(id_df_forUnique, 'PureSequence[Byonic]')
        print('Done sorting by PureSequence[Byonic] --> ', end = '')
        id_df_forUnique = sort_intstr_col(id_df_forUnique, 'mock_site[Byonic]')
        print('Done sorting by mock_site[Byonic] (added during processing)')

        # convert '-1' back to int -1 for correct glycansource analysis
        id_df_forUnique['rounded_FinalGlycans[Byonic]'] = [int(i) if i == '-1' else i for i in id_df_forUnique['rounded_FinalGlycans[Byonic]']]
        id_df_forUnique['GlycanComposition_ByonicStyle[pGlyco]'] = [int(i) if i == '-1' else i for i in id_df_forUnique['GlycanComposition_ByonicStyle[pGlyco]']]
        id_df_forUnique = id_df_forUnique[col_order]

        print('\n----- Exporting "_UniquePep" file... This may take some time, please wait. -----\n')
        threshold_masks_colorind(id_df_forUnique)
        id_df_forUnique.style.apply(bg_color, axis=None).to_excel(f'{date}_UniquePep_{filename}.xlsx', index = False)  
        print('\n----- "_UniquePep" file exported. -----\n')

        # set scan to index & concat simple_byonicbyos & simple_pglyco by scan (for nglyco)
        simple_byonicbyos_forNglyco = simple_byonicbyos_forNglyco.set_index('Scan')
        simple_pglyco_forNglyco = simple_pglyco_forNglyco.set_index('Scan')
        a1, a2 = simple_byonicbyos_forNglyco.align(simple_pglyco_forNglyco, join = 'outer', axis = 0) # row: a1 = a2
        id_df_forNglyco = pd.concat([a1,a2], axis = 1)
        id_df_forNglyco.index.name = 'Scan'
        id_df_forNglyco.reset_index(level=0, inplace=True)
        # change all nan to blank -1
        id_df_forNglyco = id_df_forNglyco.fillna(-1)
        # extract nglycan numbers in Mods & use it to calculate byonic pep mass 
        mods_nglycan_sum = [sum([ast.literal_eval(v) for v in lst]) if type(lst) == list else -1 for lst in id_df_forNglyco['Mods\r\n(variable)[Byonic]'].str.findall(r'Glycan / (\d+.\d+)').tolist()]
        id_df_forNglyco['glycan_mods_sum[Byonic]'] = mods_nglycan_sum
        id_df_forNglyco['PepMS[Byonic]'] = id_df_forNglyco['Calc.\r\nMH[Byonic]'] - id_df_forNglyco['glycan_mods_sum[Byonic]']
        id_df_forNglyco.loc[(id_df_forNglyco['glycan_mods_sum[Byonic]'] == -1), 'PepMS[Byonic]'] = -1
        move_df(id_df_forNglyco, 'ProSites[pGlyco]', 'N-site(SequonBased)[Byonic]')
        move_df(id_df_forNglyco, 'PepMS[Byonic]', 'Fragment\r\nType[Byonic]')
        # ast.literaleval byonicbyos n-site & sorting: byonic site -> pglyco site -> byonic peptide mass -> pglyco peptide mass -> byonic glycans -> pglyco glycans in byonic style -> byonic clam/z -> pglcyo calm/z
        id_df_forNglyco['N-site(SequonBased)[Byonic]'] = [ast.literal_eval(i) if type(i) == str and '(' not in i else i for i in id_df_forNglyco['N-site(SequonBased)[Byonic]'].tolist()] # convert str back to int & tuple
        id_df_forNglyco['ProSites[pGlyco]'] = [ast.literal_eval(i) if type(i) == str and '(' not in i else i for i in id_df_forNglyco['ProSites[pGlyco]'].tolist()] # convert str back to int & tuple

        id_df_forNglyco.loc[(id_df_forNglyco['N-site(SequonBased)[Byonic]'] == -1), 'mock_site[Byonic]'] = id_df_forNglyco['ProSites[pGlyco]']
        id_df_forNglyco.loc[(id_df_forNglyco['N-site(SequonBased)[Byonic]'] != -1), 'mock_site[Byonic]'] = id_df_forNglyco['N-site(SequonBased)[Byonic]']
        # drop intermediate cols
        id_df_forNglyco = id_df_forNglyco.drop(['rt_dif_reset_forUnique[Byonic]', 'rt_dif_reset_forNglyco[Byonic]', 'rt_dif_reset_forUnique[pGlyco]', 'rt_dif_reset_forNglyco[pGlyco]', 'glycan_mods_sum[Byonic]'], axis = 1) # drop the intermediate cols
        glycansource(id_df_forNglyco, 'Scan')
        move_df(id_df_forNglyco, 'mock_site[Byonic]', 'GlycanSource')
        col_order = id_df_forNglyco.columns.tolist()
        # sorting order: byonic site -> pglyco site -> byonic peptide mass -> pglyco peptide mass -> byonic glycans -> pglyco glycans in byonic style -> byonic calm/z -> pglcyo calm/z => all of this can be acheive with sort_values to multi cols.

        id_df_forNglyco = sort_intstr_col(id_df_forNglyco, 'Charge[pGlyco]')
        print('Done sorting by Charge[pGlyco] --> ', end = '')
        id_df_forNglyco = sort_intstr_col(id_df_forNglyco, 'PrecursorMH[pGlyco]')
        print('Done sorting by PrecursorMH[pGlyco] --> ', end = '')
        id_df_forNglyco = sort_intstr_col(id_df_forNglyco, 'PeptideMH[pGlyco]')
        print('Done sorting by PeptideMH[pGlyco] --> ', end = '')
        id_df_forNglyco = sort_intstr_col(id_df_forNglyco, 'GlycanComposition_ByonicStyle[pGlyco]')
        print('Done sorting by GlycanComposition_ByonicStyle[pGlyco] --> ', end = '')
        id_df_forNglyco = sort_intstr_col(id_df_forNglyco, 'Peptide[pGlyco]')
        print('Done sorting by Peptide[pGlyco] --> ', end = '')
        id_df_forNglyco = sort_intstr_col(id_df_forNglyco, 'z[Byonic]')
        print('Done sorting by z[Byonic] --> ', end = '')
        id_df_forNglyco = sort_intstr_col(id_df_forNglyco, 'Calc.\r\nMH[Byonic]')
        print('Done sorting by Calc.MH[Byonic] --> ', end = '')
        id_df_forNglyco = sort_intstr_col(id_df_forNglyco, 'PepMS[Byonic]')
        print('Done sorting by PepMS[Byonic] --> ', end = '')
        id_df_forNglyco = sort_intstr_col(id_df_forNglyco, 'rounded_FinalGlycans[Byonic]')
        print('Done sorting by rounded_FinalGlycans[Byonic] --> ', end = '')
        id_df_forNglyco = sort_intstr_col(id_df_forNglyco, 'PureSequence[Byonic]')
        print('Done sorting by PureSequence[Byonic] --> ', end = '')
        id_df_forNglyco = sort_intstr_col(id_df_forNglyco, 'mock_site[Byonic]')
        print('Done sorting by mock_site[Byonic] (added during processing)')

        # convert '-1' back to int -1 for correct glycansource analysis
        id_df_forNglyco['rounded_FinalGlycans[Byonic]'] = [int(i) if i == '-1' else i for i in id_df_forNglyco['rounded_FinalGlycans[Byonic]']]
        id_df_forNglyco['GlycanComposition_ByonicStyle[pGlyco]'] = [int(i) if i == '-1' else i for i in id_df_forNglyco['GlycanComposition_ByonicStyle[pGlyco]']]
        id_df_forNglyco = id_df_forNglyco[col_order]

        print('\n----- Exporting "_NglycoPep" file... This may take some time, please wait. -----\n')
        threshold_masks_colorind(id_df_forNglyco)
        id_df_forNglyco.style.apply(bg_color, axis=None).to_excel(f'{date}_NglycoPep_{filename}.xlsx', index = False)  
        print('\n----- "_NglycoPep" file exported. -----\n')

        print('----- Start preparing quantification file of N-glycosylation . -----\n')
        # only single sites for quant (quant_df offers important info. for later multiIndex df construction)
        quant = id_df_forNglyco[id_df_forNglyco['N-site(SequonBased)[Byonic]'].apply(lambda x: isinstance(x, int))]
        # avoid adding values from below-threshold rows: if deepgreen -> change value to -1, if deepblue -> change value to -1.(-1 to make lambda function easier to write)
        threshold_masks_colorind(quant)
        quant.loc[deepgreen_ind, ['MonoArea[pGlyco]', 'IsotopeArea[pGlyco]']] = -1
        # calculate the sum within each n-site, then normalize mono/ iso by these sum
        quant['sumpersite_mono'] = quant.groupby(['ProSites[pGlyco]'])['MonoArea[pGlyco]'].transform(lambda x: x[x!=-1].sum())
        quant['sumpersite_iso'] = quant.groupby(['ProSites[pGlyco]'])['IsotopeArea[pGlyco]'].transform(lambda x: x[x!=-1].sum())
        # convert all the 'N/A' in byonic glycans to 'Unoccupied'
        quant.loc[(quant['rounded_FinalGlycans[Byonic]'] == 'N/A'), ['rounded_FinalGlycans[Byonic]']] = 'Unoccupied'
        # using groupby transform sum function to add 8 cols (byos xicauc/byos area int/pglycomono/pglycoisotope) recording summed values & normalized values (same site & same glycan & seq can be dif)
        quant['g_sum_MonoArea[pGlyco]'] = quant.groupby(['ProSites[pGlyco]', 'GlycanComposition_ByonicStyle[pGlyco]'])['MonoArea[pGlyco]'].transform(lambda x: x[x != -1].sum())
        quant['h_sum_IsotopeArea[pGlyco]'] = quant.groupby(['ProSites[pGlyco]', 'GlycanComposition_ByonicStyle[pGlyco]'])['IsotopeArea[pGlyco]'].transform(lambda x: x[x != -1].sum())
        quant['c_norm_MonoArea[pGlyco]'] = quant['g_sum_MonoArea[pGlyco]']/quant['sumpersite_mono']
        quant['d_norm_IsotopeArea[pGlyco]'] = quant['h_sum_IsotopeArea[pGlyco]']/quant['sumpersite_iso']
        # since it's possible to have real 0 from calculation, change the real absent data back to -1 in sum_ & norm_
        quant.loc[quant['MonoArea[pGlyco]'] == -1, ['g_sum_MonoArea[pGlyco]', 'h_sum_IsotopeArea[pGlyco]', 'c_norm_MonoArea[pGlyco]', 'd_norm_IsotopeArea[pGlyco]']] = -1
        # extract the needed cols only & split byonicbyos/pglyco, drop all -1 rows, glycans as index to outer union concat data again
        quant_bb = quant[['GlycanSource', 'N-site(SequonBased)[Byonic]', 'rounded_FinalGlycans[Byonic]']].rename(columns={'GlycanSource':'GlycanSource[Byonic]'})
        quant_p = quant[['GlycanSource', 'ProSites[pGlyco]', 'GlycanComposition_ByonicStyle[pGlyco]', 'g_sum_MonoArea[pGlyco]', 'h_sum_IsotopeArea[pGlyco]', 'c_norm_MonoArea[pGlyco]', 'd_norm_IsotopeArea[pGlyco]']].rename(columns={'GlycanSource':'GlycanSource[pGlyco]'})
        quant_bb = quant_bb.loc[quant_bb['N-site(SequonBased)[Byonic]'] != -1]
        quant_p = quant_p.loc[quant_p['ProSites[pGlyco]'] != -1]
        quant_bb = quant_bb.set_index('rounded_FinalGlycans[Byonic]')
        quant_p = quant_p.set_index('GlycanComposition_ByonicStyle[pGlyco]')
        a1, a2 = quant_bb.align(quant_p, join = 'outer', axis = 0) 
        quant_glycanid = pd.concat([a1,a2], axis = 1)
        quant_glycanid.index.name = 'Glycans ↓'
        quant_glycanid.reset_index(level=0, inplace=True)
        # add N(x) col
        nx = ['N(0)' if lst == [] else 'N(%s)'%(lst[0]) for lst in quant_glycanid['Glycans ↓'].str.findall(r'HexNAc\((\d+)\)').tolist()]
        quant_glycanid.insert(0 , 'N(x) ↓', nx , True)
        move_df(quant_glycanid, 'ProSites[pGlyco]', 'N-site(SequonBased)[Byonic]')
        # change all nan to blank -1
        quant_glycanid = quant_glycanid.fillna(-1)
        quant_glycanid = quant_glycanid.drop_duplicates() # remember to drop duplicates after splitting & reconcat df

        ## onlyp
        onlyp = quant_glycanid.loc[(quant_glycanid['ProSites[pGlyco]'] != -1), [col for col in quant_glycanid.columns.tolist() if 'Byonic' not in col and 'Byos' not in col]]
        onlyp_forunion = onlyp.copy().drop_duplicates()
        onlyp = onlyp.drop('GlycanSource[pGlyco]', axis = 1).drop_duplicates()
        # clean off all no data rows
        onlyp = onlyp.loc[~((onlyp['g_sum_MonoArea[pGlyco]']==-1)&(onlyp['h_sum_IsotopeArea[pGlyco]']==-1)&(onlyp['c_norm_MonoArea[pGlyco]']==-1)&(onlyp['d_norm_IsotopeArea[pGlyco]']==-1))]
        nx = list(set(onlyp['N(x) ↓'].tolist()))
        nx = [int(re.findall(r'[0-9]+', i)[0]) if type(i) == str else i for i in nx]
        nx.sort(key=lambda v: (isinstance(v, str), v))
        p_new_nx = ['N(%s)'%(str(i)) if type(i) == int and i != -1 else i for i in nx]

        print('----- Start summary table construction. -----\n')
        # start grouping: N(X) -> byonic glycans -> byonic sites -> .agg({all 8 cols, apply .mean() & .mean()/total XXX})

        # onlyp
        onlyp_gp = onlyp.groupby(['N(x) ↓', 'Glycans ↓', 'ProSites[pGlyco]']).agg({'g_sum_MonoArea[pGlyco]': lambda x: x.mean(), 'h_sum_IsotopeArea[pGlyco]': lambda x: x.mean() \
                                                                                              , 'c_norm_MonoArea[pGlyco]': lambda x: x.mean(), 'd_norm_IsotopeArea[pGlyco]': lambda x: x.mean()})

        # fix the str '10' sorted before '2' problem
        onlyp_gp = onlyp_gp.reindex(p_new_nx, level = 0)
        # unstack byonic n-sites, pglyco n-sites
        onlyp_gp = onlyp_gp.unstack(level=-1)
        # multiIndex col swaplevel
        onlyp_gp.columns = onlyp_gp.columns.swaplevel(0, 1)
        # sort is necessary for the right table structure (but since it will sort every level, so we need to manually adjust level3 back to the original order)
        onlyp_gp.sort_index(axis=1, level=[0, 1], inplace=True)
        onlyp_gp.columns = onlyp_gp.columns.set_names(['ProSites[pGlyco] →', 'Quant. →'])
        # delete a-h in Quant. (the alphabet is just for sorting)
        onlyp_gp.columns.set_levels(['norm_MonoArea[pGlyco]', 'norm_IsotopeArea[pGlyco]', 'sum_MonoArea[pGlyco]', 'sum_IsotopeArea[pGlyco]'], level = 1, inplace=True)

        print('\n----- Exporting "_Quant" files... This may take some time, please wait. -----\n')
        
        with pd.ExcelWriter(f'{date}_Quant_{filename}.xlsx') as writer:           
            # onlyp
            onlyp_gp.replace(to_replace = -1, value = np.nan , inplace = True) 
            onlyp_gp.style.background_gradient(cmap ='Blues').highlight_null(null_color='white').to_excel(writer, sheet_name = 'Sheet2(QuantP)')
            
        print('\n----- "_Quant" file exported. -----\n')
        
        print('----- Start plotting. -----\n')

        pNORM_dfall_lstv1, pNORM_label_lstv1, pNORM_dfall_lstv2, pNORM_label_lstv2 \
        , pNORM_dfall_lstv3, pNORM_label_lstv3, pNORM_dfall_lstv4, pNORM_label_lstv4 \
        , pSUM_dfall_lstv1, pSUM_label_lstv1, pSUM_nsum_lstv1, pSUM_dfall_lstv2, pSUM_label_lstv2, pSUM_nsum_lstv2 \
        , pSUM_dfall_lstv3, pSUM_label_lstv3, pSUM_nsum_lstv3, pSUM_dfall_lstv4, pSUM_label_lstv4, pSUM_nsum_lstv4 \
        = [], [], [], [], [], [], [], [], [], [] \
        , [], [], [], [], [], [], [], [], [], [] 
        
        if plot_v2 == 'yes':  
            col_order = ['N(0)', 'N(1)', 'OligoMannose', 'Complex', 'Complex+Sia'] # col order for v2 plotting
            # necessary preprocessings for v2 plotting (modified from v1 nx)
            # onlyp v2
            onlyp.loc[(onlyp['N(x) ↓'] == 'N(0)')|(onlyp['N(x) ↓'] == 'N(1)'), 'nx_version2'] = onlyp['N(x) ↓']
            onlyp.loc[(onlyp['N(x) ↓'] == 'N(2)'), 'nx_version2'] = 'OligoMannose'
            onlyp.loc[(onlyp['N(x) ↓'] != 'N(0)')&(onlyp['N(x) ↓'] != 'N(1)')&(onlyp['N(x) ↓'] != 'N(2)'), 'nx_version2'] = 'Complex'
            onlyp.loc[(onlyp['N(x) ↓'] != 'N(0)')&(onlyp['N(x) ↓'] != 'N(1)')&(onlyp['N(x) ↓'] != 'N(2)')&(onlyp['Glycans ↓'].str.contains('NeuAc|NeuGc')), 'nx_version2'] = 'Complex+Sia'
            p_neugc_exist = sum(onlyp['Glycans ↓'].str.contains('NeuGc')) > 0 # Bool, if T, red. else purple
        
        if plot_v3 == 'yes':
            # necessary preprocessings for v3 plotting (modified from v1 nx)           
            # onlyp v3
            onlyp, onlyp_nxhy_fix_order = v3(onlyp)[0], v3(onlyp)[1]

        if plot_v4 == 'yes':
            # necessary preprocessings for v4 plotting (modified from v1 nx)            
            # onlyp v4
            onlyp, onlyp_fnxhy_fix_order = v4(onlyp)[0], v4(onlyp)[1]

        # # make sure that sites are presented as int not float
        onlyp = onlyp.astype({'ProSites[pGlyco]':'Int64'})

        # split bp_union into: xicauc_df / int_df / mono_df / iso_df for later clustered stacked plot (norm only)
        if export_mono == 'yes':
            if plot_v1 == 'yes':
                # onlyp norm v1
                p_monoNORM_df = onlyp.loc[:, ['N(x) ↓', 'ProSites[pGlyco]', 'c_norm_MonoArea[pGlyco]']]
                p_monoNORM_df.replace(to_replace = -1, value = np.nan , inplace = True)
                p_monoNORM_df = p_monoNORM_df.pivot_table(index='ProSites[pGlyco]', values='c_norm_MonoArea[pGlyco]', columns='N(x) ↓', aggfunc=np.sum).reindex(columns=p_new_nx)
                pNORM_dfall_lstv1.append(p_monoNORM_df)
                pNORM_label_lstv1.append('norm_MonoArea[pGlyco]')
                # onlyp sum v1
                p_monoSUM_df = onlyp.loc[:, ['N(x) ↓', 'ProSites[pGlyco]', 'g_sum_MonoArea[pGlyco]']]
                p_monoSUM_df.replace(to_replace = -1, value = np.nan , inplace = True)
                p_monoSUM_df = p_monoSUM_df.pivot_table(index='ProSites[pGlyco]', values='g_sum_MonoArea[pGlyco]', columns='N(x) ↓', aggfunc=np.sum).reindex(columns=p_new_nx)
                p_monoSUM_df_nsum = [p_monoSUM_df.loc[i].sum() for i in p_monoSUM_df.index.tolist()] # for A bar label
                pSUM_dfall_lstv1.append(p_monoSUM_df)
                pSUM_label_lstv1.append('sum_MonoArea[pGlyco]')
                pSUM_nsum_lstv1.append(p_monoSUM_df_nsum)

            if plot_v2 == 'yes':
                # onlyp norm v2
                p_monoNORMv2_df = onlyp.loc[:, ['nx_version2', 'ProSites[pGlyco]', 'c_norm_MonoArea[pGlyco]']]
                p_monoNORMv2_df.replace(to_replace = -1, value = np.nan , inplace = True)
                p_monoNORMv2_df = p_monoNORMv2_df.pivot_table(index='ProSites[pGlyco]', values='c_norm_MonoArea[pGlyco]', columns='nx_version2', aggfunc=np.sum).reindex(columns=col_order)
                pNORM_dfall_lstv2.append(p_monoNORMv2_df)
                pNORM_label_lstv2.append('norm_MonoArea[pGlyco]')
                # onlyp sum v2
                p_monoSUMv2_df = onlyp.loc[:, ['nx_version2', 'ProSites[pGlyco]', 'g_sum_MonoArea[pGlyco]']]
                p_monoSUMv2_df.replace(to_replace = -1, value = np.nan , inplace = True)
                p_monoSUMv2_df = p_monoSUMv2_df.pivot_table(index='ProSites[pGlyco]', values='g_sum_MonoArea[pGlyco]', columns='nx_version2', aggfunc=np.sum).reindex(columns=col_order)
                p_monoSUMv2_df_nsum = [p_monoSUMv2_df.loc[i].sum() for i in p_monoSUMv2_df.index.tolist()] # for A bar label
                pSUM_dfall_lstv2.append(p_monoSUMv2_df)
                pSUM_label_lstv2.append('sum_MonoArea[pGlyco]')
                pSUM_nsum_lstv2.append(p_monoSUMv2_df_nsum)

            if plot_v3 == 'yes':
                # onlyp norm v3
                p_monoNORMv3_df = onlyb.loc[:, ['N(x)H(y)', 'ProSites[pGlyco]', 'c_norm_MonoArea[pGlyco]']]
                p_monoNORMv3_df.replace(to_replace = -1, value = np.nan , inplace = True)
                p_monoNORMv3_df = p_monoNORMv3_df.pivot_table(index='ProSites[pGlyco]', values='c_norm_MonoArea[pGlyco]', columns='N(x)H(y)', aggfunc=np.sum).reindex(columns=onlyp_nxhy_fix_order)
                pNORM_dfall_lstv3.append(p_monoNORMv3_df)
                pNORM_label_lstv3.append('norm_MonoArea[pGlyco]')
                # onlyp sum v3
                p_monoSUMv3_df = onlyp.loc[:, ['N(x)H(y)', 'ProSites[pGlyco]', 'g_sum_MonoArea[pGlyco]']]
                p_monoSUMv3_df.replace(to_replace = -1, value = np.nan , inplace = True)
                p_monoSUMv3_df = p_monoSUMv3_df.pivot_table(index='ProSites[pGlyco]', values='g_sum_MonoArea[pGlyco]', columns='N(x)H(y)', aggfunc=np.sum).reindex(columns=onlyp_nxhy_fix_order)
                p_monoSUMv3_df_nsum = [p_monoSUMv3_df.loc[i].sum() for i in p_monoSUMv3_df.index.tolist()] # for A bar label
                pSUM_dfall_lstv3.append(p_monoSUMv3_df)
                pSUM_label_lstv3.append('sum_MonoArea[pGlyco]')
                pSUM_nsum_lstv3.append(p_monoSUMv3_df_nsum)

            if plot_v4 == 'yes':
                # onlyp norm v4
                p_monoNORMv4_df = onlyp.loc[:, ['(F)N(x)H(y)', 'ProSites[pGlyco]', 'c_norm_MonoArea[pGlyco]']]
                p_monoNORMv4_df.replace(to_replace = -1, value = np.nan , inplace = True)
                p_monoNORMv4_df = p_monoNORMv4_df.pivot_table(index='N-site(SequonBased)[Byonic]', values='c_norm_MonoArea[pGlyco]', columns='(F)N(x)H(y)', aggfunc=np.sum).reindex(columns=onlyp_fnxhy_fix_order)
                pNORM_dfall_lstv4.append(p_monoNORMv4_df)
                pNORM_label_lstv4.append('norm_MonoArea[pGlyco]')
                # onlyp sum v4
                p_monoSUMv4_df = onlyp.loc[:, ['(F)N(x)H(y)', 'ProSites[pGlyco]', 'g_sum_MonoArea[pGlyco]']]
                p_monoSUMv4_df.replace(to_replace = -1, value = np.nan , inplace = True)
                p_monoSUMv4_df = p_monoSUMv4_df.pivot_table(index='ProSites[pGlyco]', values='g_sum_MonoArea[pGlyco]', columns='(F)N(x)H(y)', aggfunc=np.sum).reindex(columns=onlyp_fnxhy_fix_order)
                p_monoSUMv4_df_nsum = [p_monoSUMv4_df.loc[i].sum() for i in p_monoSUMv4_df.index.tolist()] # for A bar label
                pSUM_dfall_lstv4.append(p_monoSUMv4_df)
                pSUM_label_lstv4.append('sum_MonoArea[pGlyco]')
                pSUM_nsum_lstv4.append(p_monoSUMv4_df_nsum)     
        else:
            pass

        if export_isotope == 'yes':
            if plot_v1 == 'yes':
                # onlyp norm v1
                p_isoNORM_df = onlyp.loc[:, ['N(x) ↓', 'ProSites[pGlyco]', 'd_norm_IsotopeArea[pGlyco]']]
                p_isoNORM_df.replace(to_replace = -1, value = np.nan , inplace = True)
                p_isoNORM_df = p_isoNORM_df.pivot_table(index='ProSites[pGlyco]', values='d_norm_IsotopeArea[pGlyco]', columns='N(x) ↓', aggfunc=np.sum).reindex(columns=p_new_nx)
                pNORM_dfall_lstv1.append(p_isoNORM_df)
                pNORM_label_lstv1.append('norm_IsotopeArea[pGlyco]')
                # onlyp sum v1
                p_isoSUM_df = onlyp.loc[:, ['N(x) ↓', 'ProSites[pGlyco]', 'h_sum_IsotopeArea[pGlyco]']] 
                p_isoSUM_df.replace(to_replace = -1, value = np.nan , inplace = True)
                p_isoSUM_df = p_isoSUM_df.pivot_table(index='ProSites[pGlyco]', values='h_sum_IsotopeArea[pGlyco]', columns='N(x) ↓', aggfunc=np.sum).reindex(columns=p_new_nx)
                p_isoSUM_df_nsum = [p_isoSUM_df.loc[i].sum() for i in p_isoSUM_df.index.tolist()] # for B bar label
                pSUM_dfall_lstv1.append(p_isoSUM_df)
                pSUM_label_lstv1.append('sum_IsotopeArea[pGlyco]')
                pSUM_nsum_lstv1.append(p_isoSUM_df_nsum)

            if plot_v2 == 'yes':
                # onlyp norm v2
                p_isoNORMv2_df = onlyp.loc[:, ['nx_version2', 'ProSites[pGlyco]', 'd_norm_IsotopeArea[pGlyco]']]
                p_isoNORMv2_df.replace(to_replace = -1, value = np.nan , inplace = True)
                p_isoNORMv2_df = p_isoNORMv2_df.pivot_table(index='ProSites[pGlyco]', values='d_norm_IsotopeArea[pGlyco]', columns='nx_version2', aggfunc=np.sum).reindex(columns=col_order)
                pNORM_dfall_lstv2.append(p_isoNORMv2_df)
                pNORM_label_lstv2.append('norm_IsotopeArea[pGlyco]')
                # onlyp sum v2
                p_isoSUMv2_df = onlyp.loc[:, ['nx_version2', 'ProSites[pGlyco]', 'h_sum_IsotopeArea[pGlyco]']] 
                p_isoSUMv2_df.replace(to_replace = -1, value = np.nan , inplace = True)
                p_isoSUMv2_df = p_isoSUMv2_df.pivot_table(index='ProSites[pGlyco]', values='h_sum_IsotopeArea[pGlyco]', columns='nx_version2', aggfunc=np.sum).reindex(columns=col_order)
                p_isoSUMv2_df_nsum = [p_isoSUMv2_df.loc[i].sum() for i in p_isoSUMv2_df.index.tolist()] # for B bar label
                pSUM_dfall_lstv2.append(p_isoSUMv2_df)
                pSUM_label_lstv2.append('sum_IsotopeArea[pGlyco]')
                pSUM_nsum_lstv2.append(p_isoSUMv2_df_nsum)

            if plot_v3 == 'yes':
                # onlyp norm v3
                p_isoNORMv3_df = onlyp.loc[:, ['N(x)H(y)', 'ProSites[pGlyco]', 'd_norm_IsotopeArea[pGlyco]']]
                p_isoNORMv3_df.replace(to_replace = -1, value = np.nan , inplace = True)
                p_isoNORMv3_df = p_isoNORMv3_df.pivot_table(index='ProSites[pGlyco]', values='d_norm_IsotopeArea[pGlyco]', columns='N(x)H(y)', aggfunc=np.sum).reindex(columns=onlyp_nxhy_fix_order)
                pNORM_dfall_lstv3.append(p_isoNORMv3_df)
                pNORM_label_lstv3.append('norm_IsotopeArea[pGlyco]')
                # onlyp sum v3
                p_isoSUMv3_df = onlyp.loc[:, ['N(x)H(y)', 'ProSites[pGlyco]', 'h_sum_IsotopeArea[pGlyco]']]
                p_isoSUMv3_df.replace(to_replace = -1, value = np.nan , inplace = True)
                p_isoSUMv3_df = p_isoSUMv3_df.pivot_table(index='ProSites[pGlyco]', values='h_sum_IsotopeArea[pGlyco]', columns='N(x)H(y)', aggfunc=np.sum).reindex(columns=onlyp_nxhy_fix_order)
                p_isoSUMv3_df_nsum = [p_isoSUMv3_df.loc[i].sum() for i in p_isoSUMv3_df.index.tolist()] # for A bar label
                pSUM_dfall_lstv3.append(p_isoSUMv3_df)
                pSUM_label_lstv3.append('sum_IsotopeArea[pGlyco]')
                pSUM_nsum_lstv3.append(p_isoSUMv3_df_nsum)

            if plot_v4 == 'yes':
                # onlyp norm v4
                p_isoNORMv4_df = onlyp.loc[:, ['(F)N(x)H(y)', 'ProSites[pGlyco]', 'd_norm_IsotopeArea[pGlyco]']]
                p_isoNORMv4_df.replace(to_replace = -1, value = np.nan , inplace = True)
                p_isoNORMv4_df = p_isoNORMv4_df.pivot_table(index='ProSites[pGlyco]', values='d_norm_IsotopeArea[pGlyco]', columns='(F)N(x)H(y)', aggfunc=np.sum).reindex(columns=onlyp_fnxhy_fix_order)
                pNORM_dfall_lstv4.append(p_isoNORMv4_df)
                pNORM_label_lstv4.append('norm_IsotopeArea[pGlyco]')
                # onlyp sum v4
                p_isoSUMv4_df = onlyp.loc[:, ['(F)N(x)H(y)', 'ProSites[pGlyco]', 'h_sum_IsotopeArea[pGlyco]']]
                p_isoSUMv4_df.replace(to_replace = -1, value = np.nan , inplace = True)
                p_isoSUMv4_df = p_isoSUMv4_df.pivot_table(index='ProSites[pGlyco]', values='h_sum_IsotopeArea[pGlyco]', columns='(F)N(x)H(y)', aggfunc=np.sum).reindex(columns=onlyp_fnxhy_fix_order)
                p_isoSUMv4_df_nsum = [p_isoSUMv4_df.loc[i].sum() for i in p_isoSUMv4_df.index.tolist()] # for A bar label
                pSUM_dfall_lstv4.append(p_isoSUMv4_df)
                pSUM_label_lstv4.append('sum_IsotopeArea[pGlyco]')
                pSUM_nsum_lstv4.append(p_isoSUMv4_df_nsum)    
        else:
            pass
        
        if export_mono == export_isotope == 'no': # no plots will be exported
            pass
        else: # plotting criterium: cannot be empty list

            if pNORM_dfall_lstv1 != []:
                # onlyp NORMv1
                fig = plot_clustered_stacked(pNORM_dfall_lstv1,[[1]*len(pNORM_dfall_lstv1[0].index) for i in range(len(pNORM_dfall_lstv1))],pNORM_label_lstv1)
                fig.tight_layout()
                fig.savefig(f'{date}_onlypNORMv1_{filename}.png', bbox_inches='tight')
            if pNORM_dfall_lstv2 != []:
                # onlyp NORMv2
                fig = plot_clustered_stacked(pNORM_dfall_lstv2,[[1]*len(pNORM_dfall_lstv2[0].index) for i in range(len(pNORM_dfall_lstv2))],pNORM_label_lstv2, neugc_exist=p_neugc_exist)
                fig.tight_layout()
                fig.savefig(f'{date}_onlypNORMv2_{filename}.png', bbox_inches='tight')
            if pNORM_dfall_lstv3 != []:
                # onlyp NORMv3
                fig = plot_clustered_stacked(pNORM_dfall_lstv3,[[1]*len(pNORM_dfall_lstv3[0].index) for i in range(len(pNORM_dfall_lstv3))],pNORM_label_lstv3, detailedHighMan=True)
                fig.tight_layout()
                fig.savefig(f'{date}_onlypNORMv3_{filename}.png', bbox_inches='tight')
            if pNORM_dfall_lstv4 != []:
                # onlyp NORMv4
                fig = plot_clustered_stacked(pNORM_dfall_lstv4,[[1]*len(pNORM_dfall_lstv4[0].index) for i in range(len(pNORM_dfall_lstv4))],pNORM_label_lstv4, detailedHighMan=True, fucosylation=True)
                fig.tight_layout()
                fig.savefig(f'{date}_onlypNORMv4_{filename}.png', bbox_inches='tight')


            if pSUM_dfall_lstv1 != []:
                # onlyp SUMv1
                fig = plot_clustered_stacked(pSUM_dfall_lstv1,pSUM_nsum_lstv1,pSUM_label_lstv1)
                fig.tight_layout()
                fig.savefig(f'{date}_onlypSUMv1_{filename}.png', bbox_inches='tight')
            if pSUM_dfall_lstv2 != []:
                # onlyp SUMv2
                fig = plot_clustered_stacked(pSUM_dfall_lstv2,pSUM_nsum_lstv2 ,pSUM_label_lstv2, neugc_exist=p_neugc_exist)
                fig.tight_layout()
                fig.savefig(f'{date}_onlypSUMv2_{filename}.png', bbox_inches='tight')
            if pSUM_dfall_lstv3 != []:
                # onlyp SUMv3
                fig = plot_clustered_stacked(pSUM_dfall_lstv3,pSUM_nsum_lstv3,pSUM_label_lstv3, detailedHighMan=True)
                fig.tight_layout()
                fig.savefig(f'{date}_onlypSUMv3_{filename}.png', bbox_inches='tight')
            if pSUM_dfall_lstv4 != []:
                # onlyp SUMv4
                fig = plot_clustered_stacked(pSUM_dfall_lstv4,pSUM_nsum_lstv4 ,pSUM_label_lstv4, detailedHighMan=True, fucosylation=True)
                fig.tight_layout()
                fig.savefig(f'{date}_onlypSUMv4_{filename}.png', bbox_inches='tight')

    elif byonicfile != '' and byosfile == '' and pglycofile == '': # only byonic
        print('----- Only Byonic file is detected. Start the corresponding processing. -----\n')

        # change the variable name
        all_combined_df = byonic_df
        # make sure scan is at the start
        all_combined_df = all_combined_df.set_index('Scan')
        all_combined_df.reset_index(level=0, inplace=True)
        ## style apply for excel export
        # color the rows below the threshold (threshold [byonic: score > BYONIC_SCORE_CUTOFF & pep2d < 0.001; pglyco: PepScore>5 & GlyScore>4])
        # color code => #ffedcc -> light orange for byonic; #add8e6 -> light blue for pglyco; #FFB6C1 -> light pink for byos w/ dif calm ^ seq from byonic; #FF1493 -> deep pink for byos w/ dif calm & seq from byonic; #FFFF00 -> yellow for byos & byonic all the same
        threshold_masks_colorind(all_combined_df)

        byonic_colored_id = list(set(lightgreen_ind + normalgreen_ind)) # byonic pass threshold (include below-threshold ethcd w/o pair) 
        byonic_colored_id_for_pairfile = byonic_colored_id.copy()
        # analyze PSM (need to pass threshold)
#         need to record the ind of below-threshold ethcd w/o pair in advance (will be excluded later, so we should not count PSM as well)
#         so here we are forced to find pair in advance (since we are forced to list psm in all file)

        if 'Pair' in all_combined_df.columns.tolist(): # pair is to identify etd with pass-threshold hcd pair for later rescue
            # ONLY FOR HCD/ETD FILES: prepare '_Pair' file (as long as byonic passes threshold & have pair)
            all_combined_df = all_combined_df.astype({'Pair': 'str'})
            pair_df = all_combined_df.loc[byonic_colored_id]
            pair_df = pair_df[pair_df['Pair'].str.contains('pair')]
        #             pair_df = pair_df[pair_df.duplicated(subset=['Pair[Byonic]'], keep=False)] # only paired data included in _Pair
            if 'ethcd' in all_combined_df['Fragment\r\nType'].tolist(): 
                hcd_pair_num = set(pair_df.loc[(pair_df['Fragment\r\nType']=='hcd'), 'Pair'].tolist())
                pair_df.drop(pair_df[(pair_df['Fragment\r\nType']=='ethcd')&(~(pair_df['Pair'].isin(hcd_pair_num)))].index, inplace=True)
        else:
            pass

        if 'ethcd' in all_combined_df['Fragment\r\nType'].tolist(): # if etd data is present (i.e. have pair)
            byonic_excluded_ethcd_ind =  all_combined_df.loc[((all_combined_df['Fragment\r\nType'] == 'ethcd')&((all_combined_df['Score'] <= BYONIC_SCORE_CUTOFF)|(all_combined_df['PEP\r\n2D'].abs() >= 0.001))&(~(all_combined_df['Pair'].isin(pair_df['Pair']))))].index.tolist()
            byonic_pass_id = [i for i in byonic_colored_id if i not in byonic_excluded_ethcd_ind]
            byonic_belowthreshold_id = [i for i in all_combined_df.index.tolist() if i not in byonic_pass_id] 
        else: # all hcd, all in (cause here all the hcd are above threshold, i.e. all colored)
            byonic_pass_id = byonic_colored_id 
            byonic_belowthreshold_id = [i for i in all_combined_df.index.tolist() if i not in byonic_pass_id] 

        # using groupby size function to count psm & add psm columns (new criterion: add fragment type, RT/scantime dif < 3 min (180s). is regarded as the same, >= 3 min. should be seen as dif.)
        # byonic & byos
        all_combined_df = all_combined_df.astype({'N-site(SequonBased)': 'str'}) # convert list & tuple to str for later groupby function
        b_df_pass = all_combined_df.loc[byonic_pass_id]
        b_df_pass_forUnique = ScantimeGp(b_df_pass, gp_by = ['Fragment\r\nType', 'N-site(SequonBased)',  'Calc.\r\nm/z','rounded_FinalGlycans', 'PureSequence'], score_col = 'Score', rt_col = 'Scan\r\nTime', software = '[Byonic]', purpose = 'forUnique')
        b_df_pass_forNglyco = ScantimeGp(b_df_pass, gp_by = ['N-site(SequonBased)',  'Calc.\r\nm/z','rounded_FinalGlycans', 'PureSequence'], score_col = 'Score', rt_col = 'Scan\r\nTime', software = '[Byonic]', purpose = 'forNglyco')
        byonic_psm = b_df_pass_forNglyco.groupby(['N-site(SequonBased)', 'Calc.\r\nm/z','rounded_FinalGlycans', 'PureSequence', 'rt_dif_reset_forNglyco[Byonic]'])['N-site(SequonBased)'].transform('size')

        all_combined_df.loc[(byonic_pass_id), 'PSM'] = byonic_psm
        move_df(all_combined_df, 'PSM', 'N-site(SequonBased)')
        all_combined_df.loc[(byonic_pass_id), 'rt_dif_reset_forUnique[Byonic]'] = b_df_pass_forUnique['rt_dif_reset_forUnique[Byonic]']
        all_combined_df.loc[(byonic_pass_id), 'rt_dif_reset_forNglyco[Byonic]'] = b_df_pass_forNglyco['rt_dif_reset_forNglyco[Byonic]']
        all_combined_df.loc[(all_combined_df['N-site(SequonBased)'] == 'N/A'), 'PSM'] = 'N/A' # if n-site is 'N/A', do not count psm
        all_combined_df.loc[byonic_belowthreshold_id, 'PSM'] = 'N/A' # do not count psm for rows below threshold
        all_combined_df.loc[(all_combined_df['N-site(SequonBased)'] == '-1'), 'PSM'] = -1 # if site is '-1', then psm set to -1
        all_forsimple_df = all_combined_df.copy() # to retain rt_dif_reset for simple df groupy and get unique row
        all_combined_df = all_combined_df.drop(['rt_dif_reset_forUnique[Byonic]', 'rt_dif_reset_forNglyco[Byonic]'], axis = 1)
        print('\n----- Exporting "_All" file... This may take some time, please wait. -----\n')
        all_combined_df.style.apply(bg_color, axis=None).to_excel(f'{date}_All_{filename}.xlsx', index = False)  
        print('\n----- "_All" file exported. -----\n')
        

        if 'Pair' in all_combined_df.columns.tolist(): # pair is to identify etd with pass-threshold hcd pair for later rescue
            # ONLY FOR HCD/ETD FILES: prepare '_Pair' file (as long as byonic passes threshold & have pair)
            all_combined_df = all_combined_df.astype({'Pair': 'str'})
            pair_df = all_combined_df.loc[byonic_colored_id_for_pairfile]
            pair_df = pair_df[pair_df['Pair'].str.contains('pair')]
        #             pair_df = pair_df[pair_df.duplicated(subset=['Pair[Byonic]'], keep=False)] # only paired data included in _Pair
            hcd_pair_num = set(pair_df.loc[(pair_df['Fragment\r\nType']=='hcd'), 'Pair'].tolist())
            pair_df.drop(pair_df[(pair_df['Fragment\r\nType']=='ethcd')&(~(pair_df['Pair'].isin(hcd_pair_num)))].index, inplace=True)
            threshold_masks_colorind(pair_df)
            print('\n----- Exporting "_Pair" file... This may take some time, please wait. -----\n')
            pair_df.style.apply(bg_color, axis=None).to_excel(f'{date}_Pair_{filename}.xlsx', index = False)  
            print('\n----- "_Pair" file exported. -----\n')
        else:
            pass

        print('----- Start preparing simplified version. -----\n')
        
  #       extract rows included in colored indices list separately
        byonicbyos_col_range = ['Scan'] + [col for col in all_forsimple_df.columns.tolist()]
        byonicbyos_col_range = [col for col in all_forsimple_df.columns.tolist()]
        
        simple_byonicbyos = all_forsimple_df.loc[byonic_pass_id, byonicbyos_col_range] # extract colored (pass threshold) rows - below-threshold ethcd w/o pair

        simple_byonicbyos_forUnique = simple_byonicbyos[(simple_byonicbyos['N-site(SequonBased)'] != '-1')] # preserve n/a sites for potential o glycan data in unique
        simple_byonicbyos_forNglyco = simple_byonicbyos[(simple_byonicbyos['N-site(SequonBased)'] != 'N/A')&(simple_byonicbyos['N-site(SequonBased)'] != '-1')] # for simplified, n-site must exist

        # clean data by dropping duplicates        
        simple_byonicbyos_forUnique = simple_byonicbyos_forUnique.drop_duplicates().reset_index(drop=True)
        simple_byonicbyos_forNglyco = simple_byonicbyos_forNglyco.drop_duplicates().reset_index(drop=True)

        # HIGHSCORE SELECTION: get unique calc.m/z by groupby & idxmax function to find highest score within each gp and preserve (remember to split the simple_df into byonicbyos & pglyco parts to avoid dropping non-target rows)
        simple_byonicbyos_forUnique = simple_byonicbyos_forUnique.groupby(['Fragment\r\nType', 'N-site(SequonBased)', 'Calc.\r\nm/z','rounded_FinalGlycans', 'PureSequence', 'rt_dif_reset_forUnique[Byonic]'], as_index=False).apply(lambda x: x.loc[x['Score'].idxmax()]).reset_index(drop=True)
        simple_byonicbyos_forNglyco = simple_byonicbyos_forNglyco.groupby(['N-site(SequonBased)', 'Calc.\r\nm/z','rounded_FinalGlycans', 'PureSequence', 'rt_dif_reset_forNglyco[Byonic]'], as_index=False).apply(lambda x: x.loc[x['Score'].idxmax()]).reset_index(drop=True)

        # change variable name
        id_df_forUnique = simple_byonicbyos_forUnique

        # change all nan to blank -1
        id_df_forUnique = id_df_forUnique.fillna(-1)

        # extract nglycan numbers in Mods & use it to calculate byonic pep mass 
        mods_nglycan_sum = [sum([ast.literal_eval(v) for v in lst]) if type(lst) == list else -1 for lst in id_df_forUnique['Mods\r\n(variable)'].str.findall(r'Glycan / (\d+.\d+)').tolist()]
        id_df_forUnique['glycan_mods_sum'] = mods_nglycan_sum
        id_df_forUnique['PepMS'] = id_df_forUnique['Calc.\r\nMH'] - id_df_forUnique['glycan_mods_sum']
        id_df_forUnique.loc[(id_df_forUnique['glycan_mods_sum'] == -1), 'PepMS'] = -1
        move_df(id_df_forUnique, 'PepMS', 'Fragment\r\nType')

        # ast.literaleval byonicbyos n-site & sorting: byonic site -> pglyco site -> byonic peptide mass -> pglyco peptide mass -> byonic glycans -> pglyco glycans in byonic style -> byonic clam/z -> pglcyo calm/z
        id_df_forUnique['N-site(SequonBased)'] = [ast.literal_eval(i) if type(i) == str and '(' not in i and i!='N/A' else i for i in id_df_forUnique['N-site(SequonBased)'].tolist()] # convert str back to int & tuple

        # drop intermediate cols
        id_df_forUnique = id_df_forUnique.drop(['rt_dif_reset_forUnique[Byonic]', 'rt_dif_reset_forNglyco[Byonic]', 'glycan_mods_sum'], axis = 1) # drop the intermediate cols

        col_order = id_df_forUnique.columns.tolist()
        # sorting order: byonic site -> pglyco site -> byonic peptide mass -> pglyco peptide mass -> byonic glycans -> pglyco glycans in byonic style -> byonic calm/z -> pglcyo calm/z => all of this can be acheive with sort_values to multi cols.

        id_df_forUnique = sort_intstr_col(id_df_forUnique, 'z')
        print('Done sorting by z --> ', end = '')
        id_df_forUnique = sort_intstr_col(id_df_forUnique, 'Calc.\r\nMH')
        print('Done sorting by Calc.MH --> ', end = '')
        id_df_forUnique = sort_intstr_col(id_df_forUnique, 'PepMS')
        print('Done sorting by PepMS --> ', end = '')
        id_df_forUnique = sort_intstr_col(id_df_forUnique, 'rounded_FinalGlycans')
        print('Done sorting by rounded_FinalGlycans --> ', end = '')
        id_df_forUnique = sort_intstr_col(id_df_forUnique, 'PureSequence')
        print('Done sorting by PureSequence --> ', end = '')

        # convert '-1' back to int -1 for correct glycansource analysis
        id_df_forUnique['rounded_FinalGlycans'] = [int(i) if i == '-1' else i for i in id_df_forUnique['rounded_FinalGlycans']]
        id_df_forUnique = id_df_forUnique[col_order]

        print('\n----- Exporting "_UniquePep" file... This may take some time, please wait. -----\n')
        threshold_masks_colorind(id_df_forUnique)
        id_df_forUnique.style.apply(bg_color, axis=None).to_excel(f'{date}_UniquePep_{filename}.xlsx', index = False)  
        print('\n----- "_UniquePep" file exported. -----\n')

        # set scan to index & concat simple_byonicbyos & simple_pglyco by scan (for nglyco)
        simple_byonicbyos_forNglyco = simple_byonicbyos_forNglyco.set_index('Scan')
        simple_byonicbyos_forNglyco.reset_index(level=0, inplace=True)

        # change variable name
        id_df_forNglyco = simple_byonicbyos_forNglyco
        # change all nan to blank -1
        id_df_forNglyco = id_df_forNglyco.fillna(-1)

        # extract nglycan numbers in Mods & use it to calculate byonic pep mass 
        mods_nglycan_sum = [sum([ast.literal_eval(v) for v in lst]) if type(lst) == list else -1 for lst in id_df_forNglyco['Mods\r\n(variable)'].str.findall(r'Glycan / (\d+.\d+)').tolist()]
        id_df_forNglyco['glycan_mods_sum'] = mods_nglycan_sum
        id_df_forNglyco['PepMS'] = id_df_forNglyco['Calc.\r\nMH'] - id_df_forNglyco['glycan_mods_sum']
        id_df_forNglyco.loc[(id_df_forNglyco['glycan_mods_sum'] == -1), 'PepMS'] = -1
        move_df(id_df_forNglyco, 'PepMS', 'Fragment\r\nType')
        # ast.literaleval byonicbyos n-site & sorting: byonic site -> pglyco site -> byonic peptide mass -> pglyco peptide mass -> byonic glycans -> pglyco glycans in byonic style -> byonic clam/z -> pglcyo calm/z
        id_df_forNglyco['N-site(SequonBased)'] = [ast.literal_eval(i) if type(i) == str and '(' not in i else i for i in id_df_forNglyco['N-site(SequonBased)'].tolist()] # convert str back to int & tuple

        # drop intermediate cols
        id_df_forNglyco = id_df_forNglyco.drop(['rt_dif_reset_forUnique[Byonic]', 'rt_dif_reset_forNglyco[Byonic]', 'glycan_mods_sum'], axis = 1) # drop the intermediate cols

        col_order = id_df_forNglyco.columns.tolist()
        # sorting order: byonic site -> pglyco site -> byonic peptide mass -> pglyco peptide mass -> byonic glycans -> pglyco glycans in byonic style -> byonic calm/z -> pglcyo calm/z => all of this can be acheive with sort_values to multi cols.

        id_df_forNglyco = sort_intstr_col(id_df_forNglyco, 'z')
        print('Done sorting by z --> ', end = '')
        id_df_forNglyco = sort_intstr_col(id_df_forNglyco, 'Calc.\r\nMH')
        print('Done sorting by Calc.MH --> ', end = '')
        id_df_forNglyco = sort_intstr_col(id_df_forNglyco, 'PepMS')
        print('Done sorting by PepMS --> ', end = '')
        id_df_forNglyco = sort_intstr_col(id_df_forNglyco, 'rounded_FinalGlycans')
        print('Done sorting by rounded_FinalGlycans --> ', end = '')
        id_df_forNglyco = sort_intstr_col(id_df_forNglyco, 'PureSequence')
        print('Done sorting by PureSequence --> ', end = '')

        # convert '-1' back to int -1 for correct glycansource analysis
        id_df_forNglyco['rounded_FinalGlycans'] = [int(i) if i == '-1' else i for i in id_df_forNglyco['rounded_FinalGlycans']]
        id_df_forNglyco = id_df_forNglyco[col_order]

        print('\n----- Exporting "_NglycoPep" file... This may take some time, please wait. -----\n')
        threshold_masks_colorind(id_df_forNglyco)
        id_df_forNglyco.style.apply(bg_color, axis=None).to_excel(f'{date}_NglycoPep_{filename}.xlsx', index = False)  
        print('\n----- "_NglycoPep" file exported. -----\n')


        
    elif byonicfile == '' and byosfile == '' and pglycofile != '': # only pglyco
        print('----- Only pGlyco file is detected. Start the corresponding processing. -----\n')
        # change the variable name
        all_combined_df = pglyco_df
        # make sure scan is at the start
        all_combined_df = all_combined_df.set_index('Scan')
        all_combined_df.reset_index(level=0, inplace=True)

        ## style apply for excel export
        # color the rows below the threshold (threshold [byonic: score > BYONIC_SCORE_CUTOFF & pep2d < 0.001; pglyco: PepScore>5 & GlyScore>4])
        # color code => #ffedcc -> light orange for byonic; #add8e6 -> light blue for pglyco; #FFB6C1 -> light pink for byos w/ dif calm ^ seq from byonic; #FF1493 -> deep pink for byos w/ dif calm & seq from byonic; #FFFF00 -> yellow for byos & byonic all the same
        threshold_masks_colorind(all_combined_df)

        pglyco_colored_id = list(set(lightblue_ind + normalblue_ind)) # pglyco pass threshold 
        pglyco_colored_id_for_pairfile = pglyco_colored_id.copy()
        pglyco_belowthreshold_id = [i for i in all_combined_df.index.tolist() if i not in pglyco_colored_id]
        ## analyze PSM (need to pass threshold)
        # need to record the ind of below-threshold ethcd w/o pair in advance (will be excluded later, so we should not count PSM as well)
        # so here we are forced to find pair in advance (since we are forced to list psm in all file)
        
        # using groupby size function to count psm & add psm columns (new criterion: add fragment type, RT/scantime dif < 3 min (180s). is regarded as the same, >= 3 min. should be seen as dif.)
        # pglyco
        all_combined_df = all_combined_df.astype({'ProSites': 'int'})
        all_combined_df = all_combined_df.astype({'ProSites': 'str'}) # convert list & tuple to str for later groupby function
        p_df_pass = all_combined_df.loc[pglyco_colored_id]
        p_df_pass_forUnique = ScantimeGp(p_df_pass, gp_by = ['FragmentType', 'ProSites', 'rounded_PrecursorMZ','GlycanComposition_ByonicStyle', 'Peptide'], score_col = 'PepScore', rt_col = 'RT', software = '[pGlyco]', purpose = 'forUnique')
        p_df_pass_forNglyco = ScantimeGp(p_df_pass, gp_by = ['ProSites', 'rounded_PrecursorMZ','GlycanComposition_ByonicStyle', 'Peptide'], score_col = 'PepScore', rt_col = 'RT', software = '[pGlyco]', purpose = 'forNglyco')
        
        pglyco_psm = p_df_pass_forNglyco.groupby(['ProSites', 'rounded_PrecursorMZ','GlycanComposition_ByonicStyle', 'Peptide', 'rt_dif_reset_forNglyco[pGlyco]'])['ProSites'].transform('size')
        all_combined_df.loc[(pglyco_colored_id), 'PSM'] = pglyco_psm
        move_df(all_combined_df, 'PSM', 'ProSites')
        all_combined_df.loc[(pglyco_colored_id), 'rt_dif_reset_forUnique[pGlyco]'] = p_df_pass_forUnique['rt_dif_reset_forUnique[pGlyco]']
        all_combined_df.loc[(pglyco_colored_id), 'rt_dif_reset_forNglyco[pGlyco]'] = p_df_pass_forNglyco['rt_dif_reset_forNglyco[pGlyco]']
        all_combined_df.loc[(all_combined_df['ProSites'] == 'N/A'), 'PSM'] = 'N/A' # if n-site is 'N/A', do not count psm
        all_combined_df.loc[pglyco_belowthreshold_id, 'PSM'] = 'N/A' # do not count psm for rows below threshold
        all_combined_df.loc[(all_combined_df['ProSites'] == '-1'), 'PSM'] = -1 # if site is '-1', then psm set to -1
        all_forsimple_df = all_combined_df.copy() # to retain rt_dif_reset for simple df groupy and get unique row
        all_combined_df = all_combined_df.drop(['rt_dif_reset_forUnique[pGlyco]', 'rt_dif_reset_forNglyco[pGlyco]'], axis = 1)
        print('\n----- Exporting "_All" file... This may take some time, please wait. -----\n')
        all_combined_df.style.apply(bg_color, axis=None).to_excel(f'{date}_All_{filename}.xlsx', index = False)  
        print('\n----- "_All" file exported. -----\n')
        
        print('----- Start preparing simplified version. -----\n')
        # extract rows included in colored indices list separately
        pglyco_col_range = [col for col in all_forsimple_df.columns.tolist()]
        simple_pglyco = all_forsimple_df.loc[pglyco_colored_id, pglyco_col_range] # extract colored (pass threshold) rows
        
        simple_pglyco_forUnique = simple_pglyco[(simple_pglyco['ProSites'] != '-1')] # preserve n/a sites for potential o glycan data in unique
        simple_pglyco_forNglyco = simple_pglyco[(simple_pglyco['ProSites'] != 'N/A')&(simple_pglyco['ProSites'] != '-1')] # for simplified, n-site must exist

        # for nglycopep
        simple_pglyco_forNglyco = simple_pglyco_forNglyco.loc[simple_pglyco_forNglyco['FragmentType'] == 'hcd'] # pglyco etd has the same values as hcd, so only let hcd enter unique file
        
        # clean data by dropping duplicates        
        simple_pglyco_forUnique = simple_pglyco_forUnique.drop_duplicates().reset_index(drop=True)
        simple_pglyco_forNglyco = simple_pglyco_forNglyco.drop_duplicates().reset_index(drop=True)
        
        # HIGHSCORE SELECTION: get unique calc.m/z by groupby & idxmax function to find highest score within each gp and preserve (remember to split the simple_df into byonicbyos & pglyco parts to avoid dropping non-target rows)
        simple_pglyco_forUnique = simple_pglyco_forUnique.groupby(['FragmentType', 'ProSites', 'rounded_PrecursorMZ','GlycanComposition_ByonicStyle', 'Peptide', 'rt_dif_reset_forUnique[pGlyco]'], as_index=False).apply(lambda x: x.loc[x['PepScore'].idxmax()]).reset_index(drop=True)
        simple_pglyco_forNglyco = simple_pglyco_forNglyco.groupby(['ProSites', 'rounded_PrecursorMZ','GlycanComposition_ByonicStyle', 'Peptide', 'rt_dif_reset_forNglyco[pGlyco]'], as_index=False).apply(lambda x: x.loc[x['PepScore'].idxmax()]).reset_index(drop=True)

        id_df_forUnique = simple_pglyco_forUnique.fillna(-1)
        # extract nglycan numbers in Mods & use it to calculate byonic pep mass 
        # ast.literaleval byonicbyos n-site & sorting: byonic site -> pglyco site -> byonic peptide mass -> pglyco peptide mass -> byonic glycans -> pglyco glycans in byonic style -> byonic clam/z -> pglcyo calm/z
        id_df_forUnique['ProSites'] = [ast.literal_eval(i) if type(i) == str and '(' not in i and i!='N/A' else i for i in id_df_forUnique['ProSites'].tolist()] # convert str back to int & tuple
        
        # drop intermediate cols
        id_df_forUnique = id_df_forUnique.drop(['rt_dif_reset_forUnique[pGlyco]', 'rt_dif_reset_forNglyco[pGlyco]'], axis = 1) # drop the intermediate cols
        col_order = id_df_forUnique.columns.tolist()
        # sorting order: byonic site -> pglyco site -> byonic peptide mass -> pglyco peptide mass -> byonic glycans -> pglyco glycans in byonic style -> byonic calm/z -> pglcyo calm/z => all of this can be acheive with sort_values to multi cols.

        id_df_forUnique = sort_intstr_col(id_df_forUnique, 'Charge')
        print('Done sorting by Charge --> ', end = '')
        id_df_forUnique = sort_intstr_col(id_df_forUnique, 'PrecursorMH')
        print('Done sorting by PrecursorMH --> ', end = '')
        id_df_forUnique = sort_intstr_col(id_df_forUnique, 'PeptideMH')
        print('Done sorting by PeptideMH --> ', end = '')
        id_df_forUnique = sort_intstr_col(id_df_forUnique, 'GlycanComposition_ByonicStyle')
        print('Done sorting by GlycanComposition_ByonicStyle --> ', end = '')
        id_df_forUnique = sort_intstr_col(id_df_forUnique, 'Peptide')
        print('Done sorting by Peptide --> ', end = '')

        # convert '-1' back to int -1 for correct glycansource analysis
        id_df_forUnique['GlycanComposition_ByonicStyle'] = [int(i) if i == '-1' else i for i in id_df_forUnique['GlycanComposition_ByonicStyle']]
        id_df_forUnique = id_df_forUnique[col_order]

        print('\n----- Exporting "_UniquePep" file... This may take some time, please wait. -----\n')
        threshold_masks_colorind(id_df_forUnique)
        id_df_forUnique.style.apply(bg_color, axis=None).to_excel(f'{date}_UniquePep_{filename}.xlsx', index = False)  
        print('\n----- "_UniquePep" file exported. -----\n')

        # set scan to index & concat simple_byonicbyos & simple_pglyco by scan (for nglyco)
        simple_pglyco_forNglyco = simple_pglyco_forNglyco.set_index('Scan')
        simple_pglyco_forNglyco.reset_index(level=0, inplace=True)
        # change all nan to blank -1
        id_df_forNglyco = simple_pglyco_forNglyco.fillna(-1)

        # extract nglycan numbers in Mods & use it to calculate byonic pep mass 
        # ast.literaleval byonicbyos n-site & sorting: byonic site -> pglyco site -> byonic peptide mass -> pglyco peptide mass -> byonic glycans -> pglyco glycans in byonic style -> byonic clam/z -> pglcyo calm/z
        id_df_forNglyco['ProSites'] = [ast.literal_eval(i) if type(i) == str and '(' not in i else i for i in id_df_forNglyco['ProSites'].tolist()] # convert str back to int & tuple

        # drop intermediate cols
        id_df_forNglyco = id_df_forNglyco.drop(['rt_dif_reset_forUnique[pGlyco]', 'rt_dif_reset_forNglyco[pGlyco]'], axis = 1) # drop the intermediate cols
        col_order = id_df_forNglyco.columns.tolist()
        # sorting order: byonic site -> pglyco site -> byonic peptide mass -> pglyco peptide mass -> byonic glycans -> pglyco glycans in byonic style -> byonic calm/z -> pglcyo calm/z => all of this can be acheive with sort_values to multi cols.

        id_df_forNglyco = sort_intstr_col(id_df_forNglyco, 'Charge')
        print('Done sorting by Charge --> ', end = '')
        id_df_forNglyco = sort_intstr_col(id_df_forNglyco, 'PrecursorMH')
        print('Done sorting by PrecursorMH --> ', end = '')
        id_df_forNglyco = sort_intstr_col(id_df_forNglyco, 'PeptideMH')
        print('Done sorting by PeptideMH --> ', end = '')
        id_df_forNglyco = sort_intstr_col(id_df_forNglyco, 'GlycanComposition_ByonicStyle')
        print('Done sorting by GlycanComposition_ByonicStyle --> ', end = '')
        id_df_forNglyco = sort_intstr_col(id_df_forNglyco, 'Peptide')
        print('Done sorting by Peptide --> ', end = '')
        
        # convert '-1' back to int -1 for correct glycansource analysis
        id_df_forNglyco['GlycanComposition_ByonicStyle'] = [int(i) if i == '-1' else i for i in id_df_forNglyco['GlycanComposition_ByonicStyle']]
        id_df_forNglyco = id_df_forNglyco[col_order]

        print('\n----- Exporting "_NglycoPep" file... This may take some time, please wait. -----\n')
        threshold_masks_colorind(id_df_forNglyco)
        id_df_forNglyco.style.apply(bg_color, axis=None).to_excel(f'{date}_NglycoPep_{filename}.xlsx', index = False)  
        print('\n----- "_NglycoPep" file exported. -----\n')

        print('----- Start preparing quantification file of N-glycosylation . -----\n')
        # only single sites for quant (quant_df offers important info. for later multiIndex df construction)
        quant = id_df_forNglyco.loc[:, ['ProSites', 'GlycanComposition_ByonicStyle', 'MonoArea', 'IsotopeArea']]
        # calculate the sum within each n-site, then normalize mono/ iso by these sum
        quant['sumpersite_mono'] = quant.groupby(['ProSites'])['MonoArea'].transform(lambda x: x[x!=-1].sum())
        quant['sumpersite_iso'] = quant.groupby(['ProSites'])['IsotopeArea'].transform(lambda x: x[x!=-1].sum())
        # using groupby transform sum function to add 8 cols (byos xicauc/byos area int/pglycomono/pglycoisotope) recording summed values & normalized values (same site & same glycan & seq can be dif)
        quant['g_sum_MonoArea'] = quant.groupby(['ProSites', 'GlycanComposition_ByonicStyle'])['MonoArea'].transform(lambda x: x[x != -1].sum())
        quant['h_sum_IsotopeArea'] = quant.groupby(['ProSites', 'GlycanComposition_ByonicStyle'])['IsotopeArea'].transform(lambda x: x[x != -1].sum())
        quant['c_norm_MonoArea'] = quant['g_sum_MonoArea']/quant['sumpersite_mono']
        quant['d_norm_IsotopeArea'] = quant['h_sum_IsotopeArea']/quant['sumpersite_iso']
        # since it's possible to have real 0 from calculation, change the real absent data back to -1 in sum_ & norm_
        quant.loc[quant['MonoArea'] == -1, ['g_sum_MonoArea', 'h_sum_IsotopeArea', 'c_norm_MonoArea', 'd_norm_IsotopeArea']] = -1
        # extract the needed cols only & split byonicbyos/pglyco, drop all -1 rows, glycans as index to outer union concat data again
        quant_glycanid = quant.set_index('GlycanComposition_ByonicStyle')
        quant_glycanid.index.name = 'Glycans ↓'
        quant_glycanid.reset_index(level=0, inplace=True)
        # add N(x) col
        nx = ['N(0)' if lst == [] else 'N(%s)'%(lst[0]) for lst in quant_glycanid['Glycans ↓'].str.findall(r'HexNAc\((\d+)\)').tolist()]
        quant_glycanid.insert(0 , 'N(x) ↓', nx , True)
        # change all nan to blank -1
        quant_glycanid = quant_glycanid.fillna(-1)
        quant_glycanid = quant_glycanid.drop_duplicates() # remember to drop duplicates after splitting & reconcat df

        ## onlyp
        onlyp = quant_glycanid.loc[(quant_glycanid['ProSites'] != -1), [col for col in quant_glycanid.columns.tolist()]].drop(['MonoArea', 'IsotopeArea', 'sumpersite_mono', 'sumpersite_iso'], axis=1).drop_duplicates()
        # clean off all no data rows
        onlyp = onlyp.loc[~((onlyp['g_sum_MonoArea']==-1)&(onlyp['h_sum_IsotopeArea']==-1)&(onlyp['c_norm_MonoArea']==-1)&(onlyp['d_norm_IsotopeArea']==-1))]
        nx = list(set(onlyp['N(x) ↓'].tolist()))
        nx = [int(re.findall(r'[0-9]+', i)[0]) if type(i) == str else i for i in nx]
        nx.sort(key=lambda v: (isinstance(v, str), v))
        p_new_nx = ['N(%s)'%(str(i)) if type(i) == int and i != -1 else i for i in nx]

        print('----- Start summary table construction. -----\n')
        # start grouping: N(X) -> byonic glycans -> byonic sites -> .agg({all 8 cols, apply .mean() & .mean()/total XXX})

        # onlyp
        onlyp_gp = onlyp.groupby(['N(x) ↓', 'Glycans ↓', 'ProSites']).agg({'g_sum_MonoArea': lambda x: x.mean(), 'h_sum_IsotopeArea': lambda x: x.mean() \
                                                                                              , 'c_norm_MonoArea': lambda x: x.mean(), 'd_norm_IsotopeArea': lambda x: x.mean()})

        # fix the str '10' sorted before '2' problem
        onlyp_gp = onlyp_gp.reindex(p_new_nx, level = 0)
        # unstack byonic n-sites, pglyco n-sites
        onlyp_gp = onlyp_gp.unstack(level=-1)
        # multiIndex col swaplevel
        onlyp_gp.columns = onlyp_gp.columns.swaplevel(0, 1)
        # sort is necessary for the right table structure (but since it will sort every level, so we need to manually adjust level3 back to the original order)
        onlyp_gp.sort_index(axis=1, level=[0, 1], inplace=True)
        onlyp_gp.columns = onlyp_gp.columns.set_names(['ProSites →', 'Quant. →'])
        # delete a-h in Quant. (the alphabet is just for sorting)
        onlyp_gp.columns.set_levels(['norm_MonoArea', 'norm_IsotopeArea', 'sum_MonoArea', 'sum_IsotopeArea'], level = 1, inplace=True)

        print('\n----- Exporting "_Quant" files... This may take some time, please wait. -----\n')
        
        with pd.ExcelWriter(f'{date}_Quant_{filename}.xlsx') as writer:           
            # onlyp
            onlyp_gp.replace(to_replace = -1, value = np.nan , inplace = True) 
            onlyp_gp.style.background_gradient(cmap ='Blues').highlight_null(null_color='white').to_excel(writer, sheet_name = 'Sheet2(QuantP)')
            
        print('\n----- "_Quant" file exported. -----\n')
        
        print('----- Start plotting. -----\n')

        pNORM_dfall_lstv1, pNORM_label_lstv1, pNORM_dfall_lstv2, pNORM_label_lstv2 \
        , pNORM_dfall_lstv3, pNORM_label_lstv3, pNORM_dfall_lstv4, pNORM_label_lstv4 \
        , pSUM_dfall_lstv1, pSUM_label_lstv1, pSUM_nsum_lstv1, pSUM_dfall_lstv2, pSUM_label_lstv2, pSUM_nsum_lstv2 \
        , pSUM_dfall_lstv3, pSUM_label_lstv3, pSUM_nsum_lstv3, pSUM_dfall_lstv4, pSUM_label_lstv4, pSUM_nsum_lstv4 \
        = [], [], [], [], [], [], [], [], [], [] \
        , [], [], [], [], [], [], [], [], [], [] 
        
        if plot_v2 == 'yes':  
            col_order = ['N(0)', 'N(1)', 'OligoMannose', 'Complex', 'Complex+Sia'] # col order for v2 plotting
            # necessary preprocessings for v2 plotting (modified from v1 nx)
            # onlyp v2
            onlyp.loc[(onlyp['N(x) ↓'] == 'N(0)')|(onlyp['N(x) ↓'] == 'N(1)'), 'nx_version2'] = onlyp['N(x) ↓']
            onlyp.loc[(onlyp['N(x) ↓'] == 'N(2)'), 'nx_version2'] = 'OligoMannose'
            onlyp.loc[(onlyp['N(x) ↓'] != 'N(0)')&(onlyp['N(x) ↓'] != 'N(1)')&(onlyp['N(x) ↓'] != 'N(2)'), 'nx_version2'] = 'Complex'
            onlyp.loc[(onlyp['N(x) ↓'] != 'N(0)')&(onlyp['N(x) ↓'] != 'N(1)')&(onlyp['N(x) ↓'] != 'N(2)')&(onlyp['Glycans ↓'].str.contains('NeuAc|NeuGc')), 'nx_version2'] = 'Complex+Sia'
            p_neugc_exist = sum(onlyp['Glycans ↓'].str.contains('NeuGc')) > 0 # Bool, if T, red. else purple
        
        if plot_v3 == 'yes':
            # necessary preprocessings for v3 plotting (modified from v1 nx)           
            # onlyp v3
            onlyp, onlyp_nxhy_fix_order = v3(onlyp)[0], v3(onlyp)[1]

        if plot_v4 == 'yes':
            # necessary preprocessings for v4 plotting (modified from v1 nx)            
            # onlyp v4
            onlyp, onlyp_fnxhy_fix_order = v4(onlyp)[0], v4(onlyp)[1]

        # # make sure that sites are presented as int not float
        onlyp = onlyp.astype({'ProSites':'Int64'})

        # split bp_union into: xicauc_df / int_df / mono_df / iso_df for later clustered stacked plot (norm only)
        if export_mono == 'yes':
            if plot_v1 == 'yes':
                # onlyp norm v1
                p_monoNORM_df = onlyp.loc[:, ['N(x) ↓', 'ProSites', 'c_norm_MonoArea']]
                p_monoNORM_df.replace(to_replace = -1, value = np.nan , inplace = True)
                p_monoNORM_df = p_monoNORM_df.pivot_table(index='ProSites', values='c_norm_MonoArea', columns='N(x) ↓', aggfunc=np.sum).reindex(columns=p_new_nx)
                pNORM_dfall_lstv1.append(p_monoNORM_df)
                pNORM_label_lstv1.append('norm_MonoArea')
                # onlyp sum v1
                p_monoSUM_df = onlyp.loc[:, ['N(x) ↓', 'ProSites', 'g_sum_MonoArea']]
                p_monoSUM_df.replace(to_replace = -1, value = np.nan , inplace = True)
                p_monoSUM_df = p_monoSUM_df.pivot_table(index='ProSites', values='g_sum_MonoArea', columns='N(x) ↓', aggfunc=np.sum).reindex(columns=p_new_nx)
                p_monoSUM_df_nsum = [p_monoSUM_df.loc[i].sum() for i in p_monoSUM_df.index.tolist()] # for A bar label
                pSUM_dfall_lstv1.append(p_monoSUM_df)
                pSUM_label_lstv1.append('sum_MonoArea')
                pSUM_nsum_lstv1.append(p_monoSUM_df_nsum)

            if plot_v2 == 'yes':
                # onlyp norm v2
                p_monoNORMv2_df = onlyp.loc[:, ['nx_version2', 'ProSites', 'c_norm_MonoArea']]
                p_monoNORMv2_df.replace(to_replace = -1, value = np.nan , inplace = True)
                p_monoNORMv2_df = p_monoNORMv2_df.pivot_table(index='ProSites', values='c_norm_MonoArea', columns='nx_version2', aggfunc=np.sum).reindex(columns=col_order)
                pNORM_dfall_lstv2.append(p_monoNORMv2_df)
                pNORM_label_lstv2.append('norm_MonoArea')
                # onlyp sum v2
                p_monoSUMv2_df = onlyp.loc[:, ['nx_version2', 'ProSites', 'g_sum_MonoArea']]
                p_monoSUMv2_df.replace(to_replace = -1, value = np.nan , inplace = True)
                p_monoSUMv2_df = p_monoSUMv2_df.pivot_table(index='ProSites', values='g_sum_MonoArea', columns='nx_version2', aggfunc=np.sum).reindex(columns=col_order)
                p_monoSUMv2_df_nsum = [p_monoSUMv2_df.loc[i].sum() for i in p_monoSUMv2_df.index.tolist()] # for A bar label
                pSUM_dfall_lstv2.append(p_monoSUMv2_df)
                pSUM_label_lstv2.append('sum_MonoArea')
                pSUM_nsum_lstv2.append(p_monoSUMv2_df_nsum)

            if plot_v3 == 'yes':
                # onlyp norm v3
                p_monoNORMv3_df = onlyb.loc[:, ['N(x)H(y)', 'ProSites', 'c_norm_MonoArea']]
                p_monoNORMv3_df.replace(to_replace = -1, value = np.nan , inplace = True)
                p_monoNORMv3_df = p_monoNORMv3_df.pivot_table(index='ProSites', values='c_norm_MonoArea', columns='N(x)H(y)', aggfunc=np.sum).reindex(columns=onlyp_nxhy_fix_order)
                pNORM_dfall_lstv3.append(p_monoNORMv3_df)
                pNORM_label_lstv3.append('norm_MonoArea')
                # onlyp sum v3
                p_monoSUMv3_df = onlyp.loc[:, ['N(x)H(y)', 'ProSites', 'g_sum_MonoArea']]
                p_monoSUMv3_df.replace(to_replace = -1, value = np.nan , inplace = True)
                p_monoSUMv3_df = p_monoSUMv3_df.pivot_table(index='ProSites', values='g_sum_MonoArea', columns='N(x)H(y)', aggfunc=np.sum).reindex(columns=onlyp_nxhy_fix_order)
                p_monoSUMv3_df_nsum = [p_monoSUMv3_df.loc[i].sum() for i in p_monoSUMv3_df.index.tolist()] # for A bar label
                pSUM_dfall_lstv3.append(p_monoSUMv3_df)
                pSUM_label_lstv3.append('sum_MonoArea')
                pSUM_nsum_lstv3.append(p_monoSUMv3_df_nsum)

            if plot_v4 == 'yes':
                # onlyp norm v4
                p_monoNORMv4_df = onlyp.loc[:, ['(F)N(x)H(y)', 'ProSites', 'c_norm_MonoArea']]
                p_monoNORMv4_df.replace(to_replace = -1, value = np.nan , inplace = True)
                p_monoNORMv4_df = p_monoNORMv4_df.pivot_table(index='N-site(SequonBased)[Byonic]', values='c_norm_MonoArea', columns='(F)N(x)H(y)', aggfunc=np.sum).reindex(columns=onlyp_fnxhy_fix_order)
                pNORM_dfall_lstv4.append(p_monoNORMv4_df)
                pNORM_label_lstv4.append('norm_MonoArea')
                # onlyp sum v4
                p_monoSUMv4_df = onlyp.loc[:, ['(F)N(x)H(y)', 'ProSites', 'g_sum_MonoArea']]
                p_monoSUMv4_df.replace(to_replace = -1, value = np.nan , inplace = True)
                p_monoSUMv4_df = p_monoSUMv4_df.pivot_table(index='ProSites', values='g_sum_MonoArea', columns='(F)N(x)H(y)', aggfunc=np.sum).reindex(columns=onlyp_fnxhy_fix_order)
                p_monoSUMv4_df_nsum = [p_monoSUMv4_df.loc[i].sum() for i in p_monoSUMv4_df.index.tolist()] # for A bar label
                pSUM_dfall_lstv4.append(p_monoSUMv4_df)
                pSUM_label_lstv4.append('sum_MonoArea')
                pSUM_nsum_lstv4.append(p_monoSUMv4_df_nsum)     
        else:
            pass

        if export_isotope == 'yes':
            if plot_v1 == 'yes':
                # onlyp norm v1
                p_isoNORM_df = onlyp.loc[:, ['N(x) ↓', 'ProSites', 'd_norm_IsotopeArea']]
                p_isoNORM_df.replace(to_replace = -1, value = np.nan , inplace = True)
                p_isoNORM_df = p_isoNORM_df.pivot_table(index='ProSites', values='d_norm_IsotopeArea', columns='N(x) ↓', aggfunc=np.sum).reindex(columns=p_new_nx)
                pNORM_dfall_lstv1.append(p_isoNORM_df)
                pNORM_label_lstv1.append('norm_IsotopeArea')
                # onlyp sum v1
                p_isoSUM_df = onlyp.loc[:, ['N(x) ↓', 'ProSites', 'h_sum_IsotopeArea']] 
                p_isoSUM_df.replace(to_replace = -1, value = np.nan , inplace = True)
                p_isoSUM_df = p_isoSUM_df.pivot_table(index='ProSites', values='h_sum_IsotopeArea', columns='N(x) ↓', aggfunc=np.sum).reindex(columns=p_new_nx)
                p_isoSUM_df_nsum = [p_isoSUM_df.loc[i].sum() for i in p_isoSUM_df.index.tolist()] # for B bar label
                pSUM_dfall_lstv1.append(p_isoSUM_df)
                pSUM_label_lstv1.append('sum_IsotopeArea')
                pSUM_nsum_lstv1.append(p_isoSUM_df_nsum)

            if plot_v2 == 'yes':
                # onlyp norm v2
                p_isoNORMv2_df = onlyp.loc[:, ['nx_version2', 'ProSites', 'd_norm_IsotopeArea']]
                p_isoNORMv2_df.replace(to_replace = -1, value = np.nan , inplace = True)
                p_isoNORMv2_df = p_isoNORMv2_df.pivot_table(index='ProSites', values='d_norm_IsotopeArea', columns='nx_version2', aggfunc=np.sum).reindex(columns=col_order)
                pNORM_dfall_lstv2.append(p_isoNORMv2_df)
                pNORM_label_lstv2.append('norm_IsotopeArea')
                # onlyp sum v2
                p_isoSUMv2_df = onlyp.loc[:, ['nx_version2', 'ProSites', 'h_sum_IsotopeArea']] 
                p_isoSUMv2_df.replace(to_replace = -1, value = np.nan , inplace = True)
                p_isoSUMv2_df = p_isoSUMv2_df.pivot_table(index='ProSites', values='h_sum_IsotopeArea', columns='nx_version2', aggfunc=np.sum).reindex(columns=col_order)
                p_isoSUMv2_df_nsum = [p_isoSUMv2_df.loc[i].sum() for i in p_isoSUMv2_df.index.tolist()] # for B bar label
                pSUM_dfall_lstv2.append(p_isoSUMv2_df)
                pSUM_label_lstv2.append('sum_IsotopeArea')
                pSUM_nsum_lstv2.append(p_isoSUMv2_df_nsum)

            if plot_v3 == 'yes':
                # onlyp norm v3
                p_isoNORMv3_df = onlyp.loc[:, ['N(x)H(y)', 'ProSites', 'd_norm_IsotopeArea']]
                p_isoNORMv3_df.replace(to_replace = -1, value = np.nan , inplace = True)
                p_isoNORMv3_df = p_isoNORMv3_df.pivot_table(index='ProSites', values='d_norm_IsotopeArea', columns='N(x)H(y)', aggfunc=np.sum).reindex(columns=onlyp_nxhy_fix_order)
                pNORM_dfall_lstv3.append(p_isoNORMv3_df)
                pNORM_label_lstv3.append('norm_IsotopeArea')
                # onlyp sum v3
                p_isoSUMv3_df = onlyp.loc[:, ['N(x)H(y)', 'ProSites', 'h_sum_IsotopeArea']]
                p_isoSUMv3_df.replace(to_replace = -1, value = np.nan , inplace = True)
                p_isoSUMv3_df = p_isoSUMv3_df.pivot_table(index='ProSites', values='h_sum_IsotopeArea', columns='N(x)H(y)', aggfunc=np.sum).reindex(columns=onlyp_nxhy_fix_order)
                p_isoSUMv3_df_nsum = [p_isoSUMv3_df.loc[i].sum() for i in p_isoSUMv3_df.index.tolist()] # for A bar label
                pSUM_dfall_lstv3.append(p_isoSUMv3_df)
                pSUM_label_lstv3.append('sum_IsotopeArea')
                pSUM_nsum_lstv3.append(p_isoSUMv3_df_nsum)

            if plot_v4 == 'yes':
                # onlyp norm v4
                p_isoNORMv4_df = onlyp.loc[:, ['(F)N(x)H(y)', 'ProSites', 'd_norm_IsotopeArea']]
                p_isoNORMv4_df.replace(to_replace = -1, value = np.nan , inplace = True)
                p_isoNORMv4_df = p_isoNORMv4_df.pivot_table(index='ProSites', values='d_norm_IsotopeArea', columns='(F)N(x)H(y)', aggfunc=np.sum).reindex(columns=onlyp_fnxhy_fix_order)
                pNORM_dfall_lstv4.append(p_isoNORMv4_df)
                pNORM_label_lstv4.append('norm_IsotopeArea')
                # onlyp sum v4
                p_isoSUMv4_df = onlyp.loc[:, ['(F)N(x)H(y)', 'ProSites', 'h_sum_IsotopeArea']]
                p_isoSUMv4_df.replace(to_replace = -1, value = np.nan , inplace = True)
                p_isoSUMv4_df = p_isoSUMv4_df.pivot_table(index='ProSites', values='h_sum_IsotopeArea', columns='(F)N(x)H(y)', aggfunc=np.sum).reindex(columns=onlyp_fnxhy_fix_order)
                p_isoSUMv4_df_nsum = [p_isoSUMv4_df.loc[i].sum() for i in p_isoSUMv4_df.index.tolist()] # for A bar label
                pSUM_dfall_lstv4.append(p_isoSUMv4_df)
                pSUM_label_lstv4.append('sum_IsotopeArea')
                pSUM_nsum_lstv4.append(p_isoSUMv4_df_nsum)    
        else:
            pass

        if export_mono == export_isotope == 'no': # no plots will be exported
            pass

        else: # plotting criterium: cannot be empty list

            if pNORM_dfall_lstv1 != []:
                # onlyp NORMv1
                fig = plot_clustered_stacked(pNORM_dfall_lstv1,[[1]*len(pNORM_dfall_lstv1[0].index) for i in range(len(pNORM_dfall_lstv1))],pNORM_label_lstv1)
                fig.tight_layout()
                fig.savefig(f'{date}_onlypNORMv1_{filename}.png', bbox_inches='tight')
            if pNORM_dfall_lstv2 != []:
                # onlyp NORMv2
                fig = plot_clustered_stacked(pNORM_dfall_lstv2,[[1]*len(pNORM_dfall_lstv2[0].index) for i in range(len(pNORM_dfall_lstv2))],pNORM_label_lstv2, neugc_exist=p_neugc_exist)
                fig.tight_layout()
                fig.savefig(f'{date}_onlypNORMv2_{filename}.png', bbox_inches='tight')
            if pNORM_dfall_lstv3 != []:
                # onlyp NORMv3
                fig = plot_clustered_stacked(pNORM_dfall_lstv3,[[1]*len(pNORM_dfall_lstv3[0].index) for i in range(len(pNORM_dfall_lstv3))],pNORM_label_lstv3, detailedHighMan=True)
                fig.tight_layout()
                fig.savefig(f'{date}_onlypNORMv3_{filename}.png', bbox_inches='tight')
            if pNORM_dfall_lstv4 != []:
                # onlyp NORMv4
                fig = plot_clustered_stacked(pNORM_dfall_lstv4,[[1]*len(pNORM_dfall_lstv4[0].index) for i in range(len(pNORM_dfall_lstv4))],pNORM_label_lstv4, detailedHighMan=True, fucosylation=True)
                fig.tight_layout()
                fig.savefig(f'{date}_onlypNORMv4_{filename}.png', bbox_inches='tight')


            if pSUM_dfall_lstv1 != []:
                # onlyp SUMv1
                fig = plot_clustered_stacked(pSUM_dfall_lstv1,pSUM_nsum_lstv1,pSUM_label_lstv1)
                fig.tight_layout()
                fig.savefig(f'{date}_onlypSUMv1_{filename}.png', bbox_inches='tight')
            if pSUM_dfall_lstv2 != []:
                # onlyp SUMv2
                fig = plot_clustered_stacked(pSUM_dfall_lstv2,pSUM_nsum_lstv2 ,pSUM_label_lstv2, neugc_exist=p_neugc_exist)
                fig.tight_layout()
                fig.savefig(f'{date}_onlypSUMv2_{filename}.png', bbox_inches='tight')
            if pSUM_dfall_lstv3 != []:
                # onlyp SUMv3
                fig = plot_clustered_stacked(pSUM_dfall_lstv3,pSUM_nsum_lstv3,pSUM_label_lstv3, detailedHighMan=True)
                fig.tight_layout()
                fig.savefig(f'{date}_onlypSUMv3_{filename}.png', bbox_inches='tight')
            if pSUM_dfall_lstv4 != []:
                # onlyp SUMv4
                fig = plot_clustered_stacked(pSUM_dfall_lstv4,pSUM_nsum_lstv4 ,pSUM_label_lstv4, detailedHighMan=True, fucosylation=True)
                fig.tight_layout()
                fig.savefig(f'{date}_onlypSUMv4_{filename}.png', bbox_inches='tight')

    elif byonicfile == '' and byosfile != '' and pglycofile == '': # only byos 
        print('----- Only Byos file is detected. Please at least submit either Byonic or pGlyco file for further processing. -----\n')
    elif byonicfile == '' and byosfile != '' and pglycofile == '': # byos + pglyco 
        print('----- Byos & pGlyco files are detected. Please submit Byonic file for further processing. -----\n')
    elif byonicfile != '' and byosfile != '' and pglycofile == '': # byonic + byos 
        print('----- Start combining Byonic & Byos. -----\n')
        # combined data based on 'Scan'
        byonic_scanasid = byonic_df.copy()
        new_byonic_col = [n + '[Byonic]' if n != 'Scan' else n for n in byonic_scanasid.columns]
        byonic_scanasid.columns = new_byonic_col
        byos_scanasid = byos_df.copy()
        new_byos_col = [n + '[Byos]' if n != 'Scan Number(s)\r\n(Posit)' else n for n in byos_scanasid.columns]
        byos_scanasid.columns = new_byos_col
        byonic_scanasid = byonic_scanasid.set_index('Scan')
        byos_scanasid = byos_scanasid.set_index('Scan Number(s)\r\n(Posit)')
        # align scan & concat (all align on row to make row number all the same)
        a1, a2 = byonic_scanasid.align(byos_scanasid, join = 'outer', axis = 0) # row: a1 = a2
        all_combined_df = pd.concat([a1,a2], axis = 1)
        all_combined_df.index.name = 'Scan'
        all_combined_df.reset_index(level=0, inplace=True)
        # change all nan to blank -1
        all_combined_df = all_combined_df.fillna(-1)
        ## result post-processing 
        print('\nCombined data shape:\nrow --> %s, column --> %s'%(all_combined_df.shape[0], all_combined_df.shape[1]))
        ## style apply for excel export
        # color the rows below the threshold (threshold [byonic: score > BYONIC_SCORE_CUTOFF & pep2d < 0.001; pglyco: PepScore>5 & GlyScore>4])
        # color code => #ffedcc -> light orange for byonic; #add8e6 -> light blue for pglyco; #FFB6C1 -> light pink for byos w/ dif calm ^ seq from byonic; #FF1493 -> deep pink for byos w/ dif calm & seq from byonic; #FFFF00 -> yellow for byos & byonic all the same
        threshold_masks_colorind(all_combined_df)
        byonic_colored_id = list(set(lightgreen_ind + normalgreen_ind)) # byonic pass threshold (include below-threshold ethcd w/o pair) 
        byonic_colored_id_for_pairfile = byonic_colored_id.copy()
        ## analyze PSM (need to pass threshold)
        # need to record the ind of below-threshold ethcd w/o pair in advance (will be excluded later, so we should not count PSM as well)
        # so here we are forced to find pair in advance (since we are forced to list psm in all file)
        if 'Pair[Byonic]' in all_combined_df.columns.tolist(): # pair is to identify etd with pass-threshold hcd pair for later rescue
            # ONLY FOR HCD/ETD FILES: prepare '_Pair' file (as long as byonic passes threshold & have pair)
            all_combined_df = all_combined_df.astype({'Pair[Byonic]': 'str'})
            pair_df = all_combined_df.loc[byonic_colored_id]
            pair_df = pair_df[pair_df['Pair[Byonic]'].str.contains('pair')]
        #             pair_df = pair_df[pair_df.duplicated(subset=['Pair[Byonic]'], keep=False)] # only paired data included in _Pair
            if 'ethcd' in all_combined_df['Fragment\r\nType[Byonic]'].tolist(): 
                hcd_pair_num = set(pair_df.loc[(pair_df['Fragment\r\nType[Byonic]']=='hcd'), 'Pair[Byonic]'].tolist())
                pair_df.drop(pair_df[(pair_df['Fragment\r\nType[Byonic]']=='ethcd')&(~(pair_df['Pair[Byonic]'].isin(hcd_pair_num)))].index, inplace=True)
        else:
            pass

        if 'ethcd' in all_combined_df['Fragment\r\nType[Byonic]'].tolist(): # if etd data is present (i.e. have pair)
            byonic_excluded_ethcd_ind =  all_combined_df.loc[((all_combined_df['Fragment\r\nType[Byonic]'] == 'ethcd')&((all_combined_df['Score[Byonic]'] <= BYONIC_SCORE_CUTOFF)|(all_combined_df['PEP\r\n2D[Byonic]'].abs() >= 0.001))&(~(all_combined_df['Pair[Byonic]'].isin(pair_df['Pair[Byonic]']))))].index.tolist()
            byonic_pass_id = [i for i in byonic_colored_id if i not in byonic_excluded_ethcd_ind]
            byonic_belowthreshold_id = [i for i in all_combined_df.index.tolist() if i not in byonic_pass_id] 
        else: # all hcd, all in (cause here all the hcd are above threshold, i.e. all colored)
            byonic_pass_id = byonic_colored_id 
            byonic_belowthreshold_id = [i for i in all_combined_df.index.tolist() if i not in byonic_pass_id] 

        # using groupby size function to count psm & add psm columns (new criterion: add fragment type, RT/scantime dif < 3 min (180s). is regarded as the same, >= 3 min. should be seen as dif.)
        # byonic & byos
        all_combined_df = all_combined_df.astype({'N-site(SequonBased)[Byonic]': 'str'}) # convert list & tuple to str for later groupby function
        b_df_pass = all_combined_df.loc[byonic_pass_id]
        b_df_pass_forUnique = ScantimeGp(b_df_pass, gp_by = ['Fragment\r\nType[Byonic]', 'N-site(SequonBased)[Byonic]',  'Calc.\r\nm/z[Byonic]','rounded_FinalGlycans[Byonic]', 'PureSequence[Byonic]'], score_col = 'Score[Byonic]', rt_col = 'Scan\r\nTime[Byonic]', software = '[Byonic]', purpose = 'forUnique')
        b_df_pass_forNglyco = ScantimeGp(b_df_pass, gp_by = ['N-site(SequonBased)[Byonic]',  'Calc.\r\nm/z[Byonic]','rounded_FinalGlycans[Byonic]', 'PureSequence[Byonic]'], score_col = 'Score[Byonic]', rt_col = 'Scan\r\nTime[Byonic]', software = '[Byonic]', purpose = 'forNglyco')
        byonic_psm = b_df_pass_forNglyco.groupby(['N-site(SequonBased)[Byonic]', 'Calc.\r\nm/z[Byonic]','rounded_FinalGlycans[Byonic]', 'PureSequence[Byonic]', 'rt_dif_reset_forNglyco[Byonic]'])['N-site(SequonBased)[Byonic]'].transform('size')

        all_combined_df.loc[(byonic_pass_id), 'PSM[Byonic]'] = byonic_psm
        move_df(all_combined_df, 'PSM[Byonic]', 'N-site(SequonBased)[Byonic]')
        all_combined_df.loc[(byonic_pass_id), 'rt_dif_reset_forUnique[Byonic]'] = b_df_pass_forUnique['rt_dif_reset_forUnique[Byonic]']
        all_combined_df.loc[(byonic_pass_id), 'rt_dif_reset_forNglyco[Byonic]'] = b_df_pass_forNglyco['rt_dif_reset_forNglyco[Byonic]']
        all_combined_df.loc[(all_combined_df['N-site(SequonBased)[Byonic]'] == 'N/A'), 'PSM[Byonic]'] = 'N/A' # if n-site is 'N/A', do not count psm
        all_combined_df.loc[byonic_belowthreshold_id, 'PSM[Byonic]'] = 'N/A' # do not count psm for rows below threshold
        all_combined_df.loc[(all_combined_df['N-site(SequonBased)[Byonic]'] == '-1'), 'PSM[Byonic]'] = -1 # if site is '-1', then psm set to -1
        all_forsimple_df = all_combined_df.copy() # to retain rt_dif_reset for simple df groupy and get unique row
        all_combined_df = all_combined_df.drop(['rt_dif_reset_forUnique[Byonic]', 'rt_dif_reset_forNglyco[Byonic]'], axis = 1)
        print('\n----- Exporting "_All" file... This may take some time, please wait. -----\n')
        all_combined_df.style.apply(bg_color, axis=None).to_excel(f'{date}_All_{filename}.xlsx', index = False)  
        print('\n----- "_All" file exported. -----\n')

        if 'Pair[Byonic]' in all_combined_df.columns.tolist(): # pair is to identify etd with pass-threshold hcd pair for later rescue
            # ONLY FOR HCD/ETD FILES: prepare '_Pair' file (as long as byonic passes threshold & have pair)
            all_combined_df = all_combined_df.astype({'Pair[Byonic]': 'str'})
            pair_df = all_combined_df.loc[byonic_colored_id_for_pairfile]
            pair_df = pair_df[pair_df['Pair[Byonic]'].str.contains('pair')]
        #             pair_df = pair_df[pair_df.duplicated(subset=['Pair[Byonic]'], keep=False)] # only paired data included in _Pair
            hcd_pair_num = set(pair_df.loc[(pair_df['Fragment\r\nType[Byonic]']=='hcd'), 'Pair[Byonic]'].tolist())
            pair_df.drop(pair_df[(pair_df['Fragment\r\nType[Byonic]']=='ethcd')&(~(pair_df['Pair[Byonic]'].isin(hcd_pair_num)))].index, inplace=True)
            threshold_masks_colorind(pair_df)
            print('\n----- Exporting "_Pair" file... This may take some time, please wait. -----\n')
            pair_df.style.apply(bg_color, axis=None).to_excel(f'{date}_Pair_{filename}.xlsx', index = False)  
            print('\n----- "_Pair" file exported. -----\n')
        else:
            pass

        print('----- Start preparing simplified version. -----\n')
        # extract rows included in colored indices list separately
        byonicbyos_col_range = ['Scan'] + [col for col in all_forsimple_df.columns.tolist() if '[Byonic]' in col or '[Byos]' in col]
        simple_byonicbyos = all_forsimple_df.loc[byonic_pass_id, byonicbyos_col_range] # extract colored (pass threshold) rows - below-threshold ethcd w/o pair

        simple_byonicbyos_forUnique = simple_byonicbyos[(simple_byonicbyos['N-site(SequonBased)[Byonic]'] != '-1')] # preserve n/a sites for potential o glycan data in unique
        simple_byonicbyos_forNglyco = simple_byonicbyos[(simple_byonicbyos['N-site(SequonBased)[Byonic]'] != 'N/A')&(simple_byonicbyos['N-site(SequonBased)[Byonic]'] != '-1')] # for simplified, n-site must exist

        # clean data by dropping duplicates        
        simple_byonicbyos_forUnique = simple_byonicbyos_forUnique.drop_duplicates().reset_index(drop=True)
        simple_byonicbyos_forNglyco = simple_byonicbyos_forNglyco.drop_duplicates().reset_index(drop=True)

        # HIGHSCORE SELECTION: get unique calc.m/z by groupby & idxmax function to find highest score within each gp and preserve (remember to split the simple_df into byonicbyos & pglyco parts to avoid dropping non-target rows)
        simple_byonicbyos_forUnique = simple_byonicbyos_forUnique.groupby(['Fragment\r\nType[Byonic]', 'N-site(SequonBased)[Byonic]', 'Calc.\r\nm/z[Byonic]','rounded_FinalGlycans[Byonic]', 'PureSequence[Byonic]', 'rt_dif_reset_forUnique[Byonic]'], as_index=False).apply(lambda x: x.loc[x['Score[Byonic]'].idxmax()]).reset_index(drop=True)
        simple_byonicbyos_forNglyco = simple_byonicbyos_forNglyco.groupby(['N-site(SequonBased)[Byonic]', 'Calc.\r\nm/z[Byonic]','rounded_FinalGlycans[Byonic]', 'PureSequence[Byonic]', 'rt_dif_reset_forNglyco[Byonic]'], as_index=False).apply(lambda x: x.loc[x['Score[Byonic]'].idxmax()]).reset_index(drop=True)

#         simple_byonicbyos_forUnique.to_excel("simple_byonicbyos_forUnique.xlsx", index=False)
#         simple_byonicbyos_forNglyco.to_excel("simple_byonicbyos_forNglyco.xlsx", index=False)

        # set scan to index & concat simple_byonicbyos & simple_pglyco by scan (for unique)
        simple_byonicbyos_forUnique = simple_byonicbyos_forUnique.set_index('Scan')

        # change variable name
        id_df_forUnique = simple_byonicbyos_forUnique
        id_df_forUnique.index.name = 'Scan'
        id_df_forUnique.reset_index(level=0, inplace=True)
        # change all nan to blank -1
        id_df_forUnique = id_df_forUnique.fillna(-1)
        # extract nglycan numbers in Mods & use it to calculate byonic pep mass 
        mods_nglycan_sum = [sum([ast.literal_eval(v) for v in lst]) if type(lst) == list else -1 for lst in id_df_forUnique['Mods\r\n(variable)[Byonic]'].str.findall(r'Glycan / (\d+.\d+)').tolist()]
        id_df_forUnique['glycan_mods_sum[Byonic]'] = mods_nglycan_sum
        id_df_forUnique['PepMS[Byonic]'] = id_df_forUnique['Calc.\r\nMH[Byonic]'] - id_df_forUnique['glycan_mods_sum[Byonic]']
        id_df_forUnique.loc[(id_df_forUnique['glycan_mods_sum[Byonic]'] == -1), 'PepMS[Byonic]'] = -1
        move_df(id_df_forUnique, 'PepMS[Byonic]', 'Fragment\r\nType[Byonic]')
        # ast.literaleval byonicbyos n-site & sorting: byonic site -> pglyco site -> byonic peptide mass -> pglyco peptide mass -> byonic glycans -> pglyco glycans in byonic style -> byonic clam/z -> pglcyo calm/z
        id_df_forUnique['N-site(SequonBased)[Byonic]'] = [ast.literal_eval(i) if type(i) == str and '(' not in i and i!='N/A' else i for i in id_df_forUnique['N-site(SequonBased)[Byonic]'].tolist()] # convert str back to int & tuple

        # drop intermediate cols
        id_df_forUnique = id_df_forUnique.drop(['rt_dif_reset_forUnique[Byonic]', 'rt_dif_reset_forNglyco[Byonic]', 'glycan_mods_sum[Byonic]'], axis = 1) # drop the intermediate cols

        col_order = id_df_forUnique.columns.tolist()
        # sorting order: byonic site -> pglyco site -> byonic peptide mass -> pglyco peptide mass -> byonic glycans -> pglyco glycans in byonic style -> byonic calm/z -> pglcyo calm/z => all of this can be acheive with sort_values to multi cols.

        id_df_forUnique = sort_intstr_col(id_df_forUnique, 'z[Byonic]')
        print('Done sorting by z[Byonic] --> ', end = '')
        id_df_forUnique = sort_intstr_col(id_df_forUnique, 'Calc.\r\nMH[Byonic]')
        print('Done sorting by Calc.MH[Byonic] --> ', end = '')
        id_df_forUnique = sort_intstr_col(id_df_forUnique, 'PepMS[Byonic]')
        print('Done sorting by PepMS[Byonic] --> ', end = '')
        id_df_forUnique = sort_intstr_col(id_df_forUnique, 'rounded_FinalGlycans[Byonic]')
        print('Done sorting by rounded_FinalGlycans[Byonic] --> ', end = '')
        id_df_forUnique = sort_intstr_col(id_df_forUnique, 'PureSequence[Byonic]')
        print('Done sorting by PureSequence[Byonic] --> ', end = '')

        # convert '-1' back to int -1 for correct glycansource analysis
        id_df_forUnique['rounded_FinalGlycans[Byonic]'] = [int(i) if i == '-1' else i for i in id_df_forUnique['rounded_FinalGlycans[Byonic]']]
        id_df_forUnique = id_df_forUnique[col_order]

        print('\n----- Exporting "_UniquePep" file... This may take some time, please wait. -----\n')
        threshold_masks_colorind(id_df_forUnique)
        id_df_forUnique.style.apply(bg_color, axis=None).to_excel(f'{date}_UniquePep_{filename}.xlsx', index = False)  
        print('\n----- "_UniquePep" file exported. -----\n')

        # set scan to index & concat simple_byonicbyos & simple_pglyco by scan (for nglyco)
        simple_byonicbyos_forNglyco = simple_byonicbyos_forNglyco.set_index('Scan')
        # change variable name
        id_df_forNglyco = simple_byonicbyos_forNglyco
        id_df_forNglyco.index.name = 'Scan'
        id_df_forNglyco.reset_index(level=0, inplace=True)
        # change all nan to blank -1
        id_df_forNglyco = id_df_forNglyco.fillna(-1)
        # extract nglycan numbers in Mods & use it to calculate byonic pep mass 
        mods_nglycan_sum = [sum([ast.literal_eval(v) for v in lst]) if type(lst) == list else -1 for lst in id_df_forNglyco['Mods\r\n(variable)[Byonic]'].str.findall(r'Glycan / (\d+.\d+)').tolist()]
        id_df_forNglyco['glycan_mods_sum[Byonic]'] = mods_nglycan_sum
        id_df_forNglyco['PepMS[Byonic]'] = id_df_forNglyco['Calc.\r\nMH[Byonic]'] - id_df_forNglyco['glycan_mods_sum[Byonic]']
        id_df_forNglyco.loc[(id_df_forNglyco['glycan_mods_sum[Byonic]'] == -1), 'PepMS[Byonic]'] = -1
        move_df(id_df_forNglyco, 'PepMS[Byonic]', 'Fragment\r\nType[Byonic]')
        # ast.literaleval byonicbyos n-site & sorting: byonic site -> pglyco site -> byonic peptide mass -> pglyco peptide mass -> byonic glycans -> pglyco glycans in byonic style -> byonic clam/z -> pglcyo calm/z
        id_df_forNglyco['N-site(SequonBased)[Byonic]'] = [ast.literal_eval(i) if type(i) == str and '(' not in i else i for i in id_df_forNglyco['N-site(SequonBased)[Byonic]'].tolist()] # convert str back to int & tuple

        # drop intermediate cols
        id_df_forNglyco = id_df_forNglyco.drop(['rt_dif_reset_forUnique[Byonic]', 'rt_dif_reset_forNglyco[Byonic]', 'glycan_mods_sum[Byonic]'], axis = 1) # drop the intermediate cols
        col_order = id_df_forNglyco.columns.tolist()
        # sorting order: byonic site -> pglyco site -> byonic peptide mass -> pglyco peptide mass -> byonic glycans -> pglyco glycans in byonic style -> byonic calm/z -> pglcyo calm/z => all of this can be acheive with sort_values to multi cols.

        id_df_forNglyco = sort_intstr_col(id_df_forNglyco, 'z[Byonic]')
        print('Done sorting by z[Byonic] --> ', end = '')
        id_df_forNglyco = sort_intstr_col(id_df_forNglyco, 'Calc.\r\nMH[Byonic]')
        print('Done sorting by Calc.MH[Byonic] --> ', end = '')
        id_df_forNglyco = sort_intstr_col(id_df_forNglyco, 'PepMS[Byonic]')
        print('Done sorting by PepMS[Byonic] --> ', end = '')
        id_df_forNglyco = sort_intstr_col(id_df_forNglyco, 'rounded_FinalGlycans[Byonic]')
        print('Done sorting by rounded_FinalGlycans[Byonic] --> ', end = '')
        id_df_forNglyco = sort_intstr_col(id_df_forNglyco, 'PureSequence[Byonic]')
        print('Done sorting by PureSequence[Byonic] --> ', end = '')

        # convert '-1' back to int -1 for correct glycansource analysis
        id_df_forNglyco['rounded_FinalGlycans[Byonic]'] = [int(i) if i == '-1' else i for i in id_df_forNglyco['rounded_FinalGlycans[Byonic]']]
        id_df_forNglyco = id_df_forNglyco[col_order]

        print('\n----- Exporting "_NglycoPep" file... This may take some time, please wait. -----\n')
        threshold_masks_colorind(id_df_forNglyco)
        id_df_forNglyco.style.apply(bg_color, axis=None).to_excel(f'{date}_NglycoPep_{filename}.xlsx', index = False)  
        print('\n----- "_NglycoPep" file exported. -----\n')

        print('----- Start preparing quantification file of N-glycosylation . -----\n')
        # only single sites for quant (quant_df offers important info. for later multiIndex df construction)
        quant = id_df_forNglyco[id_df_forNglyco['N-site(SequonBased)[Byonic]'].apply(lambda x: isinstance(x, int))]
        # avoid adding values from below-threshold rows: if deepgreen -> change value to -1, if deepblue -> change value to -1.(-1 to make lambda function easier to write)
        threshold_masks_colorind(quant)
        # calculate the sum within each n-site, then normalize xicauc/ int/ mono/ iso by these sum
        quant['sumpersite_xicauc'] = quant.groupby(['N-site(SequonBased)[Byonic]'])['XIC\r\nAUC[Byos]'].transform(lambda x: x[x!=-1].sum())
        quant['sumpersite_int'] = quant.groupby(['N-site(SequonBased)[Byonic]'])['Apex Int.\r\n(Posit)[Byos]'].transform(lambda x: x[x!=-1].sum())
        # convert all the 'N/A' in byonic glycans to 'Unoccupied'
        quant.loc[(quant['rounded_FinalGlycans[Byonic]'] == 'N/A'), ['rounded_FinalGlycans[Byonic]']] = 'Unoccupied'
        # using groupby transform sum function to add 8 cols (byos xicauc/byos area int/pglycomono/pglycoisotope) recording summed values & normalized values (same site & same glycan & seq can be dif)
        quant['e_sum_XIC\r\nAUC[Byos]'] = quant.groupby(['N-site(SequonBased)[Byonic]', 'rounded_FinalGlycans[Byonic]'])['XIC\r\nAUC[Byos]'].transform(lambda x: x[x != -1].sum())
        quant['f_sum_Apex Int.\r\n(Posit)[Byos]'] = quant.groupby(['N-site(SequonBased)[Byonic]', 'rounded_FinalGlycans[Byonic]'])['Apex Int.\r\n(Posit)[Byos]'].transform(lambda x: x[x != -1].sum())
        quant['a_norm_XIC\r\nAUC[Byos]'] = quant['e_sum_XIC\r\nAUC[Byos]']/quant['sumpersite_xicauc']
        quant['b_norm_Apex Int.\r\n(Posit)[Byos]'] = quant['f_sum_Apex Int.\r\n(Posit)[Byos]']/quant['sumpersite_int']

        # since it's possible to have real 0 from calculation, change the real absent data back to -1 in sum_ & norm_
        quant.loc[quant['XIC\r\nAUC[Byos]'] == -1, ['e_sum_XIC\r\nAUC[Byos]', 'f_sum_Apex Int.\r\n(Posit)[Byos]', 'a_norm_XIC\r\nAUC[Byos]', 'b_norm_Apex Int.\r\n(Posit)[Byos]']] = -1 
        # extract the needed cols only & split byonicbyos/pglyco, drop all -1 rows, glycans as index to outer union concat data again
        quant_bb = quant[['N-site(SequonBased)[Byonic]', 'rounded_FinalGlycans[Byonic]', 'MS2 Search\r\nAlias name[Byos]', 'e_sum_XIC\r\nAUC[Byos]', 'f_sum_Apex Int.\r\n(Posit)[Byos]', 'a_norm_XIC\r\nAUC[Byos]', 'b_norm_Apex Int.\r\n(Posit)[Byos]']]
        quant_bb = quant_bb.loc[quant_bb['N-site(SequonBased)[Byonic]'] != -1]
        quant_bb = quant_bb.set_index('rounded_FinalGlycans[Byonic]')
        # change variable name
        quant_glycanid = quant_bb
        quant_glycanid.index.name = 'Glycans ↓'
        quant_glycanid.reset_index(level=0, inplace=True)
        # add N(x) col
        nx = ['N(0)' if lst == [] else 'N(%s)'%(lst[0]) for lst in quant_glycanid['Glycans ↓'].str.findall(r'HexNAc\((\d+)\)').tolist()]
        quant_glycanid.insert(0 , 'N(x) ↓', nx , True)
        quant_glycanid = quant_glycanid.drop('MS2 Search\r\nAlias name[Byos]', axis = 1)
        # change all nan to blank -1
        quant_glycanid = quant_glycanid.fillna(-1)
        quant_glycanid = quant_glycanid.drop_duplicates() # remember to drop duplicates after splitting & reconcat df
        # generate 5 versions: bp_intersection (both sites are present at the same time <intersection>), bp_union (n site union), only b (xicauc/int), only p (mono/isotope)
        ## onlyb
        onlyb = quant_glycanid.loc[(quant_glycanid['N-site(SequonBased)[Byonic]'] != -1)]
        onlyb_forunion = onlyb.copy().drop_duplicates()
        # clean off all no data rows
        onlyb = onlyb.loc[~((onlyb['e_sum_XIC\r\nAUC[Byos]']==-1)&(onlyb['f_sum_Apex Int.\r\n(Posit)[Byos]']==-1)&(onlyb['a_norm_XIC\r\nAUC[Byos]']==-1)&(onlyb['b_norm_Apex Int.\r\n(Posit)[Byos]']==-1))]
        nx = list(set(onlyb['N(x) ↓'].tolist()))
        nx = [int(re.findall(r'[0-9]+', i)[0]) if type(i) == str else i for i in nx]
        nx.sort(key=lambda v: (isinstance(v, str), v))
        b_new_nx = ['N(%s)'%(str(i)) if type(i) == int and i != -1 else i for i in nx]

        ## onlyb_top10
        onlyb_top10 = onlyb.copy()
        # xicauc_top10 = onlyb_top10.groupby('N-site(SequonBased)[Byonic]')['a_norm_XIC\r\nAUC[Byos]'].nlargest(topN).droplevel(level = 0).index.tolist()
        xicauc_top10 = onlyb_top10.groupby('N-site(SequonBased)[Byonic]').apply(pd.DataFrame.nlargest, n=5, columns='a_norm_XIC\r\nAUC[Byos]').droplevel(level=0).index.tolist()
        int_top10 = onlyb_top10.groupby('N-site(SequonBased)[Byonic]').apply(pd.DataFrame.nlargest, n=5, columns='b_norm_Apex Int.\r\n(Posit)[Byos]').droplevel(level=0).index.tolist()
        onlyb_top10.loc[[i for i in onlyb_top10.index.tolist() if i not in xicauc_top10], 'a_norm_XIC\r\nAUC[Byos]'] = -1
        onlyb_top10.loc[[i for i in onlyb_top10.index.tolist() if i not in int_top10], 'b_norm_Apex Int.\r\n(Posit)[Byos]'] = -1
        onlyb_top10 = onlyb_top10.loc[(onlyb_top10['a_norm_XIC\r\nAUC[Byos]'] != -1)|(onlyb_top10['b_norm_Apex Int.\r\n(Posit)[Byos]'] != -1)].reset_index(drop=True).drop(['N(x) ↓', 'e_sum_XIC\r\nAUC[Byos]', 'f_sum_Apex Int.\r\n(Posit)[Byos]', 'N(x) ↓'], axis = 1)

        print('----- Start summary table construction. -----\n')
        # start grouping: N(X) -> byonic glycans -> byonic sites -> .agg({all 8 cols, apply .mean() & .mean()/total XXX})
        # onlyb
        onlyb_gp = onlyb.groupby(['N(x) ↓', 'Glycans ↓', 'N-site(SequonBased)[Byonic]']).agg({'e_sum_XIC\r\nAUC[Byos]': lambda x: x.mean(), 'f_sum_Apex Int.\r\n(Posit)[Byos]': lambda x: x.mean() \
                                                                                              , 'a_norm_XIC\r\nAUC[Byos]': lambda x: x.mean(), 'b_norm_Apex Int.\r\n(Posit)[Byos]': lambda x: x.mean()})

        # fix the str '10' sorted before '2' problem
        onlyb_gp = onlyb_gp.reindex(b_new_nx, level = 0)
        # unstack byonic n-sites
        onlyb_gp = onlyb_gp.unstack(level=-1)
        # multiIndex col swaplevel
        onlyb_gp.columns = onlyb_gp.columns.swaplevel(0, 1)
        # sort is necessary for the right table structure (but since it will sort every level, so we need to manually adjust level3 back to the original order)
        onlyb_gp.sort_index(axis=1, level=[0, 1], inplace=True)
        onlyb_gp.columns = onlyb_gp.columns.set_names(['N-site(SequonBased)[Byonic] →', 'Quant. →'])
        # delete a-h in Quant. (the alphabet is just for sorting)
#         onlyb_gp.columns.set_levels(['norm_XIC\r\nAUC[Byos]', 'norm_Apex Int.\r\n(Posit)[Byos]', 'sum_XIC\r\nAUC[Byos]', 'sum_Apex Int.\r\n(Posit)[Byos]'], level = 1, inplace=True)
        onlyb_gp.columns.set_levels(['norm_XIC\r\nAUC[Byos]', 'norm_Apex Int.\r\n(Posit)[Byos]', 'sum_XIC\r\nAUC[Byos]', 'sum_Apex Int.\r\n(Posit)[Byos]'], level = 1)
#         onlyb_gp.to_excel("onlyb_gp_setlevels.xlsx")
#         print('\n----- Exporting "_Quant" files... This may take some time, please wait. -----\n')

#         with pd.ExcelWriter(f'{date}_Quant_{filename}.xlsx') as writer:           
#             # onlyb
#             onlyb_gp.replace(to_replace = -1, value = np.nan , inplace = True)
# #             onlyb_gp.to_excel("onlyb_gp.xlsx")
#             onlyb_gp.style.background_gradient(cmap ='Greens').highlight_null(null_color='white').to_excel(writer, sheet_name = 'Sheet1(QuantB)')

#         print('\n----- "_Quant" file exported. -----\n')

        print('----- Start plotting. -----\n')
        bNORM_dfall_lstv1, bNORM_label_lstv1, bNORM_dfall_lstv2, bNORM_label_lstv2 \
        , bNORM_dfall_lstv3, bNORM_label_lstv3, bNORM_dfall_lstv4, bNORM_label_lstv4 \
        , bSUM_dfall_lstv1, bSUM_label_lstv1, bSUM_nsum_lstv1, bSUM_dfall_lstv2, bSUM_label_lstv2, bSUM_nsum_lstv2 \
        , bSUM_dfall_lstv3, bSUM_label_lstv3, bSUM_nsum_lstv3, bSUM_dfall_lstv4, bSUM_label_lstv4, bSUM_nsum_lstv4 \
        = [], [], [], [], [], [], [], [], [], [] \
        , [], [], [], [], [], [], [], [], [], [] 

        if plot_v2 == 'yes':  
            col_order = ['N(0)', 'N(1)', 'OligoMannose', 'Complex', 'Complex+Sia'] # col order for v2 plotting
            # necessary preprocessings for v2 plotting (modified from v1 nx)
            # onlyb v2
            onlyb.loc[(onlyb['N(x) ↓'] == 'N(0)')|(onlyb['N(x) ↓'] == 'N(1)'), 'nx_version2'] = onlyb['N(x) ↓']
            onlyb.loc[(onlyb['N(x) ↓'] == 'N(2)'), 'nx_version2'] = 'OligoMannose'
            onlyb.loc[(onlyb['N(x) ↓'] != 'N(0)')&(onlyb['N(x) ↓'] != 'N(1)')&(onlyb['N(x) ↓'] != 'N(2)'), 'nx_version2'] = 'Complex'
            onlyb.loc[(onlyb['N(x) ↓'] != 'N(0)')&(onlyb['N(x) ↓'] != 'N(1)')&(onlyb['N(x) ↓'] != 'N(2)')&(onlyb['Glycans ↓'].str.contains('NeuAc|NeuGc')), 'nx_version2'] = 'Complex+Sia'
            b_neugc_exist = sum(onlyb['Glycans ↓'].str.contains('NeuGc')) > 0 # Bool, if T, red. else purple

        if plot_v3 == 'yes':
            # necessary preprocessings for v3 plotting (modified from v1 nx)            
            # onlyb v3
            onlyb, onlyb_nxhy_fix_order = v3(onlyb)[0], v3(onlyb)[1] 

        if plot_v4 == 'yes':
            # necessary preprocessings for v4 plotting (modified from v1 nx)         
            # onlyb v4
            onlyb, onlyb_fnxhy_fix_order = v4(onlyb)[0], v4(onlyb)[1]    

        # make sure that sites are presented as int not float
        onlyb = onlyb.astype({'N-site(SequonBased)[Byonic]':'Int64'})

        # split bp_union into: xicauc_df / int_df / mono_df / iso_df for later clustered stacked plot (norm only)
        if export_xicauc == 'yes':
            if plot_v1 == 'yes':
                # onlyb norm v1
                b_xicaucNORM_df = onlyb.loc[:, ['N(x) ↓', 'N-site(SequonBased)[Byonic]', 'a_norm_XIC\r\nAUC[Byos]']]
                b_xicaucNORM_df.replace(to_replace = -1, value = np.nan , inplace = True)
                b_xicaucNORM_df = b_xicaucNORM_df.pivot_table(index='N-site(SequonBased)[Byonic]', values='a_norm_XIC\r\nAUC[Byos]', columns='N(x) ↓', aggfunc=np.sum).reindex(columns=b_new_nx)
                bNORM_dfall_lstv1.append(b_xicaucNORM_df)
                bNORM_label_lstv1.append('norm_XIC AUC[Byos]')
                # onlyb sum v1
                b_xicaucSUM_df = onlyb.loc[:, ['N(x) ↓', 'N-site(SequonBased)[Byonic]', 'e_sum_XIC\r\nAUC[Byos]']]
                b_xicaucSUM_df.replace(to_replace = -1, value = np.nan , inplace = True)
                b_xicaucSUM_df = b_xicaucSUM_df.pivot_table(index='N-site(SequonBased)[Byonic]', values='e_sum_XIC\r\nAUC[Byos]', columns='N(x) ↓', aggfunc=np.sum).reindex(columns=b_new_nx)
                b_xicaucSUM_df_nsum = [b_xicaucSUM_df.loc[i].sum() for i in b_xicaucSUM_df.index.tolist()] # for A bar label
                bSUM_dfall_lstv1.append(b_xicaucSUM_df)
                bSUM_label_lstv1.append('sum_XIC AUC[Byos]')
                bSUM_nsum_lstv1.append(b_xicaucSUM_df_nsum)

            if plot_v2 == 'yes':
                # onlyb norm v2
                b_xicaucNORMv2_df = onlyb.loc[:, ['nx_version2', 'N-site(SequonBased)[Byonic]', 'a_norm_XIC\r\nAUC[Byos]']]
                b_xicaucNORMv2_df.replace(to_replace = -1, value = np.nan , inplace = True)
                b_xicaucNORMv2_df = b_xicaucNORMv2_df.pivot_table(index='N-site(SequonBased)[Byonic]', values='a_norm_XIC\r\nAUC[Byos]', columns='nx_version2', aggfunc=np.sum).reindex(columns=col_order)
                bNORM_dfall_lstv2.append(b_xicaucNORMv2_df)
                bNORM_label_lstv2.append('norm_XIC AUC[Byos]')
                # onlyb sum v2
                b_xicaucSUMv2_df = onlyb.loc[:, ['nx_version2', 'N-site(SequonBased)[Byonic]', 'e_sum_XIC\r\nAUC[Byos]']]
                b_xicaucSUMv2_df.replace(to_replace = -1, value = np.nan , inplace = True)
                b_xicaucSUMv2_df = b_xicaucSUMv2_df.pivot_table(index='N-site(SequonBased)[Byonic]', values='e_sum_XIC\r\nAUC[Byos]', columns='nx_version2', aggfunc=np.sum).reindex(columns=col_order)
                b_xicaucSUMv2_df_nsum = [b_xicaucSUMv2_df.loc[i].sum() for i in b_xicaucSUMv2_df.index.tolist()] # for A bar label
                bSUM_dfall_lstv2.append(b_xicaucSUMv2_df)
                bSUM_label_lstv2.append('sum_XIC AUC[Byos]')
                bSUM_nsum_lstv2.append(b_xicaucSUMv2_df_nsum)

            if plot_v3 == 'yes':
                # onlyb norm v3
                b_xicaucNORMv3_df = onlyb.loc[:, ['N(x)H(y)', 'N-site(SequonBased)[Byonic]', 'a_norm_XIC\r\nAUC[Byos]']]
                b_xicaucNORMv3_df.replace(to_replace = -1, value = np.nan , inplace = True)
                b_xicaucNORMv3_df = b_xicaucNORMv3_df.pivot_table(index='N-site(SequonBased)[Byonic]', values='a_norm_XIC\r\nAUC[Byos]', columns='N(x)H(y)', aggfunc=np.sum).reindex(columns=onlyb_nxhy_fix_order)
                bNORM_dfall_lstv3.append(b_xicaucNORMv3_df)
                bNORM_label_lstv3.append('norm_XIC AUC[Byos]')
                # onlyb sum v3
                b_xicaucSUMv3_df = onlyb.loc[:, ['N(x)H(y)', 'N-site(SequonBased)[Byonic]', 'e_sum_XIC\r\nAUC[Byos]']]
                b_xicaucSUMv3_df.replace(to_replace = -1, value = np.nan , inplace = True)
                b_xicaucSUMv3_df = b_xicaucSUMv3_df.pivot_table(index='N-site(SequonBased)[Byonic]', values='e_sum_XIC\r\nAUC[Byos]', columns='N(x)H(y)', aggfunc=np.sum).reindex(columns=onlyb_nxhy_fix_order)
                b_xicaucSUMv3_df_nsum = [b_xicaucSUMv3_df.loc[i].sum() for i in b_xicaucSUMv3_df.index.tolist()] # for A bar label
                bSUM_dfall_lstv3.append(b_xicaucSUMv3_df)
                bSUM_label_lstv3.append('sum_XIC AUC[Byos]')
                bSUM_nsum_lstv3.append(b_xicaucSUMv3_df_nsum)

            if plot_v4 == 'yes':
                # onlyb norm v4
                b_xicaucNORMv4_df = onlyb.loc[:, ['(F)N(x)H(y)', 'N-site(SequonBased)[Byonic]', 'a_norm_XIC\r\nAUC[Byos]']]
                b_xicaucNORMv4_df.replace(to_replace = -1, value = np.nan , inplace = True)
                b_xicaucNORMv4_df = b_xicaucNORMv4_df.pivot_table(index='N-site(SequonBased)[Byonic]', values='a_norm_XIC\r\nAUC[Byos]', columns='(F)N(x)H(y)', aggfunc=np.sum).reindex(columns=onlyb_fnxhy_fix_order)
                bNORM_dfall_lstv4.append(b_xicaucNORMv4_df)
                bNORM_label_lstv4.append('norm_XIC AUC[Byos]')
                # onlyb sum v4
                b_xicaucSUMv4_df = onlyb.loc[:, ['(F)N(x)H(y)', 'N-site(SequonBased)[Byonic]', 'e_sum_XIC\r\nAUC[Byos]']]
                b_xicaucSUMv4_df.replace(to_replace = -1, value = np.nan , inplace = True)
                b_xicaucSUMv4_df = b_xicaucSUMv4_df.pivot_table(index='N-site(SequonBased)[Byonic]', values='e_sum_XIC\r\nAUC[Byos]', columns='(F)N(x)H(y)', aggfunc=np.sum).reindex(columns=onlyb_fnxhy_fix_order)
                b_xicaucSUMv4_df_nsum = [b_xicaucSUMv4_df.loc[i].sum() for i in b_xicaucSUMv4_df.index.tolist()] # for A bar label
                bSUM_dfall_lstv4.append(b_xicaucSUMv4_df)
                bSUM_label_lstv4.append('sum_XIC AUC[Byos]')
                bSUM_nsum_lstv4.append(b_xicaucSUMv4_df_nsum)    
        else:
            pass

        if export_int == 'yes': 
            if plot_v1 == 'yes':
                # onlyb norm v1 
                b_intNORM_df = onlyb.loc[:, ['N(x) ↓', 'N-site(SequonBased)[Byonic]', 'b_norm_Apex Int.\r\n(Posit)[Byos]']] 
                b_intNORM_df.replace(to_replace = -1, value = np.nan , inplace = True)
                b_intNORM_df = b_intNORM_df.pivot_table(index='N-site(SequonBased)[Byonic]', values='b_norm_Apex Int.\r\n(Posit)[Byos]', columns='N(x) ↓', aggfunc=np.sum).reindex(columns=b_new_nx)
                bNORM_dfall_lstv1.append(b_intNORM_df)
                bNORM_label_lstv1.append('norm_Apex Int.(Posit)[Byos]')
                # onlyb sum v1
                b_intSUM_df = onlyb.loc[:, ['N(x) ↓', 'N-site(SequonBased)[Byonic]', 'f_sum_Apex Int.\r\n(Posit)[Byos]']] 
                b_intSUM_df.replace(to_replace = -1, value = np.nan , inplace = True)
                b_intSUM_df = b_intSUM_df.pivot_table(index='N-site(SequonBased)[Byonic]', values='f_sum_Apex Int.\r\n(Posit)[Byos]', columns='N(x) ↓', aggfunc=np.sum).reindex(columns=b_new_nx)
                b_intSUM_df_nsum = [b_intSUM_df.loc[i].sum() for i in b_intSUM_df.index.tolist()] # for B bar label
                bSUM_dfall_lstv1.append(b_intSUM_df)
                bSUM_label_lstv1.append('sum_Apex Int.(Posit)[Byos]')
                bSUM_nsum_lstv1.append(b_intSUM_df_nsum)
            if plot_v2 == 'yes':
                # onlyb norm v2
                b_intNORMv2_df = onlyb.loc[:, ['nx_version2', 'N-site(SequonBased)[Byonic]', 'b_norm_Apex Int.\r\n(Posit)[Byos]']] 
                b_intNORMv2_df.replace(to_replace = -1, value = np.nan , inplace = True)
                b_intNORMv2_df = b_intNORMv2_df.pivot_table(index='N-site(SequonBased)[Byonic]', values='b_norm_Apex Int.\r\n(Posit)[Byos]', columns='nx_version2', aggfunc=np.sum).reindex(columns=col_order)
                bNORM_dfall_lstv2.append(b_intNORMv2_df)
                bNORM_label_lstv2.append('norm_Apex Int.(Posit)[Byos]')
                # onlyb sum v2
                b_intSUMv2_df = onlyb.loc[:, ['nx_version2', 'N-site(SequonBased)[Byonic]', 'f_sum_Apex Int.\r\n(Posit)[Byos]']] 
                b_intSUMv2_df.replace(to_replace = -1, value = np.nan , inplace = True)
                b_intSUMv2_df = b_intSUMv2_df.pivot_table(index='N-site(SequonBased)[Byonic]', values='f_sum_Apex Int.\r\n(Posit)[Byos]', columns='nx_version2', aggfunc=np.sum).reindex(columns=col_order)
                b_intSUMv2_df_nsum = [b_intSUMv2_df.loc[i].sum() for i in b_intSUMv2_df.index.tolist()] # for B bar label
                bSUM_dfall_lstv2.append(b_intSUMv2_df)
                bSUM_label_lstv2.append('sum_Apex Int.(Posit)[Byos]')
                bSUM_nsum_lstv2.append(b_intSUMv2_df_nsum)
            if plot_v3 == 'yes':
                # onlyb norm v3
                b_intNORMv3_df = onlyb.loc[:, ['N(x)H(y)', 'N-site(SequonBased)[Byonic]', 'b_norm_Apex Int.\r\n(Posit)[Byos]']]
                b_intNORMv3_df.replace(to_replace = -1, value = np.nan , inplace = True)
                b_intNORMv3_df = b_intNORMv3_df.pivot_table(index='N-site(SequonBased)[Byonic]', values='b_norm_Apex Int.\r\n(Posit)[Byos]', columns='N(x)H(y)', aggfunc=np.sum).reindex(columns=onlyb_nxhy_fix_order)
                bNORM_dfall_lstv3.append(b_intNORMv3_df)
                bNORM_label_lstv3.append('norm_Apex Int.(Posit)[Byos]')
                # onlyb sum v3
                b_intSUMv3_df = onlyb.loc[:, ['N(x)H(y)', 'N-site(SequonBased)[Byonic]', 'f_sum_Apex Int.\r\n(Posit)[Byos]']]
                b_intSUMv3_df.replace(to_replace = -1, value = np.nan , inplace = True)
                b_intSUMv3_df = b_intSUMv3_df.pivot_table(index='N-site(SequonBased)[Byonic]', values='f_sum_Apex Int.\r\n(Posit)[Byos]', columns='N(x)H(y)', aggfunc=np.sum).reindex(columns=onlyb_nxhy_fix_order)
                b_intSUMv3_df_nsum = [b_intSUMv3_df.loc[i].sum() for i in b_intSUMv3_df.index.tolist()] # for A bar label
                bSUM_dfall_lstv3.append(b_intSUMv3_df)
                bSUM_label_lstv3.append('sum_Apex Int.(Posit)[Byos]')
                bSUM_nsum_lstv3.append(b_intSUMv3_df_nsum)
            if plot_v4 == 'yes':
                # onlyb norm v4
                b_intNORMv4_df = onlyb.loc[:, ['(F)N(x)H(y)', 'N-site(SequonBased)[Byonic]', 'b_norm_Apex Int.\r\n(Posit)[Byos]']]
                b_intNORMv4_df.replace(to_replace = -1, value = np.nan , inplace = True)
                b_intNORMv4_df = b_intNORMv4_df.pivot_table(index='N-site(SequonBased)[Byonic]', values='b_norm_Apex Int.\r\n(Posit)[Byos]', columns='(F)N(x)H(y)', aggfunc=np.sum).reindex(columns=onlyb_fnxhy_fix_order)
                bNORM_dfall_lstv4.append(b_intNORMv4_df)
                bNORM_label_lstv4.append('norm_Apex Int.(Posit)[Byos]')
                # onlyb sum v4
                b_intSUMv4_df = onlyb.loc[:, ['(F)N(x)H(y)', 'N-site(SequonBased)[Byonic]', 'f_sum_Apex Int.\r\n(Posit)[Byos]']]
                b_intSUMv4_df.replace(to_replace = -1, value = np.nan , inplace = True)
                b_intSUMv4_df = b_intSUMv4_df.pivot_table(index='N-site(SequonBased)[Byonic]', values='f_sum_Apex Int.\r\n(Posit)[Byos]', columns='(F)N(x)H(y)', aggfunc=np.sum).reindex(columns=onlyb_fnxhy_fix_order)
                b_intSUMv4_df_nsum = [b_intSUMv4_df.loc[i].sum() for i in b_intSUMv4_df.index.tolist()] # for A bar label
                bSUM_dfall_lstv4.append(b_intSUMv4_df)
                bSUM_label_lstv4.append('sum_Apex Int.(Posit)[Byos]')
                bSUM_nsum_lstv4.append(b_intSUMv4_df_nsum)       
        else:
            pass

        if export_xicauc == export_int == export_mono == export_isotope == 'no': # no plots will be exported
            pass
        else: # plotting criterium: cannot be empty list

            if bNORM_dfall_lstv1 != []:
                # onlyb NORMv1
                fig = plot_clustered_stacked(bNORM_dfall_lstv1, [[1]*len(bNORM_dfall_lstv1[0].index) for i in range(len(bNORM_dfall_lstv1))],bNORM_label_lstv1)
                fig.tight_layout()
                fig.savefig(f'{date}_onlybNORMv1_{filename}.png', bbox_inches='tight')
            if bNORM_dfall_lstv2 != []:
                # onlyb NORMv2
                fig = plot_clustered_stacked(bNORM_dfall_lstv2, [[1]*len(bNORM_dfall_lstv2[0].index) for i in range(len(bNORM_dfall_lstv2))],bNORM_label_lstv2, neugc_exist=b_neugc_exist)
                fig.tight_layout()
                fig.savefig(f'{date}_onlybNORMv2_{filename}.png', bbox_inches='tight')
            if bNORM_dfall_lstv3 != []:
                # onlyb NORMv3
                fig = plot_clustered_stacked(bNORM_dfall_lstv3, [[1]*len(bNORM_dfall_lstv3[0].index) for i in range(len(bNORM_dfall_lstv3))],bNORM_label_lstv3, detailedHighMan=True)
                fig.tight_layout()
                fig.savefig(f'{date}_onlybNORMv3_{filename}.png', bbox_inches='tight')
            if bNORM_dfall_lstv4 != []:
                # onlyb NORMv4
                fig = plot_clustered_stacked(bNORM_dfall_lstv4, [[1]*len(bNORM_dfall_lstv4[0].index) for i in range(len(bNORM_dfall_lstv4))],bNORM_label_lstv4, detailedHighMan=True, fucosylation=True)
                fig.tight_layout()
                fig.savefig(f'{date}_onlybNORMv4_{filename}.png', bbox_inches='tight')


            if bSUM_dfall_lstv1 != []:
                # onlyb SUMv1
                fig = plot_clustered_stacked(bSUM_dfall_lstv1, bSUM_nsum_lstv1,bSUM_label_lstv1)
                fig.tight_layout()
                fig.savefig(f'{date}_onlybSUMv1_{filename}.png', bbox_inches='tight')
            if bSUM_dfall_lstv2 != []:
                # onlyb SUMv2
                fig = plot_clustered_stacked(bSUM_dfall_lstv2,bSUM_nsum_lstv2 ,bSUM_label_lstv2, neugc_exist=b_neugc_exist)
                fig.tight_layout()
                fig.savefig(f'{date}_onlybSUMv2_{filename}.png', bbox_inches='tight')
            if bSUM_dfall_lstv3 != []:
                # onlyb SUMv3
                fig = plot_clustered_stacked(bSUM_dfall_lstv3, bSUM_nsum_lstv3,bSUM_label_lstv3, detailedHighMan=True)
                fig.tight_layout()
                fig.savefig(f'{date}_onlybSUMv3_{filename}.png', bbox_inches='tight')
            if bSUM_dfall_lstv4 != []:
                # onlyb SUMv4
                fig = plot_clustered_stacked(bSUM_dfall_lstv4,bSUM_nsum_lstv4 ,bSUM_label_lstv4, detailedHighMan=True, fucosylation=True)
                fig.tight_layout()
                fig.savefig(f'{date}_onlybSUMv4_{filename}.png', bbox_inches='tight')

        if seq != '':
            print('----- Start preparing MajorGlycoformTable. -----\n')
            # prepare needed lists for the table cols (no./nsite/sequon are the indices, major glycoform is the col)
            result = findNPosBySequon(seq)
            nsite = [t[0] for t in result]
            nsite.append('')
            sequon = [t[1] for t in result]
            sequon.append('')
            no = [i+1 for i in range(len(nsite)-1)]
            table = [(no[i], nsite[i], sequon[i]) for i in range(len(no))]
            table = pd.DataFrame(table)
            table.columns = ['No.', 'N-site', 'Sequon']
            # rename col & extract xicauc max and its glycoform
            onlyb = onlyb.rename(columns={'Glycans ↓':'Major Glycoform'}).drop_duplicates().reset_index(drop=True)

            # (1 OF 2 WAYS TO DETERMINE MAJOR GLYCOFORM) TABLE1: CHOOSE HIGHEST XICAUC (yu-chun's version: from quant, so man4 & other below-core are not cleaned)
            # bp_union = bp_union.groupby('N-site(Byonic ∪ pGlyco) →').apply(lambda x: x.loc[x['a_norm_XIC\r\nAUC[Byos]'].idxmax(), ['N-site(Byonic ∪ pGlyco) →', 'Major Glycoform(union)']]).reset_index(drop=True)
            onlyb_forTable1 = onlyb.groupby('N-site(SequonBased)[Byonic]').apply(lambda x: x.loc[x['a_norm_XIC\r\nAUC[Byos]'].idxmax(), ['N-site(SequonBased)[Byonic]', 'Major Glycoform']]).reset_index(drop=True)    
            onlyb_forTable1 = onlyb_forTable1.set_index('N-site(SequonBased)[Byonic]')
            onlyb_forTable1.index.name = 'N-site'
            onlyb_forTable1 = onlyb_forTable1.reset_index()
            # map the result to the empty df
            table1 = table.merge(onlyb_forTable1, on=['N-site'], how='outer')

            # (2 OF 2 WAYS TO DETERMINE MAJOR GLYCOFORM) TABLE2: CHOOSE MAJOR CLASS (HIGHMAN/HYBRID/COMPLEX/UNOCCUPIED) -> CHOOSE HIGHEST XICAUC WITHIN MAJOR CLASS (danny's version: man4 & other below-core are cleaned, just like data used for xicauc bar/pie charts)
            # clean man4 & below-core structure & add glycan classes col by ModifiedGlycanTypeAnalysis func (highman/hybrid/complex/unoccupied) 
            onlyb_forTable2 = ModifiedGlycanTypeAnalysis(onlyb, 'Major Glycoform')
            onlyb_forTable2 = normalizer(onlyb_forTable2, 'N-site(SequonBased)[Byonic]')

            # find major class then find the xicauc / isotope area max within major class
            onlyb_forTable2 = findMajorClass(onlyb_forTable2, 'N-site(SequonBased)[Byonic]', 'Major Glycoform')
            onlyb_forTable2 = onlyb_forTable2.groupby('N-site(SequonBased)[Byonic]').apply(lambda x: x.loc[x['a_norm_XIC\r\nAUC[Byos]'].idxmax(), ['N-site(SequonBased)[Byonic]', 'Major Glycoform']]).reset_index(drop=True)
            onlyb_forTable2 = onlyb_forTable2.set_index('N-site(SequonBased)[Byonic]')
            onlyb_forTable2.index.name = 'N-site'
            onlyb_forTable2 = onlyb_forTable2.reset_index()

            # map the result to the empty df
            table2 = table.merge(onlyb_forTable2, on=['N-site'], how='outer')

            with pd.ExcelWriter(f'{date}_MajorGlycoformTable_{filename}.xlsx') as writer:   
                table1.to_excel(writer, sheet_name = 'XICAUC max. within site', index=False)
                table2.to_excel(writer, sheet_name = 'XICAUC max. within major class', index=False)

            print('\n----- MajorGlycoformTable exported. -----\n')
    # EXECUTION TIME
    print("\nAll tasks completed.\nExecution time: %.2f seconds"%(time.time() - start_time))
    
# RUN PROCESSING
main()

Original byonic columns:
Index(['PID', 'Prot._x000D_\nRank', 'Pos.', 'Sequence_x000D_\n(unformatted)',
       'Mods_x000D_\n(variable)', 'Glycans', 'PEP_x000D_\n2D',
       'PEP_x000D_\n1D', '|Log Prob|', 'Score', 'Delta_x000D_\nScore',
       'Delta Mod._x000D_\nScore', 'z', 'Obs._x000D_\nm/z',
       'Calc._x000D_\nm/z', 'ppm_x000D_\nerr.', 'Off-_x000D_\nBy-X',
       'Obs._x000D_\nMH', 'Calc._x000D_\nMH', 'Cleavage',
       'Glycans_x000D_\nPos.', 'Protein_x000D_\nName', 'Prot._x000D_\nId',
       'Scan_x000D_\nTime', 'Scan #', 'Mods_x000D_\n(fixed)', 'Comment',
       'Fragment_x000D_\nType', 'FDR_x000D_\n2D', 'FDR_x000D_\n1D',
       'FDR uniq._x000D_\n2D', 'FDR uniq._x000D_\n1D', 'q-value_x000D_\n2D',
       'q-value_x000D_\n1D', 'Scan info', 'MobilityValue'],
      dtype='object')

Fixed byonic columns:
Index(['PID', 'Prot.\r\nRank', 'Pos.', 'Sequence\r\n(unformatted)',
       'Mods\r\n(variable)', 'Glycans', 'PEP\r\n2D', 'PEP\r\n1D', '|Log Prob|',
       'Score', 'Delta\r\nScor

IndexError: pop index out of range